In [1]:
import mne, glob
import mne.io
from scipy.io import loadmat
import h5py
import pandas as pd
import pymatreader
import timeit
import numpy as np
import matplotlib.pyplot as plt
#mne.sys_info()
from copy import deepcopy
import os
# import our functions
from mne_methods import *
from mne.preprocessing import (ICA, corrmap, create_ecg_epochs, 
                               create_eog_epochs)

## Data sets

In [2]:
training_data = [[r'S_01/night_4\EEG_raw', 5710, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl'], ['c']],
                 [r'S_01/night_4\EEG_raw', 5810, [['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl']], ['ERI', 'EOGl'], ['e']],
                 [r'S_01/night_4\EEG_raw', 12580, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl'], ['d']],
                 [r'S_01/night_4\EEG_raw', 15140, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl'], ['e']],
                 [r'S_01/night_3\EEG_raw', 310, [['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl']], ['EOGr', 'EOGl'], ['e']],# men nu er O1 og O2 ikke ødelagte?
                 [r'S_01/night_3\EEG_raw', 1840, [['EMGr'], ['EMGr'], ['EMGr'], ['EMGr'], ['EMGr'], ['EMGr']], ['EMGr'], ['c']],
                 [r'S_01/night_3\EEG_raw', 3810, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA'], ['b']],                 
                 [r'S_01/night_1\EEG_raw', 120, [['ELC', 'EMGr', 'C3'], ['ELC', 'EMGr', 'C3'], ['C3'], ['ELC', 'EMGr', 'C3'], ['C3'], ['C3']], ['C3'], ['g']],
                 [r'S_01/night_1\EEG_raw', 410, [['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3']], ['EOGr', 'EOGl', 'C3'], ['g']],# der sker noget med øjnene
                 [r'S_01/night_1\EEG_raw', 5780, [['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'C3']], ['C3', 'ELI'], ['d']],
                 [r'S_01/night_1\EEG_raw', 10800, [[], [], [], [], [], []], [], ['a']],# fanger måske C3, men MEGET lille støj amp.
                 [r'S_03/night_1\EEG_raw', 120, [['ERA', 'ERC'], ['ERA', 'ERC', 'C4'], ['ERA', 'ERC', 'EOGl', 'EMGl'], ['ERA', 'ERC', 'EOGl', 'EMGl', 'C4'], ['ERA'], ['ERA', 'ERC']], ['ERA' ], ['g']], # både c og g.
                 [r'S_03/night_1\EEG_raw', 280, [['ERA', 'ERC'], ['ELA', 'ELE', 'ERA', 'ERC'], ['ELA', 'ERA'], ['ELA', 'ERA'], ['ELA', 'ERA'], ['ERA']], ['ERA', 'ELA'], ['g', 'h']],
                 [r'S_03/night_1\EEG_raw', 660, [['ERA', 'ERC', 'EMGc'], ['ERA', 'ERC'], ['ERA', 'ERC'], ['ELC', 'ERA', 'ERC', 'EMGc'], ['ERA', 'ERC'], ['ERA', 'ERC']], ['ERA', 'ERC'], ['d']],# skal ikke finde EMGl og EMGr                 
                 [r'S_02/night_4\EEG_raw', 750, [['EOGl', 'EMGl', 'EMGr'], ['ERC', 'ERT'], ['ERA', 'ERC', 'EOGl', 'EMGl'], ['ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'EOGl', 'EMGl', 'EMGr'], ['EOGl', 'EMGl', 'EMGr']], ['EOGl', 'EMGl', 'EMGr', 'ERC'], ['i']],
                 [r'S_02/night_4\EEG_raw', 11530, [['F3', 'F4'], ['F3'], [], ['F3', 'F4'], [], []], [], ['a']],
                 [r'S_02/night_4\EEG_raw', 12430, [['O1', 'F4', 'O2'], ['F3'], ['ELE', 'ELI'], [], [], []], [], ['a']],
                 [r'S_02/night_4\EEG_raw', 18300, [[], [], [], [], [], []], [], ['a']],
                 [r'S_02/night_4\EEG_raw', 18970, [[], [], [], [], [], []], [], ['a']],
                 [r'S_02/night_4\EEG_raw', 20820, [[], ['ELB', 'ERA', 'ERC'], ['ELB'], ['ELB'], [], []], ['ELB'], ['f', 'h']],# 3 spikes i træk                
                 [r'S_02/night_3\EEG_raw', 120, [['ERA', 'EMGc'], ['ERA'], ['ELC', 'ERA', 'EMGc'], ['ELC', 'ERA'], ['ELC', 'ERA', 'EMGc'], ['ELA', 'ERA']], ['ERA', 'ELC'], ['d']],
                 [r'S_02/night_3\EEG_raw', 4880, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA'], ['b']],
                 [r'S_02/night_3\EEG_raw', 12210, [['ERA'], ['ERA', 'F3'], ['ELA', 'ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA'], ['b']],                 
                 [r'S_01/night_2\EEG_raw', 10, [['EMGr'], ['ELC', 'EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['ELC', 'EMGl', 'EMGr', 'EMGc'], ['EMGl', 'EMGr', 'EMGc'], ['EMGr']],  ['EMGr', 'EMGl'], ['d']],
                 [r'S_01/night_2\EEG_raw', 5854, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3'], ['c']],
                 [r'S_01/night_2\EEG_raw', 7610, [['F3', 'C3'], ['F3', 'F4', 'C4'], ['ELI', 'F3'], ['F3', 'F4'], ['F3'], ['F3']],  ['F3'], ['c']],
                 [r'S_01/night_2\EEG_raw', 10580, [['ELI'], ['ERA', 'ERC', 'ERT'], ['ELE', 'ELI', 'C4'], ['ERA'], ['ELI', 'O2'], ['O2']], [], ['f']],
                 [r'S_01/night_2\EEG_raw', 11920, [['ELI', 'EMGr'], [], ['ELI'], ['EMGr'], ['ELC', 'ELE', 'ELI', 'EMGr'], ['ELC', 'ELE']], [], ['f']],
                 [r'S_01/night_2\EEG_raw', 12640, [['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE']], ['ELE'], ['c']],
                 [r'S_01/night_2\EEG_raw', 13000, [['F3'], ['F3'], ['F3'], ['F3'], ['F3 '], ['ELE', 'F3']], ['F3'], ['c']],
                 [r'S_01/night_2\EEG_raw', 16210, [['EMGr', 'F3'], ['F3'], ['ERI', 'F3'], ['ELI', 'ERI', 'F3', 'C4'], ['ELI', 'F3'], ['F3']], ['F3'], ['g']],
                 [r'S_01/night_2\EEG_raw', 19760, [['F3'], ['ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'EMGc'], [], ['F3'], ['F3'], []], [], ['f']],
                 [r'S_01/night_2\EEG_raw', 21070, [['F3'], ['F3'], ['F3', 'O1'], ['F3', 'O1'], ['F3'], ['F3']], ['F3'], ['b']], # Malene -> Andreas
                [r'S_11/night_1\EEG_raw', 10, [['EOGl','F4'], ['EOGl','F4'], ['EOGl','F4'], ['EOGl','F4'], ['EOGl'], ['EOGl','F4']], ['EOGl','F4'], ["e", "h"]],
                [r'S_11/night_1\EEG_raw', 100, [['EOGl','F4'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl'], ["g", "h"]],
                [r'S_11/night_1\EEG_raw', 570, [['EOGl'], ['EOGl','F4'], ['ELC','EOGl','F4'], ['ELC','EOGl'], ['EOGl'], ['EOGl']], ['EOGl'], ["g", "h"]],
                [r'S_11/night_1\EEG_raw', 900, [['EOGl','F4'], ['EOGl','F4'], ['EOGl','F4'], ['EOGl','F4'], ['EOGl','F4'], ['EOGl','F4']], ['EOGl','F4'], ["d", "h"]],
                [r'S_11/night_1\EEG_raw', 3720, [['EOGl','F4'], ['EOGl','F4'], ['ELE','EOGl','F4'], ['EOGl','F4'], ['EOGl','F4'], ['ERA','ERC','EOGl','F4']], ['EOGl','F4'], ["e", "h"]],
                [r'S_11/night_1\EEG_raw', 4170, [['EOGl','F4'], ['EOGl','F4'], ['EOGl','F4'], ['EOGl','F4'], ['EOGl','F4'], ['EOGl','F4']], ['EOGl','F4'], ["d", "h"]],
                [r'S_11/night_1\EEG_raw', 4560, [['EOGl','F4'], ['EOGl','F4'], ['EOGl','F4'], ['EOGl','F4'], ['EOGl','F4'], ['EOGl','F4']], ['EOGl','F4'], ["e", "h"]],
                [r'S_11/night_1\EEG_raw', 4840, [['EOGl','F4'], ['EOGl','F4'], ['EOGl','F4'], ['EOGl','F4'], ['EOGl','F4'], ['EOGl','F4']], ['EOGl','F4'], ["e", "h"]],
                [r'S_11/night_1\EEG_raw', 6400, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl','C4'], ["b", "h"]],
                [r'S_11/night_1\EEG_raw', 6660, [['EOGl','F4'], ['EOGl','F4'], ['EOGl','F4'], ['EOGl','F4'], ['EOGl','F4'], ['F4']], ['EOGl','F4'], ["d", "h"]],
                [r'S_13/night_1\EEG_raw', 10, [['ERE','EMGl','EMGr'], ['ERE','EMGl','EMGr','EMGc'], ['ERE','EMGl','EMGr','EMGc'], [], ['ERE','EMGl','EMGr'], ['ERE','EMGl','EMGr','EMGc']], ['ERE'], ["g"]],
                [r'S_13/night_1\EEG_raw', 1000, [['ERE'], ['ERE'], ['ELB','ERB','ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE'], ["g"]],
                [r'S_13/night_1\EEG_raw', 6000, [[], ['ELB','ELC','ERC','O2'], ['ERC','ERE','ERI'], [], [], []], [], ["f"]],
                [r'S_13/night_1\EEG_raw', 12000, [['EMGl','EMGc'], ['EMGr','EMGl','EMGc'], [], [], [], []], [], ["f"]],
                [r'S_14/night_1\EEG_raw', 1500, [['F4','O2'], ['F4','O2'], ['F4','O2'], ['F4','O2'], ['F4','O2'], ['F4','O2']], ['F4','O2'], ["d", "h"]], #Missing whole channel.
                [r'S_14/night_1\EEG_raw', 3000, [['ELA','F4'], ['ELA','ERB','F4'], ['ELA','ERB','F4'], ['ELA','F4'], ['ELA','F4'], ['ELA','F4']], ['ELA','F4'], ["e", "h"]], #Missing whole channel.
                [r'S_14/night_1\EEG_raw', 4000, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4'], ["g", "h"]], #Missing whole channel.
                [r'S_14/night_1\EEG_raw', 5500, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4'], ["b", "h"]], #Missing whole channel.
                [r'S_14/night_1\EEG_raw', 7500, [[], [], ['F4'], ['F4'], ['F4'], []], ['F4'], ["g", "h"]], #Missing whole channel.
                [r'S_15/night_1\EEG_raw', 5000, [['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE'], ["c", "h"]], #Missing whole channels.
                [r'S_17/night_1\EEG_raw', 100, [[], [], [], ['ERA','EMGl','EMGr'], [], []], [], ["a", "h"]], # Will proably find some artifacts.
                [r'S_17/night_1\EEG_raw', 1000, [[], [], [], [], ['ELA','ELE'], []], [], ["f", "h"]],
                [r'S_17/night_1\EEG_raw', 2000, [[], [], [], ['EMGl','F4','EMGc'], [], []], [], ["f", "h"]],
                [r'S_17/night_1\EEG_raw', 3000, [[], [], [], [], [], []], [], ["a", "h"]],
                [r'S_17/night_1\EEG_raw', 4000, [[], [], [], [], [], []], [], ["a", "h"]],
                [r'S_17/night_1\EEG_raw', 5000, [[], [], [], [], [], ['F3']], [], ["a", "h"]],
                [r'S_17/night_1\EEG_raw', 6000, [[], [], [], ['EOGl'], [], []], [], ["a", "h"]],
                [r'S_17/night_1\EEG_raw', 7000, [[], [], [], [], [], []], [], ["a", "h"]],
                [r'S_17/night_1\EEG_raw', 8000, [[], [], [], [], [], []], [], ["a", "h"]],
                [r'S_17/night_1\EEG_raw', 9000, [[], [], [], [], [], []], [], ["a"]],
                [r'S_17/night_1\EEG_raw', 10000, [[], [], [], ['F3','F4'], [], []], [], ["f"]],
                [r'S_17/night_1\EEG_raw', 11000, [[], [], [], [], [], []], [], ["a"]]
                  ]

In [3]:
test_data = [[r'S_01/night_1\EEG_raw', 170, [['ERB', 'EMGr', 'C3'], ['C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3'], ['c']],
             [r'S_01/night_1\EEG_raw', 960, [['ERB', 'C3'], ['ERB', 'C3'], ['ERB', 'C3'], ['ERB', 'C3'], ['ERB', 'C3'], ['C3']], ['ERB', 'C3'], ['d']], # måske samme amp. men de er forskelligt ødelagte
             [r'S_01/night_1\EEG_raw', 1830, [['C3', 'C4'], ['C3', 'C4'], ['C3', 'C4'], ['C3', 'C4'], ['C3', 'C4'], ['C3', 'C4']], ['C3', 'C4'], ['d']], # C3 skaber spikes i alle andre kanaler
             [r'S_01/night_1\EEG_raw', 6290, [['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'C3']], ['C3', 'ELI'], ['d']],
             [r'S_01/night_1\EEG_raw', 7050, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3'], ['b']],
             [r'S_01/night_3\EEG_raw', 240, [['O1', 'O2'], ['O1', 'O2'], ['O1', 'O2'], ['O1', 'O2'], ['O1', 'O2'], ['O1', 'O2']], ['O1', 'O2'], ['c']], # MEGET lille amp., små spikes på andre kanaler
             [r'S_01/night_4\EEG_raw', 15320, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl'], ['e']],
             [r'S_01/night_4\EEG_raw', 1670, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl'], ['c']], # Malene -> Andreas
             [r'S_12/night_1\EEG_raw', 10, [['EOGr','F4'], ['EOGr','F4'], ['EOGr','F4'], ['EOGr','F4'], ['EOGr','F4'], ['EOGr','F4']], ['EOGr','F4'], ["d"]],
             [r'S_12/night_1\EEG_raw', 510, [['EOGr','F4'], ['EOGr','F4'], ['EOGr','F4'], ['EOGr','F4'], ['EOGr','F4'], ['EOGr','F4']], ['EOGr','F4'], ["g"]],
             [r'S_12/night_1\EEG_raw', 2000, [['EOGr','F4'], ['EOGr','F4'], ['EOGr','F4'], ['EOGr','F4'], ['EOGr','F4'], ['EOGr','F4']], ['EOGr','F4'], ["d"]],
             [r'S_12/night_1\EEG_raw', 3000, [['EOGr','F4'], ['EOGr','F4'], ['EOGr','F4'], ['EOGr','F4'], ['EOGr','F4'], ['EOGr','F4']], ['EOGr','F4'], ["g"]],
             [r'S_13/night_1\EEG_raw', 8000, [['F3','F4'], [], [], [], [], []], [], ["f"]],
             [r'S_13/night_1\EEG_raw', 10000, [[], ['ELC','EMGl','EMGr','EMGc'], [], [], [], []], [], ["f"]],
             [r'S_14/night_1\EEG_raw', 2500, [['ELC'], ['ELC'], ['ELC'], ['ELC'], ['ELC'], ['ELC']], ['ELC'], ["b", "h"]], #Missing whole channel.
             [r'S_14/night_1\EEG_raw', 4500, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4'], ["b", "h"]] #Missing whole channel.
            ] 
    

In [4]:
print(len(training_data))
print(len(test_data))

65
16


## Helper functions

In [5]:
def check_file_exists(data_set, simulation):
    if simulation:
        local_folder = r"D:\Bachelor-Project-Data\Manual_data_28_05_2023"
    else:
        local_folder = r"D:\Bachelor-Project-Data\Kaare\study_1A_mat_simple"
    
    for data in data_set:
        folder_path = local_folder + "\ "[:-1] + data[0]
        
        if not os.path.isfile(folder_path + ".mat"):
            print(folder_path)

In [6]:
check_file_exists(training_data, False)
check_file_exists(test_data, False)

In [7]:
def count_types(data_set):
    catogory_dictonary = {}
    
    # Runs through each data point in the data set
    for data_point in range(len(data_set)):
        
        #Runs through each category type:
        for category_type in data_set[data_point][4]:
            if category_type in catogory_dictonary.keys():
                # Add 1 to the type
                catogory_dictonary.update({category_type: catogory_dictonary.get(category_type) + 1})
            else:
                # Add a new type to the dictonary
                catogory_dictonary.update({category_type: 1})
    return catogory_dictonary

In [8]:
count_types(training_data)

{'c': 7, 'e': 8, 'd': 9, 'b': 6, 'g': 11, 'a': 14, 'h': 27, 'i': 1, 'f': 9}

In [9]:
count_types(test_data)

{'c': 3, 'd': 5, 'b': 3, 'e': 1, 'g': 2, 'f': 2, 'h': 2}

# Optimisation

Parameters to optimise:
* Relative procent  {1.00, 1.5, 2.00, 2.5, 3.00, 4.00, 5.00}
* Measure of cross {M_idx, M_max}
* Matrix {MM, UM}
* Number of 10 s. intervals in a row {1, 2, 3, 4, 5, 6}

$7\cdot2\cdot2\cdot6 = 168$ grid search

In [10]:
relative_procents = [1.00, 1.5, 2.00, 2.5, 3.00, 4.00, 5.00]
measures = ['M_idx','M_max']
matrices = ['mm','um']
artifact_intervals = [1, 2, 3, 4, 5, 6]
parameters_op = [measures, matrices, relative_procents, artifact_intervals]

## Optimisation Broken Electrode classifier

In [11]:
def optimise_set(data_set, simulation, parameters):
    # Empty list to the output.
    outputs = [] 
    
    start_cont = timeit.default_timer() # starts timer
    for data in data_set:
        #Running through every data point
        start = timeit.default_timer() # starts timer
        
        # Extracting the information of the data.
        if simulation:
            local_folder = r"D:\Bachelor-Project-Data\Manual_data_28_05_2023"
        else:
            local_folder = r"D:\Bachelor-Project-Data\Kaare\study_1A_mat_simple"
        folder_path = local_folder + "\ "[:-1] + data[0] #S_17/night_1\EEG_raw
        time_start = data[1]
        artifact_labels = data[2]
        broken_labels = data[3]
        
        # Loading the full data.
        raw_all = load_mne.file_to_raw(folder_path + ".mat")
        
        # Cropping it to the 60 seconds + buffer interval.
        raw_all = raw_all.crop(tmin=time_start - 4, tmax = time_start + 64)
        
        if simulation:
            #No preprocess
            raw_all_processed = raw_all.crop(tmin=4, tmax=64)

        else:
            # Preprocess
            raw_all_processed, NaN = load_mne.preprocess(raw_all)
            # Remove bufer interval.
            raw_all_processed = raw_all_processed.crop(tmin=4, tmax=64)
        
        # Prepares to loop through intervals.
        intv_count = 6
        intv_size = 10
        intv_end = intv_count * intv_size + 1 #Etc. 6*10 + 1
        time_intv = [x for x in range(0, intv_end, intv_size)]
        
        # ICA on intervals
        for intv_idx in range(intv_count):
            # Running through each interval
            folder_int = folder_path + "_time=" + str(time_start + time_intv[intv_idx]) + "-" + str(time_start +time_intv[intv_idx + 1])
            
            # Cropping to the interval to perform multiple functions [0, 10], [10, 20] etc.
            raw_crop_int = raw_all_processed.copy().crop(tmin = time_intv[intv_idx], tmax = time_intv[intv_idx + 1])
            
            # Runs multiple ICA for 2 - 15 number of components, both measures and matrices are used.
            ICA_subfunctions.multiple_ica(raw_crop_int, 2, 15, folder_int, matrix=parameters[1], cross_measure=parameters[0])
            
            # Runs multi_plot to get a summary of all 2 - 15 components.
            if type(parameters[1]) is not list and type(parameters[0]) is not list:
                ICA_subfunctions.multi_plot(2, 15, folder_int, matrix_list=[parameters[1]], cross_measure=[parameters[0]])
            else:
                ICA_subfunctions.multi_plot(2, 15, folder_int, matrix_list=parameters[1], cross_measure=parameters[0])
        
        if type(parameters[3]) is not list and type(parameters[2]) is not list and type(parameters[1]) is not list and type(parameters[0]) is not list:
            # Running the artifact classifier with relative procent on each ICA interval
            artifact_intervals = []
            for intv_idx in range(intv_count):
                folder_int = folder_path + "_time=" + str(time_start + time_intv[intv_idx]) + "-" + str(time_start +time_intv[intv_idx + 1])
                artifact_intervals.append(ICA_subfunctions.artifact_classifier(folder_int, parameters[2], cross=parameters[0], matrix=parameters[1]))

            # Running the broken electrode classifier with number of 10 s. intervals in row.
            broken_pred = ICA_subfunctions.broken_channel_classifier(artifact_intervals, broken_interval_size = parameters[3])

            # Output the predictions for the combination of parameters.
            outputs.append([data[0], time_start, parameters[0], parameters[1], parameters[2], parameters[3], artifact_intervals, broken_pred])
        else:
            # Artifact classifier
            for art_int in parameters[3]:
                for rel_pro in parameters[2]:
                    for matrix in parameters[1]:
                        for cross in parameters[0]:
                            # Running the artifact classifier with relative procent on each ICA interval
                            artifact_intervals = []
                            for intv_idx in range(intv_count):
                                folder_int = folder_path + "_time=" + str(time_start + time_intv[intv_idx]) + "-" + str(time_start +time_intv[intv_idx + 1])
                                artifact_intervals.append(ICA_subfunctions.artifact_classifier(folder_int, rel_pro, cross=cross, matrix=matrix))

                            # Running the broken electrode classifier with number of 10 s. intervals in row.
                            broken_pred = ICA_subfunctions.broken_channel_classifier(artifact_intervals, broken_interval_size = art_int)

                            # Output the predictions for the combination of parameters.
                            outputs.append([data[0], time_start, cross, matrix, rel_pro, art_int, artifact_intervals, broken_pred])
        
        # ends timer
        stop = timeit.default_timer() 
        total_time = (stop - start_cont)/60
        time_estimated = (stop - start)*len(data_set)/60
        print(str(total_time)[:4] + "min. Estimated time left with this speed: " + str(time_estimated - total_time)[:4] + "min.")
    return outputs

In [12]:
#training_outputs = optimise_set(training_data, False, parameters_op)
#print(training_outputs)

## Predictions

Measured training

In [13]:
training_predictions = [['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERC', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGl', 'EMGr', 'F3', 'EMGc']], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 1.0, 1, [['ELB', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERC', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'C4']], ['ELB', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGr', 'EMGc', 'ERC', 'ELA']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'EOGr', 'EOGl']], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERI', 'EMGl', 'EMGr', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERI', 'EMGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERI', 'EMGl', 'EMGr', 'C3', 'F4', 'EMGc', 'EOGr', 'C4', 'O2', 'O1', 'F3']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 1.5, 1, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['ERI', 'EOGl']], ['EOGl', 'ERI']], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 1.5, 1, [['ELB', 'EOGl', 'F3', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F3', 'F4'], ['EOGl'], ['EOGl'], ['ERI', 'EOGl']], ['ELB', 'EOGl', 'F3', 'F4', 'ERI']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 1.5, 1, [['ELB'], ['EMGr'], [], [], [], []], ['ELB', 'EMGr']], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 1.5, 1, [['ELB', 'ELC'], ['ELA', 'ERC', 'ERI', 'EMGr'], ['ERI', 'EMGc'], ['ELA', 'ELB', 'ERI'], ['ERC', 'ERI'], ['ELA', 'ELC']], ['ELB', 'ELC', 'ELA', 'ERC', 'ERI', 'EMGr', 'EMGc']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 2.0, 1, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['ERI', 'EOGl']], ['EOGl', 'ERI']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 2.0, 1, [[], ['EMGr'], ['EMGc'], [], [], []], ['EMGr', 'EMGc']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 2.5, 1, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['ERI', 'EOGl']], ['EOGl', 'ERI']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 3.0, 1, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['ERI', 'EOGl']], ['EOGl', 'ERI']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 4.0, 1, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 5.0, 1, [['EOGl'], [], [], [], [], ['EOGl']], ['EOGl']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERC', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGl', 'EMGr', 'F3']], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 1.0, 2, [['ELB', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERC', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'C4']], ['ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERI', 'EMGl', 'EMGr', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERI', 'EMGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERI', 'EMGl', 'EMGr', 'C3', 'F4', 'EMGc', 'EOGr', 'C4', 'O2']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 1.5, 2, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['ERI', 'EOGl']], ['EOGl']], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 1.5, 2, [['ELB', 'EOGl', 'F3', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F3', 'F4'], ['EOGl'], ['EOGl'], ['ERI', 'EOGl']], ['EOGl', 'F4']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 1.5, 2, [['ELB'], ['EMGr'], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 1.5, 2, [['ELB', 'ELC'], ['ELA', 'ERC', 'ERI', 'EMGr'], ['ERI', 'EMGc'], ['ELA', 'ELB', 'ERI'], ['ERC', 'ERI'], ['ELA', 'ELC']], ['ERI']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 2.0, 2, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['ERI', 'EOGl']], ['EOGl']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 2.0, 2, [[], ['EMGr'], ['EMGc'], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 2.5, 2, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['ERI', 'EOGl']], ['EOGl']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 3.0, 2, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['ERI', 'EOGl']], ['EOGl']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 4.0, 2, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 5.0, 2, [['EOGl'], [], [], [], [], ['EOGl']], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERC', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELC', 'ERC', 'ERI', 'EOGl', 'C3', 'O1', 'F4', 'C4', 'O2', 'F3']], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 1.0, 3, [['ELB', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERC', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'C4']], ['ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERI', 'EMGl', 'EMGr', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERI', 'EMGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'C3', 'EMGc']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 1.5, 3, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['ERI', 'EOGl']], ['EOGl']], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 1.5, 3, [['ELB', 'EOGl', 'F3', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F3', 'F4'], ['EOGl'], ['EOGl'], ['ERI', 'EOGl']], ['EOGl', 'F4']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 1.5, 3, [['ELB'], ['EMGr'], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 1.5, 3, [['ELB', 'ELC'], ['ELA', 'ERC', 'ERI', 'EMGr'], ['ERI', 'EMGc'], ['ELA', 'ELB', 'ERI'], ['ERC', 'ERI'], ['ELA', 'ELC']], ['ERI']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 2.0, 3, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['ERI', 'EOGl']], ['EOGl']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 2.0, 3, [[], ['EMGr'], ['EMGc'], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 2.5, 3, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['ERI', 'EOGl']], ['EOGl']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 3.0, 3, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['ERI', 'EOGl']], ['EOGl']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 4.0, 3, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 5.0, 3, [['EOGl'], [], [], [], [], ['EOGl']], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERC', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELC', 'ERC', 'ERI', 'EOGl', 'C3', 'O1', 'F4', 'C4', 'O2', 'F3']], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 1.0, 4, [['ELB', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERC', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'C4']], ['ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERI', 'EMGl', 'EMGr', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERI', 'EMGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'EMGc']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 1.5, 4, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['ERI', 'EOGl']], ['EOGl']], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 1.5, 4, [['ELB', 'EOGl', 'F3', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F3', 'F4'], ['EOGl'], ['EOGl'], ['ERI', 'EOGl']], ['EOGl']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 1.5, 4, [['ELB'], ['EMGr'], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 1.5, 4, [['ELB', 'ELC'], ['ELA', 'ERC', 'ERI', 'EMGr'], ['ERI', 'EMGc'], ['ELA', 'ELB', 'ERI'], ['ERC', 'ERI'], ['ELA', 'ELC']], ['ERI']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 2.0, 4, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['ERI', 'EOGl']], ['EOGl']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 2.0, 4, [[], ['EMGr'], ['EMGc'], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 2.5, 4, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['ERI', 'EOGl']], ['EOGl']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 3.0, 4, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['ERI', 'EOGl']], ['EOGl']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 4.0, 4, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 5.0, 4, [['EOGl'], [], [], [], [], ['EOGl']], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERC', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELC', 'ERC', 'ERI', 'EOGl', 'C3', 'O1', 'F4', 'C4', 'O2', 'F3']], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 1.0, 5, [['ELB', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERC', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'C4']], ['EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2']], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERI', 'EMGl', 'EMGr', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERI', 'EMGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 1.5, 5, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['ERI', 'EOGl']], ['EOGl']], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 1.5, 5, [['ELB', 'EOGl', 'F3', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F3', 'F4'], ['EOGl'], ['EOGl'], ['ERI', 'EOGl']], ['EOGl']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 1.5, 5, [['ELB'], ['EMGr'], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 1.5, 5, [['ELB', 'ELC'], ['ELA', 'ERC', 'ERI', 'EMGr'], ['ERI', 'EMGc'], ['ELA', 'ELB', 'ERI'], ['ERC', 'ERI'], ['ELA', 'ELC']], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 2.0, 5, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['ERI', 'EOGl']], ['EOGl']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 2.0, 5, [[], ['EMGr'], ['EMGc'], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 2.5, 5, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['ERI', 'EOGl']], ['EOGl']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 3.0, 5, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['ERI', 'EOGl']], ['EOGl']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 4.0, 5, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 5.0, 5, [['EOGl'], [], [], [], [], ['EOGl']], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERC', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELC', 'ERC', 'ERI', 'EOGl', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 1.0, 6, [['ELB', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERC', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'C4']], ['EOGl', 'F3', 'C3', 'F4', 'C4']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'F3', 'C3', 'C4', 'O2']], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERI', 'EMGl', 'EMGr', 'C3', 'F4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERI', 'EMGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EMGl']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 1.5, 6, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['ERI', 'EOGl']], ['EOGl']], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 1.5, 6, [['ELB', 'EOGl', 'F3', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F3', 'F4'], ['EOGl'], ['EOGl'], ['ERI', 'EOGl']], ['EOGl']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 1.5, 6, [['ELB'], ['EMGr'], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 1.5, 6, [['ELB', 'ELC'], ['ELA', 'ERC', 'ERI', 'EMGr'], ['ERI', 'EMGc'], ['ELA', 'ELB', 'ERI'], ['ERC', 'ERI'], ['ELA', 'ELC']], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 2.0, 6, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['ERI', 'EOGl']], ['EOGl']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 2.0, 6, [[], ['EMGr'], ['EMGc'], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 2.5, 6, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['ERI', 'EOGl']], ['EOGl']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 3.0, 6, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['ERI', 'EOGl']], ['EOGl']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 4.0, 6, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'mm', 5.0, 6, [['EOGl'], [], [], [], [], ['EOGl']], []], ['S_01/night_4\\EEG_raw', 5710, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5710, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'M1', 'M2', 'EMGl']], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 1.0, 1, [['ELB', 'ERI', 'EOGl', 'F3', 'F4', 'C4', 'O2'], ['ERI', 'EOGl', 'F3', 'F4'], ['ERI', 'EOGl', 'M1', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ERI', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ERI', 'EOGl', 'F3', 'C3'], ['ERI', 'EOGl', 'F4']], ['ELB', 'ERI', 'EOGl', 'F3', 'F4', 'C4', 'O2', 'M1', 'O1', 'C3']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ERB', 'M1', 'M2']], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc', 'C3', 'F4', 'O2', 'M1', 'M2', 'ELE']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 1.5, 1, [['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI'], ['ERI', 'EOGl']], ['ERI', 'EOGl']], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 1.5, 1, [['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl']], ['ERI', 'EOGl']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], ['EOGl']], ['EOGl']], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 1.5, 1, [['ELB', 'EOGr'], ['ELA', 'ELB'], [], ['ELB', 'ELC', 'ERC'], ['ELA', 'ELC', 'ERA', 'ERC'], ['ELA', 'ELC', 'ERA', 'ERC']], ['ELB', 'EOGr', 'ELA', 'ELC', 'ERC', 'ERA']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 2.0, 1, [[], ['ERI'], [], [], [], []], ['ERI']], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 2.0, 1, [['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl']], ['ERI', 'EOGl']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 2.0, 1, [['EOGr'], [], [], ['ELB'], [], []], ['EOGr', 'ELB']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 2.5, 1, [['ERI', 'EOGl'], ['ERI'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl']], ['ERI', 'EOGl']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 3.0, 1, [['ERI', 'EOGl'], ['ERI'], ['ERI'], ['ERI', 'EOGl'], ['ERI'], ['ERI']], ['ERI', 'EOGl']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 4.0, 1, [[], ['ERI'], [], [], ['ERI'], ['ERI']], ['ERI']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 5.0, 1, [[], ['ERI'], [], [], ['ERI'], ['ERI']], ['ERI']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2']], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 1.0, 2, [['ELB', 'ERI', 'EOGl', 'F3', 'F4', 'C4', 'O2'], ['ERI', 'EOGl', 'F3', 'F4'], ['ERI', 'EOGl', 'M1', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ERI', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ERI', 'EOGl', 'F3', 'C3'], ['ERI', 'EOGl', 'F4']], ['ERI', 'EOGl', 'F3', 'F4', 'C4', 'O2']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C4', 'EMGc', 'C3', 'F4', 'O2']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 1.5, 2, [['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI'], ['ERI', 'EOGl']], ['ERI', 'EOGl']], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 1.5, 2, [['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl']], ['ERI', 'EOGl']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], ['EOGl']], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 1.5, 2, [['ELB', 'EOGr'], ['ELA', 'ELB'], [], ['ELB', 'ELC', 'ERC'], ['ELA', 'ELC', 'ERA', 'ERC'], ['ELA', 'ELC', 'ERA', 'ERC']], ['ELB', 'ELA', 'ELC', 'ERC', 'ERA']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 2.0, 2, [[], ['ERI'], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 2.0, 2, [['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl']], ['ERI', 'EOGl']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 2.0, 2, [['EOGr'], [], [], ['ELB'], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 2.5, 2, [['ERI', 'EOGl'], ['ERI'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl']], ['ERI', 'EOGl']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 3.0, 2, [['ERI', 'EOGl'], ['ERI'], ['ERI'], ['ERI', 'EOGl'], ['ERI'], ['ERI']], ['ERI']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 4.0, 2, [[], ['ERI'], [], [], ['ERI'], ['ERI']], ['ERI']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 5.0, 2, [[], ['ERI'], [], [], ['ERI'], ['ERI']], ['ERI']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2']], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 1.0, 3, [['ELB', 'ERI', 'EOGl', 'F3', 'F4', 'C4', 'O2'], ['ERI', 'EOGl', 'F3', 'F4'], ['ERI', 'EOGl', 'M1', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ERI', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ERI', 'EOGl', 'F3', 'C3'], ['ERI', 'EOGl', 'F4']], ['ERI', 'EOGl', 'F3', 'F4']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C4', 'C3', 'F4', 'O2']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 1.5, 3, [['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI'], ['ERI', 'EOGl']], ['ERI', 'EOGl']], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 1.5, 3, [['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl']], ['ERI', 'EOGl']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], ['EOGl']], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 1.5, 3, [['ELB', 'EOGr'], ['ELA', 'ELB'], [], ['ELB', 'ELC', 'ERC'], ['ELA', 'ELC', 'ERA', 'ERC'], ['ELA', 'ELC', 'ERA', 'ERC']], ['ELC', 'ERC']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 2.0, 3, [[], ['ERI'], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 2.0, 3, [['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl']], ['ERI', 'EOGl']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 2.0, 3, [['EOGr'], [], [], ['ELB'], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 2.5, 3, [['ERI', 'EOGl'], ['ERI'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl']], ['ERI', 'EOGl']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 3.0, 3, [['ERI', 'EOGl'], ['ERI'], ['ERI'], ['ERI', 'EOGl'], ['ERI'], ['ERI']], ['ERI']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 4.0, 3, [[], ['ERI'], [], [], ['ERI'], ['ERI']], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 5.0, 3, [[], ['ERI'], [], [], ['ERI'], ['ERI']], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ERC', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2']], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 1.0, 4, [['ELB', 'ERI', 'EOGl', 'F3', 'F4', 'C4', 'O2'], ['ERI', 'EOGl', 'F3', 'F4'], ['ERI', 'EOGl', 'M1', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ERI', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ERI', 'EOGl', 'F3', 'C3'], ['ERI', 'EOGl', 'F4']], ['ERI', 'EOGl', 'F3', 'F4']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EMGr', 'C4', 'C3', 'O2']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 1.5, 4, [['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI'], ['ERI', 'EOGl']], ['ERI', 'EOGl']], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 1.5, 4, [['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl']], ['ERI', 'EOGl']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], ['EOGl']], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 1.5, 4, [['ELB', 'EOGr'], ['ELA', 'ELB'], [], ['ELB', 'ELC', 'ERC'], ['ELA', 'ELC', 'ERA', 'ERC'], ['ELA', 'ELC', 'ERA', 'ERC']], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 2.0, 4, [[], ['ERI'], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 2.0, 4, [['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl']], ['ERI', 'EOGl']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 2.0, 4, [['EOGr'], [], [], ['ELB'], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 2.5, 4, [['ERI', 'EOGl'], ['ERI'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl']], ['ERI', 'EOGl']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 3.0, 4, [['ERI', 'EOGl'], ['ERI'], ['ERI'], ['ERI', 'EOGl'], ['ERI'], ['ERI']], ['ERI']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 4.0, 4, [[], ['ERI'], [], [], ['ERI'], ['ERI']], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 5.0, 4, [[], ['ERI'], [], [], ['ERI'], ['ERI']], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERI', 'EOGl', 'F3', 'C3', 'F4', 'C4']], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 1.0, 5, [['ELB', 'ERI', 'EOGl', 'F3', 'F4', 'C4', 'O2'], ['ERI', 'EOGl', 'F3', 'F4'], ['ERI', 'EOGl', 'M1', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ERI', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ERI', 'EOGl', 'F3', 'C3'], ['ERI', 'EOGl', 'F4']], ['ERI', 'EOGl', 'F3']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'ERC', 'F3', 'C3', 'F4', 'C4', 'O2']], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EMGr', 'C4', 'C3']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 1.5, 5, [['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI'], ['ERI', 'EOGl']], ['ERI']], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 1.5, 5, [['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl']], ['ERI', 'EOGl']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], ['EOGl']], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 1.5, 5, [['ELB', 'EOGr'], ['ELA', 'ELB'], [], ['ELB', 'ELC', 'ERC'], ['ELA', 'ELC', 'ERA', 'ERC'], ['ELA', 'ELC', 'ERA', 'ERC']], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 2.0, 5, [[], ['ERI'], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 2.0, 5, [['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl']], ['ERI', 'EOGl']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 2.0, 5, [['EOGr'], [], [], ['ELB'], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 2.5, 5, [['ERI', 'EOGl'], ['ERI'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl']], ['ERI']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 3.0, 5, [['ERI', 'EOGl'], ['ERI'], ['ERI'], ['ERI', 'EOGl'], ['ERI'], ['ERI']], ['ERI']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 4.0, 5, [[], ['ERI'], [], [], ['ERI'], ['ERI']], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 5.0, 5, [[], ['ERI'], [], [], ['ERI'], ['ERI']], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERI', 'EOGl', 'F3', 'C3', 'F4', 'C4']], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 1.0, 6, [['ELB', 'ERI', 'EOGl', 'F3', 'F4', 'C4', 'O2'], ['ERI', 'EOGl', 'F3', 'F4'], ['ERI', 'EOGl', 'M1', 'F3', 'F4', 'C4', 'O2'], ['ELB', 'ERI', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ERI', 'EOGl', 'F3', 'C3'], ['ERI', 'EOGl', 'F4']], ['ERI', 'EOGl']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'ERC', 'F3', 'C3', 'F4', 'C4', 'O2']], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ERA', 'ERC', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EMGr', 'C4']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 1.5, 6, [['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI'], ['ERI', 'EOGl']], ['ERI']], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 1.5, 6, [['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl']], ['ERI', 'EOGl']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], ['EOGl']], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 1.5, 6, [['ELB', 'EOGr'], ['ELA', 'ELB'], [], ['ELB', 'ELC', 'ERC'], ['ELA', 'ELC', 'ERA', 'ERC'], ['ELA', 'ELC', 'ERA', 'ERC']], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 2.0, 6, [[], ['ERI'], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 2.0, 6, [['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl']], ['ERI', 'EOGl']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 2.0, 6, [['EOGr'], [], [], ['ELB'], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 2.5, 6, [['ERI', 'EOGl'], ['ERI'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl'], ['ERI', 'EOGl']], ['ERI']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 3.0, 6, [['ERI', 'EOGl'], ['ERI'], ['ERI'], ['ERI', 'EOGl'], ['ERI'], ['ERI']], ['ERI']], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 4.0, 6, [[], ['ERI'], [], [], ['ERI'], ['ERI']], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'mm', 5.0, 6, [[], ['ERI'], [], [], ['ERI'], ['ERI']], []], ['S_01/night_4\\EEG_raw', 5810, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 5810, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ERA', 'ERI', 'EOGl', 'F3', 'C3', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'ERC', 'ERI', 'EOGl', 'F4', 'O2'], ['ERA', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ELC', 'ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4']], ['ELA', 'ERA', 'ERI', 'EOGl', 'F3', 'C3', 'C4', 'O2', 'ELC', 'F4', 'ERC', 'EMGl', 'O1', 'ERB', 'EOGr']], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 1.0, 1, [['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'O2'], ['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2', 'ERI']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EOGr', 'EMGr', 'EMGc']], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EOGr', 'O1', 'EMGc']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 1.5, 1, [['ERA', 'EOGl'], ['ERA', 'ERI', 'C3'], ['ERA', 'ERI'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl', 'ERI', 'C3']], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 1.5, 1, [['ERA', 'EOGl'], ['ERA'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl', 'F3', 'F4']], ['ERA', 'EOGl', 'F3', 'F4']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 1.5, 1, [[], [], [], [], ['ERI'], []], ['ERI']], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 1.5, 1, [['ELA', 'ERB', 'ERI'], ['ERB', 'ERI'], ['ELA', 'ERB', 'ERI'], ['ERI'], ['ELA', 'ERI'], ['ELA', 'ERI', 'EOGr']], ['ELA', 'ERB', 'ERI', 'EOGr']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 2.0, 1, [['ERA', 'EOGl'], ['ERA'], ['ERA', 'ERI'], ['ERA'], ['ERA', 'EOGl'], ['ERA']], ['ERA', 'EOGl', 'ERI']], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 2.0, 1, [['ERA', 'EOGl'], ['ERA'], ['ERA'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 2.0, 1, [[], [], [], [], ['ERI'], []], ['ERI']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 2.5, 1, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], []], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 2.5, 1, [['ERA', 'EOGl'], ['ERA'], ['ERA'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 3.0, 1, [[], ['ERA'], ['ERA'], [], [], []], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 3.0, 1, [['ERA', 'EOGl'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 4.0, 1, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 5.0, 1, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ERA', 'ERI', 'EOGl', 'F3', 'C3', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'ERC', 'ERI', 'EOGl', 'F4', 'O2'], ['ERA', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ELC', 'ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4']], ['ELA', 'ERA', 'ERI', 'EOGl', 'F3', 'C3', 'C4', 'O2', 'F4', 'O1', 'ERB']], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 1.0, 2, [['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'O2'], ['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EOGr', 'EMGr', 'EMGc']], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EOGr', 'EMGc']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 1.5, 2, [['ERA', 'EOGl'], ['ERA', 'ERI', 'C3'], ['ERA', 'ERI'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl', 'ERI']], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 1.5, 2, [['ERA', 'EOGl'], ['ERA'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl', 'F3', 'F4']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 1.5, 2, [[], [], [], [], ['ERI'], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 1.5, 2, [['ELA', 'ERB', 'ERI'], ['ERB', 'ERI'], ['ELA', 'ERB', 'ERI'], ['ERI'], ['ELA', 'ERI'], ['ELA', 'ERI', 'EOGr']], ['ELA', 'ERB', 'ERI']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 2.0, 2, [['ERA', 'EOGl'], ['ERA'], ['ERA', 'ERI'], ['ERA'], ['ERA', 'EOGl'], ['ERA']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 2.0, 2, [['ERA', 'EOGl'], ['ERA'], ['ERA'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 2.0, 2, [[], [], [], [], ['ERI'], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 2.5, 2, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], []], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 2.5, 2, [['ERA', 'EOGl'], ['ERA'], ['ERA'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 3.0, 2, [[], ['ERA'], ['ERA'], [], [], []], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 3.0, 2, [['ERA', 'EOGl'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 4.0, 2, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 5.0, 2, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ERA', 'ERI', 'EOGl', 'F3', 'C3', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'ERC', 'ERI', 'EOGl', 'F4', 'O2'], ['ERA', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ELC', 'ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4']], ['ERA', 'ERI', 'EOGl', 'F3', 'C3', 'C4', 'F4']], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 1.0, 3, [['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'O2'], ['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ERA', 'EOGl', 'F3', 'C3', 'F4']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EOGr']], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EOGr', 'EMGc']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 1.5, 3, [['ERA', 'EOGl'], ['ERA', 'ERI', 'C3'], ['ERA', 'ERI'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 1.5, 3, [['ERA', 'EOGl'], ['ERA'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl', 'F3', 'F4']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 1.5, 3, [[], [], [], [], ['ERI'], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 1.5, 3, [['ELA', 'ERB', 'ERI'], ['ERB', 'ERI'], ['ELA', 'ERB', 'ERI'], ['ERI'], ['ELA', 'ERI'], ['ELA', 'ERI', 'EOGr']], ['ERB', 'ERI']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 2.0, 3, [['ERA', 'EOGl'], ['ERA'], ['ERA', 'ERI'], ['ERA'], ['ERA', 'EOGl'], ['ERA']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 2.0, 3, [['ERA', 'EOGl'], ['ERA'], ['ERA'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 2.0, 3, [[], [], [], [], ['ERI'], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 2.5, 3, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], []], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 2.5, 3, [['ERA', 'EOGl'], ['ERA'], ['ERA'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 3.0, 3, [[], ['ERA'], ['ERA'], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 3.0, 3, [['ERA', 'EOGl'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 4.0, 3, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 5.0, 3, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ERA', 'ERI', 'EOGl', 'F3', 'C3', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'ERC', 'ERI', 'EOGl', 'F4', 'O2'], ['ERA', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ELC', 'ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4']], ['ERA', 'ERI', 'EOGl', 'F4']], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 1.0, 4, [['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'O2'], ['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ERB', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'F4', 'C4', 'O2', 'EOGr']], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EOGr']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 1.5, 4, [['ERA', 'EOGl'], ['ERA', 'ERI', 'C3'], ['ERA', 'ERI'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 1.5, 4, [['ERA', 'EOGl'], ['ERA'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl', 'F3', 'F4']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 1.5, 4, [[], [], [], [], ['ERI'], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 1.5, 4, [['ELA', 'ERB', 'ERI'], ['ERB', 'ERI'], ['ELA', 'ERB', 'ERI'], ['ERI'], ['ELA', 'ERI'], ['ELA', 'ERI', 'EOGr']], ['ERI']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 2.0, 4, [['ERA', 'EOGl'], ['ERA'], ['ERA', 'ERI'], ['ERA'], ['ERA', 'EOGl'], ['ERA']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 2.0, 4, [['ERA', 'EOGl'], ['ERA'], ['ERA'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 2.0, 4, [[], [], [], [], ['ERI'], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 2.5, 4, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], []], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 2.5, 4, [['ERA', 'EOGl'], ['ERA'], ['ERA'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 3.0, 4, [[], ['ERA'], ['ERA'], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 3.0, 4, [['ERA', 'EOGl'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 4.0, 4, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 5.0, 4, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ERA', 'ERI', 'EOGl', 'F3', 'C3', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'ERC', 'ERI', 'EOGl', 'F4', 'O2'], ['ERA', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ELC', 'ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4']], ['ERA', 'ERI', 'EOGl', 'F4']], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 1.0, 5, [['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'O2'], ['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ERB', 'ERI', 'EMGl', 'F3', 'C3', 'F4', 'C4', 'O2', 'EOGr']], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'EOGr']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 1.5, 5, [['ERA', 'EOGl'], ['ERA', 'ERI', 'C3'], ['ERA', 'ERI'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 1.5, 5, [['ERA', 'EOGl'], ['ERA'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl', 'F3', 'F4']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 1.5, 5, [[], [], [], [], ['ERI'], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 1.5, 5, [['ELA', 'ERB', 'ERI'], ['ERB', 'ERI'], ['ELA', 'ERB', 'ERI'], ['ERI'], ['ELA', 'ERI'], ['ELA', 'ERI', 'EOGr']], ['ERI']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 2.0, 5, [['ERA', 'EOGl'], ['ERA'], ['ERA', 'ERI'], ['ERA'], ['ERA', 'EOGl'], ['ERA']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 2.0, 5, [['ERA', 'EOGl'], ['ERA'], ['ERA'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 2.0, 5, [[], [], [], [], ['ERI'], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 2.5, 5, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], []], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 2.5, 5, [['ERA', 'EOGl'], ['ERA'], ['ERA'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 3.0, 5, [[], ['ERA'], ['ERA'], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 3.0, 5, [['ERA', 'EOGl'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 4.0, 5, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 5.0, 5, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ERA', 'ERI', 'EOGl', 'F3', 'C3', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'ERC', 'ERI', 'EOGl', 'F4', 'O2'], ['ERA', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'ERI', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ELC', 'ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4']], ['ERA', 'ERI', 'EOGl']], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 1.0, 6, [['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'ERI', 'EOGl', 'F3', 'C3', 'F4', 'O2'], ['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ERB', 'ERI', 'EMGl', 'F3', 'C3', 'F4', 'C4', 'O2']], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELC', 'ERB', 'ERC', 'ERI', 'EMGr', 'F3', 'C3', 'F4', 'C4']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 1.5, 6, [['ERA', 'EOGl'], ['ERA', 'ERI', 'C3'], ['ERA', 'ERI'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 1.5, 6, [['ERA', 'EOGl'], ['ERA'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl', 'F3', 'F4']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 1.5, 6, [[], [], [], [], ['ERI'], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 1.5, 6, [['ELA', 'ERB', 'ERI'], ['ERB', 'ERI'], ['ELA', 'ERB', 'ERI'], ['ERI'], ['ELA', 'ERI'], ['ELA', 'ERI', 'EOGr']], ['ERI']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 2.0, 6, [['ERA', 'EOGl'], ['ERA'], ['ERA', 'ERI'], ['ERA'], ['ERA', 'EOGl'], ['ERA']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 2.0, 6, [['ERA', 'EOGl'], ['ERA'], ['ERA'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 2.0, 6, [[], [], [], [], ['ERI'], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 2.5, 6, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 2.5, 6, [['ERA', 'EOGl'], ['ERA'], ['ERA'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 3.0, 6, [[], ['ERA'], ['ERA'], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 3.0, 6, [['ERA', 'EOGl'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 4.0, 6, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'mm', 5.0, 6, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_4\\EEG_raw', 12580, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 12580, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ERA', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGl', 'F3', 'F4', 'C4', 'O2'], ['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'ERI', 'EOGl', 'F3', 'F4', 'C4', 'O2'], ['ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ERA', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 1.0, 1, [['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EOGl', 'F3', 'F4'], ['ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGl', 'F3', 'F4', 'C4'], ['ERA', 'EOGl']], ['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O1', 'O2', 'ERI']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 1.0, 1, [['ELA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERB', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ELA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'ERB', 'ELC', 'ERA']], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 1.0, 1, [['ELA', 'ERE', 'ERI', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ERE', 'ERI', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGl', 'M1', 'EMGc', 'ELB', 'ELC', 'EMGr']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 1.5, 1, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl', 'F4'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl', 'F3', 'F4', 'C4']], ['ERA', 'EOGl', 'F4', 'F3', 'C4']], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 1.5, 1, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 1.5, 1, [['ELA', 'ERI', 'O1', 'O2'], ['ERI', 'C4'], ['ERI', 'EOGr'], ['ELA', 'ERI', 'EOGr', 'EMGl'], ['ELA', 'ERI', 'O1'], ['ELA', 'ERI']], ['ELA', 'ERI', 'O1', 'O2', 'C4', 'EOGr', 'EMGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 2.0, 1, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 2.0, 1, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], ['ERI']], ['ERI']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 2.5, 1, [[], ['ERA'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 2.5, 1, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 3.0, 1, [[], [], [], ['EOGl'], [], []], ['EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 3.0, 1, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 4.0, 1, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA'], ['EOGl'], ['ERA', 'EOGl'], ['EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 5.0, 1, [['EOGl'], ['EOGl'], [], [], [], ['EOGl']], ['EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ERA', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGl', 'F3', 'F4', 'C4', 'O2'], ['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'ERI', 'EOGl', 'F3', 'F4', 'C4', 'O2'], ['ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 1.0, 2, [['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EOGl', 'F3', 'F4'], ['ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGl', 'F3', 'F4', 'C4'], ['ERA', 'EOGl']], ['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 1.0, 2, [['ELA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERB', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ELA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 1.0, 2, [['ELA', 'ERE', 'ERI', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ERI', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGl', 'EMGc', 'ELB', 'ELC', 'EMGr']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 1.5, 2, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl', 'F4'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl', 'F3', 'F4', 'C4']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 1.5, 2, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 1.5, 2, [['ELA', 'ERI', 'O1', 'O2'], ['ERI', 'C4'], ['ERI', 'EOGr'], ['ELA', 'ERI', 'EOGr', 'EMGl'], ['ELA', 'ERI', 'O1'], ['ELA', 'ERI']], ['ELA', 'ERI', 'EOGr']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 2.0, 2, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 2.0, 2, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], ['ERI']], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 2.5, 2, [[], ['ERA'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 2.5, 2, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 3.0, 2, [[], [], [], ['EOGl'], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 3.0, 2, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 4.0, 2, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA'], ['EOGl'], ['ERA', 'EOGl'], ['EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 5.0, 2, [['EOGl'], ['EOGl'], [], [], [], ['EOGl']], ['EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ERA', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGl', 'F3', 'F4', 'C4', 'O2'], ['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'ERI', 'EOGl', 'F3', 'F4', 'C4', 'O2'], ['ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'ERI', 'EOGl', 'F3', 'F4', 'C4', 'O2']], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 1.0, 3, [['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EOGl', 'F3', 'F4'], ['ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGl', 'F3', 'F4', 'C4'], ['ERA', 'EOGl']], ['ERA', 'EOGl', 'F3', 'F4', 'C4']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 1.0, 3, [['ELA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERB', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ELA', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 1.0, 3, [['ELA', 'ERE', 'ERI', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ERI', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGl', 'EMGc', 'ELB', 'ELC']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 1.5, 3, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl', 'F4'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl', 'F3', 'F4', 'C4']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 1.5, 3, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 1.5, 3, [['ELA', 'ERI', 'O1', 'O2'], ['ERI', 'C4'], ['ERI', 'EOGr'], ['ELA', 'ERI', 'EOGr', 'EMGl'], ['ELA', 'ERI', 'O1'], ['ELA', 'ERI']], ['ELA', 'ERI']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 2.0, 3, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 2.0, 3, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], ['ERI']], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 2.5, 3, [[], ['ERA'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 2.5, 3, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 3.0, 3, [[], [], [], ['EOGl'], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 3.0, 3, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 4.0, 3, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA'], ['EOGl'], ['ERA', 'EOGl'], ['EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 5.0, 3, [['EOGl'], ['EOGl'], [], [], [], ['EOGl']], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ERA', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGl', 'F3', 'F4', 'C4', 'O2'], ['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'ERI', 'EOGl', 'F3', 'F4', 'C4', 'O2'], ['ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'EOGl', 'F3', 'F4', 'C4']], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 1.0, 4, [['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EOGl', 'F3', 'F4'], ['ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGl', 'F3', 'F4', 'C4'], ['ERA', 'EOGl']], ['ERA', 'EOGl', 'F3', 'F4']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 1.0, 4, [['ELA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERB', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ELA', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 1.0, 4, [['ELA', 'ERE', 'ERI', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ERI', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGl', 'ELB', 'ELC']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 1.5, 4, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl', 'F4'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl', 'F3', 'F4', 'C4']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 1.5, 4, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 1.5, 4, [['ELA', 'ERI', 'O1', 'O2'], ['ERI', 'C4'], ['ERI', 'EOGr'], ['ELA', 'ERI', 'EOGr', 'EMGl'], ['ELA', 'ERI', 'O1'], ['ELA', 'ERI']], ['ERI']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 2.0, 4, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 2.0, 4, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], ['ERI']], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 2.5, 4, [[], ['ERA'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 2.5, 4, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 3.0, 4, [[], [], [], ['EOGl'], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 3.0, 4, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 4.0, 4, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA'], ['EOGl'], ['ERA', 'EOGl'], ['EOGl']], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 5.0, 4, [['EOGl'], ['EOGl'], [], [], [], ['EOGl']], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ERA', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGl', 'F3', 'F4', 'C4', 'O2'], ['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'ERI', 'EOGl', 'F3', 'F4', 'C4', 'O2'], ['ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'EOGl', 'F3', 'F4', 'C4']], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 1.0, 5, [['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EOGl', 'F3', 'F4'], ['ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGl', 'F3', 'F4', 'C4'], ['ERA', 'EOGl']], ['ERA', 'EOGl', 'F3', 'F4']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 1.0, 5, [['ELA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERB', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ELA', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 1.0, 5, [['ELA', 'ERE', 'ERI', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ERI', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 1.5, 5, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl', 'F4'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl', 'F3', 'F4', 'C4']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 1.5, 5, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 1.5, 5, [['ELA', 'ERI', 'O1', 'O2'], ['ERI', 'C4'], ['ERI', 'EOGr'], ['ELA', 'ERI', 'EOGr', 'EMGl'], ['ELA', 'ERI', 'O1'], ['ELA', 'ERI']], ['ERI']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 2.0, 5, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 2.0, 5, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], ['ERI']], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 2.5, 5, [[], ['ERA'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA']], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 2.5, 5, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 3.0, 5, [[], [], [], ['EOGl'], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 3.0, 5, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 4.0, 5, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA'], ['EOGl'], ['ERA', 'EOGl'], ['EOGl']], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 5.0, 5, [['EOGl'], ['EOGl'], [], [], [], ['EOGl']], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ERA', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGl', 'F3', 'F4', 'C4', 'O2'], ['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'ERI', 'EOGl', 'F3', 'F4', 'C4', 'O2'], ['ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'EOGl', 'F3', 'F4', 'C4']], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 1.0, 6, [['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EOGl', 'F3', 'F4'], ['ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGl', 'F3', 'F4', 'C4'], ['ERA', 'EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 1.0, 6, [['ELA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERB', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ELA', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 1.0, 6, [['ELA', 'ERE', 'ERI', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ERI', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'O2']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 1.5, 6, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl', 'F4'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl', 'F3', 'F4', 'C4']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 1.5, 6, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 1.5, 6, [['ELA', 'ERI', 'O1', 'O2'], ['ERI', 'C4'], ['ERI', 'EOGr'], ['ELA', 'ERI', 'EOGr', 'EMGl'], ['ELA', 'ERI', 'O1'], ['ELA', 'ERI']], ['ERI']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 2.0, 6, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 2.0, 6, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], ['ERI']], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 2.5, 6, [[], ['ERA'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 2.5, 6, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 3.0, 6, [[], [], [], ['EOGl'], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 3.0, 6, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['EOGl']], ['EOGl']], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 4.0, 6, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA'], ['EOGl'], ['ERA', 'EOGl'], ['EOGl']], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'mm', 5.0, 6, [['EOGl'], ['EOGl'], [], [], [], ['EOGl']], []], ['S_01/night_4\\EEG_raw', 15140, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_4\\EEG_raw', 15140, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 1.0, 1, [['ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc', 'EMGl', 'C4', 'F4']], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 1.0, 1, [['EOGr', 'EOGl', 'M2'], ['ERT ', 'ERE', 'EOGr', 'EOGl'], ['EOGr', 'EOGl', 'M2'], ['ELT', 'ERT ', 'ERE', 'EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['ERT ', 'EOGr', 'EOGl']], ['EOGr', 'EOGl', 'M2', 'ERT ', 'ERE', 'ELT']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 1.0, 1, [['ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGl', 'EMGr', 'C4', 'EMGc', 'F4', 'ELB']], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 1.0, 1, [['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELB', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'ERT ', 'EMGr', 'EMGc']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 1.5, 1, [['EOGr', 'EOGl', 'O1', 'M2'], ['EOGr', 'EOGl', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'O1', 'M2']], ['EOGr', 'EOGl', 'O1', 'M2', 'O2', 'M1']], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 1.5, 1, [['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl']], ['EOGr', 'EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 1.5, 1, [['ERB', 'O1', 'M2', 'O2'], ['O1', 'M2', 'O2'], ['O1', 'M2', 'O2'], ['O1', 'M2', 'O2'], ['O1', 'M2', 'O2'], ['O1', 'M2']], ['ERB', 'O1', 'M2', 'O2']], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 1.5, 1, [['ERA', 'ERB', 'M1', 'O1', 'O2'], ['M1', 'O1', 'C4', 'O2'], ['ERA', 'M1', 'O1', 'O2'], ['ERA', 'M1', 'O1', 'C4', 'O2'], ['ELB', 'ERA', 'M1', 'O1', 'C4', 'O2'], ['M1', 'O1', 'O2']], ['ERA', 'ERB', 'M1', 'O1', 'O2', 'C4', 'ELB']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 2.0, 1, [[], ['EOGl', 'M2'], [], ['EOGl'], ['EOGl'], []], ['EOGl', 'M2']], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 2.0, 1, [['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl']], ['EOGr', 'EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 2.5, 1, [['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl']], ['EOGr', 'EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 3.0, 1, [['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl']], ['EOGr', 'EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 4.0, 1, [['EOGr', 'EOGl'], ['EOGr', 'EOGl'], [], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl']], ['EOGr', 'EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 5.0, 1, [[], ['EOGl'], [], ['EOGl'], ['EOGl'], ['EOGr', 'EOGl']], ['EOGl', 'EOGr']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 1.0, 2, [['ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc', 'EMGl']], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 1.0, 2, [['EOGr', 'EOGl', 'M2'], ['ERT ', 'ERE', 'EOGr', 'EOGl'], ['EOGr', 'EOGl', 'M2'], ['ELT', 'ERT ', 'ERE', 'EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['ERT ', 'EOGr', 'EOGl']], ['EOGr', 'EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 1.0, 2, [['ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGr', 'C4', 'EMGc', 'ELB']], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 1.0, 2, [['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELB', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGr', 'EMGc']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 1.5, 2, [['EOGr', 'EOGl', 'O1', 'M2'], ['EOGr', 'EOGl', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'O1', 'M2']], ['EOGr', 'EOGl', 'O1', 'M2', 'O2']], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 1.5, 2, [['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl']], ['EOGr', 'EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 1.5, 2, [['ERB', 'O1', 'M2', 'O2'], ['O1', 'M2', 'O2'], ['O1', 'M2', 'O2'], ['O1', 'M2', 'O2'], ['O1', 'M2', 'O2'], ['O1', 'M2']], ['O1', 'M2', 'O2']], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 1.5, 2, [['ERA', 'ERB', 'M1', 'O1', 'O2'], ['M1', 'O1', 'C4', 'O2'], ['ERA', 'M1', 'O1', 'O2'], ['ERA', 'M1', 'O1', 'C4', 'O2'], ['ELB', 'ERA', 'M1', 'O1', 'C4', 'O2'], ['M1', 'O1', 'O2']], ['ERA', 'M1', 'O1', 'O2', 'C4']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 2.0, 2, [[], ['EOGl', 'M2'], [], ['EOGl'], ['EOGl'], []], ['EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 2.0, 2, [['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl']], ['EOGr', 'EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 2.5, 2, [['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl']], ['EOGr', 'EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 3.0, 2, [['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl']], ['EOGr', 'EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 4.0, 2, [['EOGr', 'EOGl'], ['EOGr', 'EOGl'], [], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl']], ['EOGr', 'EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 5.0, 2, [[], ['EOGl'], [], ['EOGl'], ['EOGl'], ['EOGr', 'EOGl']], ['EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 1.0, 3, [['ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc', 'EMGl']], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 1.0, 3, [['EOGr', 'EOGl', 'M2'], ['ERT ', 'ERE', 'EOGr', 'EOGl'], ['EOGr', 'EOGl', 'M2'], ['ELT', 'ERT ', 'ERE', 'EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['ERT ', 'EOGr', 'EOGl']], ['EOGr', 'EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 1.0, 3, [['ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGr', 'C4', 'EMGc']], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 1.0, 3, [['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELB', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGr']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 1.5, 3, [['EOGr', 'EOGl', 'O1', 'M2'], ['EOGr', 'EOGl', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'O1', 'M2']], ['EOGr', 'EOGl', 'O1', 'M2', 'O2']], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 1.5, 3, [['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl']], ['EOGr', 'EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 1.5, 3, [['ERB', 'O1', 'M2', 'O2'], ['O1', 'M2', 'O2'], ['O1', 'M2', 'O2'], ['O1', 'M2', 'O2'], ['O1', 'M2', 'O2'], ['O1', 'M2']], ['O1', 'M2', 'O2']], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 1.5, 3, [['ERA', 'ERB', 'M1', 'O1', 'O2'], ['M1', 'O1', 'C4', 'O2'], ['ERA', 'M1', 'O1', 'O2'], ['ERA', 'M1', 'O1', 'C4', 'O2'], ['ELB', 'ERA', 'M1', 'O1', 'C4', 'O2'], ['M1', 'O1', 'O2']], ['ERA', 'M1', 'O1', 'O2']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 2.0, 3, [[], ['EOGl', 'M2'], [], ['EOGl'], ['EOGl'], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 2.0, 3, [['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl']], ['EOGr', 'EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 2.5, 3, [['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl']], ['EOGr', 'EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 3.0, 3, [['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl']], ['EOGr', 'EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 4.0, 3, [['EOGr', 'EOGl'], ['EOGr', 'EOGl'], [], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl']], ['EOGr', 'EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 5.0, 3, [[], ['EOGl'], [], ['EOGl'], ['EOGl'], ['EOGr', 'EOGl']], ['EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 1.0, 4, [['ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'O2']], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 1.0, 4, [['EOGr', 'EOGl', 'M2'], ['ERT ', 'ERE', 'EOGr', 'EOGl'], ['EOGr', 'EOGl', 'M2'], ['ELT', 'ERT ', 'ERE', 'EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['ERT ', 'EOGr', 'EOGl']], ['EOGr', 'EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 1.0, 4, [['ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGr', 'C4']], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 1.0, 4, [['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELB', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELB', 'ERA', 'EOGl', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGr']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 1.5, 4, [['EOGr', 'EOGl', 'O1', 'M2'], ['EOGr', 'EOGl', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'O1', 'M2']], ['EOGr', 'EOGl', 'O1', 'M2', 'O2']], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 1.5, 4, [['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl']], ['EOGr', 'EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 1.5, 4, [['ERB', 'O1', 'M2', 'O2'], ['O1', 'M2', 'O2'], ['O1', 'M2', 'O2'], ['O1', 'M2', 'O2'], ['O1', 'M2', 'O2'], ['O1', 'M2']], ['O1', 'M2', 'O2']], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 1.5, 4, [['ERA', 'ERB', 'M1', 'O1', 'O2'], ['M1', 'O1', 'C4', 'O2'], ['ERA', 'M1', 'O1', 'O2'], ['ERA', 'M1', 'O1', 'C4', 'O2'], ['ELB', 'ERA', 'M1', 'O1', 'C4', 'O2'], ['M1', 'O1', 'O2']], ['M1', 'O1', 'O2']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 2.0, 4, [[], ['EOGl', 'M2'], [], ['EOGl'], ['EOGl'], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 2.0, 4, [['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl']], ['EOGr', 'EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 2.5, 4, [['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl']], ['EOGr', 'EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 3.0, 4, [['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl']], ['EOGr', 'EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 4.0, 4, [['EOGr', 'EOGl'], ['EOGr', 'EOGl'], [], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl']], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 5.0, 4, [[], ['EOGl'], [], ['EOGl'], ['EOGl'], ['EOGr', 'EOGl']], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 1.0, 5, [['ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'O2']], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 1.0, 5, [['EOGr', 'EOGl', 'M2'], ['ERT ', 'ERE', 'EOGr', 'EOGl'], ['EOGr', 'EOGl', 'M2'], ['ELT', 'ERT ', 'ERE', 'EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['ERT ', 'EOGr', 'EOGl']], ['EOGr', 'EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 1.0, 5, [['ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGr', 'C4']], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 1.0, 5, [['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELB', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELB', 'ERA', 'EOGl', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGr']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 1.5, 5, [['EOGr', 'EOGl', 'O1', 'M2'], ['EOGr', 'EOGl', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'O1', 'M2']], ['EOGr', 'EOGl', 'O1', 'M2']], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 1.5, 5, [['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl']], ['EOGr', 'EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 1.5, 5, [['ERB', 'O1', 'M2', 'O2'], ['O1', 'M2', 'O2'], ['O1', 'M2', 'O2'], ['O1', 'M2', 'O2'], ['O1', 'M2', 'O2'], ['O1', 'M2']], ['O1', 'M2', 'O2']], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 1.5, 5, [['ERA', 'ERB', 'M1', 'O1', 'O2'], ['M1', 'O1', 'C4', 'O2'], ['ERA', 'M1', 'O1', 'O2'], ['ERA', 'M1', 'O1', 'C4', 'O2'], ['ELB', 'ERA', 'M1', 'O1', 'C4', 'O2'], ['M1', 'O1', 'O2']], ['M1', 'O1', 'O2']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 2.0, 5, [[], ['EOGl', 'M2'], [], ['EOGl'], ['EOGl'], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 2.0, 5, [['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl']], ['EOGr', 'EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 2.5, 5, [['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl']], ['EOGr', 'EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 3.0, 5, [['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl']], ['EOGr', 'EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 4.0, 5, [['EOGr', 'EOGl'], ['EOGr', 'EOGl'], [], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl']], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 5.0, 5, [[], ['EOGl'], [], ['EOGl'], ['EOGl'], ['EOGr', 'EOGl']], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 1.0, 6, [['ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'O2']], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 1.0, 6, [['EOGr', 'EOGl', 'M2'], ['ERT ', 'ERE', 'EOGr', 'EOGl'], ['EOGr', 'EOGl', 'M2'], ['ELT', 'ERT ', 'ERE', 'EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['ERT ', 'EOGr', 'EOGl']], ['EOGr', 'EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 1.0, 6, [['ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'O2']], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 1.0, 6, [['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELB', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELB', 'ERA', 'EOGl', 'M1', 'O1', 'M2', 'C4', 'O2']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 1.5, 6, [['EOGr', 'EOGl', 'O1', 'M2'], ['EOGr', 'EOGl', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2'], ['EOGr', 'EOGl', 'O1', 'M2']], ['EOGr', 'EOGl', 'O1', 'M2']], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 1.5, 6, [['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl']], ['EOGr', 'EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 1.5, 6, [['ERB', 'O1', 'M2', 'O2'], ['O1', 'M2', 'O2'], ['O1', 'M2', 'O2'], ['O1', 'M2', 'O2'], ['O1', 'M2', 'O2'], ['O1', 'M2']], ['O1', 'M2']], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 1.5, 6, [['ERA', 'ERB', 'M1', 'O1', 'O2'], ['M1', 'O1', 'C4', 'O2'], ['ERA', 'M1', 'O1', 'O2'], ['ERA', 'M1', 'O1', 'C4', 'O2'], ['ELB', 'ERA', 'M1', 'O1', 'C4', 'O2'], ['M1', 'O1', 'O2']], ['M1', 'O1', 'O2']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 2.0, 6, [[], ['EOGl', 'M2'], [], ['EOGl'], ['EOGl'], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 2.0, 6, [['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl']], ['EOGr', 'EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 2.5, 6, [['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl']], ['EOGr', 'EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 3.0, 6, [['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl']], ['EOGr', 'EOGl']], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 4.0, 6, [['EOGr', 'EOGl'], ['EOGr', 'EOGl'], [], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl']], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'mm', 5.0, 6, [[], ['EOGl'], [], ['EOGl'], ['EOGl'], ['EOGr', 'EOGl']], []], ['S_01/night_3\\EEG_raw', 310, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 310, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 1.0, 1, [['ERA', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc', 'EOGr', 'O2', 'M2', 'ELB', 'ERB']], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 1.0, 1, [['ERA', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ERA', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'EOGr', 'EOGl', 'O1', 'M1', 'O2', 'EMGc', 'ERB']], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc', 'O2', 'ERB', 'M2']], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 1.0, 1, [['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc']], ['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'M2', 'ERB', 'EMGl']], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 1.5, 1, [[], ['EMGr'], ['EMGr'], ['EMGr'], [], ['ERA', 'EMGr']], ['EMGr', 'ERA']], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 1.5, 1, [['ERA', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['EMGr', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EMGr', 'M1', 'F3'], ['EMGr', 'F3', 'F4'], ['ERA', 'EMGr']], ['ERA', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'M1']], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 1.5, 1, [[], ['ERA'], [], [], [], ['ERA', 'ERB']], ['ERA', 'ERB']], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 1.5, 1, [['M1', 'EMGc'], ['EOGl', 'M1'], ['M1'], ['ERA', 'M1'], ['ERA', 'M1'], ['ERB', 'M1', 'M2', 'EMGc']], ['M1', 'EMGc', 'EOGl', 'ERA', 'ERB', 'M2']], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 2.0, 1, [['F3'], ['EMGr', 'F3', 'F4'], ['EMGr', 'F4'], ['EMGr'], ['EMGr'], ['ERA', 'EMGr']], ['F3', 'EMGr', 'F4', 'ERA']], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 2.0, 1, [[], ['M1'], ['M1'], ['M1'], ['M1'], []], ['M1']], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 2.5, 1, [[], ['EMGr', 'F3'], [], ['EMGr'], [], ['ERA', 'EMGr']], ['EMGr', 'F3', 'ERA']], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 1.0, 2, [['ERA', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc', 'EOGr', 'O2', 'ERB']], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 1.0, 2, [['ERA', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ERA', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O1', 'M1', 'O2', 'EMGc']], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc', 'O2', 'ERB', 'M2']], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 1.0, 2, [['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc']], ['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc', 'M2', 'ERB']], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 1.5, 2, [[], ['EMGr'], ['EMGr'], ['EMGr'], [], ['ERA', 'EMGr']], ['EMGr']], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 1.5, 2, [['ERA', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['EMGr', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EMGr', 'M1', 'F3'], ['EMGr', 'F3', 'F4'], ['ERA', 'EMGr']], ['ERA', 'EMGr', 'F3', 'C3', 'F4', 'C4']], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 1.5, 2, [[], ['ERA'], [], [], [], ['ERA', 'ERB']], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 1.5, 2, [['M1', 'EMGc'], ['EOGl', 'M1'], ['M1'], ['ERA', 'M1'], ['ERA', 'M1'], ['ERB', 'M1', 'M2', 'EMGc']], ['M1', 'ERA']], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 2.0, 2, [['F3'], ['EMGr', 'F3', 'F4'], ['EMGr', 'F4'], ['EMGr'], ['EMGr'], ['ERA', 'EMGr']], ['F3', 'EMGr', 'F4']], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 2.0, 2, [[], ['M1'], ['M1'], ['M1'], ['M1'], []], ['M1']], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 2.5, 2, [[], ['EMGr', 'F3'], [], ['EMGr'], [], ['ERA', 'EMGr']], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 1.0, 3, [['ERA', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc', 'EOGr', 'O2']], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 1.0, 3, [['ERA', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ERA', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O1', 'M1', 'O2', 'EMGc']], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc', 'O2', 'ERB']], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 1.0, 3, [['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc']], ['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc', 'M2', 'ERB']], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 1.5, 3, [[], ['EMGr'], ['EMGr'], ['EMGr'], [], ['ERA', 'EMGr']], ['EMGr']], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 1.5, 3, [['ERA', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['EMGr', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EMGr', 'M1', 'F3'], ['EMGr', 'F3', 'F4'], ['ERA', 'EMGr']], ['EMGr', 'F3', 'C3', 'F4', 'C4']], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 1.5, 3, [[], ['ERA'], [], [], [], ['ERA', 'ERB']], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 1.5, 3, [['M1', 'EMGc'], ['EOGl', 'M1'], ['M1'], ['ERA', 'M1'], ['ERA', 'M1'], ['ERB', 'M1', 'M2', 'EMGc']], ['M1']], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 2.0, 3, [['F3'], ['EMGr', 'F3', 'F4'], ['EMGr', 'F4'], ['EMGr'], ['EMGr'], ['ERA', 'EMGr']], ['EMGr']], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 2.0, 3, [[], ['M1'], ['M1'], ['M1'], ['M1'], []], ['M1']], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 2.5, 3, [[], ['EMGr', 'F3'], [], ['EMGr'], [], ['ERA', 'EMGr']], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 1.0, 4, [['ERA', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EOGr', 'O2']], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 1.0, 4, [['ERA', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ERA', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O1']], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc', 'ERB']], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 1.0, 4, [['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc']], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'EMGc', 'M2', 'ERB']], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 1.5, 4, [[], ['EMGr'], ['EMGr'], ['EMGr'], [], ['ERA', 'EMGr']], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 1.5, 4, [['ERA', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['EMGr', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EMGr', 'M1', 'F3'], ['EMGr', 'F3', 'F4'], ['ERA', 'EMGr']], ['EMGr', 'F3']], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 1.5, 4, [[], ['ERA'], [], [], [], ['ERA', 'ERB']], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 1.5, 4, [['M1', 'EMGc'], ['EOGl', 'M1'], ['M1'], ['ERA', 'M1'], ['ERA', 'M1'], ['ERB', 'M1', 'M2', 'EMGc']], ['M1']], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 2.0, 4, [['F3'], ['EMGr', 'F3', 'F4'], ['EMGr', 'F4'], ['EMGr'], ['EMGr'], ['ERA', 'EMGr']], ['EMGr']], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 2.0, 4, [[], ['M1'], ['M1'], ['M1'], ['M1'], []], ['M1']], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 2.5, 4, [[], ['EMGr', 'F3'], [], ['EMGr'], [], ['ERA', 'EMGr']], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 1.0, 5, [['ERA', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EOGr', 'O2']], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 1.0, 5, [['ERA', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ERA', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'EMGr', 'F3', 'C3', 'F4', 'C4']], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc']], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 1.0, 5, [['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc']], ['ERA', 'EOGl', 'EMGr', 'M1', 'F4', 'C4', 'EMGc', 'M2']], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 1.5, 5, [[], ['EMGr'], ['EMGr'], ['EMGr'], [], ['ERA', 'EMGr']], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 1.5, 5, [['ERA', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['EMGr', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EMGr', 'M1', 'F3'], ['EMGr', 'F3', 'F4'], ['ERA', 'EMGr']], ['EMGr', 'F3']], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 1.5, 5, [[], ['ERA'], [], [], [], ['ERA', 'ERB']], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 1.5, 5, [['M1', 'EMGc'], ['EOGl', 'M1'], ['M1'], ['ERA', 'M1'], ['ERA', 'M1'], ['ERB', 'M1', 'M2', 'EMGc']], ['M1']], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 2.0, 5, [['F3'], ['EMGr', 'F3', 'F4'], ['EMGr', 'F4'], ['EMGr'], ['EMGr'], ['ERA', 'EMGr']], ['EMGr']], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 2.0, 5, [[], ['M1'], ['M1'], ['M1'], ['M1'], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 2.5, 5, [[], ['EMGr', 'F3'], [], ['EMGr'], [], ['ERA', 'EMGr']], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 1.0, 6, [['ERA', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4']], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 1.0, 6, [['ERA', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ERA', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'EMGr', 'F3', 'C3', 'F4', 'C4']], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc']], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 1.0, 6, [['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'C4', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc']], ['ERA', 'EMGr', 'M1', 'EMGc']], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 1.5, 6, [[], ['EMGr'], ['EMGr'], ['EMGr'], [], ['ERA', 'EMGr']], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 1.5, 6, [['ERA', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['EMGr', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'EMGr', 'M1', 'F3'], ['EMGr', 'F3', 'F4'], ['ERA', 'EMGr']], ['EMGr']], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 1.5, 6, [[], ['ERA'], [], [], [], ['ERA', 'ERB']], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 1.5, 6, [['M1', 'EMGc'], ['EOGl', 'M1'], ['M1'], ['ERA', 'M1'], ['ERA', 'M1'], ['ERB', 'M1', 'M2', 'EMGc']], ['M1']], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 2.0, 6, [['F3'], ['EMGr', 'F3', 'F4'], ['EMGr', 'F4'], ['EMGr'], ['EMGr'], ['ERA', 'EMGr']], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 2.0, 6, [[], ['M1'], ['M1'], ['M1'], ['M1'], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 2.5, 6, [[], ['EMGr', 'F3'], [], ['EMGr'], [], ['ERA', 'EMGr']], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 1840, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 1.0, 1, [['ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELA', 'EMGc']], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 1.0, 1, [['ERA', 'F3', 'F4', 'C4'], ['ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'F3', 'F4', 'C4', 'EMGc'], ['ERA'], ['ERA', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'F3', 'F4', 'C4', 'EOGr', 'C3', 'O1', 'O2', 'EMGr', 'EMGc']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 1.0, 1, [['ELA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERT ', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ELA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ERT ', 'EMGc', 'M1', 'M2']], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 1.0, 1, [['ELA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc']], ['ELA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ELB', 'M2', 'ERT ', 'M1']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 1.5, 1, [['ERA'], ['ERA'], ['ERA'], ['ERA', 'C4'], ['ERA'], ['ERA']], ['ERA', 'C4']], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 1.5, 1, [['ERA', 'F4', 'C4'], ['ERA', 'F3', 'F4', 'C4'], ['ERA', 'F3', 'C3'], ['ERA'], ['ERA'], ['ERA', 'F3', 'C3']], ['ERA', 'F4', 'C4', 'F3', 'C3']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 1.5, 1, [[], [], ['C3'], [], [], []], ['C3']], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 1.5, 1, [['EOGr', 'EMGr'], ['EMGr', 'C4'], ['ERT ', 'EOGr', 'EOGl', 'EMGr'], ['ERT ', 'F4'], ['EMGl', 'C4', 'EMGc'], ['EOGr', 'EOGl', 'EMGl']], ['EOGr', 'EMGr', 'C4', 'ERT ', 'EOGl', 'F4', 'EMGl', 'EMGc']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 2.0, 1, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 2.0, 1, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 2.0, 1, [[], [], [], ['ERT '], [], []], ['ERT ']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 2.5, 1, [[], ['ERA'], ['ERA'], ['ERA'], ['ERA'], []], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 2.5, 1, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 3.0, 1, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 4.0, 1, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 5.0, 1, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], []], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 1.0, 2, [['ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2']], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 1.0, 2, [['ERA', 'F3', 'F4', 'C4'], ['ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'F3', 'F4', 'C4', 'EMGc'], ['ERA'], ['ERA', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'F3', 'F4', 'C4', 'EOGr', 'C3']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 1.0, 2, [['ELA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERT ', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ERT ', 'EMGc']], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 1.0, 2, [['ELA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc']], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ELB', 'ERT ', 'M1']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 1.5, 2, [['ERA'], ['ERA'], ['ERA'], ['ERA', 'C4'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 1.5, 2, [['ERA', 'F4', 'C4'], ['ERA', 'F3', 'F4', 'C4'], ['ERA', 'F3', 'C3'], ['ERA'], ['ERA'], ['ERA', 'F3', 'C3']], ['ERA', 'F4', 'C4', 'F3']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 1.5, 2, [[], [], ['C3'], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 1.5, 2, [['EOGr', 'EMGr'], ['EMGr', 'C4'], ['ERT ', 'EOGr', 'EOGl', 'EMGr'], ['ERT ', 'F4'], ['EMGl', 'C4', 'EMGc'], ['EOGr', 'EOGl', 'EMGl']], ['EMGr', 'ERT ', 'EMGl']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 2.0, 2, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 2.0, 2, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 2.0, 2, [[], [], [], ['ERT '], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 2.5, 2, [[], ['ERA'], ['ERA'], ['ERA'], ['ERA'], []], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 2.5, 2, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 3.0, 2, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 4.0, 2, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 5.0, 2, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], []], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 1.0, 3, [['ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2']], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 1.0, 3, [['ERA', 'F3', 'F4', 'C4'], ['ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'F3', 'F4', 'C4', 'EMGc'], ['ERA'], ['ERA', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'F3', 'F4', 'C4']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 1.0, 3, [['ELA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERT ', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 1.0, 3, [['ELA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc']], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'M1']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 1.5, 3, [['ERA'], ['ERA'], ['ERA'], ['ERA', 'C4'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 1.5, 3, [['ERA', 'F4', 'C4'], ['ERA', 'F3', 'F4', 'C4'], ['ERA', 'F3', 'C3'], ['ERA'], ['ERA'], ['ERA', 'F3', 'C3']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 1.5, 3, [[], [], ['C3'], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 1.5, 3, [['EOGr', 'EMGr'], ['EMGr', 'C4'], ['ERT ', 'EOGr', 'EOGl', 'EMGr'], ['ERT ', 'F4'], ['EMGl', 'C4', 'EMGc'], ['EOGr', 'EOGl', 'EMGl']], ['EMGr']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 2.0, 3, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 2.0, 3, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 2.0, 3, [[], [], [], ['ERT '], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 2.5, 3, [[], ['ERA'], ['ERA'], ['ERA'], ['ERA'], []], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 2.5, 3, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 3.0, 3, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 4.0, 3, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 5.0, 3, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], []], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 1.0, 4, [['ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'EOGr', 'EMGr', 'F3', 'C3', 'F4', 'C4']], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 1.0, 4, [['ERA', 'F3', 'F4', 'C4'], ['ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'F3', 'F4', 'C4', 'EMGc'], ['ERA'], ['ERA', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'F3', 'F4', 'C4']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 1.0, 4, [['ELA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERT ', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 1.0, 4, [['ELA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc']], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 1.5, 4, [['ERA'], ['ERA'], ['ERA'], ['ERA', 'C4'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 1.5, 4, [['ERA', 'F4', 'C4'], ['ERA', 'F3', 'F4', 'C4'], ['ERA', 'F3', 'C3'], ['ERA'], ['ERA'], ['ERA', 'F3', 'C3']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 1.5, 4, [[], [], ['C3'], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 1.5, 4, [['EOGr', 'EMGr'], ['EMGr', 'C4'], ['ERT ', 'EOGr', 'EOGl', 'EMGr'], ['ERT ', 'F4'], ['EMGl', 'C4', 'EMGc'], ['EOGr', 'EOGl', 'EMGl']], []], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 2.0, 4, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 2.0, 4, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 2.0, 4, [[], [], [], ['ERT '], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 2.5, 4, [[], ['ERA'], ['ERA'], ['ERA'], ['ERA'], []], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 2.5, 4, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 3.0, 4, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 4.0, 4, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 5.0, 4, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], []], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 1.0, 5, [['ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'EOGr', 'EMGr', 'F3', 'C3', 'F4', 'C4']], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 1.0, 5, [['ERA', 'F3', 'F4', 'C4'], ['ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'F3', 'F4', 'C4', 'EMGc'], ['ERA'], ['ERA', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 1.0, 5, [['ELA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERT ', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 1.0, 5, [['ELA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc']], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 1.5, 5, [['ERA'], ['ERA'], ['ERA'], ['ERA', 'C4'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 1.5, 5, [['ERA', 'F4', 'C4'], ['ERA', 'F3', 'F4', 'C4'], ['ERA', 'F3', 'C3'], ['ERA'], ['ERA'], ['ERA', 'F3', 'C3']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 1.5, 5, [[], [], ['C3'], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 1.5, 5, [['EOGr', 'EMGr'], ['EMGr', 'C4'], ['ERT ', 'EOGr', 'EOGl', 'EMGr'], ['ERT ', 'F4'], ['EMGl', 'C4', 'EMGc'], ['EOGr', 'EOGl', 'EMGl']], []], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 2.0, 5, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 2.0, 5, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 2.0, 5, [[], [], [], ['ERT '], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 2.5, 5, [[], ['ERA'], ['ERA'], ['ERA'], ['ERA'], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 2.5, 5, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 3.0, 5, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 4.0, 5, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 5.0, 5, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], []], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 1.0, 6, [['ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'EOGr', 'EMGr', 'F3', 'C3', 'F4', 'C4']], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 1.0, 6, [['ERA', 'F3', 'F4', 'C4'], ['ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ERA', 'F3', 'F4', 'C4', 'EMGc'], ['ERA'], ['ERA', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 1.0, 6, [['ELA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERT ', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 1.0, 6, [['ELA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc']], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 1.5, 6, [['ERA'], ['ERA'], ['ERA'], ['ERA', 'C4'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 1.5, 6, [['ERA', 'F4', 'C4'], ['ERA', 'F3', 'F4', 'C4'], ['ERA', 'F3', 'C3'], ['ERA'], ['ERA'], ['ERA', 'F3', 'C3']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 1.5, 6, [[], [], ['C3'], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 1.5, 6, [['EOGr', 'EMGr'], ['EMGr', 'C4'], ['ERT ', 'EOGr', 'EOGl', 'EMGr'], ['ERT ', 'F4'], ['EMGl', 'C4', 'EMGc'], ['EOGr', 'EOGl', 'EMGl']], []], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 2.0, 6, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 2.0, 6, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 2.0, 6, [[], [], [], ['ERT '], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 2.5, 6, [[], ['ERA'], ['ERA'], ['ERA'], ['ERA'], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 2.5, 6, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 3.0, 6, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 4.0, 6, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'mm', 5.0, 6, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_3\\EEG_raw', 3810, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 1.0, 1, [['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'M2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc']], ['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc', 'ERC', 'ERA', 'ERB', 'M1', 'O1', 'O2', 'M2']], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 1.0, 1, [['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3'], ['ELC', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'C3'], ['ERB', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ERB', 'EOGr', 'EMGr', 'C3']], ['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc', 'ERB']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 1.0, 1, [['ELC', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc']], ['ELC', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'O2', 'EMGc', 'ELE', 'ERA', 'M2', 'EMGr', 'C3']], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 1.0, 1, [['ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ERB', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc']], ['ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc', 'ELC', 'ELE', 'ERA', 'ERI', 'M2', 'ELI', 'ELT', 'EMGr']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 1.5, 1, [['ELC', 'EOGr', 'EOGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'EMGl', 'EMGr', 'C3'], ['ELC', 'EMGr', 'C3'], ['ELC', 'ERB', 'EMGr', 'C3'], ['ERB', 'EMGr', 'C3'], ['ERB', 'EMGr', 'C3']], ['ELC', 'EOGr', 'EOGl', 'EMGr', 'C3', 'EMGc', 'EMGl', 'ERB']], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 1.5, 1, [['ELC', 'EOGl', 'EMGr', 'C3'], ['ELC', 'EMGr', 'C3'], ['ELC', 'ERB', 'EMGr', 'C3'], ['ELC', 'EOGl', 'EMGr', 'C3'], ['ERB', 'C3'], ['ERB', 'EOGr', 'EMGr', 'C3']], ['ELC', 'EOGl', 'EMGr', 'C3', 'ERB', 'EOGr']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 1.5, 1, [['ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGc'], ['EOGr', 'EOGl', 'EMGl'], ['ELC', 'EOGr', 'EMGl'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl'], ['ERB', 'EOGr', 'EOGl', 'EMGr', 'EMGc'], ['ERB', 'EMGr']], ['ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGc', 'ELC', 'EMGr']], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 1.5, 1, [['ERB', 'ERC', 'M1', 'O1', 'O2'], ['ELE', 'ERC', 'M2', 'EMGc'], ['ERA', 'EMGl', 'O1', 'O2'], ['ELT', 'M2'], ['ELC', 'EOGr', 'M1', 'O1', 'O2'], ['ELC', 'EMGl', 'M1', 'O1', 'M2', 'O2', 'EMGc']], ['ERB', 'ERC', 'M1', 'O1', 'O2', 'ELE', 'M2', 'EMGc', 'ERA', 'EMGl', 'ELT', 'ELC', 'EOGr']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 2.0, 1, [['ELC', 'EMGr', 'C3'], ['EMGr', 'C3'], ['C3'], ['EMGr', 'C3'], ['C3'], ['C3']], ['ELC', 'EMGr', 'C3']], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 2.0, 1, [['ELC', 'EMGr', 'C3'], ['ELC', 'EMGr', 'C3'], ['EMGr', 'C3'], ['ELC', 'EMGr', 'C3'], ['ERB', 'C3'], ['EMGr', 'C3']], ['ELC', 'EMGr', 'C3', 'ERB']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 2.0, 1, [['ERB'], [], [], [], [], []], ['ERB']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 2.5, 1, [[], [], [], ['C3'], [], []], ['C3']], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 2.5, 1, [['ELC', 'EMGr', 'C3'], ['EMGr', 'C3'], ['EMGr', 'C3'], ['EMGr', 'C3'], ['C3'], ['C3']], ['ELC', 'EMGr', 'C3']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 3.0, 1, [['ELC', 'C3'], ['EMGr', 'C3'], ['EMGr', 'C3'], ['EMGr', 'C3'], ['C3'], ['C3']], ['ELC', 'C3', 'EMGr']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 4.0, 1, [['C3'], ['EMGr', 'C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3', 'EMGr']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 5.0, 1, [[], [], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 1.0, 2, [['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'M2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc']], ['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc', 'ERB', 'O1', 'O2']], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 1.0, 2, [['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3'], ['ELC', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'C3'], ['ERB', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ERB', 'EOGr', 'EMGr', 'C3']], ['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'ERB']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 1.0, 2, [['ELC', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc']], ['ELC', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'O2', 'EMGc', 'ERA', 'EMGr']], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 1.0, 2, [['ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ERB', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc']], ['ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'O2', 'EMGc', 'ELC', 'ERA', 'EMGr']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 1.5, 2, [['ELC', 'EOGr', 'EOGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'EMGl', 'EMGr', 'C3'], ['ELC', 'EMGr', 'C3'], ['ELC', 'ERB', 'EMGr', 'C3'], ['ERB', 'EMGr', 'C3'], ['ERB', 'EMGr', 'C3']], ['ELC', 'EMGr', 'C3', 'ERB']], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 1.5, 2, [['ELC', 'EOGl', 'EMGr', 'C3'], ['ELC', 'EMGr', 'C3'], ['ELC', 'ERB', 'EMGr', 'C3'], ['ELC', 'EOGl', 'EMGr', 'C3'], ['ERB', 'C3'], ['ERB', 'EOGr', 'EMGr', 'C3']], ['ELC', 'EMGr', 'C3', 'ERB']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 1.5, 2, [['ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGc'], ['EOGr', 'EOGl', 'EMGl'], ['ELC', 'EOGr', 'EMGl'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl'], ['ERB', 'EOGr', 'EOGl', 'EMGr', 'EMGc'], ['ERB', 'EMGr']], ['ERB', 'EOGr', 'EOGl', 'EMGl', 'ELC', 'EMGr']], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 1.5, 2, [['ERB', 'ERC', 'M1', 'O1', 'O2'], ['ELE', 'ERC', 'M2', 'EMGc'], ['ERA', 'EMGl', 'O1', 'O2'], ['ELT', 'M2'], ['ELC', 'EOGr', 'M1', 'O1', 'O2'], ['ELC', 'EMGl', 'M1', 'O1', 'M2', 'O2', 'EMGc']], ['ERC', 'M1', 'O1', 'O2', 'ELC']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 2.0, 2, [['ELC', 'EMGr', 'C3'], ['EMGr', 'C3'], ['C3'], ['EMGr', 'C3'], ['C3'], ['C3']], ['EMGr', 'C3']], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 2.0, 2, [['ELC', 'EMGr', 'C3'], ['ELC', 'EMGr', 'C3'], ['EMGr', 'C3'], ['ELC', 'EMGr', 'C3'], ['ERB', 'C3'], ['EMGr', 'C3']], ['ELC', 'EMGr', 'C3']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 2.0, 2, [['ERB'], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 2.5, 2, [[], [], [], ['C3'], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 2.5, 2, [['ELC', 'EMGr', 'C3'], ['EMGr', 'C3'], ['EMGr', 'C3'], ['EMGr', 'C3'], ['C3'], ['C3']], ['EMGr', 'C3']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 3.0, 2, [['ELC', 'C3'], ['EMGr', 'C3'], ['EMGr', 'C3'], ['EMGr', 'C3'], ['C3'], ['C3']], ['C3', 'EMGr']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 4.0, 2, [['C3'], ['EMGr', 'C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 5.0, 2, [[], [], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 1.0, 3, [['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'M2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc']], ['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc', 'ERB']], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 1.0, 3, [['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3'], ['ELC', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'C3'], ['ERB', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ERB', 'EOGr', 'EMGr', 'C3']], ['ELC', 'EOGr', 'EMGl', 'EMGr', 'C3', 'ERB']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 1.0, 3, [['ELC', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc']], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'O1', 'O2', 'EMGc', 'EMGr']], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 1.0, 3, [['ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ERB', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc']], ['ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'O2', 'EMGc', 'ELC']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 1.5, 3, [['ELC', 'EOGr', 'EOGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'EMGl', 'EMGr', 'C3'], ['ELC', 'EMGr', 'C3'], ['ELC', 'ERB', 'EMGr', 'C3'], ['ERB', 'EMGr', 'C3'], ['ERB', 'EMGr', 'C3']], ['ELC', 'EMGr', 'C3', 'ERB']], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 1.5, 3, [['ELC', 'EOGl', 'EMGr', 'C3'], ['ELC', 'EMGr', 'C3'], ['ELC', 'ERB', 'EMGr', 'C3'], ['ELC', 'EOGl', 'EMGr', 'C3'], ['ERB', 'C3'], ['ERB', 'EOGr', 'EMGr', 'C3']], ['ELC', 'EMGr', 'C3']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 1.5, 3, [['ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGc'], ['EOGr', 'EOGl', 'EMGl'], ['ELC', 'EOGr', 'EMGl'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl'], ['ERB', 'EOGr', 'EOGl', 'EMGr', 'EMGc'], ['ERB', 'EMGr']], ['ERB', 'EOGr', 'EMGl']], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 1.5, 3, [['ERB', 'ERC', 'M1', 'O1', 'O2'], ['ELE', 'ERC', 'M2', 'EMGc'], ['ERA', 'EMGl', 'O1', 'O2'], ['ELT', 'M2'], ['ELC', 'EOGr', 'M1', 'O1', 'O2'], ['ELC', 'EMGl', 'M1', 'O1', 'M2', 'O2', 'EMGc']], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 2.0, 3, [['ELC', 'EMGr', 'C3'], ['EMGr', 'C3'], ['C3'], ['EMGr', 'C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 2.0, 3, [['ELC', 'EMGr', 'C3'], ['ELC', 'EMGr', 'C3'], ['EMGr', 'C3'], ['ELC', 'EMGr', 'C3'], ['ERB', 'C3'], ['EMGr', 'C3']], ['EMGr', 'C3']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 2.0, 3, [['ERB'], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 2.5, 3, [[], [], [], ['C3'], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 2.5, 3, [['ELC', 'EMGr', 'C3'], ['EMGr', 'C3'], ['EMGr', 'C3'], ['EMGr', 'C3'], ['C3'], ['C3']], ['EMGr', 'C3']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 3.0, 3, [['ELC', 'C3'], ['EMGr', 'C3'], ['EMGr', 'C3'], ['EMGr', 'C3'], ['C3'], ['C3']], ['C3', 'EMGr']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 4.0, 3, [['C3'], ['EMGr', 'C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 5.0, 3, [[], [], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 1.0, 4, [['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'M2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc']], ['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc', 'ERB']], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 1.0, 4, [['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3'], ['ELC', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'C3'], ['ERB', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ERB', 'EOGr', 'EMGr', 'C3']], ['ELC', 'EMGl', 'EMGr', 'C3', 'ERB']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 1.0, 4, [['ELC', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc']], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'O1', 'O2', 'EMGc', 'EMGr']], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 1.0, 4, [['ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ERB', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc']], ['ERB', 'EOGr', 'EMGl', 'M1', 'O1', 'O2', 'EMGc', 'ELC']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 1.5, 4, [['ELC', 'EOGr', 'EOGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'EMGl', 'EMGr', 'C3'], ['ELC', 'EMGr', 'C3'], ['ELC', 'ERB', 'EMGr', 'C3'], ['ERB', 'EMGr', 'C3'], ['ERB', 'EMGr', 'C3']], ['ELC', 'EMGr', 'C3']], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 1.5, 4, [['ELC', 'EOGl', 'EMGr', 'C3'], ['ELC', 'EMGr', 'C3'], ['ELC', 'ERB', 'EMGr', 'C3'], ['ELC', 'EOGl', 'EMGr', 'C3'], ['ERB', 'C3'], ['ERB', 'EOGr', 'EMGr', 'C3']], ['ELC', 'EMGr', 'C3']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 1.5, 4, [['ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGc'], ['EOGr', 'EOGl', 'EMGl'], ['ELC', 'EOGr', 'EMGl'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl'], ['ERB', 'EOGr', 'EOGl', 'EMGr', 'EMGc'], ['ERB', 'EMGr']], ['EOGr', 'EMGl']], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 1.5, 4, [['ERB', 'ERC', 'M1', 'O1', 'O2'], ['ELE', 'ERC', 'M2', 'EMGc'], ['ERA', 'EMGl', 'O1', 'O2'], ['ELT', 'M2'], ['ELC', 'EOGr', 'M1', 'O1', 'O2'], ['ELC', 'EMGl', 'M1', 'O1', 'M2', 'O2', 'EMGc']], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 2.0, 4, [['ELC', 'EMGr', 'C3'], ['EMGr', 'C3'], ['C3'], ['EMGr', 'C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 2.0, 4, [['ELC', 'EMGr', 'C3'], ['ELC', 'EMGr', 'C3'], ['EMGr', 'C3'], ['ELC', 'EMGr', 'C3'], ['ERB', 'C3'], ['EMGr', 'C3']], ['EMGr', 'C3']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 2.0, 4, [['ERB'], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 2.5, 4, [[], [], [], ['C3'], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 2.5, 4, [['ELC', 'EMGr', 'C3'], ['EMGr', 'C3'], ['EMGr', 'C3'], ['EMGr', 'C3'], ['C3'], ['C3']], ['EMGr', 'C3']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 3.0, 4, [['ELC', 'C3'], ['EMGr', 'C3'], ['EMGr', 'C3'], ['EMGr', 'C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 4.0, 4, [['C3'], ['EMGr', 'C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 5.0, 4, [[], [], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 1.0, 5, [['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'M2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc']], ['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc']], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 1.0, 5, [['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3'], ['ELC', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'C3'], ['ERB', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ERB', 'EOGr', 'EMGr', 'C3']], ['EMGl', 'EMGr', 'C3']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 1.0, 5, [['ELC', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc']], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'O1', 'O2', 'EMGc']], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 1.0, 5, [['ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ERB', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc']], ['EOGr', 'EMGl', 'O1', 'O2', 'EMGc', 'ELC']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 1.5, 5, [['ELC', 'EOGr', 'EOGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'EMGl', 'EMGr', 'C3'], ['ELC', 'EMGr', 'C3'], ['ELC', 'ERB', 'EMGr', 'C3'], ['ERB', 'EMGr', 'C3'], ['ERB', 'EMGr', 'C3']], ['EMGr', 'C3']], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 1.5, 5, [['ELC', 'EOGl', 'EMGr', 'C3'], ['ELC', 'EMGr', 'C3'], ['ELC', 'ERB', 'EMGr', 'C3'], ['ELC', 'EOGl', 'EMGr', 'C3'], ['ERB', 'C3'], ['ERB', 'EOGr', 'EMGr', 'C3']], ['C3']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 1.5, 5, [['ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGc'], ['EOGr', 'EOGl', 'EMGl'], ['ELC', 'EOGr', 'EMGl'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl'], ['ERB', 'EOGr', 'EOGl', 'EMGr', 'EMGc'], ['ERB', 'EMGr']], ['EOGr']], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 1.5, 5, [['ERB', 'ERC', 'M1', 'O1', 'O2'], ['ELE', 'ERC', 'M2', 'EMGc'], ['ERA', 'EMGl', 'O1', 'O2'], ['ELT', 'M2'], ['ELC', 'EOGr', 'M1', 'O1', 'O2'], ['ELC', 'EMGl', 'M1', 'O1', 'M2', 'O2', 'EMGc']], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 2.0, 5, [['ELC', 'EMGr', 'C3'], ['EMGr', 'C3'], ['C3'], ['EMGr', 'C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 2.0, 5, [['ELC', 'EMGr', 'C3'], ['ELC', 'EMGr', 'C3'], ['EMGr', 'C3'], ['ELC', 'EMGr', 'C3'], ['ERB', 'C3'], ['EMGr', 'C3']], ['C3']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 2.0, 5, [['ERB'], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 2.5, 5, [[], [], [], ['C3'], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 2.5, 5, [['ELC', 'EMGr', 'C3'], ['EMGr', 'C3'], ['EMGr', 'C3'], ['EMGr', 'C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 3.0, 5, [['ELC', 'C3'], ['EMGr', 'C3'], ['EMGr', 'C3'], ['EMGr', 'C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 4.0, 5, [['C3'], ['EMGr', 'C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 5.0, 5, [[], [], ['C3'], ['C3'], ['C3'], ['C3']], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 1.0, 6, [['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'M2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc']], ['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc']], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 1.0, 6, [['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3'], ['ELC', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'C3'], ['ERB', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ERB', 'EOGr', 'EMGr', 'C3']], ['EMGr', 'C3']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 1.0, 6, [['ELC', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'O2', 'EMGc']], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'O1', 'O2', 'EMGc']], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 1.0, 6, [['ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ELT', 'ELE', 'ERB', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc']], ['EOGr', 'EMGl', 'O1', 'O2', 'EMGc']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 1.5, 6, [['ELC', 'EOGr', 'EOGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'EMGl', 'EMGr', 'C3'], ['ELC', 'EMGr', 'C3'], ['ELC', 'ERB', 'EMGr', 'C3'], ['ERB', 'EMGr', 'C3'], ['ERB', 'EMGr', 'C3']], ['EMGr', 'C3']], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 1.5, 6, [['ELC', 'EOGl', 'EMGr', 'C3'], ['ELC', 'EMGr', 'C3'], ['ELC', 'ERB', 'EMGr', 'C3'], ['ELC', 'EOGl', 'EMGr', 'C3'], ['ERB', 'C3'], ['ERB', 'EOGr', 'EMGr', 'C3']], ['C3']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 1.5, 6, [['ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGc'], ['EOGr', 'EOGl', 'EMGl'], ['ELC', 'EOGr', 'EMGl'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl'], ['ERB', 'EOGr', 'EOGl', 'EMGr', 'EMGc'], ['ERB', 'EMGr']], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 1.5, 6, [['ERB', 'ERC', 'M1', 'O1', 'O2'], ['ELE', 'ERC', 'M2', 'EMGc'], ['ERA', 'EMGl', 'O1', 'O2'], ['ELT', 'M2'], ['ELC', 'EOGr', 'M1', 'O1', 'O2'], ['ELC', 'EMGl', 'M1', 'O1', 'M2', 'O2', 'EMGc']], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 2.0, 6, [['ELC', 'EMGr', 'C3'], ['EMGr', 'C3'], ['C3'], ['EMGr', 'C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 2.0, 6, [['ELC', 'EMGr', 'C3'], ['ELC', 'EMGr', 'C3'], ['EMGr', 'C3'], ['ELC', 'EMGr', 'C3'], ['ERB', 'C3'], ['EMGr', 'C3']], ['C3']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 2.0, 6, [['ERB'], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 2.5, 6, [[], [], [], ['C3'], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 2.5, 6, [['ELC', 'EMGr', 'C3'], ['EMGr', 'C3'], ['EMGr', 'C3'], ['EMGr', 'C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 3.0, 6, [['ELC', 'C3'], ['EMGr', 'C3'], ['EMGr', 'C3'], ['EMGr', 'C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 4.0, 6, [['C3'], ['EMGr', 'C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'mm', 5.0, 6, [[], [], ['C3'], ['C3'], ['C3'], ['C3']], []], ['S_01/night_1\\EEG_raw', 120, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 120, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 1.0, 1, [['ERB', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2'], ['ERB', 'EOGr', 'EOGl', 'M1', 'C3', 'M2'], ['ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'M2'], ['ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4'], ['EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'O2'], ['ERB', 'ERC', 'EOGr', 'EOGl', 'M1', 'C3', 'M2']], ['ERB', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'F3', 'EMGr', 'F4', 'O2', 'ERC']], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 1.0, 1, [['ERB', 'EOGr', 'EOGl', 'C3', 'M2'], ['EOGr', 'EOGl', 'F3', 'C3', 'M2'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['ERB', 'ERC', 'EOGr', 'EOGl', 'C3', 'M2']], ['ERB', 'EOGr', 'EOGl', 'C3', 'M2', 'F3', 'ERC']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 1.0, 1, [['ELB', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2'], ['ELB', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['ELB', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc', 'F3', 'F4', 'ERI', 'ERA']], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 1.0, 1, [['ELB', 'ELC', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc']], ['ELB', 'ELC', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc', 'ERA', 'EOGr', 'F3', 'O2', 'C4']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 1.5, 1, [['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3']], ['EOGr', 'EOGl', 'C3']], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 1.5, 1, [['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3']], ['EOGr', 'EOGl', 'C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 1.5, 1, [['M2'], ['M2'], ['M2'], [], ['M2'], ['ERB', 'M2']], ['M2', 'ERB']], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 1.5, 1, [['ELB', 'ERB', 'M1', 'M2'], ['ELB', 'ERB', 'M1', 'O1', 'M2'], ['ELB', 'ERB', 'M1', 'M2'], ['ERB', 'M1', 'M2'], ['ERB', 'EMGl', 'M1', 'M2'], ['ERB', 'M1', 'M2', 'EMGc']], ['ELB', 'ERB', 'M1', 'M2', 'O1', 'EMGl', 'EMGc']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 2.0, 1, [['EOGr', 'C3'], ['C3'], ['EOGr', 'C3'], ['C3'], ['EOGr', 'EOGl', 'C3'], ['C3']], ['EOGr', 'C3', 'EOGl']], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 2.0, 1, [['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'C3']], ['EOGr', 'EOGl', 'C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 2.0, 1, [['M2'], ['ERB'], ['M2'], [], ['M1'], ['M1']], ['M2', 'ERB', 'M1']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 2.5, 1, [['C3'], ['C3'], ['C3'], [], ['EOGr', 'C3'], []], ['C3', 'EOGr']], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 2.5, 1, [['C3'], ['EOGr', 'C3'], ['C3'], ['EOGr', 'C3'], ['C3'], ['C3']], ['C3', 'EOGr']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 3.0, 1, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 4.0, 1, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 5.0, 1, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], []], ['C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 1.0, 2, [['ERB', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2'], ['ERB', 'EOGr', 'EOGl', 'M1', 'C3', 'M2'], ['ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'M2'], ['ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4'], ['EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'O2'], ['ERB', 'ERC', 'EOGr', 'EOGl', 'M1', 'C3', 'M2']], ['ERB', 'EOGr', 'EOGl', 'M1', 'C3', 'M2', 'F3']], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 1.0, 2, [['ERB', 'EOGr', 'EOGl', 'C3', 'M2'], ['EOGr', 'EOGl', 'F3', 'C3', 'M2'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['ERB', 'ERC', 'EOGr', 'EOGl', 'C3', 'M2']], ['EOGr', 'EOGl', 'C3', 'M2']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 1.0, 2, [['ELB', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2'], ['ELB', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['ELB', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc', 'F3']], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 1.0, 2, [['ELB', 'ELC', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc']], ['ELB', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc', 'ERA', 'EOGr', 'O2']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 1.5, 2, [['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3']], ['EOGr', 'EOGl', 'C3']], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 1.5, 2, [['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3']], ['EOGr', 'EOGl', 'C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 1.5, 2, [['M2'], ['M2'], ['M2'], [], ['M2'], ['ERB', 'M2']], ['M2']], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 1.5, 2, [['ELB', 'ERB', 'M1', 'M2'], ['ELB', 'ERB', 'M1', 'O1', 'M2'], ['ELB', 'ERB', 'M1', 'M2'], ['ERB', 'M1', 'M2'], ['ERB', 'EMGl', 'M1', 'M2'], ['ERB', 'M1', 'M2', 'EMGc']], ['ELB', 'ERB', 'M1', 'M2']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 2.0, 2, [['EOGr', 'C3'], ['C3'], ['EOGr', 'C3'], ['C3'], ['EOGr', 'EOGl', 'C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 2.0, 2, [['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'C3']], ['EOGr', 'EOGl', 'C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 2.0, 2, [['M2'], ['ERB'], ['M2'], [], ['M1'], ['M1']], ['M1']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 2.5, 2, [['C3'], ['C3'], ['C3'], [], ['EOGr', 'C3'], []], ['C3']], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 2.5, 2, [['C3'], ['EOGr', 'C3'], ['C3'], ['EOGr', 'C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 3.0, 2, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 4.0, 2, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 5.0, 2, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], []], ['C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 1.0, 3, [['ERB', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2'], ['ERB', 'EOGr', 'EOGl', 'M1', 'C3', 'M2'], ['ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'M2'], ['ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4'], ['EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'O2'], ['ERB', 'ERC', 'EOGr', 'EOGl', 'M1', 'C3', 'M2']], ['ERB', 'EOGr', 'EOGl', 'C3', 'M2']], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 1.0, 3, [['ERB', 'EOGr', 'EOGl', 'C3', 'M2'], ['EOGr', 'EOGl', 'F3', 'C3', 'M2'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['ERB', 'ERC', 'EOGr', 'EOGl', 'C3', 'M2']], ['EOGr', 'EOGl', 'C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 1.0, 3, [['ELB', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2'], ['ELB', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['ELB', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'F3']], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 1.0, 3, [['ELB', 'ELC', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc']], ['ELB', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EOGr']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 1.5, 3, [['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3']], ['EOGr', 'EOGl', 'C3']], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 1.5, 3, [['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3']], ['EOGr', 'EOGl', 'C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 1.5, 3, [['M2'], ['M2'], ['M2'], [], ['M2'], ['ERB', 'M2']], ['M2']], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 1.5, 3, [['ELB', 'ERB', 'M1', 'M2'], ['ELB', 'ERB', 'M1', 'O1', 'M2'], ['ELB', 'ERB', 'M1', 'M2'], ['ERB', 'M1', 'M2'], ['ERB', 'EMGl', 'M1', 'M2'], ['ERB', 'M1', 'M2', 'EMGc']], ['ELB', 'ERB', 'M1', 'M2']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 2.0, 3, [['EOGr', 'C3'], ['C3'], ['EOGr', 'C3'], ['C3'], ['EOGr', 'EOGl', 'C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 2.0, 3, [['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'C3']], ['EOGr', 'C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 2.0, 3, [['M2'], ['ERB'], ['M2'], [], ['M1'], ['M1']], []], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 2.5, 3, [['C3'], ['C3'], ['C3'], [], ['EOGr', 'C3'], []], ['C3']], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 2.5, 3, [['C3'], ['EOGr', 'C3'], ['C3'], ['EOGr', 'C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 3.0, 3, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 4.0, 3, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 5.0, 3, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], []], ['C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 1.0, 4, [['ERB', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2'], ['ERB', 'EOGr', 'EOGl', 'M1', 'C3', 'M2'], ['ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'M2'], ['ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4'], ['EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'O2'], ['ERB', 'ERC', 'EOGr', 'EOGl', 'M1', 'C3', 'M2']], ['ERB', 'EOGr', 'EOGl', 'C3', 'M2']], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 1.0, 4, [['ERB', 'EOGr', 'EOGl', 'C3', 'M2'], ['EOGr', 'EOGl', 'F3', 'C3', 'M2'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['ERB', 'ERC', 'EOGr', 'EOGl', 'C3', 'M2']], ['EOGr', 'EOGl', 'C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 1.0, 4, [['ELB', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2'], ['ELB', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['ELB', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2']], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 1.0, 4, [['ELB', 'ELC', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc']], ['ELB', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EOGr']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 1.5, 4, [['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3']], ['EOGr', 'EOGl', 'C3']], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 1.5, 4, [['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3']], ['EOGr', 'EOGl', 'C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 1.5, 4, [['M2'], ['M2'], ['M2'], [], ['M2'], ['ERB', 'M2']], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 1.5, 4, [['ELB', 'ERB', 'M1', 'M2'], ['ELB', 'ERB', 'M1', 'O1', 'M2'], ['ELB', 'ERB', 'M1', 'M2'], ['ERB', 'M1', 'M2'], ['ERB', 'EMGl', 'M1', 'M2'], ['ERB', 'M1', 'M2', 'EMGc']], ['ERB', 'M1', 'M2']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 2.0, 4, [['EOGr', 'C3'], ['C3'], ['EOGr', 'C3'], ['C3'], ['EOGr', 'EOGl', 'C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 2.0, 4, [['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'C3']], ['EOGr', 'C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 2.0, 4, [['M2'], ['ERB'], ['M2'], [], ['M1'], ['M1']], []], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 2.5, 4, [['C3'], ['C3'], ['C3'], [], ['EOGr', 'C3'], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 2.5, 4, [['C3'], ['EOGr', 'C3'], ['C3'], ['EOGr', 'C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 3.0, 4, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 4.0, 4, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 5.0, 4, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], []], ['C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 1.0, 5, [['ERB', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2'], ['ERB', 'EOGr', 'EOGl', 'M1', 'C3', 'M2'], ['ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'M2'], ['ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4'], ['EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'O2'], ['ERB', 'ERC', 'EOGr', 'EOGl', 'M1', 'C3', 'M2']], ['EOGr', 'EOGl', 'C3', 'M2']], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 1.0, 5, [['ERB', 'EOGr', 'EOGl', 'C3', 'M2'], ['EOGr', 'EOGl', 'F3', 'C3', 'M2'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['ERB', 'ERC', 'EOGr', 'EOGl', 'C3', 'M2']], ['EOGr', 'EOGl', 'C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 1.0, 5, [['ELB', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2'], ['ELB', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['ELB', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2']], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 1.0, 5, [['ELB', 'ELC', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc']], ['ELB', 'ERB', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'EOGr']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 1.5, 5, [['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3']], ['EOGr', 'EOGl', 'C3']], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 1.5, 5, [['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3']], ['EOGr', 'EOGl', 'C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 1.5, 5, [['M2'], ['M2'], ['M2'], [], ['M2'], ['ERB', 'M2']], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 1.5, 5, [['ELB', 'ERB', 'M1', 'M2'], ['ELB', 'ERB', 'M1', 'O1', 'M2'], ['ELB', 'ERB', 'M1', 'M2'], ['ERB', 'M1', 'M2'], ['ERB', 'EMGl', 'M1', 'M2'], ['ERB', 'M1', 'M2', 'EMGc']], ['ERB', 'M1', 'M2']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 2.0, 5, [['EOGr', 'C3'], ['C3'], ['EOGr', 'C3'], ['C3'], ['EOGr', 'EOGl', 'C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 2.0, 5, [['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'C3']], ['EOGr', 'C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 2.0, 5, [['M2'], ['ERB'], ['M2'], [], ['M1'], ['M1']], []], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 2.5, 5, [['C3'], ['C3'], ['C3'], [], ['EOGr', 'C3'], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 2.5, 5, [['C3'], ['EOGr', 'C3'], ['C3'], ['EOGr', 'C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 3.0, 5, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 4.0, 5, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 5.0, 5, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], []], ['C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 1.0, 6, [['ERB', 'EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2'], ['ERB', 'EOGr', 'EOGl', 'M1', 'C3', 'M2'], ['ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'M2'], ['ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'M2', 'F4'], ['EOGr', 'EOGl', 'M1', 'C3', 'O1', 'M2', 'O2'], ['ERB', 'ERC', 'EOGr', 'EOGl', 'M1', 'C3', 'M2']], ['EOGr', 'EOGl', 'C3', 'M2']], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 1.0, 6, [['ERB', 'EOGr', 'EOGl', 'C3', 'M2'], ['EOGr', 'EOGl', 'F3', 'C3', 'M2'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['ERB', 'ERC', 'EOGr', 'EOGl', 'C3', 'M2']], ['EOGr', 'EOGl', 'C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 1.0, 6, [['ELB', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2'], ['ELB', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['ELB', 'ERB', 'EOGr', 'EOGl', 'M1', 'O1', 'M2']], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 1.0, 6, [['ELB', 'ELC', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'M2', 'C4', 'EMGc']], ['ELB', 'ERB', 'EOGl', 'M1', 'O1', 'M2']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 1.5, 6, [['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3']], ['EOGr', 'EOGl', 'C3']], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 1.5, 6, [['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3']], ['EOGr', 'EOGl', 'C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 1.5, 6, [['M2'], ['M2'], ['M2'], [], ['M2'], ['ERB', 'M2']], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 1.5, 6, [['ELB', 'ERB', 'M1', 'M2'], ['ELB', 'ERB', 'M1', 'O1', 'M2'], ['ELB', 'ERB', 'M1', 'M2'], ['ERB', 'M1', 'M2'], ['ERB', 'EMGl', 'M1', 'M2'], ['ERB', 'M1', 'M2', 'EMGc']], ['ERB', 'M1', 'M2']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 2.0, 6, [['EOGr', 'C3'], ['C3'], ['EOGr', 'C3'], ['C3'], ['EOGr', 'EOGl', 'C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 2.0, 6, [['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'EOGl', 'C3'], ['EOGr', 'C3']], ['EOGr', 'C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 2.0, 6, [['M2'], ['ERB'], ['M2'], [], ['M1'], ['M1']], []], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 2.5, 6, [['C3'], ['C3'], ['C3'], [], ['EOGr', 'C3'], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 2.5, 6, [['C3'], ['EOGr', 'C3'], ['C3'], ['EOGr', 'C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 3.0, 6, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 4.0, 6, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'mm', 5.0, 6, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 410, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 1.0, 1, [['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'C3'], ['ELI', 'ERA', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELI', 'ERA', 'ERB', 'ERC', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'C4']], ['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELB', 'EOGl', 'ELC']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 1.0, 1, [['ERB', 'C3'], ['ELI', 'ERB', 'C3', 'O2'], ['ELI', 'C3'], ['ELI', 'ERB', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELI', 'ERB', 'C3', 'O1', 'O2'], ['ELI', 'ERB', 'C3', 'O1', 'O2']], ['ERB', 'C3', 'ELI', 'O2', 'F3', 'O1', 'F4']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 1.0, 1, [['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'ELC', 'EOGl', 'EMGc', 'M1']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 1.0, 1, [['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C4', 'O2']], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2', 'ELB', 'F3', 'EMGc', 'M1']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 1.5, 1, [['ELI', 'C3'], ['C3'], ['C3'], ['ELI', 'ERB', 'C3'], ['ELI', 'C3'], ['ELI', 'ERB', 'C3']], ['ELI', 'C3', 'ERB']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 1.5, 1, [['C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'C3', 'O2'], ['ELI', 'C3'], ['ELI', 'ERB', 'C3']], ['C3', 'ELI', 'O2', 'ERB']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 1.5, 1, [['ELI'], ['ELI'], ['ELI'], ['ELI', 'ERB'], ['ELI', 'O1', 'O2'], ['ELI']], ['ELI', 'ERB', 'O1', 'O2']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 1.5, 1, [['ELI', 'ERA', 'EOGr', 'EMGl'], ['EOGr', 'EMGl', 'EMGc'], ['EOGr', 'C4'], ['ELI', 'EMGl'], ['ERA', 'EOGl'], ['ELI', 'ERA', 'ERC', 'EOGr']], ['ELI', 'ERA', 'EOGr', 'EMGl', 'EMGc', 'C4', 'EOGl', 'ERC']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 2.0, 1, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 2.0, 1, [['C3'], ['C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'ERB', 'C3']], ['C3', 'ELI', 'ERB']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 2.5, 1, [['C3'], ['C3'], ['C3'], ['C3'], [], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 2.5, 1, [['C3'], ['C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'ERB', 'C3']], ['C3', 'ELI', 'ERB']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 3.0, 1, [['C3'], ['C3'], [], [], [], []], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 3.0, 1, [['C3'], ['C3'], ['C3'], ['C3'], ['ELI', 'C3'], ['ERB', 'C3']], ['C3', 'ELI', 'ERB']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 4.0, 1, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 5.0, 1, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 1.0, 2, [['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'C3'], ['ELI', 'ERA', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELI', 'ERA', 'ERB', 'ERC', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'C4']], ['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EOGl']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 1.0, 2, [['ERB', 'C3'], ['ELI', 'ERB', 'C3', 'O2'], ['ELI', 'C3'], ['ELI', 'ERB', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELI', 'ERB', 'C3', 'O1', 'O2'], ['ELI', 'ERB', 'C3', 'O1', 'O2']], ['ERB', 'C3', 'ELI', 'O2', 'O1']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 1.0, 2, [['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EOGl']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 1.0, 2, [['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C4', 'O2']], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2', 'F3']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 1.5, 2, [['ELI', 'C3'], ['C3'], ['C3'], ['ELI', 'ERB', 'C3'], ['ELI', 'C3'], ['ELI', 'ERB', 'C3']], ['ELI', 'C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 1.5, 2, [['C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'C3', 'O2'], ['ELI', 'C3'], ['ELI', 'ERB', 'C3']], ['C3', 'ELI']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 1.5, 2, [['ELI'], ['ELI'], ['ELI'], ['ELI', 'ERB'], ['ELI', 'O1', 'O2'], ['ELI']], ['ELI']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 1.5, 2, [['ELI', 'ERA', 'EOGr', 'EMGl'], ['EOGr', 'EMGl', 'EMGc'], ['EOGr', 'C4'], ['ELI', 'EMGl'], ['ERA', 'EOGl'], ['ELI', 'ERA', 'ERC', 'EOGr']], ['ERA', 'EOGr', 'EMGl']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 2.0, 2, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 2.0, 2, [['C3'], ['C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'ERB', 'C3']], ['C3', 'ELI']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 2.5, 2, [['C3'], ['C3'], ['C3'], ['C3'], [], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 2.5, 2, [['C3'], ['C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'ERB', 'C3']], ['C3', 'ELI']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 3.0, 2, [['C3'], ['C3'], [], [], [], []], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 3.0, 2, [['C3'], ['C3'], ['C3'], ['C3'], ['ELI', 'C3'], ['ERB', 'C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 4.0, 2, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 5.0, 2, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 1.0, 3, [['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'C3'], ['ELI', 'ERA', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELI', 'ERA', 'ERB', 'ERC', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'C4']], ['ELI', 'ERA', 'ERB', 'ERC', 'EMGl', 'F3', 'C3', 'C4', 'O2']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 1.0, 3, [['ERB', 'C3'], ['ELI', 'ERB', 'C3', 'O2'], ['ELI', 'C3'], ['ELI', 'ERB', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELI', 'ERB', 'C3', 'O1', 'O2'], ['ELI', 'ERB', 'C3', 'O1', 'O2']], ['ERB', 'C3', 'ELI', 'O2', 'O1']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 1.0, 3, [['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EOGl']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 1.0, 3, [['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C4', 'O2']], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 1.5, 3, [['ELI', 'C3'], ['C3'], ['C3'], ['ELI', 'ERB', 'C3'], ['ELI', 'C3'], ['ELI', 'ERB', 'C3']], ['ELI', 'C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 1.5, 3, [['C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'C3', 'O2'], ['ELI', 'C3'], ['ELI', 'ERB', 'C3']], ['C3', 'ELI']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 1.5, 3, [['ELI'], ['ELI'], ['ELI'], ['ELI', 'ERB'], ['ELI', 'O1', 'O2'], ['ELI']], ['ELI']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 1.5, 3, [['ELI', 'ERA', 'EOGr', 'EMGl'], ['EOGr', 'EMGl', 'EMGc'], ['EOGr', 'C4'], ['ELI', 'EMGl'], ['ERA', 'EOGl'], ['ELI', 'ERA', 'ERC', 'EOGr']], ['EOGr']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 2.0, 3, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 2.0, 3, [['C3'], ['C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'ERB', 'C3']], ['C3', 'ELI']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 2.5, 3, [['C3'], ['C3'], ['C3'], ['C3'], [], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 2.5, 3, [['C3'], ['C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'ERB', 'C3']], ['C3', 'ELI']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 3.0, 3, [['C3'], ['C3'], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 3.0, 3, [['C3'], ['C3'], ['C3'], ['C3'], ['ELI', 'C3'], ['ERB', 'C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 4.0, 3, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 5.0, 3, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 1.0, 4, [['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'C3'], ['ELI', 'ERA', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELI', 'ERA', 'ERB', 'ERC', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'C4']], ['ELI', 'ERB', 'EMGl', 'F3', 'C3', 'C4']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 1.0, 4, [['ERB', 'C3'], ['ELI', 'ERB', 'C3', 'O2'], ['ELI', 'C3'], ['ELI', 'ERB', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELI', 'ERB', 'C3', 'O1', 'O2'], ['ELI', 'ERB', 'C3', 'O1', 'O2']], ['C3', 'ELI']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 1.0, 4, [['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EOGl']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 1.0, 4, [['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C4', 'O2']], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 1.5, 4, [['ELI', 'C3'], ['C3'], ['C3'], ['ELI', 'ERB', 'C3'], ['ELI', 'C3'], ['ELI', 'ERB', 'C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 1.5, 4, [['C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'C3', 'O2'], ['ELI', 'C3'], ['ELI', 'ERB', 'C3']], ['C3', 'ELI']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 1.5, 4, [['ELI'], ['ELI'], ['ELI'], ['ELI', 'ERB'], ['ELI', 'O1', 'O2'], ['ELI']], ['ELI']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 1.5, 4, [['ELI', 'ERA', 'EOGr', 'EMGl'], ['EOGr', 'EMGl', 'EMGc'], ['EOGr', 'C4'], ['ELI', 'EMGl'], ['ERA', 'EOGl'], ['ELI', 'ERA', 'ERC', 'EOGr']], []], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 2.0, 4, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 2.0, 4, [['C3'], ['C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'ERB', 'C3']], ['C3', 'ELI']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 2.5, 4, [['C3'], ['C3'], ['C3'], ['C3'], [], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 2.5, 4, [['C3'], ['C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'ERB', 'C3']], ['C3', 'ELI']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 3.0, 4, [['C3'], ['C3'], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 3.0, 4, [['C3'], ['C3'], ['C3'], ['C3'], ['ELI', 'C3'], ['ERB', 'C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 4.0, 4, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 5.0, 4, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 1.0, 5, [['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'C3'], ['ELI', 'ERA', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELI', 'ERA', 'ERB', 'ERC', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'C4']], ['ELI', 'ERB', 'C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 1.0, 5, [['ERB', 'C3'], ['ELI', 'ERB', 'C3', 'O2'], ['ELI', 'C3'], ['ELI', 'ERB', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELI', 'ERB', 'C3', 'O1', 'O2'], ['ELI', 'ERB', 'C3', 'O1', 'O2']], ['C3', 'ELI']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 1.0, 5, [['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EOGl']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 1.0, 5, [['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C4', 'O2']], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 1.5, 5, [['ELI', 'C3'], ['C3'], ['C3'], ['ELI', 'ERB', 'C3'], ['ELI', 'C3'], ['ELI', 'ERB', 'C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 1.5, 5, [['C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'C3', 'O2'], ['ELI', 'C3'], ['ELI', 'ERB', 'C3']], ['C3', 'ELI']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 1.5, 5, [['ELI'], ['ELI'], ['ELI'], ['ELI', 'ERB'], ['ELI', 'O1', 'O2'], ['ELI']], ['ELI']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 1.5, 5, [['ELI', 'ERA', 'EOGr', 'EMGl'], ['EOGr', 'EMGl', 'EMGc'], ['EOGr', 'C4'], ['ELI', 'EMGl'], ['ERA', 'EOGl'], ['ELI', 'ERA', 'ERC', 'EOGr']], []], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 2.0, 5, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 2.0, 5, [['C3'], ['C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'ERB', 'C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 2.5, 5, [['C3'], ['C3'], ['C3'], ['C3'], [], ['C3']], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 2.5, 5, [['C3'], ['C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'ERB', 'C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 3.0, 5, [['C3'], ['C3'], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 3.0, 5, [['C3'], ['C3'], ['C3'], ['C3'], ['ELI', 'C3'], ['ERB', 'C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 4.0, 5, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 5.0, 5, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 1.0, 6, [['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'C3'], ['ELI', 'ERA', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELI', 'ERA', 'ERB', 'ERC', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'C4']], ['ELI', 'ERB', 'C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 1.0, 6, [['ERB', 'C3'], ['ELI', 'ERB', 'C3', 'O2'], ['ELI', 'C3'], ['ELI', 'ERB', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELI', 'ERB', 'C3', 'O1', 'O2'], ['ELI', 'ERB', 'C3', 'O1', 'O2']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 1.0, 6, [['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'F3', 'O1', 'C4', 'O2']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 1.0, 6, [['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C4', 'O2']], ['ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O2']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 1.5, 6, [['ELI', 'C3'], ['C3'], ['C3'], ['ELI', 'ERB', 'C3'], ['ELI', 'C3'], ['ELI', 'ERB', 'C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 1.5, 6, [['C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'C3', 'O2'], ['ELI', 'C3'], ['ELI', 'ERB', 'C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 1.5, 6, [['ELI'], ['ELI'], ['ELI'], ['ELI', 'ERB'], ['ELI', 'O1', 'O2'], ['ELI']], ['ELI']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 1.5, 6, [['ELI', 'ERA', 'EOGr', 'EMGl'], ['EOGr', 'EMGl', 'EMGc'], ['EOGr', 'C4'], ['ELI', 'EMGl'], ['ERA', 'EOGl'], ['ELI', 'ERA', 'ERC', 'EOGr']], []], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 2.0, 6, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 2.0, 6, [['C3'], ['C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'ERB', 'C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 2.5, 6, [['C3'], ['C3'], ['C3'], ['C3'], [], ['C3']], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 2.5, 6, [['C3'], ['C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'ERB', 'C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 3.0, 6, [['C3'], ['C3'], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 3.0, 6, [['C3'], ['C3'], ['C3'], ['C3'], ['ELI', 'C3'], ['ERB', 'C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 4.0, 6, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'mm', 5.0, 6, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 5780, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 5780, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 1.0, 1, [['ELI', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELI', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EOGr', 'EOGl', 'ELB', 'ELC']], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 1.0, 1, [['ELI', 'ERT ', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERT ', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELI', 'ERT ', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EOGr', 'EOGl', 'EMGl']], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 1.0, 1, [['ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELI', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ERB', 'EOGl', 'ELT', 'EMGc']], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 1.0, 1, [['ELC', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ELB', 'ELC', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4'], ['ELC', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELC', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'O2', 'ELB', 'F4', 'O1', 'EMGc']], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 1.5, 1, [[], ['C3'], ['C3'], [], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 1.5, 1, [['ELI', 'F3', 'C3'], ['ELI', 'F3', 'C3'], ['F3', 'C3', 'O1', 'C4'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4', 'C4'], ['ELI', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELI', 'F3', 'C3', 'O1', 'C4', 'F4', 'O2']], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 1.5, 1, [['ERT '], [], [], [], [], ['O2']], ['ERT ', 'O2']], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 1.5, 1, [['ERT ', 'F3'], ['F3'], [], ['ELI', 'EOGl'], ['ELI'], []], ['ERT ', 'F3', 'ELI', 'EOGl']], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 2.0, 1, [['C3'], ['C3'], ['C3'], [], ['F3', 'C3', 'F4', 'C4'], ['C4']], ['C3', 'F3', 'F4', 'C4']], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 2.5, 1, [[], [], ['C3'], [], [], []], ['C3']], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 3.0, 1, [[], [], ['C3'], [], [], []], ['C3']], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 1.0, 2, [['ELI', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELI', 'ERC', 'ERT ', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EOGr', 'EOGl', 'ELB']], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 1.0, 2, [['ELI', 'ERT ', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERT ', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELI', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EOGr']], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 1.0, 2, [['ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELI', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ERB', 'EOGl']], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 1.0, 2, [['ELC', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ELB', 'ELC', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4'], ['ELC', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELC', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'O2', 'ELB', 'F4']], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 1.5, 2, [[], ['C3'], ['C3'], [], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 1.5, 2, [['ELI', 'F3', 'C3'], ['ELI', 'F3', 'C3'], ['F3', 'C3', 'O1', 'C4'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4', 'C4'], ['ELI', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELI', 'F3', 'C3', 'C4', 'F4']], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 1.5, 2, [['ERT '], [], [], [], [], ['O2']], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 1.5, 2, [['ERT ', 'F3'], ['F3'], [], ['ELI', 'EOGl'], ['ELI'], []], ['F3', 'ELI']], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 2.0, 2, [['C3'], ['C3'], ['C3'], [], ['F3', 'C3', 'F4', 'C4'], ['C4']], ['C3', 'C4']], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 2.5, 2, [[], [], ['C3'], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 3.0, 2, [[], [], ['C3'], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 1.0, 3, [['ELI', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELI', 'ERC', 'ERT ', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EOGr', 'EOGl', 'ELB']], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 1.0, 3, [['ELI', 'ERT ', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERT ', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELI', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 1.0, 3, [['ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELI', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 1.0, 3, [['ELC', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ELB', 'ELC', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4'], ['ELC', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELC', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'O2', 'F4']], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 1.5, 3, [[], ['C3'], ['C3'], [], ['C3'], ['C3']], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 1.5, 3, [['ELI', 'F3', 'C3'], ['ELI', 'F3', 'C3'], ['F3', 'C3', 'O1', 'C4'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4', 'C4'], ['ELI', 'F3', 'C3', 'F4', 'C4', 'O2']], ['F3', 'C3', 'C4', 'F4']], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 1.5, 3, [['ERT '], [], [], [], [], ['O2']], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 1.5, 3, [['ERT ', 'F3'], ['F3'], [], ['ELI', 'EOGl'], ['ELI'], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 2.0, 3, [['C3'], ['C3'], ['C3'], [], ['F3', 'C3', 'F4', 'C4'], ['C4']], ['C3']], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 2.5, 3, [[], [], ['C3'], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 3.0, 3, [[], [], ['C3'], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 1.0, 4, [['ELI', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERT ', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EOGr']], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 1.0, 4, [['ELI', 'ERT ', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERT ', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 1.0, 4, [['ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELI', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELI', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 1.0, 4, [['ELC', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ELB', 'ELC', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4'], ['ELC', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELC', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C4']], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 1.5, 4, [[], ['C3'], ['C3'], [], ['C3'], ['C3']], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 1.5, 4, [['ELI', 'F3', 'C3'], ['ELI', 'F3', 'C3'], ['F3', 'C3', 'O1', 'C4'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4', 'C4'], ['ELI', 'F3', 'C3', 'F4', 'C4', 'O2']], ['F3', 'C3', 'C4']], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 1.5, 4, [['ERT '], [], [], [], [], ['O2']], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 1.5, 4, [['ERT ', 'F3'], ['F3'], [], ['ELI', 'EOGl'], ['ELI'], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 2.0, 4, [['C3'], ['C3'], ['C3'], [], ['F3', 'C3', 'F4', 'C4'], ['C4']], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 2.5, 4, [[], [], ['C3'], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 3.0, 4, [[], [], ['C3'], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 1.0, 5, [['ELI', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERT ', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EOGr']], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 1.0, 5, [['ELI', 'ERT ', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERT ', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 1.0, 5, [['ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELI', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELI', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 1.0, 5, [['ELC', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ELB', 'ELC', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4'], ['ELC', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELC', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C4']], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 1.5, 5, [[], ['C3'], ['C3'], [], ['C3'], ['C3']], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 1.5, 5, [['ELI', 'F3', 'C3'], ['ELI', 'F3', 'C3'], ['F3', 'C3', 'O1', 'C4'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4', 'C4'], ['ELI', 'F3', 'C3', 'F4', 'C4', 'O2']], ['F3', 'C3']], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 1.5, 5, [['ERT '], [], [], [], [], ['O2']], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 1.5, 5, [['ERT ', 'F3'], ['F3'], [], ['ELI', 'EOGl'], ['ELI'], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 2.0, 5, [['C3'], ['C3'], ['C3'], [], ['F3', 'C3', 'F4', 'C4'], ['C4']], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 2.5, 5, [[], [], ['C3'], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 3.0, 5, [[], [], ['C3'], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 1.0, 6, [['ELI', 'ERC', 'ERT ', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERT ', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 1.0, 6, [['ELI', 'ERT ', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERT ', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 1.0, 6, [['ELB', 'ELC', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELT', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELI', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELI', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 1.0, 6, [['ELC', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'O2'], ['ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ELB', 'ELC', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4'], ['ELC', 'ELI', 'ERB', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2']], ['ELC', 'ELI', 'ERC', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C4']], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 1.5, 6, [[], ['C3'], ['C3'], [], ['C3'], ['C3']], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 1.5, 6, [['ELI', 'F3', 'C3'], ['ELI', 'F3', 'C3'], ['F3', 'C3', 'O1', 'C4'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4', 'C4'], ['ELI', 'F3', 'C3', 'F4', 'C4', 'O2']], ['F3', 'C3']], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 1.5, 6, [['ERT '], [], [], [], [], ['O2']], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 1.5, 6, [['ERT ', 'F3'], ['F3'], [], ['ELI', 'EOGl'], ['ELI'], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 2.0, 6, [['C3'], ['C3'], ['C3'], [], ['F3', 'C3', 'F4', 'C4'], ['C4']], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 2.5, 6, [[], [], ['C3'], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 3.0, 6, [[], [], ['C3'], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_1\\EEG_raw', 10800, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 1.0, 1, [['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'C4', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C4', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'O2', 'EMGc']], ['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'O2', 'EMGc', 'C4', 'ELA', 'M1', 'F3', 'O1', 'ELE', 'ERE']], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 1.0, 1, [['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4'], ['ERC', 'EOGl', 'C4'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGl', 'C4'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl']], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'C4', 'ELA', 'ELC', 'EMGc']], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 1.0, 1, [['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C4', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'O2', 'EMGc']], ['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc', 'M1', 'C4', 'ELA', 'ELE', 'ERE']], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 1.0, 1, [['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O2', 'EMGc']], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc', 'ELE', 'ERA', 'ERE', 'ERI', 'F4', 'ERC']], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 1.5, 1, [['ERA', 'ERC', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'C4'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr'], ['ELC', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'C4', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'ERC', 'EOGl']], ['ERA', 'ERC', 'EMGl', 'EMGr', 'EMGc', 'ELC', 'EOGl', 'C4', 'EOGr']], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 1.5, 1, [['ERA', 'ERC', 'EOGr'], ['EOGl', 'C4'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGc'], ['C4'], ['ERA', 'EMGr'], ['ELC', 'ERA', 'ERC', 'EOGl']], ['ERA', 'ERC', 'EOGr', 'EOGl', 'C4', 'ELC', 'EMGl', 'EMGc', 'EMGr']], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 1.5, 1, [['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'C4'], ['ERA', 'EMGl', 'EMGc'], ['ELC', 'ERA', 'EMGl', 'EMGr', 'C4', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'EOGl']], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'ELC', 'ERC', 'C4']], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 1.5, 1, [['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'M1', 'M2', 'O2'], ['M1', 'O2'], ['ELA', 'ELC', 'ERA', 'EMGr', 'M1', 'O1', 'O2'], ['ELC', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'EMGl', 'EMGr', 'O2']], ['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'O2', 'EMGc', 'ELA', 'ERA', 'M1', 'O1', 'ERC', 'ERE']], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 2.0, 1, [['ERA', 'ERC'], ['ERC', 'EOGl', 'C4'], ['EOGl', 'EMGl'], ['EOGl', 'EMGl', 'C4'], ['ERA'], ['ERA']], ['ERA', 'ERC', 'EOGl', 'C4', 'EMGl']], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 2.0, 1, [['ERA', 'ERC'], ['C4'], ['ERA', 'ERC', 'EOGl', 'EMGl'], ['C4'], ['ERA'], ['ERA', 'ERC']], ['ERA', 'ERC', 'C4', 'EOGl', 'EMGl']], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 2.0, 1, [[], [], ['EMGl'], ['C4'], [], []], ['EMGl', 'C4']], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], ['ELA']], ['ELA']], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 2.5, 1, [[], ['EOGl', 'C4'], [], ['EOGl', 'C4'], [], []], ['EOGl', 'C4']], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 2.5, 1, [['ERA', 'ERC'], ['C4'], ['ERC', 'EOGl', 'EMGl'], ['C4'], ['ERA'], ['ERA', 'ERC']], ['ERA', 'ERC', 'C4', 'EOGl', 'EMGl']], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 3.0, 1, [[], ['C4'], [], ['C4'], [], []], ['C4']], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 3.0, 1, [['ERA', 'ERC'], ['C4'], ['EOGl', 'EMGl'], ['C4'], ['ERA'], ['ERA', 'ERC']], ['ERA', 'ERC', 'C4', 'EOGl', 'EMGl']], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 4.0, 1, [['ERA', 'ERC'], ['C4'], ['EOGl'], ['C4'], ['ERA'], ['ERC']], ['ERA', 'ERC', 'C4', 'EOGl']], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 5.0, 1, [['ERC'], ['C4'], [], ['C4'], ['ERA'], []], ['ERC', 'C4', 'ERA']], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 1.0, 2, [['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'C4', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C4', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'O2', 'EMGc']], ['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'O2', 'EMGc', 'F3']], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 1.0, 2, [['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4'], ['ERC', 'EOGl', 'C4'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGl', 'C4'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl']], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl']], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 1.0, 2, [['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C4', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'O2', 'EMGc']], ['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc', 'M1', 'ELA']], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 1.0, 2, [['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O2', 'EMGc']], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc', 'ELE', 'ERA', 'ERC']], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 1.5, 2, [['ERA', 'ERC', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'C4'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr'], ['ELC', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'C4', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'ERC', 'EOGl']], ['ERA', 'ERC', 'EMGl', 'EMGr', 'EMGc', 'ELC', 'EOGl']], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 1.5, 2, [['ERA', 'ERC', 'EOGr'], ['EOGl', 'C4'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGc'], ['C4'], ['ERA', 'EMGr'], ['ELC', 'ERA', 'ERC', 'EOGl']], ['ERA', 'EOGl']], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 1.5, 2, [['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'C4'], ['ERA', 'EMGl', 'EMGc'], ['ELC', 'ERA', 'EMGl', 'EMGr', 'C4', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'EOGl']], ['ERA', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 1.5, 2, [['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'M1', 'M2', 'O2'], ['M1', 'O2'], ['ELA', 'ELC', 'ERA', 'EMGr', 'M1', 'O1', 'O2'], ['ELC', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'EMGl', 'EMGr', 'O2']], ['ELC', 'EMGl', 'EMGr', 'M2', 'O2', 'M1']], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 2.0, 2, [['ERA', 'ERC'], ['ERC', 'EOGl', 'C4'], ['EOGl', 'EMGl'], ['EOGl', 'EMGl', 'C4'], ['ERA'], ['ERA']], ['ERA', 'ERC', 'EOGl', 'EMGl']], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 2.0, 2, [['ERA', 'ERC'], ['C4'], ['ERA', 'ERC', 'EOGl', 'EMGl'], ['C4'], ['ERA'], ['ERA', 'ERC']], ['ERA']], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 2.0, 2, [[], [], ['EMGl'], ['C4'], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], ['ELA']], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 2.5, 2, [[], ['EOGl', 'C4'], [], ['EOGl', 'C4'], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 2.5, 2, [['ERA', 'ERC'], ['C4'], ['ERC', 'EOGl', 'EMGl'], ['C4'], ['ERA'], ['ERA', 'ERC']], ['ERA']], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 3.0, 2, [[], ['C4'], [], ['C4'], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 3.0, 2, [['ERA', 'ERC'], ['C4'], ['EOGl', 'EMGl'], ['C4'], ['ERA'], ['ERA', 'ERC']], ['ERA']], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 4.0, 2, [['ERA', 'ERC'], ['C4'], ['EOGl'], ['C4'], ['ERA'], ['ERC']], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 5.0, 2, [['ERC'], ['C4'], [], ['C4'], ['ERA'], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 1.0, 3, [['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'C4', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C4', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'O2', 'EMGc']], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 1.0, 3, [['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4'], ['ERC', 'EOGl', 'C4'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGl', 'C4'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl']], ['ERC', 'EOGl', 'EMGl']], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 1.0, 3, [['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C4', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'O2', 'EMGc']], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'O2', 'EMGc', 'M1']], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 1.0, 3, [['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O2', 'EMGc']], ['ELA', 'ELC', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc', 'ELE', 'ERA']], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 1.5, 3, [['ERA', 'ERC', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'C4'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr'], ['ELC', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'C4', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'ERC', 'EOGl']], ['ERA', 'ERC', 'EMGl', 'EMGr', 'ELC', 'EOGl']], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 1.5, 3, [['ERA', 'ERC', 'EOGr'], ['EOGl', 'C4'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGc'], ['C4'], ['ERA', 'EMGr'], ['ELC', 'ERA', 'ERC', 'EOGl']], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 1.5, 3, [['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'C4'], ['ERA', 'EMGl', 'EMGc'], ['ELC', 'ERA', 'EMGl', 'EMGr', 'C4', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'EOGl']], ['ERA', 'EMGl', 'EMGc']], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 1.5, 3, [['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'M1', 'M2', 'O2'], ['M1', 'O2'], ['ELA', 'ELC', 'ERA', 'EMGr', 'M1', 'O1', 'O2'], ['ELC', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'EMGl', 'EMGr', 'O2']], ['ELC', 'EMGr', 'O2', 'M1']], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 2.0, 3, [['ERA', 'ERC'], ['ERC', 'EOGl', 'C4'], ['EOGl', 'EMGl'], ['EOGl', 'EMGl', 'C4'], ['ERA'], ['ERA']], ['EOGl']], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 2.0, 3, [['ERA', 'ERC'], ['C4'], ['ERA', 'ERC', 'EOGl', 'EMGl'], ['C4'], ['ERA'], ['ERA', 'ERC']], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 2.0, 3, [[], [], ['EMGl'], ['C4'], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], ['ELA']], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 2.5, 3, [[], ['EOGl', 'C4'], [], ['EOGl', 'C4'], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 2.5, 3, [['ERA', 'ERC'], ['C4'], ['ERC', 'EOGl', 'EMGl'], ['C4'], ['ERA'], ['ERA', 'ERC']], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 3.0, 3, [[], ['C4'], [], ['C4'], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 3.0, 3, [['ERA', 'ERC'], ['C4'], ['EOGl', 'EMGl'], ['C4'], ['ERA'], ['ERA', 'ERC']], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 4.0, 3, [['ERA', 'ERC'], ['C4'], ['EOGl'], ['C4'], ['ERA'], ['ERC']], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 5.0, 3, [['ERC'], ['C4'], [], ['C4'], ['ERA'], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 1.0, 4, [['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'C4', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C4', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'O2', 'EMGc']], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 1.0, 4, [['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4'], ['ERC', 'EOGl', 'C4'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGl', 'C4'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl']], ['EOGl']], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 1.0, 4, [['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C4', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'O2', 'EMGc']], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'O2', 'EMGc']], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 1.0, 4, [['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O2', 'EMGc']], ['ELA', 'ELC', 'EOGr', 'EMGr', 'M1', 'O1', 'O2', 'EMGc']], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 1.5, 4, [['ERA', 'ERC', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'C4'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr'], ['ELC', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'C4', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'ERC', 'EOGl']], ['ERC', 'EMGl', 'EMGr', 'EOGl']], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 1.5, 4, [['ERA', 'ERC', 'EOGr'], ['EOGl', 'C4'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGc'], ['C4'], ['ERA', 'EMGr'], ['ELC', 'ERA', 'ERC', 'EOGl']], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 1.5, 4, [['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'C4'], ['ERA', 'EMGl', 'EMGc'], ['ELC', 'ERA', 'EMGl', 'EMGr', 'C4', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'EOGl']], ['ERA', 'EMGl']], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 1.5, 4, [['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'M1', 'M2', 'O2'], ['M1', 'O2'], ['ELA', 'ELC', 'ERA', 'EMGr', 'M1', 'O1', 'O2'], ['ELC', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'EMGl', 'EMGr', 'O2']], ['O2']], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 2.0, 4, [['ERA', 'ERC'], ['ERC', 'EOGl', 'C4'], ['EOGl', 'EMGl'], ['EOGl', 'EMGl', 'C4'], ['ERA'], ['ERA']], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 2.0, 4, [['ERA', 'ERC'], ['C4'], ['ERA', 'ERC', 'EOGl', 'EMGl'], ['C4'], ['ERA'], ['ERA', 'ERC']], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 2.0, 4, [[], [], ['EMGl'], ['C4'], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], ['ELA']], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 2.5, 4, [[], ['EOGl', 'C4'], [], ['EOGl', 'C4'], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 2.5, 4, [['ERA', 'ERC'], ['C4'], ['ERC', 'EOGl', 'EMGl'], ['C4'], ['ERA'], ['ERA', 'ERC']], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 3.0, 4, [[], ['C4'], [], ['C4'], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 3.0, 4, [['ERA', 'ERC'], ['C4'], ['EOGl', 'EMGl'], ['C4'], ['ERA'], ['ERA', 'ERC']], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 4.0, 4, [['ERA', 'ERC'], ['C4'], ['EOGl'], ['C4'], ['ERA'], ['ERC']], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 5.0, 4, [['ERC'], ['C4'], [], ['C4'], ['ERA'], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 1.0, 5, [['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'C4', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C4', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'O2', 'EMGc']], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 1.0, 5, [['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4'], ['ERC', 'EOGl', 'C4'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGl', 'C4'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl']], ['EOGl']], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 1.0, 5, [['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C4', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'O2', 'EMGc']], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'O2', 'EMGc']], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 1.0, 5, [['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O2', 'EMGc']], ['ELC', 'EOGr', 'EMGr', 'M1', 'O1', 'O2', 'EMGc']], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 1.5, 5, [['ERA', 'ERC', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'C4'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr'], ['ELC', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'C4', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'ERC', 'EOGl']], ['ERC', 'EMGl', 'EMGr', 'EOGl']], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 1.5, 5, [['ERA', 'ERC', 'EOGr'], ['EOGl', 'C4'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGc'], ['C4'], ['ERA', 'EMGr'], ['ELC', 'ERA', 'ERC', 'EOGl']], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 1.5, 5, [['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'C4'], ['ERA', 'EMGl', 'EMGc'], ['ELC', 'ERA', 'EMGl', 'EMGr', 'C4', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'EOGl']], ['ERA', 'EMGl']], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 1.5, 5, [['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'M1', 'M2', 'O2'], ['M1', 'O2'], ['ELA', 'ELC', 'ERA', 'EMGr', 'M1', 'O1', 'O2'], ['ELC', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'EMGl', 'EMGr', 'O2']], ['O2']], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 2.0, 5, [['ERA', 'ERC'], ['ERC', 'EOGl', 'C4'], ['EOGl', 'EMGl'], ['EOGl', 'EMGl', 'C4'], ['ERA'], ['ERA']], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 2.0, 5, [['ERA', 'ERC'], ['C4'], ['ERA', 'ERC', 'EOGl', 'EMGl'], ['C4'], ['ERA'], ['ERA', 'ERC']], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 2.0, 5, [[], [], ['EMGl'], ['C4'], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], ['ELA']], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 2.5, 5, [[], ['EOGl', 'C4'], [], ['EOGl', 'C4'], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 2.5, 5, [['ERA', 'ERC'], ['C4'], ['ERC', 'EOGl', 'EMGl'], ['C4'], ['ERA'], ['ERA', 'ERC']], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 3.0, 5, [[], ['C4'], [], ['C4'], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 3.0, 5, [['ERA', 'ERC'], ['C4'], ['EOGl', 'EMGl'], ['C4'], ['ERA'], ['ERA', 'ERC']], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 4.0, 5, [['ERA', 'ERC'], ['C4'], ['EOGl'], ['C4'], ['ERA'], ['ERC']], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 5.0, 5, [['ERC'], ['C4'], [], ['C4'], ['ERA'], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 1.0, 6, [['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'C4', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C4', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'O2', 'EMGc']], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 1.0, 6, [['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4'], ['ERC', 'EOGl', 'C4'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGl', 'C4'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl']], ['EOGl']], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 1.0, 6, [['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C4', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'O2', 'EMGc']], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'O2', 'EMGc']], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 1.0, 6, [['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O2', 'EMGc']], ['ELC', 'EOGr', 'EMGr', 'O2', 'EMGc']], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 1.5, 6, [['ERA', 'ERC', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'C4'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr'], ['ELC', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'C4', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'ERC', 'EOGl']], ['ERC']], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 1.5, 6, [['ERA', 'ERC', 'EOGr'], ['EOGl', 'C4'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'EMGc'], ['C4'], ['ERA', 'EMGr'], ['ELC', 'ERA', 'ERC', 'EOGl']], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 1.5, 6, [['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EOGl', 'EMGl', 'C4'], ['ERA', 'EMGl', 'EMGc'], ['ELC', 'ERA', 'EMGl', 'EMGr', 'C4', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'EOGl']], ['ERA']], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 1.5, 6, [['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'M1', 'M2', 'O2'], ['M1', 'O2'], ['ELA', 'ELC', 'ERA', 'EMGr', 'M1', 'O1', 'O2'], ['ELC', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'EMGl', 'EMGr', 'O2']], ['O2']], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 2.0, 6, [['ERA', 'ERC'], ['ERC', 'EOGl', 'C4'], ['EOGl', 'EMGl'], ['EOGl', 'EMGl', 'C4'], ['ERA'], ['ERA']], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 2.0, 6, [['ERA', 'ERC'], ['C4'], ['ERA', 'ERC', 'EOGl', 'EMGl'], ['C4'], ['ERA'], ['ERA', 'ERC']], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 2.0, 6, [[], [], ['EMGl'], ['C4'], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], ['ELA']], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 2.5, 6, [[], ['EOGl', 'C4'], [], ['EOGl', 'C4'], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 2.5, 6, [['ERA', 'ERC'], ['C4'], ['ERC', 'EOGl', 'EMGl'], ['C4'], ['ERA'], ['ERA', 'ERC']], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 3.0, 6, [[], ['C4'], [], ['C4'], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 3.0, 6, [['ERA', 'ERC'], ['C4'], ['EOGl', 'EMGl'], ['C4'], ['ERA'], ['ERA', 'ERC']], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 4.0, 6, [['ERA', 'ERC'], ['C4'], ['EOGl'], ['C4'], ['ERA'], ['ERC']], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'mm', 5.0, 6, [['ERC'], ['C4'], [], ['C4'], ['ERA'], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 120, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 1.0, 1, [['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'F4'], ['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'O2', 'EMGc', 'ELA', 'ELE', 'C3', 'F4', 'C4', 'M2']], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 1.0, 1, [['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'F4'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGl', 'C3'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'C3', 'C4'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'F3', 'F4'], ['ERA', 'EOGr', 'EOGl', 'F3', 'M2', 'F4']], ['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'F4', 'ELA', 'ELE', 'EMGl', 'C3', 'C4', 'EMGc', 'M2']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 1.0, 1, [['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4'], ['ELA', 'ELC', 'ELE', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'O2', 'EMGc', 'ELA', 'ELE', 'C3', 'C4', 'ERE', 'F4', 'M2']], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 1.0, 1, [['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELC', 'ELE', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc', 'C3', 'ELE', 'ERC', 'C4', 'F4']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 1.5, 1, [['ERA', 'ERC', 'EOGr', 'EOGl'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr'], ['ELA', 'ELE', 'ERA', 'ERC'], ['ELA', 'ELE', 'ERA', 'C3'], ['ELA', 'ERA'], ['ERA', 'EOGr']], ['ERA', 'ERC', 'EOGr', 'EOGl', 'ELA', 'ELE', 'C3']], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 1.5, 1, [['ERA', 'ERC', 'EOGr'], ['ELA', 'ELE', 'ERA', 'ERC'], ['ELA', 'ELE', 'ERA', 'ERC'], ['ELA', 'ELE', 'ERA'], ['ELA', 'ERA'], ['ERA']], ['ERA', 'ERC', 'EOGr', 'ELA', 'ELE']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 1.5, 1, [['ELC', 'ERA', 'EOGr', 'EOGl'], ['ERA', 'ERC', 'EOGr'], ['ERA', 'EOGr', 'C3'], ['ERA', 'ERC', 'C3', 'C4'], ['ERC', 'EOGr'], ['EOGr', 'M2']], ['ELC', 'ERA', 'EOGr', 'EOGl', 'ERC', 'C3', 'C4', 'M2']], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 1.5, 1, [['ELC', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['EOGl', 'EMGl', 'EMGr', 'O2', 'EMGc'], ['ELC', 'ELE', 'EOGr', 'EOGl', 'EMGr', 'C3', 'C4'], ['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3'], ['ELC', 'M2', 'EMGc'], ['ELC', 'ERC', 'O1', 'M2', 'O2', 'EMGc']], ['ELC', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc', 'EOGl', 'ELE', 'EOGr', 'C3', 'C4', 'M2', 'ERC']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 2.0, 1, [['ERA', 'ERC'], [], ['ERA'], [], ['ERA'], ['ERA']], ['ERA', 'ERC']], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 2.0, 1, [['ERA', 'ERC'], ['ELA', 'ELE', 'ERA', 'ERC'], ['ELA', 'ERA'], ['ELA', 'ERA'], ['ELA', 'ERA'], ['ERA']], ['ERA', 'ERC', 'ELA', 'ELE']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 2.0, 1, [['EMGl'], [], [], [], [], ['ELC']], ['EMGl', 'ELC']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 2.5, 1, [['ERA', 'ERC'], ['ELA', 'ERC'], ['ELA', 'ERA'], ['ELA', 'ERA'], ['ERA'], ['ERA']], ['ERA', 'ERC', 'ELA']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 3.0, 1, [['ERA', 'ERC'], [], ['ELA', 'ERA'], ['ELA', 'ERA'], ['ERA'], ['ERA']], ['ERA', 'ERC', 'ELA']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 4.0, 1, [['ERA', 'ERC'], [], [], ['ELA'], ['ERA'], ['ERA']], ['ERA', 'ERC', 'ELA']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 5.0, 1, [['ERA'], [], [], [], ['ERA'], ['ERA']], ['ERA']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 1.0, 2, [['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'F4'], ['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'EMGc', 'ELA', 'ELE', 'C3', 'F4', 'C4']], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 1.0, 2, [['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'F4'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGl', 'C3'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'C3', 'C4'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'F3', 'F4'], ['ERA', 'EOGr', 'EOGl', 'F3', 'M2', 'F4']], ['ERA', 'ERC', 'EOGr', 'F3', 'F4', 'ELA', 'ELE', 'EMGl', 'C3']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 1.0, 2, [['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4'], ['ELA', 'ELC', 'ELE', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'O2', 'EMGc', 'ELA', 'ELE', 'C3', 'C4', 'M2']], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 1.0, 2, [['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELC', 'ELE', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc', 'C3', 'ELE', 'ERC', 'C4']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 1.5, 2, [['ERA', 'ERC', 'EOGr', 'EOGl'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr'], ['ELA', 'ELE', 'ERA', 'ERC'], ['ELA', 'ELE', 'ERA', 'C3'], ['ELA', 'ERA'], ['ERA', 'EOGr']], ['ERA', 'ERC', 'EOGr', 'ELA', 'ELE']], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 1.5, 2, [['ERA', 'ERC', 'EOGr'], ['ELA', 'ELE', 'ERA', 'ERC'], ['ELA', 'ELE', 'ERA', 'ERC'], ['ELA', 'ELE', 'ERA'], ['ELA', 'ERA'], ['ERA']], ['ERA', 'ERC', 'ELA', 'ELE']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 1.5, 2, [['ELC', 'ERA', 'EOGr', 'EOGl'], ['ERA', 'ERC', 'EOGr'], ['ERA', 'EOGr', 'C3'], ['ERA', 'ERC', 'C3', 'C4'], ['ERC', 'EOGr'], ['EOGr', 'M2']], ['ERA', 'EOGr', 'ERC', 'C3']], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 1.5, 2, [['ELC', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['EOGl', 'EMGl', 'EMGr', 'O2', 'EMGc'], ['ELC', 'ELE', 'EOGr', 'EOGl', 'EMGr', 'C3', 'C4'], ['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3'], ['ELC', 'M2', 'EMGc'], ['ELC', 'ERC', 'O1', 'M2', 'O2', 'EMGc']], ['ELC', 'EMGl', 'EMGr', 'O2', 'EMGc', 'EOGl', 'EOGr', 'C3', 'M2']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 2.0, 2, [['ERA', 'ERC'], [], ['ERA'], [], ['ERA'], ['ERA']], ['ERA']], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 2.0, 2, [['ERA', 'ERC'], ['ELA', 'ELE', 'ERA', 'ERC'], ['ELA', 'ERA'], ['ELA', 'ERA'], ['ELA', 'ERA'], ['ERA']], ['ERA', 'ERC', 'ELA']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 2.0, 2, [['EMGl'], [], [], [], [], ['ELC']], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 2.5, 2, [['ERA', 'ERC'], ['ELA', 'ERC'], ['ELA', 'ERA'], ['ELA', 'ERA'], ['ERA'], ['ERA']], ['ERA', 'ERC', 'ELA']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 3.0, 2, [['ERA', 'ERC'], [], ['ELA', 'ERA'], ['ELA', 'ERA'], ['ERA'], ['ERA']], ['ERA', 'ELA']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 4.0, 2, [['ERA', 'ERC'], [], [], ['ELA'], ['ERA'], ['ERA']], ['ERA']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 5.0, 2, [['ERA'], [], [], [], ['ERA'], ['ERA']], ['ERA']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 1.0, 3, [['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'F4'], ['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGc', 'ELA', 'ELE', 'C3', 'C4']], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 1.0, 3, [['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'F4'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGl', 'C3'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'C3', 'C4'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'F3', 'F4'], ['ERA', 'EOGr', 'EOGl', 'F3', 'M2', 'F4']], ['ERA', 'ERC', 'EOGr', 'ELA', 'ELE', 'C3']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 1.0, 3, [['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4'], ['ELA', 'ELC', 'ELE', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'ELE', 'C3', 'C4']], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 1.0, 3, [['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELC', 'ELE', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc', 'C3', 'ERC']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 1.5, 3, [['ERA', 'ERC', 'EOGr', 'EOGl'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr'], ['ELA', 'ELE', 'ERA', 'ERC'], ['ELA', 'ELE', 'ERA', 'C3'], ['ELA', 'ERA'], ['ERA', 'EOGr']], ['ERA', 'ERC', 'ELA', 'ELE']], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 1.5, 3, [['ERA', 'ERC', 'EOGr'], ['ELA', 'ELE', 'ERA', 'ERC'], ['ELA', 'ELE', 'ERA', 'ERC'], ['ELA', 'ELE', 'ERA'], ['ELA', 'ERA'], ['ERA']], ['ERA', 'ERC', 'ELA', 'ELE']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 1.5, 3, [['ELC', 'ERA', 'EOGr', 'EOGl'], ['ERA', 'ERC', 'EOGr'], ['ERA', 'EOGr', 'C3'], ['ERA', 'ERC', 'C3', 'C4'], ['ERC', 'EOGr'], ['EOGr', 'M2']], ['ERA', 'EOGr']], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 1.5, 3, [['ELC', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['EOGl', 'EMGl', 'EMGr', 'O2', 'EMGc'], ['ELC', 'ELE', 'EOGr', 'EOGl', 'EMGr', 'C3', 'C4'], ['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3'], ['ELC', 'M2', 'EMGc'], ['ELC', 'ERC', 'O1', 'M2', 'O2', 'EMGc']], ['ELC', 'EMGr', 'EOGl']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 2.0, 3, [['ERA', 'ERC'], [], ['ERA'], [], ['ERA'], ['ERA']], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 2.0, 3, [['ERA', 'ERC'], ['ELA', 'ELE', 'ERA', 'ERC'], ['ELA', 'ERA'], ['ELA', 'ERA'], ['ELA', 'ERA'], ['ERA']], ['ERA', 'ELA']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 2.0, 3, [['EMGl'], [], [], [], [], ['ELC']], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 2.5, 3, [['ERA', 'ERC'], ['ELA', 'ERC'], ['ELA', 'ERA'], ['ELA', 'ERA'], ['ERA'], ['ERA']], ['ERA', 'ELA']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 3.0, 3, [['ERA', 'ERC'], [], ['ELA', 'ERA'], ['ELA', 'ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 4.0, 3, [['ERA', 'ERC'], [], [], ['ELA'], ['ERA'], ['ERA']], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 5.0, 3, [['ERA'], [], [], [], ['ERA'], ['ERA']], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 1.0, 4, [['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'F4'], ['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGc', 'ELA', 'ELE', 'C3']], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 1.0, 4, [['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'F4'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGl', 'C3'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'C3', 'C4'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'F3', 'F4'], ['ERA', 'EOGr', 'EOGl', 'F3', 'M2', 'F4']], ['ERA', 'ERC', 'EOGr', 'ELA', 'ELE']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 1.0, 4, [['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4'], ['ELA', 'ELC', 'ELE', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'ELE', 'C3']], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 1.0, 4, [['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELC', 'ELE', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'ERC']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 1.5, 4, [['ERA', 'ERC', 'EOGr', 'EOGl'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr'], ['ELA', 'ELE', 'ERA', 'ERC'], ['ELA', 'ELE', 'ERA', 'C3'], ['ELA', 'ERA'], ['ERA', 'EOGr']], ['ERA', 'ELA']], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 1.5, 4, [['ERA', 'ERC', 'EOGr'], ['ELA', 'ELE', 'ERA', 'ERC'], ['ELA', 'ELE', 'ERA', 'ERC'], ['ELA', 'ELE', 'ERA'], ['ELA', 'ERA'], ['ERA']], ['ERA', 'ELA']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 1.5, 4, [['ELC', 'ERA', 'EOGr', 'EOGl'], ['ERA', 'ERC', 'EOGr'], ['ERA', 'EOGr', 'C3'], ['ERA', 'ERC', 'C3', 'C4'], ['ERC', 'EOGr'], ['EOGr', 'M2']], ['ERA']], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 1.5, 4, [['ELC', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['EOGl', 'EMGl', 'EMGr', 'O2', 'EMGc'], ['ELC', 'ELE', 'EOGr', 'EOGl', 'EMGr', 'C3', 'C4'], ['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3'], ['ELC', 'M2', 'EMGc'], ['ELC', 'ERC', 'O1', 'M2', 'O2', 'EMGc']], ['ELC', 'EMGr']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 2.0, 4, [['ERA', 'ERC'], [], ['ERA'], [], ['ERA'], ['ERA']], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 2.0, 4, [['ERA', 'ERC'], ['ELA', 'ELE', 'ERA', 'ERC'], ['ELA', 'ERA'], ['ELA', 'ERA'], ['ELA', 'ERA'], ['ERA']], ['ERA', 'ELA']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 2.0, 4, [['EMGl'], [], [], [], [], ['ELC']], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 2.5, 4, [['ERA', 'ERC'], ['ELA', 'ERC'], ['ELA', 'ERA'], ['ELA', 'ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 3.0, 4, [['ERA', 'ERC'], [], ['ELA', 'ERA'], ['ELA', 'ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 4.0, 4, [['ERA', 'ERC'], [], [], ['ELA'], ['ERA'], ['ERA']], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 5.0, 4, [['ERA'], [], [], [], ['ERA'], ['ERA']], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 1.0, 5, [['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'F4'], ['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['ERA', 'ERC', 'EOGr']], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 1.0, 5, [['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'F4'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGl', 'C3'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'C3', 'C4'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'F3', 'F4'], ['ERA', 'EOGr', 'EOGl', 'F3', 'M2', 'F4']], ['ERA', 'ERC', 'EOGr']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 1.0, 5, [['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4'], ['ELA', 'ELC', 'ELE', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr']], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 1.0, 5, [['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELC', 'ELE', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 1.5, 5, [['ERA', 'ERC', 'EOGr', 'EOGl'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr'], ['ELA', 'ELE', 'ERA', 'ERC'], ['ELA', 'ELE', 'ERA', 'C3'], ['ELA', 'ERA'], ['ERA', 'EOGr']], ['ERA']], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 1.5, 5, [['ERA', 'ERC', 'EOGr'], ['ELA', 'ELE', 'ERA', 'ERC'], ['ELA', 'ELE', 'ERA', 'ERC'], ['ELA', 'ELE', 'ERA'], ['ELA', 'ERA'], ['ERA']], ['ERA']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 1.5, 5, [['ELC', 'ERA', 'EOGr', 'EOGl'], ['ERA', 'ERC', 'EOGr'], ['ERA', 'EOGr', 'C3'], ['ERA', 'ERC', 'C3', 'C4'], ['ERC', 'EOGr'], ['EOGr', 'M2']], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 1.5, 5, [['ELC', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['EOGl', 'EMGl', 'EMGr', 'O2', 'EMGc'], ['ELC', 'ELE', 'EOGr', 'EOGl', 'EMGr', 'C3', 'C4'], ['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3'], ['ELC', 'M2', 'EMGc'], ['ELC', 'ERC', 'O1', 'M2', 'O2', 'EMGc']], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 2.0, 5, [['ERA', 'ERC'], [], ['ERA'], [], ['ERA'], ['ERA']], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 2.0, 5, [['ERA', 'ERC'], ['ELA', 'ELE', 'ERA', 'ERC'], ['ELA', 'ERA'], ['ELA', 'ERA'], ['ELA', 'ERA'], ['ERA']], ['ERA']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 2.0, 5, [['EMGl'], [], [], [], [], ['ELC']], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 2.5, 5, [['ERA', 'ERC'], ['ELA', 'ERC'], ['ELA', 'ERA'], ['ELA', 'ERA'], ['ERA'], ['ERA']], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 3.0, 5, [['ERA', 'ERC'], [], ['ELA', 'ERA'], ['ELA', 'ERA'], ['ERA'], ['ERA']], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 4.0, 5, [['ERA', 'ERC'], [], [], ['ELA'], ['ERA'], ['ERA']], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 5.0, 5, [['ERA'], [], [], [], ['ERA'], ['ERA']], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 1.0, 6, [['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'F4'], ['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['ERA', 'ERC', 'EOGr']], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 1.0, 6, [['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'F4'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'EMGl', 'C3'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'C3', 'C4'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'F3', 'F4'], ['ERA', 'EOGr', 'EOGl', 'F3', 'M2', 'F4']], ['ERA', 'EOGr']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 1.0, 6, [['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4'], ['ELA', 'ELC', 'ELE', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc']], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr']], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 1.0, 6, [['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELC', 'ELE', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 1.5, 6, [['ERA', 'ERC', 'EOGr', 'EOGl'], ['ELA', 'ELE', 'ERA', 'ERC', 'EOGr'], ['ELA', 'ELE', 'ERA', 'ERC'], ['ELA', 'ELE', 'ERA', 'C3'], ['ELA', 'ERA'], ['ERA', 'EOGr']], ['ERA']], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 1.5, 6, [['ERA', 'ERC', 'EOGr'], ['ELA', 'ELE', 'ERA', 'ERC'], ['ELA', 'ELE', 'ERA', 'ERC'], ['ELA', 'ELE', 'ERA'], ['ELA', 'ERA'], ['ERA']], ['ERA']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 1.5, 6, [['ELC', 'ERA', 'EOGr', 'EOGl'], ['ERA', 'ERC', 'EOGr'], ['ERA', 'EOGr', 'C3'], ['ERA', 'ERC', 'C3', 'C4'], ['ERC', 'EOGr'], ['EOGr', 'M2']], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 1.5, 6, [['ELC', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['EOGl', 'EMGl', 'EMGr', 'O2', 'EMGc'], ['ELC', 'ELE', 'EOGr', 'EOGl', 'EMGr', 'C3', 'C4'], ['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3'], ['ELC', 'M2', 'EMGc'], ['ELC', 'ERC', 'O1', 'M2', 'O2', 'EMGc']], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 2.0, 6, [['ERA', 'ERC'], [], ['ERA'], [], ['ERA'], ['ERA']], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 2.0, 6, [['ERA', 'ERC'], ['ELA', 'ELE', 'ERA', 'ERC'], ['ELA', 'ERA'], ['ELA', 'ERA'], ['ELA', 'ERA'], ['ERA']], ['ERA']], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 2.0, 6, [['EMGl'], [], [], [], [], ['ELC']], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 2.5, 6, [['ERA', 'ERC'], ['ELA', 'ERC'], ['ELA', 'ERA'], ['ELA', 'ERA'], ['ERA'], ['ERA']], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 3.0, 6, [['ERA', 'ERC'], [], ['ELA', 'ERA'], ['ELA', 'ERA'], ['ERA'], ['ERA']], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 4.0, 6, [['ERA', 'ERC'], [], [], ['ELA'], ['ERA'], ['ERA']], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'mm', 5.0, 6, [['ERA'], [], [], [], ['ERA'], ['ERA']], []], ['S_03/night_1\\EEG_raw', 280, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 280, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc']], ['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'ERT ', 'C3']], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 1.0, 1, [['ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'ERC', 'EMGr', 'EMGc'], ['ERA', 'ERC']], ['ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'ELC', 'EOGr', 'ELE', 'ERE']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc']], ['ELA', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'ERC', 'ERT ', 'C3']], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 1.0, 1, [['ELA', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'EMGc'], ['ELC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'EMGc'], ['ELC', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc']], ['ELA', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'EMGc', 'O1', 'ERT ', 'C3']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 1.5, 1, [['ERA', 'ERC', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'ERC', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'EMGl', 'EMGr'], ['ELC', 'ELE', 'ERA', 'ERC', 'EMGc'], ['ERC', 'EOGl', 'EMGc'], ['ERC', 'EMGr', 'EMGc']], ['ERA', 'ERC', 'EMGl', 'EMGr', 'EMGc', 'ELC', 'ELE', 'EOGl']], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 1.5, 1, [['ERA', 'ERC', 'EMGc'], ['ERA', 'ERC', 'EMGr', 'EMGc'], ['ERA', 'ERC'], ['ELC', 'ERC', 'EMGl', 'EMGc'], ['ERA', 'ERC', 'EMGc'], ['ERA', 'ERC']], ['ERA', 'ERC', 'EMGc', 'EMGr', 'ELC', 'EMGl']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 1.5, 1, [['ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ELE', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'C3', 'EMGc']], ['ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'EMGc', 'ELC', 'EOGl', 'ELE', 'C3']], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 1.5, 1, [['ELA', 'ELC', 'EOGr', 'EMGl', 'EMGr'], ['ELC', 'ELE', 'EMGl', 'EMGr'], ['ELC', 'ERE', 'ERI', 'EMGl', 'EMGr', 'EMGc'], ['ELE', 'ERA', 'EOGl', 'EMGr'], ['EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'C3', 'EMGc']], ['ELA', 'ELC', 'EOGr', 'EMGl', 'EMGr', 'ELE', 'ERE', 'ERI', 'EMGc', 'ERA', 'EOGl', 'C3']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 2.0, 1, [['ERC'], ['ERA', 'ERC'], ['ERC'], ['ERC'], ['ERC'], ['ERC']], ['ERC', 'ERA']], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 2.0, 1, [['ERA', 'ERC', 'EMGc'], ['ERA', 'ERC'], ['ERA', 'ERC'], ['ELC', 'ERC', 'EMGc'], ['ERA', 'ERC'], ['ERA', 'ERC']], ['ERA', 'ERC', 'EMGc', 'ELC']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 2.0, 1, [[], ['EMGr'], [], [], [], []], ['EMGr']], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], ['ERC'], []], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 2.5, 1, [['ERC', 'EMGc'], ['ERA', 'ERC'], ['ERA', 'ERC'], ['ERC', 'EMGc'], ['ERA', 'ERC'], ['ERC']], ['ERC', 'EMGc', 'ERA']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 3.0, 1, [['ERC', 'EMGc'], ['ERC'], ['ERA', 'ERC'], ['ERC'], ['ERA', 'ERC'], ['ERC']], ['ERC', 'EMGc', 'ERA']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 4.0, 1, [['ERC'], ['ERC'], ['ERC'], ['ERC'], ['ERC'], ['ERC']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 5.0, 1, [['ERC'], ['ERC'], ['ERC'], ['ERC'], ['ERC'], ['ERC']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc']], ['ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'ERT ']], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 1.0, 2, [['ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'ERC', 'EMGr', 'EMGc'], ['ERA', 'ERC']], ['ERA', 'ERC', 'EMGl', 'EMGr', 'EMGc']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc']], ['ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'ERT ', 'C3']], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 1.0, 2, [['ELA', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'EMGc'], ['ELC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'EMGc'], ['ELC', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc']], ['ELA', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'EMGc', 'O1', 'ERT ', 'C3']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 1.5, 2, [['ERA', 'ERC', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'ERC', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'EMGl', 'EMGr'], ['ELC', 'ELE', 'ERA', 'ERC', 'EMGc'], ['ERC', 'EOGl', 'EMGc'], ['ERC', 'EMGr', 'EMGc']], ['ERA', 'ERC', 'EMGl', 'EMGr', 'EMGc']], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 1.5, 2, [['ERA', 'ERC', 'EMGc'], ['ERA', 'ERC', 'EMGr', 'EMGc'], ['ERA', 'ERC'], ['ELC', 'ERC', 'EMGl', 'EMGc'], ['ERA', 'ERC', 'EMGc'], ['ERA', 'ERC']], ['ERA', 'ERC', 'EMGc']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 1.5, 2, [['ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ELE', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'C3', 'EMGc']], ['ERA', 'EMGl', 'EMGr', 'EMGc', 'ELC']], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 1.5, 2, [['ELA', 'ELC', 'EOGr', 'EMGl', 'EMGr'], ['ELC', 'ELE', 'EMGl', 'EMGr'], ['ELC', 'ERE', 'ERI', 'EMGl', 'EMGr', 'EMGc'], ['ELE', 'ERA', 'EOGl', 'EMGr'], ['EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'C3', 'EMGc']], ['ELC', 'EMGl', 'EMGr', 'EMGc']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 2.0, 2, [['ERC'], ['ERA', 'ERC'], ['ERC'], ['ERC'], ['ERC'], ['ERC']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 2.0, 2, [['ERA', 'ERC', 'EMGc'], ['ERA', 'ERC'], ['ERA', 'ERC'], ['ELC', 'ERC', 'EMGc'], ['ERA', 'ERC'], ['ERA', 'ERC']], ['ERA', 'ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 2.0, 2, [[], ['EMGr'], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], ['ERC'], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 2.5, 2, [['ERC', 'EMGc'], ['ERA', 'ERC'], ['ERA', 'ERC'], ['ERC', 'EMGc'], ['ERA', 'ERC'], ['ERC']], ['ERC', 'ERA']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 3.0, 2, [['ERC', 'EMGc'], ['ERC'], ['ERA', 'ERC'], ['ERC'], ['ERA', 'ERC'], ['ERC']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 4.0, 2, [['ERC'], ['ERC'], ['ERC'], ['ERC'], ['ERC'], ['ERC']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 5.0, 2, [['ERC'], ['ERC'], ['ERC'], ['ERC'], ['ERC'], ['ERC']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc']], ['ELC', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'ERT ']], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 1.0, 3, [['ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'ERC', 'EMGr', 'EMGc'], ['ERA', 'ERC']], ['ERA', 'ERC', 'EMGc']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc']], ['ELC', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'ERT ']], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 1.0, 3, [['ELA', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'EMGc'], ['ELC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'EMGc'], ['ELC', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc']], ['ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'O1', 'ERT ']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 1.5, 3, [['ERA', 'ERC', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'ERC', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'EMGl', 'EMGr'], ['ELC', 'ELE', 'ERA', 'ERC', 'EMGc'], ['ERC', 'EOGl', 'EMGc'], ['ERC', 'EMGr', 'EMGc']], ['ERC', 'EMGl', 'EMGr', 'EMGc']], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 1.5, 3, [['ERA', 'ERC', 'EMGc'], ['ERA', 'ERC', 'EMGr', 'EMGc'], ['ERA', 'ERC'], ['ELC', 'ERC', 'EMGl', 'EMGc'], ['ERA', 'ERC', 'EMGc'], ['ERA', 'ERC']], ['ERA', 'ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 1.5, 3, [['ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ELE', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'C3', 'EMGc']], ['ERA', 'EMGl', 'EMGr', 'EMGc', 'ELC']], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 1.5, 3, [['ELA', 'ELC', 'EOGr', 'EMGl', 'EMGr'], ['ELC', 'ELE', 'EMGl', 'EMGr'], ['ELC', 'ERE', 'ERI', 'EMGl', 'EMGr', 'EMGc'], ['ELE', 'ERA', 'EOGl', 'EMGr'], ['EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'C3', 'EMGc']], ['ELC', 'EMGl', 'EMGr']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 2.0, 3, [['ERC'], ['ERA', 'ERC'], ['ERC'], ['ERC'], ['ERC'], ['ERC']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 2.0, 3, [['ERA', 'ERC', 'EMGc'], ['ERA', 'ERC'], ['ERA', 'ERC'], ['ELC', 'ERC', 'EMGc'], ['ERA', 'ERC'], ['ERA', 'ERC']], ['ERA', 'ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 2.0, 3, [[], ['EMGr'], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], ['ERC'], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 2.5, 3, [['ERC', 'EMGc'], ['ERA', 'ERC'], ['ERA', 'ERC'], ['ERC', 'EMGc'], ['ERA', 'ERC'], ['ERC']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 3.0, 3, [['ERC', 'EMGc'], ['ERC'], ['ERA', 'ERC'], ['ERC'], ['ERA', 'ERC'], ['ERC']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 4.0, 3, [['ERC'], ['ERC'], ['ERC'], ['ERC'], ['ERC'], ['ERC']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 5.0, 3, [['ERC'], ['ERC'], ['ERC'], ['ERC'], ['ERC'], ['ERC']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc']], ['ELC', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'ERT ']], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 1.0, 4, [['ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'ERC', 'EMGr', 'EMGc'], ['ERA', 'ERC']], ['ERA', 'ERC', 'EMGc']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc']], ['ELC', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'ERT ']], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 1.0, 4, [['ELA', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'EMGc'], ['ELC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'EMGc'], ['ELC', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc']], ['ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'O1', 'ERT ']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 1.5, 4, [['ERA', 'ERC', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'ERC', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'EMGl', 'EMGr'], ['ELC', 'ELE', 'ERA', 'ERC', 'EMGc'], ['ERC', 'EOGl', 'EMGc'], ['ERC', 'EMGr', 'EMGc']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 1.5, 4, [['ERA', 'ERC', 'EMGc'], ['ERA', 'ERC', 'EMGr', 'EMGc'], ['ERA', 'ERC'], ['ELC', 'ERC', 'EMGl', 'EMGc'], ['ERA', 'ERC', 'EMGc'], ['ERA', 'ERC']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 1.5, 4, [['ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ELE', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'C3', 'EMGc']], ['EMGl', 'EMGr', 'EMGc']], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 1.5, 4, [['ELA', 'ELC', 'EOGr', 'EMGl', 'EMGr'], ['ELC', 'ELE', 'EMGl', 'EMGr'], ['ELC', 'ERE', 'ERI', 'EMGl', 'EMGr', 'EMGc'], ['ELE', 'ERA', 'EOGl', 'EMGr'], ['EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'C3', 'EMGc']], ['EMGr']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 2.0, 4, [['ERC'], ['ERA', 'ERC'], ['ERC'], ['ERC'], ['ERC'], ['ERC']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 2.0, 4, [['ERA', 'ERC', 'EMGc'], ['ERA', 'ERC'], ['ERA', 'ERC'], ['ELC', 'ERC', 'EMGc'], ['ERA', 'ERC'], ['ERA', 'ERC']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 2.0, 4, [[], ['EMGr'], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], ['ERC'], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 2.5, 4, [['ERC', 'EMGc'], ['ERA', 'ERC'], ['ERA', 'ERC'], ['ERC', 'EMGc'], ['ERA', 'ERC'], ['ERC']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 3.0, 4, [['ERC', 'EMGc'], ['ERC'], ['ERA', 'ERC'], ['ERC'], ['ERA', 'ERC'], ['ERC']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 4.0, 4, [['ERC'], ['ERC'], ['ERC'], ['ERC'], ['ERC'], ['ERC']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 5.0, 4, [['ERC'], ['ERC'], ['ERC'], ['ERC'], ['ERC'], ['ERC']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc']], ['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 1.0, 5, [['ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'ERC', 'EMGr', 'EMGc'], ['ERA', 'ERC']], ['ERA', 'ERC', 'EMGc']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc']], ['ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 1.0, 5, [['ELA', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'EMGc'], ['ELC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'EMGc'], ['ELC', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc']], ['ELC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 1.5, 5, [['ERA', 'ERC', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'ERC', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'EMGl', 'EMGr'], ['ELC', 'ELE', 'ERA', 'ERC', 'EMGc'], ['ERC', 'EOGl', 'EMGc'], ['ERC', 'EMGr', 'EMGc']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 1.5, 5, [['ERA', 'ERC', 'EMGc'], ['ERA', 'ERC', 'EMGr', 'EMGc'], ['ERA', 'ERC'], ['ELC', 'ERC', 'EMGl', 'EMGc'], ['ERA', 'ERC', 'EMGc'], ['ERA', 'ERC']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 1.5, 5, [['ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ELE', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'C3', 'EMGc']], ['EMGl', 'EMGr', 'EMGc']], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 1.5, 5, [['ELA', 'ELC', 'EOGr', 'EMGl', 'EMGr'], ['ELC', 'ELE', 'EMGl', 'EMGr'], ['ELC', 'ERE', 'ERI', 'EMGl', 'EMGr', 'EMGc'], ['ELE', 'ERA', 'EOGl', 'EMGr'], ['EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'C3', 'EMGc']], ['EMGr']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 2.0, 5, [['ERC'], ['ERA', 'ERC'], ['ERC'], ['ERC'], ['ERC'], ['ERC']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 2.0, 5, [['ERA', 'ERC', 'EMGc'], ['ERA', 'ERC'], ['ERA', 'ERC'], ['ELC', 'ERC', 'EMGc'], ['ERA', 'ERC'], ['ERA', 'ERC']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 2.0, 5, [[], ['EMGr'], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], ['ERC'], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 2.5, 5, [['ERC', 'EMGc'], ['ERA', 'ERC'], ['ERA', 'ERC'], ['ERC', 'EMGc'], ['ERA', 'ERC'], ['ERC']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 3.0, 5, [['ERC', 'EMGc'], ['ERC'], ['ERA', 'ERC'], ['ERC'], ['ERA', 'ERC'], ['ERC']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 4.0, 5, [['ERC'], ['ERC'], ['ERC'], ['ERC'], ['ERC'], ['ERC']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 5.0, 5, [['ERC'], ['ERC'], ['ERC'], ['ERC'], ['ERC'], ['ERC']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc']], ['ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 1.0, 6, [['ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERC', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'ERC', 'EMGr', 'EMGc'], ['ERA', 'ERC']], ['ERA', 'ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELC', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc']], ['ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 1.0, 6, [['ELA', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'EMGc'], ['ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'EMGc'], ['ELA', 'ELC', 'ELE', 'ERA', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'EMGc'], ['ELC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'EMGc'], ['ELC', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc']], ['ELC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 1.5, 6, [['ERA', 'ERC', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'ERC', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'EMGl', 'EMGr'], ['ELC', 'ELE', 'ERA', 'ERC', 'EMGc'], ['ERC', 'EOGl', 'EMGc'], ['ERC', 'EMGr', 'EMGc']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 1.5, 6, [['ERA', 'ERC', 'EMGc'], ['ERA', 'ERC', 'EMGr', 'EMGc'], ['ERA', 'ERC'], ['ELC', 'ERC', 'EMGl', 'EMGc'], ['ERA', 'ERC', 'EMGc'], ['ERA', 'ERC']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 1.5, 6, [['ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ELE', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'C3', 'EMGc']], ['EMGr', 'EMGc']], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 1.5, 6, [['ELA', 'ELC', 'EOGr', 'EMGl', 'EMGr'], ['ELC', 'ELE', 'EMGl', 'EMGr'], ['ELC', 'ERE', 'ERI', 'EMGl', 'EMGr', 'EMGc'], ['ELE', 'ERA', 'EOGl', 'EMGr'], ['EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'C3', 'EMGc']], ['EMGr']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 2.0, 6, [['ERC'], ['ERA', 'ERC'], ['ERC'], ['ERC'], ['ERC'], ['ERC']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 2.0, 6, [['ERA', 'ERC', 'EMGc'], ['ERA', 'ERC'], ['ERA', 'ERC'], ['ELC', 'ERC', 'EMGc'], ['ERA', 'ERC'], ['ERA', 'ERC']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 2.0, 6, [[], ['EMGr'], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], ['ERC'], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 2.5, 6, [['ERC', 'EMGc'], ['ERA', 'ERC'], ['ERA', 'ERC'], ['ERC', 'EMGc'], ['ERA', 'ERC'], ['ERC']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 3.0, 6, [['ERC', 'EMGc'], ['ERC'], ['ERA', 'ERC'], ['ERC'], ['ERA', 'ERC'], ['ERC']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 4.0, 6, [['ERC'], ['ERC'], ['ERC'], ['ERC'], ['ERC'], ['ERC']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'mm', 5.0, 6, [['ERC'], ['ERC'], ['ERC'], ['ERC'], ['ERC'], ['ERC']], ['ERC']], ['S_03/night_1\\EEG_raw', 660, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_03/night_1\\EEG_raw', 660, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'O2'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc']], ['ELA', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc', 'ERA', 'ERT ', 'F3', 'F4', 'O1', 'M2', 'ERE', 'ERI']], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 1.0, 1, [['ELA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'F3'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['ELA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'ELI', 'ERA', 'ERT ', 'F3', 'M2', 'ERE', 'ERI']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc']], ['ELA', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc', 'ERA', 'ERT ', 'F3', 'F4', 'ERE', 'ERI']], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 1.0, 1, [['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'M1', 'F3', 'M2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'EMGc']], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc', 'EOGl', 'EMGl', 'F3', 'F4', 'ERT ', 'ERE', 'ERI', 'ELT']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 1.5, 1, [['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'ERT '], ['ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr'], ['ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'ERT ', 'ERA', 'ELA', 'ERE']], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 1.5, 1, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERC', 'ERT ', 'EOGr'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl'], ['ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr']], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'ELI', 'ERC', 'ERT ', 'ERA', 'ELA', 'ERE', 'ERI']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 1.5, 1, [['ELA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'EOGl'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr'], ['ERC', 'EOGr', 'EOGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M2'], ['ELA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGr', 'EMGc']], ['ELA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'ERA', 'M2', 'ERE', 'ERI']], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 1.5, 1, [['ELA', 'ELI', 'ERC', 'M1', 'O1', 'M2', 'O2'], ['EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'O2'], ['ELA', 'EOGr', 'EMGr', 'M2'], ['ELA', 'ERT ', 'EOGr', 'M1', 'M2'], ['ERA', 'M1', 'M2'], ['ELA', 'ERC', 'EOGr', 'M1', 'M2', 'EMGc']], ['ELA', 'ELI', 'ERC', 'M1', 'O1', 'M2', 'O2', 'EOGl', 'EMGl', 'EMGr', 'F3', 'EOGr', 'ERT ', 'ERA', 'EMGc']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 2.0, 1, [['EOGl', 'EMGl'], ['ERC', 'ERT '], ['ERA', 'ERC'], [], ['EMGl'], ['EOGl', 'EMGl']], ['EOGl', 'EMGl', 'ERC', 'ERT ', 'ERA']], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 2.0, 1, [['EOGl', 'EMGl', 'EMGr'], ['ERC', 'ERT '], ['ERA', 'ERC', 'EOGl', 'EMGl'], ['ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'EOGl', 'EMGl', 'EMGr'], ['EOGl', 'EMGl', 'EMGr']], ['EOGl', 'EMGl', 'EMGr', 'ERC', 'ERT ', 'ERA', 'EMGc']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 2.0, 1, [['O2'], ['EOGl'], [], ['M2'], [], []], ['O2', 'EOGl', 'M2']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 2.5, 1, [['EOGl', 'EMGl'], ['ERC', 'ERT '], ['ERA', 'ERC', 'EOGl', 'EMGl'], ['EOGl', 'EMGl', 'EMGr'], ['ERC', 'EMGl', 'EMGr'], ['EOGl', 'EMGl']], ['EOGl', 'EMGl', 'ERC', 'ERT ', 'ERA', 'EMGr']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 3.0, 1, [['EOGl', 'EMGl'], ['ERC', 'ERT '], ['ERA', 'ERC'], ['EOGl', 'EMGl'], ['EMGl'], ['EMGl']], ['EOGl', 'EMGl', 'ERC', 'ERT ', 'ERA']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 4.0, 1, [['EOGl', 'EMGl'], ['ERC'], [], [], [], ['EMGl']], ['EOGl', 'EMGl', 'ERC']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'O2'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc']], ['ELA', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc', 'ERA', 'ERT ', 'M2']], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 1.0, 2, [['ELA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'F3'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['ELA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'ERA', 'M2']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc']], ['ELA', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc', 'ERA', 'ERT ', 'ERE']], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 1.0, 2, [['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'M1', 'F3', 'M2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'EMGc']], ['ELA', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc', 'EOGl', 'EMGl', 'F3', 'ERT ', 'ERE', 'ERI']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 1.5, 2, [['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'ERT '], ['ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr'], ['ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 1.5, 2, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERC', 'ERT ', 'EOGr'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl'], ['ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr']], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'ERC']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 1.5, 2, [['ELA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'EOGl'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr'], ['ERC', 'EOGr', 'EOGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M2'], ['ELA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGr', 'EMGc']], ['ELA', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'ERA', 'M2']], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 1.5, 2, [['ELA', 'ELI', 'ERC', 'M1', 'O1', 'M2', 'O2'], ['EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'O2'], ['ELA', 'EOGr', 'EMGr', 'M2'], ['ELA', 'ERT ', 'EOGr', 'M1', 'M2'], ['ERA', 'M1', 'M2'], ['ELA', 'ERC', 'EOGr', 'M1', 'M2', 'EMGc']], ['ELA', 'M1', 'M2', 'O2', 'EMGr', 'EOGr']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 2.0, 2, [['EOGl', 'EMGl'], ['ERC', 'ERT '], ['ERA', 'ERC'], [], ['EMGl'], ['EOGl', 'EMGl']], ['EMGl', 'ERC']], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 2.0, 2, [['EOGl', 'EMGl', 'EMGr'], ['ERC', 'ERT '], ['ERA', 'ERC', 'EOGl', 'EMGl'], ['ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'EOGl', 'EMGl', 'EMGr'], ['EOGl', 'EMGl', 'EMGr']], ['EOGl', 'EMGl', 'EMGr', 'ERC']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 2.0, 2, [['O2'], ['EOGl'], [], ['M2'], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 2.5, 2, [['EOGl', 'EMGl'], ['ERC', 'ERT '], ['ERA', 'ERC', 'EOGl', 'EMGl'], ['EOGl', 'EMGl', 'EMGr'], ['ERC', 'EMGl', 'EMGr'], ['EOGl', 'EMGl']], ['EOGl', 'EMGl', 'ERC', 'EMGr']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 3.0, 2, [['EOGl', 'EMGl'], ['ERC', 'ERT '], ['ERA', 'ERC'], ['EOGl', 'EMGl'], ['EMGl'], ['EMGl']], ['EMGl', 'ERC']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 4.0, 2, [['EOGl', 'EMGl'], ['ERC'], [], [], [], ['EMGl']], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'O2'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc']], ['ELA', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O2', 'EMGc', 'M2']], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 1.0, 3, [['ELA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'F3'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['ELA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc']], ['ELA', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc']], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 1.0, 3, [['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'M1', 'F3', 'M2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc', 'EOGl', 'ERT ', 'ERE', 'ERI']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 1.5, 3, [['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'ERT '], ['ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr'], ['ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 1.5, 3, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERC', 'ERT ', 'EOGr'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl'], ['ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr']], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'ERC']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 1.5, 3, [['ELA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'EOGl'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr'], ['ERC', 'EOGr', 'EOGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M2'], ['ELA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGr', 'EMGc']], ['ERC', 'EOGl', 'EMGr']], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 1.5, 3, [['ELA', 'ELI', 'ERC', 'M1', 'O1', 'M2', 'O2'], ['EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'O2'], ['ELA', 'EOGr', 'EMGr', 'M2'], ['ELA', 'ERT ', 'EOGr', 'M1', 'M2'], ['ERA', 'M1', 'M2'], ['ELA', 'ERC', 'EOGr', 'M1', 'M2', 'EMGc']], ['M1', 'M2']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 2.0, 3, [['EOGl', 'EMGl'], ['ERC', 'ERT '], ['ERA', 'ERC'], [], ['EMGl'], ['EOGl', 'EMGl']], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 2.0, 3, [['EOGl', 'EMGl', 'EMGr'], ['ERC', 'ERT '], ['ERA', 'ERC', 'EOGl', 'EMGl'], ['ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'EOGl', 'EMGl', 'EMGr'], ['EOGl', 'EMGl', 'EMGr']], ['EOGl', 'EMGl', 'EMGr', 'ERC']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 2.0, 3, [['O2'], ['EOGl'], [], ['M2'], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 2.5, 3, [['EOGl', 'EMGl'], ['ERC', 'ERT '], ['ERA', 'ERC', 'EOGl', 'EMGl'], ['EOGl', 'EMGl', 'EMGr'], ['ERC', 'EMGl', 'EMGr'], ['EOGl', 'EMGl']], ['EMGl']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 3.0, 3, [['EOGl', 'EMGl'], ['ERC', 'ERT '], ['ERA', 'ERC'], ['EOGl', 'EMGl'], ['EMGl'], ['EMGl']], ['EMGl']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 4.0, 3, [['EOGl', 'EMGl'], ['ERC'], [], [], [], ['EMGl']], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'O2'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc']], ['ELA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'M2']], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 1.0, 4, [['ELA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'F3'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc']], ['ELA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc']], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 1.0, 4, [['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'M1', 'F3', 'M2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M2', 'EMGc', 'EOGl', 'ERT ', 'ERE']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 1.5, 4, [['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'ERT '], ['ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr'], ['ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['ERC', 'EOGl', 'EMGl', 'EMGr']], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 1.5, 4, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERC', 'ERT ', 'EOGr'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl'], ['ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr']], ['EOGl', 'EMGl', 'ERC']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 1.5, 4, [['ELA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'EOGl'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr'], ['ERC', 'EOGr', 'EOGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M2'], ['ELA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGr', 'EMGc']], ['ERC', 'EOGl', 'EMGr']], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 1.5, 4, [['ELA', 'ELI', 'ERC', 'M1', 'O1', 'M2', 'O2'], ['EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'O2'], ['ELA', 'EOGr', 'EMGr', 'M2'], ['ELA', 'ERT ', 'EOGr', 'M1', 'M2'], ['ERA', 'M1', 'M2'], ['ELA', 'ERC', 'EOGr', 'M1', 'M2', 'EMGc']], ['M2']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 2.0, 4, [['EOGl', 'EMGl'], ['ERC', 'ERT '], ['ERA', 'ERC'], [], ['EMGl'], ['EOGl', 'EMGl']], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 2.0, 4, [['EOGl', 'EMGl', 'EMGr'], ['ERC', 'ERT '], ['ERA', 'ERC', 'EOGl', 'EMGl'], ['ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'EOGl', 'EMGl', 'EMGr'], ['EOGl', 'EMGl', 'EMGr']], ['EOGl', 'EMGl', 'ERC']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 2.0, 4, [['O2'], ['EOGl'], [], ['M2'], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 2.5, 4, [['EOGl', 'EMGl'], ['ERC', 'ERT '], ['ERA', 'ERC', 'EOGl', 'EMGl'], ['EOGl', 'EMGl', 'EMGr'], ['ERC', 'EMGl', 'EMGr'], ['EOGl', 'EMGl']], ['EMGl']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 3.0, 4, [['EOGl', 'EMGl'], ['ERC', 'ERT '], ['ERA', 'ERC'], ['EOGl', 'EMGl'], ['EMGl'], ['EMGl']], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 4.0, 4, [['EOGl', 'EMGl'], ['ERC'], [], [], [], ['EMGl']], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'O2'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc']], ['ELA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr']], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 1.0, 5, [['ELA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'F3'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['ERC', 'EOGr', 'EOGl']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc']], ['ELA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2']], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 1.0, 5, [['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'M1', 'F3', 'M2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'EMGc']], ['ELA', 'ERA', 'EOGr', 'M2', 'EMGc']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 1.5, 5, [['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'ERT '], ['ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr'], ['ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['ERC']], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 1.5, 5, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERC', 'ERT ', 'EOGr'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl'], ['ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr']], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 1.5, 5, [['ELA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'EOGl'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr'], ['ERC', 'EOGr', 'EOGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M2'], ['ELA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGr', 'EMGc']], ['EOGl']], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 1.5, 5, [['ELA', 'ELI', 'ERC', 'M1', 'O1', 'M2', 'O2'], ['EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'O2'], ['ELA', 'EOGr', 'EMGr', 'M2'], ['ELA', 'ERT ', 'EOGr', 'M1', 'M2'], ['ERA', 'M1', 'M2'], ['ELA', 'ERC', 'EOGr', 'M1', 'M2', 'EMGc']], ['M2']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 2.0, 5, [['EOGl', 'EMGl'], ['ERC', 'ERT '], ['ERA', 'ERC'], [], ['EMGl'], ['EOGl', 'EMGl']], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 2.0, 5, [['EOGl', 'EMGl', 'EMGr'], ['ERC', 'ERT '], ['ERA', 'ERC', 'EOGl', 'EMGl'], ['ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'EOGl', 'EMGl', 'EMGr'], ['EOGl', 'EMGl', 'EMGr']], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 2.0, 5, [['O2'], ['EOGl'], [], ['M2'], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 2.5, 5, [['EOGl', 'EMGl'], ['ERC', 'ERT '], ['ERA', 'ERC', 'EOGl', 'EMGl'], ['EOGl', 'EMGl', 'EMGr'], ['ERC', 'EMGl', 'EMGr'], ['EOGl', 'EMGl']], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 3.0, 5, [['EOGl', 'EMGl'], ['ERC', 'ERT '], ['ERA', 'ERC'], ['EOGl', 'EMGl'], ['EMGl'], ['EMGl']], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 4.0, 5, [['EOGl', 'EMGl'], ['ERC'], [], [], [], ['EMGl']], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'O2'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc']], ['ELA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr']], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 1.0, 6, [['ELA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'F3'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['ERC', 'EOGl']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc']], ['ELA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2']], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 1.0, 6, [['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELT', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'M1', 'F3', 'M2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'M1', 'F3', 'M2', 'EMGc']], ['ELA', 'ERA', 'EOGr', 'M2', 'EMGc']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 1.5, 6, [['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'ERT '], ['ERA', 'ERC', 'EOGl', 'EMGl', 'EMGr'], ['ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['ERC']], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 1.5, 6, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERC', 'ERT ', 'EOGr'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl'], ['ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr']], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 1.5, 6, [['ELA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'EOGl'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr'], ['ERC', 'EOGr', 'EOGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M2'], ['ELA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGr', 'EMGc']], ['EOGl']], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 1.5, 6, [['ELA', 'ELI', 'ERC', 'M1', 'O1', 'M2', 'O2'], ['EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'O2'], ['ELA', 'EOGr', 'EMGr', 'M2'], ['ELA', 'ERT ', 'EOGr', 'M1', 'M2'], ['ERA', 'M1', 'M2'], ['ELA', 'ERC', 'EOGr', 'M1', 'M2', 'EMGc']], ['M2']], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 2.0, 6, [['EOGl', 'EMGl'], ['ERC', 'ERT '], ['ERA', 'ERC'], [], ['EMGl'], ['EOGl', 'EMGl']], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 2.0, 6, [['EOGl', 'EMGl', 'EMGr'], ['ERC', 'ERT '], ['ERA', 'ERC', 'EOGl', 'EMGl'], ['ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'EOGl', 'EMGl', 'EMGr'], ['EOGl', 'EMGl', 'EMGr']], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 2.0, 6, [['O2'], ['EOGl'], [], ['M2'], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 2.5, 6, [['EOGl', 'EMGl'], ['ERC', 'ERT '], ['ERA', 'ERC', 'EOGl', 'EMGl'], ['EOGl', 'EMGl', 'EMGr'], ['ERC', 'EMGl', 'EMGr'], ['EOGl', 'EMGl']], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 3.0, 6, [['EOGl', 'EMGl'], ['ERC', 'ERT '], ['ERA', 'ERC'], ['EOGl', 'EMGl'], ['EMGl'], ['EMGl']], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 4.0, 6, [['EOGl', 'EMGl'], ['ERC'], [], [], [], ['EMGl']], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 750, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ERA', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ERB', 'EMGl', 'M1', 'ELC']], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 1.0, 1, [['ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ERB', 'EMGr', 'EOGl', 'EMGl', 'ELA']], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGl', 'M2', 'M1', 'ELC']], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ERE', 'EMGl', 'M1', 'EMGc']], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 1.5, 1, [['F3', 'C3', 'F4', 'C4'], ['ELB', 'F3', 'O1', 'F4'], ['F3', 'C3', 'O1', 'F4', 'C4'], ['F3', 'C3', 'F4'], ['ELB', 'C3', 'F4', 'C4'], ['C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'C3', 'F4', 'C4', 'ELB', 'O1', 'O2']], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 1.5, 1, [['ELB', 'ERA', 'EOGr'], ['ELB'], ['ELB'], ['ELB'], [], ['ELB']], ['ELB', 'ERA', 'EOGr']], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 2.0, 1, [['F3', 'F4'], ['F3'], [], ['F3', 'F4'], [], []], ['F3', 'F4']], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 2.5, 1, [[], [], [], ['F3', 'F4'], [], []], ['F3', 'F4']], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 3.0, 1, [[], [], [], ['F3'], [], []], ['F3']], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ERA', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 1.0, 2, [['ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGl', 'M1']], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGl', 'M1']], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 1.5, 2, [['F3', 'C3', 'F4', 'C4'], ['ELB', 'F3', 'O1', 'F4'], ['F3', 'C3', 'O1', 'F4', 'C4'], ['F3', 'C3', 'F4'], ['ELB', 'C3', 'F4', 'C4'], ['C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'C3', 'F4', 'C4', 'O1']], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 1.5, 2, [['ELB', 'ERA', 'EOGr'], ['ELB'], ['ELB'], ['ELB'], [], ['ELB']], ['ELB']], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 2.0, 2, [['F3', 'F4'], ['F3'], [], ['F3', 'F4'], [], []], ['F3']], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 2.5, 2, [[], [], [], ['F3', 'F4'], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 3.0, 2, [[], [], [], ['F3'], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ERA', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 1.0, 3, [['ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGl']], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGl']], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 1.5, 3, [['F3', 'C3', 'F4', 'C4'], ['ELB', 'F3', 'O1', 'F4'], ['F3', 'C3', 'O1', 'F4', 'C4'], ['F3', 'C3', 'F4'], ['ELB', 'C3', 'F4', 'C4'], ['C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'C3', 'F4']], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 1.5, 3, [['ELB', 'ERA', 'EOGr'], ['ELB'], ['ELB'], ['ELB'], [], ['ELB']], ['ELB']], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 2.0, 3, [['F3', 'F4'], ['F3'], [], ['F3', 'F4'], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 2.5, 3, [[], [], [], ['F3', 'F4'], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 3.0, 3, [[], [], [], ['F3'], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ERA', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 1.0, 4, [['ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'C3', 'O1', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 1.5, 4, [['F3', 'C3', 'F4', 'C4'], ['ELB', 'F3', 'O1', 'F4'], ['F3', 'C3', 'O1', 'F4', 'C4'], ['F3', 'C3', 'F4'], ['ELB', 'C3', 'F4', 'C4'], ['C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'C3', 'F4']], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 1.5, 4, [['ELB', 'ERA', 'EOGr'], ['ELB'], ['ELB'], ['ELB'], [], ['ELB']], ['ELB']], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 2.0, 4, [['F3', 'F4'], ['F3'], [], ['F3', 'F4'], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 2.5, 4, [[], [], [], ['F3', 'F4'], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 3.0, 4, [[], [], [], ['F3'], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ERA', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 1.0, 5, [['ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'ERA', 'EOGr', 'C3', 'O1', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 1.5, 5, [['F3', 'C3', 'F4', 'C4'], ['ELB', 'F3', 'O1', 'F4'], ['F3', 'C3', 'O1', 'F4', 'C4'], ['F3', 'C3', 'F4'], ['ELB', 'C3', 'F4', 'C4'], ['C3', 'O1', 'F4', 'C4', 'O2']], ['F4']], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 1.5, 5, [['ELB', 'ERA', 'EOGr'], ['ELB'], ['ELB'], ['ELB'], [], ['ELB']], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 2.0, 5, [['F3', 'F4'], ['F3'], [], ['F3', 'F4'], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 2.5, 5, [[], [], [], ['F3', 'F4'], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 3.0, 5, [[], [], [], ['F3'], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ERA', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 1.0, 6, [['ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'ERA', 'EOGr', 'C3', 'O1', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 1.5, 6, [['F3', 'C3', 'F4', 'C4'], ['ELB', 'F3', 'O1', 'F4'], ['F3', 'C3', 'O1', 'F4', 'C4'], ['F3', 'C3', 'F4'], ['ELB', 'C3', 'F4', 'C4'], ['C3', 'O1', 'F4', 'C4', 'O2']], ['F4']], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 1.5, 6, [['ELB', 'ERA', 'EOGr'], ['ELB'], ['ELB'], ['ELB'], [], ['ELB']], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 2.0, 6, [['F3', 'F4'], ['F3'], [], ['F3', 'F4'], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 2.5, 6, [[], [], [], ['F3', 'F4'], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 3.0, 6, [[], [], [], ['F3'], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 11530, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELI', 'ERA', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELI', 'ERA', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ELE', 'EOGr', 'EOGl', 'M1', 'ELC']], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELB', 'ERA', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELE', 'ELI', 'EOGr', 'M1']], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELI', 'ERA', 'ERC', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELI', 'ERA', 'ERC', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ELE', 'EOGr', 'M1', 'EOGl']], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELI', 'ERA', 'ERC', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELI', 'ERA', 'ERC', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc', 'ELC', 'ELE', 'EOGr', 'O2', 'EOGl']], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 1.5, 1, [['O1', 'O2'], ['F3'], ['ELB', 'ELI'], [], ['ELB'], ['ELA']], ['O1', 'O2', 'F3', 'ELB', 'ELI', 'ELA']], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 1.5, 1, [['F3', 'O1', 'F4', 'O2'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'F3'], ['ELB', 'F3', 'O1', 'F4', 'O2'], ['ELB', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELA', 'ELB', 'F3', 'C3', 'O1', 'F4', 'O2']], ['F3', 'O1', 'F4', 'O2', 'C3', 'C4', 'ELB', 'ELE', 'ELI', 'ELA']], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 1.5, 1, [[], [], ['F3'], ['O1'], [], ['ELA']], ['F3', 'O1', 'ELA']], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 1.5, 1, [['ELA', 'ELB', 'ERA', 'EMGc'], ['ELA', 'ELB', 'ERA'], ['ELA', 'ERA', 'M1', 'EMGc'], ['ELB', 'M1'], ['ELA', 'ELB', 'ERA', 'M1'], ['ELA', 'ERA', 'M1', 'EMGc']], ['ELA', 'ELB', 'ERA', 'EMGc', 'M1']], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 2.0, 1, [['O1', 'F4', 'O2'], ['F3', 'F4'], ['ELE', 'ELI'], [], [], []], ['O1', 'F4', 'O2', 'F3', 'ELE', 'ELI']], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 2.5, 1, [[], ['F3'], ['ELE', 'ELI'], [], [], []], ['F3', 'ELE', 'ELI']], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 3.0, 1, [[], [], ['ELE', 'ELI'], [], [], []], ['ELE', 'ELI']], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 4.0, 1, [[], [], ['ELI'], [], [], []], ['ELI']], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELI', 'ERA', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELI', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ELE', 'EOGl', 'M1']], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELB', 'ERA', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELE', 'ELI', 'M1']], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELI', 'ERA', 'ERC', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELI', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ELE', 'M1']], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELI', 'ERA', 'ERC', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc', 'ELC', 'O2', 'EOGl']], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 1.5, 2, [['O1', 'O2'], ['F3'], ['ELB', 'ELI'], [], ['ELB'], ['ELA']], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 1.5, 2, [['F3', 'O1', 'F4', 'O2'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'F3'], ['ELB', 'F3', 'O1', 'F4', 'O2'], ['ELB', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELA', 'ELB', 'F3', 'C3', 'O1', 'F4', 'O2']], ['F3', 'O1', 'F4', 'O2', 'C3', 'ELB']], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 1.5, 2, [[], [], ['F3'], ['O1'], [], ['ELA']], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 1.5, 2, [['ELA', 'ELB', 'ERA', 'EMGc'], ['ELA', 'ELB', 'ERA'], ['ELA', 'ERA', 'M1', 'EMGc'], ['ELB', 'M1'], ['ELA', 'ELB', 'ERA', 'M1'], ['ELA', 'ERA', 'M1', 'EMGc']], ['ELA', 'ELB', 'ERA', 'M1']], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 2.0, 2, [['O1', 'F4', 'O2'], ['F3', 'F4'], ['ELE', 'ELI'], [], [], []], ['F4']], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 2.5, 2, [[], ['F3'], ['ELE', 'ELI'], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 3.0, 2, [[], [], ['ELE', 'ELI'], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 4.0, 2, [[], [], ['ELI'], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELI', 'ERA', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELI', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ELE', 'M1']], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELB', 'ERA', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELE', 'ELI']], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELI', 'ERA', 'ERC', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELI', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ELE', 'M1']], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELI', 'ERA', 'ERC', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc', 'O2']], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 1.5, 3, [['O1', 'O2'], ['F3'], ['ELB', 'ELI'], [], ['ELB'], ['ELA']], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 1.5, 3, [['F3', 'O1', 'F4', 'O2'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'F3'], ['ELB', 'F3', 'O1', 'F4', 'O2'], ['ELB', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELA', 'ELB', 'F3', 'C3', 'O1', 'F4', 'O2']], ['F3', 'O1', 'F4', 'O2', 'ELB']], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 1.5, 3, [[], [], ['F3'], ['O1'], [], ['ELA']], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 1.5, 3, [['ELA', 'ELB', 'ERA', 'EMGc'], ['ELA', 'ELB', 'ERA'], ['ELA', 'ERA', 'M1', 'EMGc'], ['ELB', 'M1'], ['ELA', 'ELB', 'ERA', 'M1'], ['ELA', 'ERA', 'M1', 'EMGc']], ['ELA', 'ERA', 'M1']], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 2.0, 3, [['O1', 'F4', 'O2'], ['F3', 'F4'], ['ELE', 'ELI'], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 2.5, 3, [[], ['F3'], ['ELE', 'ELI'], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 3.0, 3, [[], [], ['ELE', 'ELI'], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 4.0, 3, [[], [], ['ELI'], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELI', 'ERA', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELI', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELB', 'ERA', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'ERA', 'F3', 'C3', 'F4', 'O2', 'ELI']], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELI', 'ERA', 'ERC', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELI', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ELE', 'M1']], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELI', 'ERA', 'ERC', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc', 'O2']], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 1.5, 4, [['O1', 'O2'], ['F3'], ['ELB', 'ELI'], [], ['ELB'], ['ELA']], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 1.5, 4, [['F3', 'O1', 'F4', 'O2'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'F3'], ['ELB', 'F3', 'O1', 'F4', 'O2'], ['ELB', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELA', 'ELB', 'F3', 'C3', 'O1', 'F4', 'O2']], ['F3', 'ELB']], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 1.5, 4, [[], [], ['F3'], ['O1'], [], ['ELA']], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 1.5, 4, [['ELA', 'ELB', 'ERA', 'EMGc'], ['ELA', 'ELB', 'ERA'], ['ELA', 'ERA', 'M1', 'EMGc'], ['ELB', 'M1'], ['ELA', 'ELB', 'ERA', 'M1'], ['ELA', 'ERA', 'M1', 'EMGc']], ['M1']], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 2.0, 4, [['O1', 'F4', 'O2'], ['F3', 'F4'], ['ELE', 'ELI'], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 2.5, 4, [[], ['F3'], ['ELE', 'ELI'], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 3.0, 4, [[], [], ['ELE', 'ELI'], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 4.0, 4, [[], [], ['ELI'], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELI', 'ERA', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELI', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELB', 'ERA', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'ERA', 'F3', 'C3', 'F4', 'O2']], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELI', 'ERA', 'ERC', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELI', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ELE', 'M1']], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELI', 'ERA', 'ERC', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'M1', 'C3', 'O1', 'C4', 'EMGc', 'O2']], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 1.5, 5, [['O1', 'O2'], ['F3'], ['ELB', 'ELI'], [], ['ELB'], ['ELA']], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 1.5, 5, [['F3', 'O1', 'F4', 'O2'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'F3'], ['ELB', 'F3', 'O1', 'F4', 'O2'], ['ELB', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELA', 'ELB', 'F3', 'C3', 'O1', 'F4', 'O2']], ['F3']], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 1.5, 5, [[], [], ['F3'], ['O1'], [], ['ELA']], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 1.5, 5, [['ELA', 'ELB', 'ERA', 'EMGc'], ['ELA', 'ELB', 'ERA'], ['ELA', 'ERA', 'M1', 'EMGc'], ['ELB', 'M1'], ['ELA', 'ELB', 'ERA', 'M1'], ['ELA', 'ERA', 'M1', 'EMGc']], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 2.0, 5, [['O1', 'F4', 'O2'], ['F3', 'F4'], ['ELE', 'ELI'], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 2.5, 5, [[], ['F3'], ['ELE', 'ELI'], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 3.0, 5, [[], [], ['ELE', 'ELI'], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 4.0, 5, [[], [], ['ELI'], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELI', 'ERA', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELI', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELI', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELB', 'ERA', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'ERA', 'F3', 'C3', 'F4', 'O2']], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELI', 'ERA', 'ERC', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELI', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELI', 'ERA', 'ERC', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERA', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'M1', 'C3', 'O1', 'C4', 'EMGc']], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 1.5, 6, [['O1', 'O2'], ['F3'], ['ELB', 'ELI'], [], ['ELB'], ['ELA']], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 1.5, 6, [['F3', 'O1', 'F4', 'O2'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'F3'], ['ELB', 'F3', 'O1', 'F4', 'O2'], ['ELB', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELA', 'ELB', 'F3', 'C3', 'O1', 'F4', 'O2']], ['F3']], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 1.5, 6, [[], [], ['F3'], ['O1'], [], ['ELA']], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 1.5, 6, [['ELA', 'ELB', 'ERA', 'EMGc'], ['ELA', 'ELB', 'ERA'], ['ELA', 'ERA', 'M1', 'EMGc'], ['ELB', 'M1'], ['ELA', 'ELB', 'ERA', 'M1'], ['ELA', 'ERA', 'M1', 'EMGc']], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 2.0, 6, [['O1', 'F4', 'O2'], ['F3', 'F4'], ['ELE', 'ELI'], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 2.5, 6, [[], ['F3'], ['ELE', 'ELI'], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 3.0, 6, [[], [], ['ELE', 'ELI'], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 4.0, 6, [[], [], ['ELI'], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 12430, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELI']], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 1.0, 1, [['ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELB']], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELC']], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'F3', 'ELC']], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 1.5, 1, [['F3', 'O1', 'F4', 'O2'], ['EOGr', 'F3', 'O1', 'F4', 'O2'], ['F3', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'O1', 'F4', 'O2'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'O1', 'F4', 'O2', 'EOGr', 'C4', 'C3']], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 1.5, 1, [[], [], [], [], ['EMGr'], []], ['EMGr']], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 1.5, 1, [['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['ERA', 'EMGr'], ['ERA', 'EOGr', 'EMGr'], ['EMGr'], ['ERA', 'EOGr']], ['EMGl', 'EMGr', 'ERA', 'EOGr']], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 2.0, 1, [['F3', 'O2'], [], [], ['F3', 'F4'], [], ['F4']], ['F3', 'O2', 'F4']], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 2.5, 1, [[], [], [], ['F3', 'F4'], [], []], ['F3', 'F4']], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 1.0, 2, [['ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'F3']], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 1.5, 2, [['F3', 'O1', 'F4', 'O2'], ['EOGr', 'F3', 'O1', 'F4', 'O2'], ['F3', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'O1', 'F4', 'O2'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'O1', 'F4', 'O2', 'C4']], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 1.5, 2, [[], [], [], [], ['EMGr'], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 1.5, 2, [['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['ERA', 'EMGr'], ['ERA', 'EOGr', 'EMGr'], ['EMGr'], ['ERA', 'EOGr']], ['EMGl', 'EMGr', 'ERA']], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 2.0, 2, [['F3', 'O2'], [], [], ['F3', 'F4'], [], ['F4']], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 2.5, 2, [[], [], [], ['F3', 'F4'], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 1.0, 3, [['ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'F3']], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 1.5, 3, [['F3', 'O1', 'F4', 'O2'], ['EOGr', 'F3', 'O1', 'F4', 'O2'], ['F3', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'O1', 'F4', 'O2'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'F4', 'O2']], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 1.5, 3, [[], [], [], [], ['EMGr'], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 1.5, 3, [['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['ERA', 'EMGr'], ['ERA', 'EOGr', 'EMGr'], ['EMGr'], ['ERA', 'EOGr']], ['EMGr']], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 2.0, 3, [['F3', 'O2'], [], [], ['F3', 'F4'], [], ['F4']], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 2.5, 3, [[], [], [], ['F3', 'F4'], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 1.0, 4, [['ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'C3', 'O1', 'C4', 'O2', 'F3']], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 1.5, 4, [['F3', 'O1', 'F4', 'O2'], ['EOGr', 'F3', 'O1', 'F4', 'O2'], ['F3', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'O1', 'F4', 'O2'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'F4']], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 1.5, 4, [[], [], [], [], ['EMGr'], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 1.5, 4, [['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['ERA', 'EMGr'], ['ERA', 'EOGr', 'EMGr'], ['EMGr'], ['ERA', 'EOGr']], ['EMGr']], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 2.0, 4, [['F3', 'O2'], [], [], ['F3', 'F4'], [], ['F4']], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 2.5, 4, [[], [], [], ['F3', 'F4'], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 1.0, 5, [['ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'ERA', 'ERB', 'EOGr', 'EMGr', 'C3', 'C4', 'F3']], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 1.5, 5, [['F3', 'O1', 'F4', 'O2'], ['EOGr', 'F3', 'O1', 'F4', 'O2'], ['F3', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'O1', 'F4', 'O2'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'F4']], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 1.5, 5, [[], [], [], [], ['EMGr'], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 1.5, 5, [['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['ERA', 'EMGr'], ['ERA', 'EOGr', 'EMGr'], ['EMGr'], ['ERA', 'EOGr']], ['EMGr']], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 2.0, 5, [['F3', 'O2'], [], [], ['F3', 'F4'], [], ['F4']], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 2.5, 5, [[], [], [], ['F3', 'F4'], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 1.0, 6, [['ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'ERA', 'EOGr', 'EMGr', 'C3', 'C4']], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 1.5, 6, [['F3', 'O1', 'F4', 'O2'], ['EOGr', 'F3', 'O1', 'F4', 'O2'], ['F3', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'O1', 'F4', 'O2'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'F4']], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 1.5, 6, [[], [], [], [], ['EMGr'], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 1.5, 6, [['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['ERA', 'EMGr'], ['ERA', 'EOGr', 'EMGr'], ['EMGr'], ['ERA', 'EOGr']], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 2.0, 6, [['F3', 'O2'], [], [], ['F3', 'F4'], [], ['F4']], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 2.5, 6, [[], [], [], ['F3', 'F4'], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18300, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 1.0, 1, [['ERA', 'ERB', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERB', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'ERB', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELA', 'ELB', 'EOGl', 'EMGl']], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 1.0, 1, [['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELA', 'EMGl']], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 1.0, 1, [['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELA', 'EMGl', 'M1']], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELC', 'M2', 'M1']], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], ['F4', 'O2'], []], ['F4', 'O2']], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 1.5, 1, [['C3', 'F4', 'C4', 'O2'], ['C3', 'O1', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'O1', 'F4', 'O2'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4']], ['C3', 'F4', 'C4', 'O2', 'O1', 'F3']], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 1.5, 1, [['ERA', 'EMGr'], ['ERA', 'EMGr', 'C3'], ['EMGr'], [], ['ELB', 'ERA', 'EOGr', 'EMGr'], ['EMGr']], ['ERA', 'EMGr', 'C3', 'ELB', 'EOGr']], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 2.0, 1, [[], [], ['F3', 'C3', 'F4'], ['O1'], [], ['F3', 'F4']], ['F3', 'C3', 'F4', 'O1']], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 2.5, 1, [[], [], ['F3'], [], [], []], ['F3']], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 1.0, 2, [['ERA', 'ERB', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERB', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'ERB', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELA', 'ELB', 'EOGl']], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 1.0, 2, [['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 1.0, 2, [['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELA', 'EMGl']], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELC']], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], ['F4', 'O2'], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 1.5, 2, [['C3', 'F4', 'C4', 'O2'], ['C3', 'O1', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'O1', 'F4', 'O2'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4']], ['C3', 'F4', 'C4', 'O2', 'F3']], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 1.5, 2, [['ERA', 'EMGr'], ['ERA', 'EMGr', 'C3'], ['EMGr'], [], ['ELB', 'ERA', 'EOGr', 'EMGr'], ['EMGr']], ['ERA', 'EMGr']], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 2.0, 2, [[], [], ['F3', 'C3', 'F4'], ['O1'], [], ['F3', 'F4']], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 2.5, 2, [[], [], ['F3'], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 1.0, 3, [['ERA', 'ERB', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERB', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'ERB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 1.0, 3, [['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 1.0, 3, [['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELA']], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], ['F4', 'O2'], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 1.5, 3, [['C3', 'F4', 'C4', 'O2'], ['C3', 'O1', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'O1', 'F4', 'O2'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4']], ['C3', 'F4', 'C4', 'F3']], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 1.5, 3, [['ERA', 'EMGr'], ['ERA', 'EMGr', 'C3'], ['EMGr'], [], ['ELB', 'ERA', 'EOGr', 'EMGr'], ['EMGr']], ['EMGr']], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 2.0, 3, [[], [], ['F3', 'C3', 'F4'], ['O1'], [], ['F3', 'F4']], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 2.5, 3, [[], [], ['F3'], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 1.0, 4, [['ERA', 'ERB', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERB', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 1.0, 4, [['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 1.0, 4, [['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'ERA', 'ERB', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELA']], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'ERB', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], ['F4', 'O2'], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 1.5, 4, [['C3', 'F4', 'C4', 'O2'], ['C3', 'O1', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'O1', 'F4', 'O2'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4']], ['F4', 'F3']], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 1.5, 4, [['ERA', 'EMGr'], ['ERA', 'EMGr', 'C3'], ['EMGr'], [], ['ELB', 'ERA', 'EOGr', 'EMGr'], ['EMGr']], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 2.0, 4, [[], [], ['F3', 'C3', 'F4'], ['O1'], [], ['F3', 'F4']], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 2.5, 4, [[], [], ['F3'], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 1.0, 5, [['ERA', 'ERB', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERB', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 1.0, 5, [['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 1.0, 5, [['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'ERA', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], ['F4', 'O2'], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 1.5, 5, [['C3', 'F4', 'C4', 'O2'], ['C3', 'O1', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'O1', 'F4', 'O2'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4']], ['F4']], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 1.5, 5, [['ERA', 'EMGr'], ['ERA', 'EMGr', 'C3'], ['EMGr'], [], ['ELB', 'ERA', 'EOGr', 'EMGr'], ['EMGr']], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 2.0, 5, [[], [], ['F3', 'C3', 'F4'], ['O1'], [], ['F3', 'F4']], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 2.5, 5, [[], [], ['F3'], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 1.0, 6, [['ERA', 'ERB', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERB', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 1.0, 6, [['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 1.0, 6, [['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'ERA', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], ['F4', 'O2'], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 1.5, 6, [['C3', 'F4', 'C4', 'O2'], ['C3', 'O1', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'O1', 'F4', 'O2'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4']], ['F4']], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 1.5, 6, [['ERA', 'EMGr'], ['ERA', 'EMGr', 'C3'], ['EMGr'], [], ['ELB', 'ERA', 'EOGr', 'EMGr'], ['EMGr']], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 2.0, 6, [[], [], ['F3', 'C3', 'F4'], ['O1'], [], ['F3', 'F4']], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 2.5, 6, [[], [], ['F3'], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 18970, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'C4'], ['ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERC'], ['ELA', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'ERT ']], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERC'], ['ELB', 'ERC'], ['ELB', 'ERC'], ['ERA', 'ERB', 'ERC', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERB', 'ERC', 'EOGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ERA', 'ERC', 'ERB', 'M1']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'C4'], ['ELA', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'ELE', 'ELI', 'ERC', 'ERE', 'ERI', 'EMGr', 'ERT ', 'EMGl']], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ERA', 'ERB', 'ERT ', 'ERE', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'ELE', 'ELI', 'ERT ', 'ERE', 'EMGr', 'F4', 'EMGc', 'ERI', 'EOGl', 'EMGl']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 1.5, 1, [[], ['ELB', 'ERB', 'ERC'], ['ELB', 'ERA', 'EOGr'], ['ELB', 'ERC'], ['ERA', 'ERB', 'ERC'], ['ERA', 'ERB']], ['ELB', 'ERB', 'ERC', 'ERA', 'EOGr']], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 1.5, 1, [['F3', 'C3', 'O1', 'F4', 'O2'], ['ELB', 'ERA', 'ERC'], ['ELB', 'ERC'], ['ELB', 'ERC'], ['ERA', 'ERB', 'ERC', 'F3', 'F4'], ['ERA', 'ERB', 'ERC', 'F3', 'F4']], ['F3', 'C3', 'O1', 'F4', 'O2', 'ELB', 'ERA', 'ERC', 'ERB']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 1.5, 1, [[], ['ERB', 'ERE', 'EOGr', 'EMGr', 'C4'], ['ERA', 'EOGr'], ['ERA', 'ERB', 'ERE'], ['ERC'], ['ERA']], ['ERB', 'ERE', 'EOGr', 'EMGr', 'C4', 'ERA', 'ERC']], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 1.5, 1, [['ELB', 'ERA', 'ERB'], ['ELA', 'ELC', 'ERT ', 'EMGr'], ['ERB', 'ERE', 'M1', 'C3'], ['ERA', 'ERB', 'ERT ', 'ERE', 'M1'], ['ELA', 'ERT ', 'M1'], ['ERA', 'EMGl', 'M1']], ['ELB', 'ERA', 'ERB', 'ELA', 'ELC', 'ERT ', 'EMGr', 'ERE', 'M1', 'C3', 'EMGl']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 2.0, 1, [[], ['ELB'], ['ELB', 'EOGr'], ['ELB'], [], []], ['ELB', 'EOGr']], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 2.0, 1, [['F4', 'O2'], ['ELB', 'ERA', 'ERC'], ['ELB', 'ERC'], ['ELB'], ['ERA', 'ERC'], ['ERA', 'ERB']], ['F4', 'O2', 'ELB', 'ERA', 'ERC', 'ERB']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 2.0, 1, [[], [], [], [], ['M1'], []], ['M1']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 2.5, 1, [[], ['ELB'], ['ELB'], ['ELB'], [], []], ['ELB']], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 2.5, 1, [[], ['ELB', 'ERC'], ['ELB'], ['ELB'], ['ERA'], []], ['ELB', 'ERC', 'ERA']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 3.0, 1, [[], ['ELB'], ['ELB'], ['ELB'], [], []], ['ELB']], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 3.0, 1, [[], ['ELB', 'ERC'], ['ELB'], ['ELB'], [], []], ['ELB', 'ERC']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 4.0, 1, [[], [], [], ['ELB'], [], []], ['ELB']], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 4.0, 1, [[], ['ELB'], ['ELB'], ['ELB'], [], []], ['ELB']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 5.0, 1, [[], [], [], ['ELB'], [], []], ['ELB']], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 5.0, 1, [[], ['ELB'], ['ELB'], ['ELB'], [], []], ['ELB']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'C4'], ['ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERC'], ['ELA', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'ERB', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ERC', 'EOGr']], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERC'], ['ELB', 'ERC'], ['ELB', 'ERC'], ['ERA', 'ERB', 'ERC', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERB', 'ERC', 'EOGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'F3', 'O1', 'F4', 'C4', 'O2', 'ERA', 'ERC', 'ERB']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'C4'], ['ELA', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELC', 'ERA', 'ERB', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ERC', 'ERE', 'ERI', 'ERT ']], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ERA', 'ERB', 'ERT ', 'ERE', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'ERT ', 'ERE', 'F4']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 1.5, 2, [[], ['ELB', 'ERB', 'ERC'], ['ELB', 'ERA', 'EOGr'], ['ELB', 'ERC'], ['ERA', 'ERB', 'ERC'], ['ERA', 'ERB']], ['ELB', 'ERB', 'ERC', 'ERA']], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 1.5, 2, [['F3', 'C3', 'O1', 'F4', 'O2'], ['ELB', 'ERA', 'ERC'], ['ELB', 'ERC'], ['ELB', 'ERC'], ['ERA', 'ERB', 'ERC', 'F3', 'F4'], ['ERA', 'ERB', 'ERC', 'F3', 'F4']], ['F3', 'F4', 'ELB', 'ERA', 'ERC', 'ERB']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 1.5, 2, [[], ['ERB', 'ERE', 'EOGr', 'EMGr', 'C4'], ['ERA', 'EOGr'], ['ERA', 'ERB', 'ERE'], ['ERC'], ['ERA']], ['EOGr', 'ERA']], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 1.5, 2, [['ELB', 'ERA', 'ERB'], ['ELA', 'ELC', 'ERT ', 'EMGr'], ['ERB', 'ERE', 'M1', 'C3'], ['ERA', 'ERB', 'ERT ', 'ERE', 'M1'], ['ELA', 'ERT ', 'M1'], ['ERA', 'EMGl', 'M1']], ['ERB', 'ERT ', 'ERE', 'M1']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 2.0, 2, [[], ['ELB'], ['ELB', 'EOGr'], ['ELB'], [], []], ['ELB']], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 2.0, 2, [['F4', 'O2'], ['ELB', 'ERA', 'ERC'], ['ELB', 'ERC'], ['ELB'], ['ERA', 'ERC'], ['ERA', 'ERB']], ['ELB', 'ERA', 'ERC']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 2.0, 2, [[], [], [], [], ['M1'], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 2.5, 2, [[], ['ELB'], ['ELB'], ['ELB'], [], []], ['ELB']], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 2.5, 2, [[], ['ELB', 'ERC'], ['ELB'], ['ELB'], ['ERA'], []], ['ELB']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 3.0, 2, [[], ['ELB'], ['ELB'], ['ELB'], [], []], ['ELB']], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 3.0, 2, [[], ['ELB', 'ERC'], ['ELB'], ['ELB'], [], []], ['ELB']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 4.0, 2, [[], [], [], ['ELB'], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 4.0, 2, [[], ['ELB'], ['ELB'], ['ELB'], [], []], ['ELB']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 5.0, 2, [[], [], [], ['ELB'], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 5.0, 2, [[], ['ELB'], ['ELB'], ['ELB'], [], []], ['ELB']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'C4'], ['ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERC'], ['ELA', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'ERA', 'ERB', 'C4', 'ERC']], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERC'], ['ELB', 'ERC'], ['ELB', 'ERC'], ['ERA', 'ERB', 'ERC', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERB', 'ERC', 'EOGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'ERC']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'C4'], ['ELA', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ERA', 'ERB', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ERC', 'ERE']], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ERA', 'ERB', 'ERT ', 'ERE', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ELA', 'ERB', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'ERE', 'F4']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 1.5, 3, [[], ['ELB', 'ERB', 'ERC'], ['ELB', 'ERA', 'EOGr'], ['ELB', 'ERC'], ['ERA', 'ERB', 'ERC'], ['ERA', 'ERB']], ['ELB']], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 1.5, 3, [['F3', 'C3', 'O1', 'F4', 'O2'], ['ELB', 'ERA', 'ERC'], ['ELB', 'ERC'], ['ELB', 'ERC'], ['ERA', 'ERB', 'ERC', 'F3', 'F4'], ['ERA', 'ERB', 'ERC', 'F3', 'F4']], ['ELB', 'ERC']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 1.5, 3, [[], ['ERB', 'ERE', 'EOGr', 'EMGr', 'C4'], ['ERA', 'EOGr'], ['ERA', 'ERB', 'ERE'], ['ERC'], ['ERA']], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 1.5, 3, [['ELB', 'ERA', 'ERB'], ['ELA', 'ELC', 'ERT ', 'EMGr'], ['ERB', 'ERE', 'M1', 'C3'], ['ERA', 'ERB', 'ERT ', 'ERE', 'M1'], ['ELA', 'ERT ', 'M1'], ['ERA', 'EMGl', 'M1']], ['M1']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 2.0, 3, [[], ['ELB'], ['ELB', 'EOGr'], ['ELB'], [], []], ['ELB']], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 2.0, 3, [['F4', 'O2'], ['ELB', 'ERA', 'ERC'], ['ELB', 'ERC'], ['ELB'], ['ERA', 'ERC'], ['ERA', 'ERB']], ['ELB']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 2.0, 3, [[], [], [], [], ['M1'], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 2.5, 3, [[], ['ELB'], ['ELB'], ['ELB'], [], []], ['ELB']], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 2.5, 3, [[], ['ELB', 'ERC'], ['ELB'], ['ELB'], ['ERA'], []], ['ELB']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 3.0, 3, [[], ['ELB'], ['ELB'], ['ELB'], [], []], ['ELB']], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 3.0, 3, [[], ['ELB', 'ERC'], ['ELB'], ['ELB'], [], []], ['ELB']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 4.0, 3, [[], [], [], ['ELB'], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 4.0, 3, [[], ['ELB'], ['ELB'], ['ELB'], [], []], ['ELB']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 5.0, 3, [[], [], [], ['ELB'], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 5.0, 3, [[], ['ELB'], ['ELB'], ['ELB'], [], []], ['ELB']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'C4'], ['ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERC'], ['ELA', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'ERC']], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERC'], ['ELB', 'ERC'], ['ELB', 'ERC'], ['ERA', 'ERB', 'ERC', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERB', 'ERC', 'EOGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'ERC']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'C4'], ['ELA', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ERA', 'ERB', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ERC']], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ERA', 'ERB', 'ERT ', 'ERE', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ELA', 'ERB', 'M1', 'C3', 'C4', 'ERE', 'F4']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 1.5, 4, [[], ['ELB', 'ERB', 'ERC'], ['ELB', 'ERA', 'EOGr'], ['ELB', 'ERC'], ['ERA', 'ERB', 'ERC'], ['ERA', 'ERB']], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 1.5, 4, [['F3', 'C3', 'O1', 'F4', 'O2'], ['ELB', 'ERA', 'ERC'], ['ELB', 'ERC'], ['ELB', 'ERC'], ['ERA', 'ERB', 'ERC', 'F3', 'F4'], ['ERA', 'ERB', 'ERC', 'F3', 'F4']], ['ERC']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 1.5, 4, [[], ['ERB', 'ERE', 'EOGr', 'EMGr', 'C4'], ['ERA', 'EOGr'], ['ERA', 'ERB', 'ERE'], ['ERC'], ['ERA']], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 1.5, 4, [['ELB', 'ERA', 'ERB'], ['ELA', 'ELC', 'ERT ', 'EMGr'], ['ERB', 'ERE', 'M1', 'C3'], ['ERA', 'ERB', 'ERT ', 'ERE', 'M1'], ['ELA', 'ERT ', 'M1'], ['ERA', 'EMGl', 'M1']], ['M1']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 2.0, 4, [[], ['ELB'], ['ELB', 'EOGr'], ['ELB'], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 2.0, 4, [['F4', 'O2'], ['ELB', 'ERA', 'ERC'], ['ELB', 'ERC'], ['ELB'], ['ERA', 'ERC'], ['ERA', 'ERB']], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 2.0, 4, [[], [], [], [], ['M1'], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 2.5, 4, [[], ['ELB'], ['ELB'], ['ELB'], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 2.5, 4, [[], ['ELB', 'ERC'], ['ELB'], ['ELB'], ['ERA'], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 3.0, 4, [[], ['ELB'], ['ELB'], ['ELB'], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 3.0, 4, [[], ['ELB', 'ERC'], ['ELB'], ['ELB'], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 4.0, 4, [[], [], [], ['ELB'], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 4.0, 4, [[], ['ELB'], ['ELB'], ['ELB'], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 5.0, 4, [[], [], [], ['ELB'], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 5.0, 4, [[], ['ELB'], ['ELB'], ['ELB'], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'C4'], ['ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERC'], ['ELA', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERC']], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERC'], ['ELB', 'ERC'], ['ELB', 'ERC'], ['ERA', 'ERB', 'ERC', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERB', 'ERC', 'EOGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ERC']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'C4'], ['ELA', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ERA', 'ERB', 'C4', 'ERC']], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ERA', 'ERB', 'ERT ', 'ERE', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ELA', 'ERB', 'C3', 'C4', 'F4']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 1.5, 5, [[], ['ELB', 'ERB', 'ERC'], ['ELB', 'ERA', 'EOGr'], ['ELB', 'ERC'], ['ERA', 'ERB', 'ERC'], ['ERA', 'ERB']], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 1.5, 5, [['F3', 'C3', 'O1', 'F4', 'O2'], ['ELB', 'ERA', 'ERC'], ['ELB', 'ERC'], ['ELB', 'ERC'], ['ERA', 'ERB', 'ERC', 'F3', 'F4'], ['ERA', 'ERB', 'ERC', 'F3', 'F4']], ['ERC']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 1.5, 5, [[], ['ERB', 'ERE', 'EOGr', 'EMGr', 'C4'], ['ERA', 'EOGr'], ['ERA', 'ERB', 'ERE'], ['ERC'], ['ERA']], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 1.5, 5, [['ELB', 'ERA', 'ERB'], ['ELA', 'ELC', 'ERT ', 'EMGr'], ['ERB', 'ERE', 'M1', 'C3'], ['ERA', 'ERB', 'ERT ', 'ERE', 'M1'], ['ELA', 'ERT ', 'M1'], ['ERA', 'EMGl', 'M1']], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 2.0, 5, [[], ['ELB'], ['ELB', 'EOGr'], ['ELB'], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 2.0, 5, [['F4', 'O2'], ['ELB', 'ERA', 'ERC'], ['ELB', 'ERC'], ['ELB'], ['ERA', 'ERC'], ['ERA', 'ERB']], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 2.0, 5, [[], [], [], [], ['M1'], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 2.5, 5, [[], ['ELB'], ['ELB'], ['ELB'], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 2.5, 5, [[], ['ELB', 'ERC'], ['ELB'], ['ELB'], ['ERA'], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 3.0, 5, [[], ['ELB'], ['ELB'], ['ELB'], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 3.0, 5, [[], ['ELB', 'ERC'], ['ELB'], ['ELB'], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 4.0, 5, [[], [], [], ['ELB'], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 4.0, 5, [[], ['ELB'], ['ELB'], ['ELB'], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 5.0, 5, [[], [], [], ['ELB'], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 5.0, 5, [[], ['ELB'], ['ELB'], ['ELB'], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'C4'], ['ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERC'], ['ELA', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERA', 'ERC'], ['ELB', 'ERC'], ['ELB', 'ERC'], ['ERA', 'ERB', 'ERC', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERB', 'ERC', 'EOGr', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2']], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGr', 'C4'], ['ELA', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ERA', 'ERB', 'C4']], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ERA', 'ERB', 'ERT ', 'ERE', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ELA', 'ERB', 'C3', 'C4']], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 1.5, 6, [[], ['ELB', 'ERB', 'ERC'], ['ELB', 'ERA', 'EOGr'], ['ELB', 'ERC'], ['ERA', 'ERB', 'ERC'], ['ERA', 'ERB']], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 1.5, 6, [['F3', 'C3', 'O1', 'F4', 'O2'], ['ELB', 'ERA', 'ERC'], ['ELB', 'ERC'], ['ELB', 'ERC'], ['ERA', 'ERB', 'ERC', 'F3', 'F4'], ['ERA', 'ERB', 'ERC', 'F3', 'F4']], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 1.5, 6, [[], ['ERB', 'ERE', 'EOGr', 'EMGr', 'C4'], ['ERA', 'EOGr'], ['ERA', 'ERB', 'ERE'], ['ERC'], ['ERA']], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 1.5, 6, [['ELB', 'ERA', 'ERB'], ['ELA', 'ELC', 'ERT ', 'EMGr'], ['ERB', 'ERE', 'M1', 'C3'], ['ERA', 'ERB', 'ERT ', 'ERE', 'M1'], ['ELA', 'ERT ', 'M1'], ['ERA', 'EMGl', 'M1']], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 2.0, 6, [[], ['ELB'], ['ELB', 'EOGr'], ['ELB'], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 2.0, 6, [['F4', 'O2'], ['ELB', 'ERA', 'ERC'], ['ELB', 'ERC'], ['ELB'], ['ERA', 'ERC'], ['ERA', 'ERB']], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 2.0, 6, [[], [], [], [], ['M1'], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 2.5, 6, [[], ['ELB'], ['ELB'], ['ELB'], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 2.5, 6, [[], ['ELB', 'ERC'], ['ELB'], ['ELB'], ['ERA'], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 3.0, 6, [[], ['ELB'], ['ELB'], ['ELB'], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 3.0, 6, [[], ['ELB', 'ERC'], ['ELB'], ['ELB'], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 4.0, 6, [[], [], [], ['ELB'], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 4.0, 6, [[], ['ELB'], ['ELB'], ['ELB'], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'mm', 5.0, 6, [[], [], [], ['ELB'], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'mm', 5.0, 6, [[], ['ELB'], ['ELB'], ['ELB'], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_02/night_4\\EEG_raw', 20820, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 1.0, 1, [['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc']], ['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'O2', 'EMGc', 'ERE', 'ELB', 'M2', 'F4', 'ELA', 'ERI', 'C3', 'C4']], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 1.0, 1, [['ELC', 'ERA', 'ERB', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl'], ['ELC', 'ERA', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGl', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGl', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'F3', 'C3', 'F4', 'C4']], ['ELC', 'ERA', 'ERB', 'EMGc', 'EOGr', 'EOGl', 'O2', 'ELA', 'ERE', 'F3', 'C3', 'F4', 'C4']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 1.0, 1, [['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELC', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'EMGc']], ['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'O2', 'EMGc', 'F4', 'ERE', 'ELB', 'M2', 'ERI', 'C3', 'C4']], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'O2'], ['ELB', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELB', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'O2', 'M2', 'EMGc', 'ERE', 'C3', 'F4', 'C4']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 1.5, 1, [['ERA', 'EMGc'], ['ELC', 'ERA', 'O1', 'O2'], ['ELC', 'ERA', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EMGc'], ['ELA', 'ERA']], ['ERA', 'EMGc', 'ELC', 'O1', 'O2', 'ERB', 'ELA']], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 1.5, 1, [['ERA', 'EMGc'], ['ELC', 'ERA'], ['ELC', 'ERA', 'EMGc'], ['ELC', 'ERA', 'ERB'], ['ELC', 'ERA', 'ERB', 'EMGc'], ['ELA', 'ERA']], ['ERA', 'EMGc', 'ELC', 'ERB', 'ELA']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 1.5, 1, [['ELC', 'ERB', 'EMGr', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGl', 'O2'], ['EMGc'], ['ELC', 'ERB', 'O2', 'EMGc'], ['EMGc'], ['ERA']], ['ELC', 'ERB', 'EMGr', 'O2', 'EMGc', 'EOGl', 'ERA']], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 1.5, 1, [['ERB', 'EMGl', 'EMGr', 'M1', 'O1', 'O2'], ['ERB', 'M1', 'O1', 'O2', 'EMGc'], ['EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELB', 'EMGl', 'O1', 'M2', 'O2', 'EMGc'], ['EMGl', 'EMGr', 'M1', 'M2', 'O2'], ['ELC', 'C4', 'O2', 'EMGc']], ['ERB', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc', 'M2', 'ELB', 'ELC', 'C4']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 2.0, 1, [['ERA', 'EMGc'], ['ERA'], ['ERA'], ['ERA'], ['ERA', 'EMGc'], ['ELA']], ['ERA', 'EMGc', 'ELA']], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 2.0, 1, [['ERA', 'EMGc'], ['ELC', 'ERA'], ['ERA', 'EMGc'], ['ELC', 'ERA'], ['ELC', 'ERA', 'EMGc'], ['ELA', 'ERA']], ['ERA', 'EMGc', 'ELC', 'ELA']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 2.0, 1, [['EMGl', 'O2'], [], [], ['EMGl'], ['M1'], []], ['EMGl', 'O2', 'M1']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 2.5, 1, [['ERA', 'EMGc'], ['ERA'], ['ERA', 'EMGc'], ['ERA'], ['ERA'], ['ELA', 'ERA']], ['ERA', 'EMGc', 'ELA']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 3.0, 1, [['ERA', 'EMGc'], ['ERA'], ['ERA', 'EMGc'], ['ERA'], ['ERA'], ['ELA', 'ERA']], ['ERA', 'EMGc', 'ELA']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 4.0, 1, [['ERA', 'EMGc'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ELA']], ['ERA', 'EMGc', 'ELA']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 5.0, 1, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ELA']], ['ERA', 'ELA']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 1.0, 2, [['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc']], ['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'O2', 'EMGc', 'F4']], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 1.0, 2, [['ELC', 'ERA', 'ERB', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl'], ['ELC', 'ERA', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGl', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGl', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'F3', 'C3', 'F4', 'C4']], ['ELC', 'ERA', 'ERB', 'EMGc', 'EOGl']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 1.0, 2, [['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELC', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'EMGc']], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'O2', 'EMGc', 'F4', 'M2']], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'O2'], ['ELB', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELB', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELB', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'M2', 'EMGc']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 1.5, 2, [['ERA', 'EMGc'], ['ELC', 'ERA', 'O1', 'O2'], ['ELC', 'ERA', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EMGc'], ['ELA', 'ERA']], ['ERA', 'EMGc', 'ELC', 'ERB']], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 1.5, 2, [['ERA', 'EMGc'], ['ELC', 'ERA'], ['ELC', 'ERA', 'EMGc'], ['ELC', 'ERA', 'ERB'], ['ELC', 'ERA', 'ERB', 'EMGc'], ['ELA', 'ERA']], ['ERA', 'ELC', 'ERB']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 1.5, 2, [['ELC', 'ERB', 'EMGr', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGl', 'O2'], ['EMGc'], ['ELC', 'ERB', 'O2', 'EMGc'], ['EMGc'], ['ERA']], ['ELC', 'ERB', 'O2', 'EMGc']], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 1.5, 2, [['ERB', 'EMGl', 'EMGr', 'M1', 'O1', 'O2'], ['ERB', 'M1', 'O1', 'O2', 'EMGc'], ['EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELB', 'EMGl', 'O1', 'M2', 'O2', 'EMGc'], ['EMGl', 'EMGr', 'M1', 'M2', 'O2'], ['ELC', 'C4', 'O2', 'EMGc']], ['ERB', 'EMGl', 'M1', 'O1', 'O2', 'M2']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 2.0, 2, [['ERA', 'EMGc'], ['ERA'], ['ERA'], ['ERA'], ['ERA', 'EMGc'], ['ELA']], ['ERA']], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 2.0, 2, [['ERA', 'EMGc'], ['ELC', 'ERA'], ['ERA', 'EMGc'], ['ELC', 'ERA'], ['ELC', 'ERA', 'EMGc'], ['ELA', 'ERA']], ['ERA', 'ELC']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 2.0, 2, [['EMGl', 'O2'], [], [], ['EMGl'], ['M1'], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 2.5, 2, [['ERA', 'EMGc'], ['ERA'], ['ERA', 'EMGc'], ['ERA'], ['ERA'], ['ELA', 'ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 3.0, 2, [['ERA', 'EMGc'], ['ERA'], ['ERA', 'EMGc'], ['ERA'], ['ERA'], ['ELA', 'ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 4.0, 2, [['ERA', 'EMGc'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ELA']], ['ERA']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 5.0, 2, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ELA']], ['ERA']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 1.0, 3, [['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc']], ['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'O1', 'O2', 'EMGc']], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 1.0, 3, [['ELC', 'ERA', 'ERB', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl'], ['ELC', 'ERA', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGl', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGl', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'F3', 'C3', 'F4', 'C4']], ['ELC', 'ERA', 'EMGc']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 1.0, 3, [['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELC', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'EMGc']], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'O2', 'EMGc', 'F4']], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'O2'], ['ELB', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELB', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELB', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'M2', 'EMGc']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 1.5, 3, [['ERA', 'EMGc'], ['ELC', 'ERA', 'O1', 'O2'], ['ELC', 'ERA', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EMGc'], ['ELA', 'ERA']], ['ERA', 'EMGc', 'ELC']], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 1.5, 3, [['ERA', 'EMGc'], ['ELC', 'ERA'], ['ELC', 'ERA', 'EMGc'], ['ELC', 'ERA', 'ERB'], ['ELC', 'ERA', 'ERB', 'EMGc'], ['ELA', 'ERA']], ['ERA', 'ELC']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 1.5, 3, [['ELC', 'ERB', 'EMGr', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGl', 'O2'], ['EMGc'], ['ELC', 'ERB', 'O2', 'EMGc'], ['EMGc'], ['ERA']], ['EMGc']], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 1.5, 3, [['ERB', 'EMGl', 'EMGr', 'M1', 'O1', 'O2'], ['ERB', 'M1', 'O1', 'O2', 'EMGc'], ['EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELB', 'EMGl', 'O1', 'M2', 'O2', 'EMGc'], ['EMGl', 'EMGr', 'M1', 'M2', 'O2'], ['ELC', 'C4', 'O2', 'EMGc']], ['EMGl', 'M1', 'O1', 'O2', 'M2']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 2.0, 3, [['ERA', 'EMGc'], ['ERA'], ['ERA'], ['ERA'], ['ERA', 'EMGc'], ['ELA']], ['ERA']], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 2.0, 3, [['ERA', 'EMGc'], ['ELC', 'ERA'], ['ERA', 'EMGc'], ['ELC', 'ERA'], ['ELC', 'ERA', 'EMGc'], ['ELA', 'ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 2.0, 3, [['EMGl', 'O2'], [], [], ['EMGl'], ['M1'], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 2.5, 3, [['ERA', 'EMGc'], ['ERA'], ['ERA', 'EMGc'], ['ERA'], ['ERA'], ['ELA', 'ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 3.0, 3, [['ERA', 'EMGc'], ['ERA'], ['ERA', 'EMGc'], ['ERA'], ['ERA'], ['ELA', 'ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 4.0, 3, [['ERA', 'EMGc'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ELA']], ['ERA']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 5.0, 3, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ELA']], ['ERA']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 1.0, 4, [['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc']], ['ELC', 'ERA', 'ERB', 'EOGl', 'EMGl', 'O1', 'O2', 'EMGc']], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 1.0, 4, [['ELC', 'ERA', 'ERB', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl'], ['ELC', 'ERA', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGl', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGl', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'F3', 'C3', 'F4', 'C4']], ['ELC', 'ERA']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 1.0, 4, [['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELC', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'EMGc']], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'O2', 'EMGc', 'F4']], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'O2'], ['ELB', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELB', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'M2']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 1.5, 4, [['ERA', 'EMGc'], ['ELC', 'ERA', 'O1', 'O2'], ['ELC', 'ERA', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EMGc'], ['ELA', 'ERA']], ['ERA', 'ELC']], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 1.5, 4, [['ERA', 'EMGc'], ['ELC', 'ERA'], ['ELC', 'ERA', 'EMGc'], ['ELC', 'ERA', 'ERB'], ['ELC', 'ERA', 'ERB', 'EMGc'], ['ELA', 'ERA']], ['ERA', 'ELC']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 1.5, 4, [['ELC', 'ERB', 'EMGr', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGl', 'O2'], ['EMGc'], ['ELC', 'ERB', 'O2', 'EMGc'], ['EMGc'], ['ERA']], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 1.5, 4, [['ERB', 'EMGl', 'EMGr', 'M1', 'O1', 'O2'], ['ERB', 'M1', 'O1', 'O2', 'EMGc'], ['EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELB', 'EMGl', 'O1', 'M2', 'O2', 'EMGc'], ['EMGl', 'EMGr', 'M1', 'M2', 'O2'], ['ELC', 'C4', 'O2', 'EMGc']], ['O1', 'O2']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 2.0, 4, [['ERA', 'EMGc'], ['ERA'], ['ERA'], ['ERA'], ['ERA', 'EMGc'], ['ELA']], ['ERA']], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 2.0, 4, [['ERA', 'EMGc'], ['ELC', 'ERA'], ['ERA', 'EMGc'], ['ELC', 'ERA'], ['ELC', 'ERA', 'EMGc'], ['ELA', 'ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 2.0, 4, [['EMGl', 'O2'], [], [], ['EMGl'], ['M1'], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 2.5, 4, [['ERA', 'EMGc'], ['ERA'], ['ERA', 'EMGc'], ['ERA'], ['ERA'], ['ELA', 'ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 3.0, 4, [['ERA', 'EMGc'], ['ERA'], ['ERA', 'EMGc'], ['ERA'], ['ERA'], ['ELA', 'ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 4.0, 4, [['ERA', 'EMGc'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ELA']], ['ERA']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 5.0, 4, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ELA']], ['ERA']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 1.0, 5, [['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc']], ['ELC', 'ERA', 'ERB', 'EOGl', 'EMGl', 'O1', 'O2', 'EMGc']], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 1.0, 5, [['ELC', 'ERA', 'ERB', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl'], ['ELC', 'ERA', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGl', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGl', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'F3', 'C3', 'F4', 'C4']], ['ELC', 'ERA']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 1.0, 5, [['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELC', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'EMGc']], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'O2', 'EMGc', 'F4']], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'O2'], ['ELB', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELB', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 1.5, 5, [['ERA', 'EMGc'], ['ELC', 'ERA', 'O1', 'O2'], ['ELC', 'ERA', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EMGc'], ['ELA', 'ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 1.5, 5, [['ERA', 'EMGc'], ['ELC', 'ERA'], ['ELC', 'ERA', 'EMGc'], ['ELC', 'ERA', 'ERB'], ['ELC', 'ERA', 'ERB', 'EMGc'], ['ELA', 'ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 1.5, 5, [['ELC', 'ERB', 'EMGr', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGl', 'O2'], ['EMGc'], ['ELC', 'ERB', 'O2', 'EMGc'], ['EMGc'], ['ERA']], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 1.5, 5, [['ERB', 'EMGl', 'EMGr', 'M1', 'O1', 'O2'], ['ERB', 'M1', 'O1', 'O2', 'EMGc'], ['EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELB', 'EMGl', 'O1', 'M2', 'O2', 'EMGc'], ['EMGl', 'EMGr', 'M1', 'M2', 'O2'], ['ELC', 'C4', 'O2', 'EMGc']], ['O2']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 2.0, 5, [['ERA', 'EMGc'], ['ERA'], ['ERA'], ['ERA'], ['ERA', 'EMGc'], ['ELA']], ['ERA']], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 2.0, 5, [['ERA', 'EMGc'], ['ELC', 'ERA'], ['ERA', 'EMGc'], ['ELC', 'ERA'], ['ELC', 'ERA', 'EMGc'], ['ELA', 'ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 2.0, 5, [['EMGl', 'O2'], [], [], ['EMGl'], ['M1'], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 2.5, 5, [['ERA', 'EMGc'], ['ERA'], ['ERA', 'EMGc'], ['ERA'], ['ERA'], ['ELA', 'ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 3.0, 5, [['ERA', 'EMGc'], ['ERA'], ['ERA', 'EMGc'], ['ERA'], ['ERA'], ['ELA', 'ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 4.0, 5, [['ERA', 'EMGc'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ELA']], ['ERA']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 5.0, 5, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ELA']], ['ERA']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 1.0, 6, [['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc']], ['ELC', 'ERA', 'ERB', 'EMGc']], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 1.0, 6, [['ELC', 'ERA', 'ERB', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl'], ['ELC', 'ERA', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGl', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EOGl', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'F3', 'C3', 'F4', 'C4']], ['ERA']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 1.0, 6, [['ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELC', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'EMGc']], ['ELC', 'ERB', 'EOGr', 'EOGl', 'F3', 'EMGc']], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'O2'], ['ELB', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELB', 'ERB', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ERB', 'EOGl', 'O1', 'O2']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 1.5, 6, [['ERA', 'EMGc'], ['ELC', 'ERA', 'O1', 'O2'], ['ELC', 'ERA', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EMGc'], ['ELC', 'ERA', 'ERB', 'EMGc'], ['ELA', 'ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 1.5, 6, [['ERA', 'EMGc'], ['ELC', 'ERA'], ['ELC', 'ERA', 'EMGc'], ['ELC', 'ERA', 'ERB'], ['ELC', 'ERA', 'ERB', 'EMGc'], ['ELA', 'ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 1.5, 6, [['ELC', 'ERB', 'EMGr', 'O2', 'EMGc'], ['ELC', 'ERB', 'EOGl', 'O2'], ['EMGc'], ['ELC', 'ERB', 'O2', 'EMGc'], ['EMGc'], ['ERA']], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 1.5, 6, [['ERB', 'EMGl', 'EMGr', 'M1', 'O1', 'O2'], ['ERB', 'M1', 'O1', 'O2', 'EMGc'], ['EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2'], ['ELB', 'EMGl', 'O1', 'M2', 'O2', 'EMGc'], ['EMGl', 'EMGr', 'M1', 'M2', 'O2'], ['ELC', 'C4', 'O2', 'EMGc']], ['O2']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 2.0, 6, [['ERA', 'EMGc'], ['ERA'], ['ERA'], ['ERA'], ['ERA', 'EMGc'], ['ELA']], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 2.0, 6, [['ERA', 'EMGc'], ['ELC', 'ERA'], ['ERA', 'EMGc'], ['ELC', 'ERA'], ['ELC', 'ERA', 'EMGc'], ['ELA', 'ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 2.0, 6, [['EMGl', 'O2'], [], [], ['EMGl'], ['M1'], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 2.5, 6, [['ERA', 'EMGc'], ['ERA'], ['ERA', 'EMGc'], ['ERA'], ['ERA'], ['ELA', 'ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 3.0, 6, [['ERA', 'EMGc'], ['ERA'], ['ERA', 'EMGc'], ['ERA'], ['ERA'], ['ELA', 'ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 4.0, 6, [['ERA', 'EMGc'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ELA']], []], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'mm', 5.0, 6, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ELA']], []], ['S_02/night_3\\EEG_raw', 120, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 120, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELC', 'ERE', 'M1', 'M2', 'ERB']], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 1.0, 1, [['ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ERA', 'ERE']], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 1.0, 1, [['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ERE']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 1.5, 1, [['ERA', 'F3'], ['ERA', 'O2'], ['ERA', 'C3'], ['ERA'], ['ERA', 'F4'], ['ERA']], ['ERA', 'F3', 'O2', 'C3', 'F4']], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 1.5, 1, [['ERA', 'F3', 'F4', 'O2'], ['ERA', 'O1', 'F4', 'O2'], ['ERA', 'F3', 'O1', 'F4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'F4', 'O2'], ['ERA', 'F3', 'O1', 'O2']], ['ERA', 'F3', 'F4', 'O2', 'O1', 'C3', 'C4']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 1.5, 1, [[], ['O1'], [], [], ['C3'], []], ['O1', 'C3']], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 1.5, 1, [['EOGr', 'M1'], ['C3', 'O1', 'C4', 'O2'], ['C4', 'O2', 'EMGc'], ['EOGr', 'M1', 'C3'], ['C3', 'C4'], ['M1']], ['EOGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 2.0, 1, [[], [], [], ['ERA'], [], []], ['ERA']], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 2.0, 1, [['ERA', 'O2'], ['ERA', 'O2'], ['ERA', 'F3'], ['ERA', 'F4'], ['ERA'], ['ERA']], ['ERA', 'O2', 'F3', 'F4']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 2.5, 1, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 3.0, 1, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 4.0, 1, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 5.0, 1, [[], ['ERA'], [], ['ERA'], [], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELC', 'M1']], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 1.0, 2, [['ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 1.0, 2, [['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 1.5, 2, [['ERA', 'F3'], ['ERA', 'O2'], ['ERA', 'C3'], ['ERA'], ['ERA', 'F4'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 1.5, 2, [['ERA', 'F3', 'F4', 'O2'], ['ERA', 'O1', 'F4', 'O2'], ['ERA', 'F3', 'O1', 'F4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'F4', 'O2'], ['ERA', 'F3', 'O1', 'O2']], ['ERA', 'F3', 'F4', 'O2', 'O1']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 1.5, 2, [[], ['O1'], [], [], ['C3'], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 1.5, 2, [['EOGr', 'M1'], ['C3', 'O1', 'C4', 'O2'], ['C4', 'O2', 'EMGc'], ['EOGr', 'M1', 'C3'], ['C3', 'C4'], ['M1']], ['C3', 'C4', 'O2']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 2.0, 2, [[], [], [], ['ERA'], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 2.0, 2, [['ERA', 'O2'], ['ERA', 'O2'], ['ERA', 'F3'], ['ERA', 'F4'], ['ERA'], ['ERA']], ['ERA', 'O2']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 2.5, 2, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 3.0, 2, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 4.0, 2, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 5.0, 2, [[], ['ERA'], [], ['ERA'], [], ['ERA']], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'M1']], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 1.0, 3, [['ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 1.0, 3, [['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 1.5, 3, [['ERA', 'F3'], ['ERA', 'O2'], ['ERA', 'C3'], ['ERA'], ['ERA', 'F4'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 1.5, 3, [['ERA', 'F3', 'F4', 'O2'], ['ERA', 'O1', 'F4', 'O2'], ['ERA', 'F3', 'O1', 'F4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'F4', 'O2'], ['ERA', 'F3', 'O1', 'O2']], ['ERA', 'F3', 'F4', 'O2', 'O1']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 1.5, 3, [[], ['O1'], [], [], ['C3'], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 1.5, 3, [['EOGr', 'M1'], ['C3', 'O1', 'C4', 'O2'], ['C4', 'O2', 'EMGc'], ['EOGr', 'M1', 'C3'], ['C3', 'C4'], ['M1']], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 2.0, 3, [[], [], [], ['ERA'], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 2.0, 3, [['ERA', 'O2'], ['ERA', 'O2'], ['ERA', 'F3'], ['ERA', 'F4'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 2.5, 3, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 3.0, 3, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 4.0, 3, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 5.0, 3, [[], ['ERA'], [], ['ERA'], [], ['ERA']], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 1.0, 4, [['ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 1.0, 4, [['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 1.5, 4, [['ERA', 'F3'], ['ERA', 'O2'], ['ERA', 'C3'], ['ERA'], ['ERA', 'F4'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 1.5, 4, [['ERA', 'F3', 'F4', 'O2'], ['ERA', 'O1', 'F4', 'O2'], ['ERA', 'F3', 'O1', 'F4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'F4', 'O2'], ['ERA', 'F3', 'O1', 'O2']], ['ERA', 'F3', 'F4', 'O2']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 1.5, 4, [[], ['O1'], [], [], ['C3'], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 1.5, 4, [['EOGr', 'M1'], ['C3', 'O1', 'C4', 'O2'], ['C4', 'O2', 'EMGc'], ['EOGr', 'M1', 'C3'], ['C3', 'C4'], ['M1']], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 2.0, 4, [[], [], [], ['ERA'], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 2.0, 4, [['ERA', 'O2'], ['ERA', 'O2'], ['ERA', 'F3'], ['ERA', 'F4'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 2.5, 4, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 3.0, 4, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 4.0, 4, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 5.0, 4, [[], ['ERA'], [], ['ERA'], [], ['ERA']], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 1.0, 5, [['ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 1.0, 5, [['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 1.5, 5, [['ERA', 'F3'], ['ERA', 'O2'], ['ERA', 'C3'], ['ERA'], ['ERA', 'F4'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 1.5, 5, [['ERA', 'F3', 'F4', 'O2'], ['ERA', 'O1', 'F4', 'O2'], ['ERA', 'F3', 'O1', 'F4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'F4', 'O2'], ['ERA', 'F3', 'O1', 'O2']], ['ERA', 'F4', 'O2']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 1.5, 5, [[], ['O1'], [], [], ['C3'], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 1.5, 5, [['EOGr', 'M1'], ['C3', 'O1', 'C4', 'O2'], ['C4', 'O2', 'EMGc'], ['EOGr', 'M1', 'C3'], ['C3', 'C4'], ['M1']], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 2.0, 5, [[], [], [], ['ERA'], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 2.0, 5, [['ERA', 'O2'], ['ERA', 'O2'], ['ERA', 'F3'], ['ERA', 'F4'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 2.5, 5, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 3.0, 5, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 4.0, 5, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 5.0, 5, [[], ['ERA'], [], ['ERA'], [], ['ERA']], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 1.0, 6, [['ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 1.0, 6, [['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 1.5, 6, [['ERA', 'F3'], ['ERA', 'O2'], ['ERA', 'C3'], ['ERA'], ['ERA', 'F4'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 1.5, 6, [['ERA', 'F3', 'F4', 'O2'], ['ERA', 'O1', 'F4', 'O2'], ['ERA', 'F3', 'O1', 'F4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'F4', 'O2'], ['ERA', 'F3', 'O1', 'O2']], ['ERA', 'O2']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 1.5, 6, [[], ['O1'], [], [], ['C3'], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 1.5, 6, [['EOGr', 'M1'], ['C3', 'O1', 'C4', 'O2'], ['C4', 'O2', 'EMGc'], ['EOGr', 'M1', 'C3'], ['C3', 'C4'], ['M1']], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 2.0, 6, [[], [], [], ['ERA'], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 2.0, 6, [['ERA', 'O2'], ['ERA', 'O2'], ['ERA', 'F3'], ['ERA', 'F4'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 2.5, 6, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 3.0, 6, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 4.0, 6, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'mm', 5.0, 6, [[], ['ERA'], [], ['ERA'], [], ['ERA']], []], ['S_02/night_3\\EEG_raw', 4880, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 4880, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELC', 'ERA', 'ERE', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGr', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELC', 'ERA', 'ERE', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc', 'EOGr', 'M1', 'C4', 'ERI', 'EMGl', 'M2']], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 1.0, 1, [['ERA', 'F3', 'F4'], ['ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'ERI', 'C4'], ['ELA', 'ERA', 'EOGl', 'F3', 'C3', 'O1', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'O2']], ['ERA', 'F3', 'F4', 'EOGl', 'C3', 'O1', 'C4', 'O2', 'ELA', 'ERE', 'ERI']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc', 'ERC', 'EOGr', 'M1', 'C4', 'ERI', 'EMGl', 'M2']], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 1.0, 1, [['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'EOGr', 'EMGl', 'C3', 'EMGr', 'M2']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 1.5, 1, [['ERA'], ['ERA'], ['ELA', 'ERA', 'ERE', 'ERI'], ['ERA', 'O1', 'O2'], ['ERA'], ['ERA']], ['ERA', 'ELA', 'ERE', 'ERI', 'O1', 'O2']], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 1.5, 1, [['ERA', 'F4'], ['ERA', 'F3'], ['ELA', 'ERA', 'ERE', 'ERI'], ['ERA', 'O1', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'O2'], ['ERA', 'F3', 'O1', 'F4']], ['ERA', 'F4', 'F3', 'ELA', 'ERE', 'ERI', 'O1', 'O2', 'C3']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 1.5, 1, [['ELA'], ['F3'], ['ELA'], [], ['F3', 'C3'], ['F3', 'C3']], ['ELA', 'F3', 'C3']], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 1.5, 1, [['ELA', 'ELC', 'ERB', 'ERE', 'EMGc'], ['ELC', 'ERE'], ['EOGl', 'C4'], ['EOGl', 'M1', 'EMGc'], ['ELA', 'M1', 'C3', 'EMGc'], ['ELA', 'ERE', 'F3', 'C3']], ['ELA', 'ELC', 'ERB', 'ERE', 'EMGc', 'EOGl', 'C4', 'M1', 'C3', 'F3']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 2.0, 1, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 2.0, 1, [['ERA'], ['ERA'], ['ELA', 'ERA'], ['ERA', 'O1', 'O2'], ['ERA'], ['ERA', 'F3']], ['ERA', 'ELA', 'O1', 'O2', 'F3']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 2.0, 1, [[], ['ELC'], [], [], ['EMGc'], []], ['ELC', 'EMGc']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 2.5, 1, [[], [], [], ['ERA'], [], []], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 2.5, 1, [['ERA'], ['ERA'], ['ERA'], ['ERA', 'O2'], ['ERA'], ['ERA', 'F3']], ['ERA', 'O2', 'F3']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 3.0, 1, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 4.0, 1, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 5.0, 1, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELC', 'ERA', 'ERE', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGr', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELC', 'ERA', 'ERE', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc', 'M1', 'C4']], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 1.0, 2, [['ERA', 'F3', 'F4'], ['ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'ERI', 'C4'], ['ELA', 'ERA', 'EOGl', 'F3', 'C3', 'O1', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'O2']], ['ERA', 'F3', 'F4', 'C3', 'O1', 'C4', 'O2', 'ELA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc', 'EOGr', 'M1', 'C4', 'EMGl', 'M2']], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 1.0, 2, [['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERE', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'EOGr', 'EMGl', 'C3', 'EMGr', 'M2']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 1.5, 2, [['ERA'], ['ERA'], ['ELA', 'ERA', 'ERE', 'ERI'], ['ERA', 'O1', 'O2'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 1.5, 2, [['ERA', 'F4'], ['ERA', 'F3'], ['ELA', 'ERA', 'ERE', 'ERI'], ['ERA', 'O1', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'O2'], ['ERA', 'F3', 'O1', 'F4']], ['ERA', 'F3', 'O1', 'O2']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 1.5, 2, [['ELA'], ['F3'], ['ELA'], [], ['F3', 'C3'], ['F3', 'C3']], ['F3', 'C3']], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 1.5, 2, [['ELA', 'ELC', 'ERB', 'ERE', 'EMGc'], ['ELC', 'ERE'], ['EOGl', 'C4'], ['EOGl', 'M1', 'EMGc'], ['ELA', 'M1', 'C3', 'EMGc'], ['ELA', 'ERE', 'F3', 'C3']], ['ELA', 'ELC', 'ERE', 'EMGc', 'EOGl', 'M1', 'C3']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 2.0, 2, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 2.0, 2, [['ERA'], ['ERA'], ['ELA', 'ERA'], ['ERA', 'O1', 'O2'], ['ERA'], ['ERA', 'F3']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 2.0, 2, [[], ['ELC'], [], [], ['EMGc'], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 2.5, 2, [[], [], [], ['ERA'], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 2.5, 2, [['ERA'], ['ERA'], ['ERA'], ['ERA', 'O2'], ['ERA'], ['ERA', 'F3']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 3.0, 2, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 4.0, 2, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 5.0, 2, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELC', 'ERA', 'ERE', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGr', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ERA', 'ERE', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc', 'M1']], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 1.0, 3, [['ERA', 'F3', 'F4'], ['ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'ERI', 'C4'], ['ELA', 'ERA', 'EOGl', 'F3', 'C3', 'O1', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'O2']], ['ERA', 'F3', 'C3', 'O1', 'O2']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc', 'M1', 'C4', 'EMGl', 'M2']], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 1.0, 3, [['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERE', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'EMGl', 'C3', 'EMGr', 'M2']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 1.5, 3, [['ERA'], ['ERA'], ['ELA', 'ERA', 'ERE', 'ERI'], ['ERA', 'O1', 'O2'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 1.5, 3, [['ERA', 'F4'], ['ERA', 'F3'], ['ELA', 'ERA', 'ERE', 'ERI'], ['ERA', 'O1', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'O2'], ['ERA', 'F3', 'O1', 'F4']], ['ERA', 'O1']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 1.5, 3, [['ELA'], ['F3'], ['ELA'], [], ['F3', 'C3'], ['F3', 'C3']], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 1.5, 3, [['ELA', 'ELC', 'ERB', 'ERE', 'EMGc'], ['ELC', 'ERE'], ['EOGl', 'C4'], ['EOGl', 'M1', 'EMGc'], ['ELA', 'M1', 'C3', 'EMGc'], ['ELA', 'ERE', 'F3', 'C3']], []], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 2.0, 3, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 2.0, 3, [['ERA'], ['ERA'], ['ELA', 'ERA'], ['ERA', 'O1', 'O2'], ['ERA'], ['ERA', 'F3']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 2.0, 3, [[], ['ELC'], [], [], ['EMGc'], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 2.5, 3, [[], [], [], ['ERA'], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 2.5, 3, [['ERA'], ['ERA'], ['ERA'], ['ERA', 'O2'], ['ERA'], ['ERA', 'F3']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 3.0, 3, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 4.0, 3, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 5.0, 3, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELC', 'ERA', 'ERE', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGr', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'EOGl', 'O2']], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 1.0, 4, [['ERA', 'F3', 'F4'], ['ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'ERI', 'C4'], ['ELA', 'ERA', 'EOGl', 'F3', 'C3', 'O1', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'O2']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERE', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc', 'C4', 'EMGl']], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 1.0, 4, [['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc', 'EMGl', 'C3', 'EMGr']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 1.5, 4, [['ERA'], ['ERA'], ['ELA', 'ERA', 'ERE', 'ERI'], ['ERA', 'O1', 'O2'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 1.5, 4, [['ERA', 'F4'], ['ERA', 'F3'], ['ELA', 'ERA', 'ERE', 'ERI'], ['ERA', 'O1', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'O2'], ['ERA', 'F3', 'O1', 'F4']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 1.5, 4, [['ELA'], ['F3'], ['ELA'], [], ['F3', 'C3'], ['F3', 'C3']], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 1.5, 4, [['ELA', 'ELC', 'ERB', 'ERE', 'EMGc'], ['ELC', 'ERE'], ['EOGl', 'C4'], ['EOGl', 'M1', 'EMGc'], ['ELA', 'M1', 'C3', 'EMGc'], ['ELA', 'ERE', 'F3', 'C3']], []], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 2.0, 4, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 2.0, 4, [['ERA'], ['ERA'], ['ELA', 'ERA'], ['ERA', 'O1', 'O2'], ['ERA'], ['ERA', 'F3']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 2.0, 4, [[], ['ELC'], [], [], ['EMGc'], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 2.5, 4, [[], [], [], ['ERA'], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 2.5, 4, [['ERA'], ['ERA'], ['ERA'], ['ERA', 'O2'], ['ERA'], ['ERA', 'F3']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 3.0, 4, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 4.0, 4, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 5.0, 4, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELC', 'ERA', 'ERE', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGr', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'EOGl', 'O2']], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 1.0, 5, [['ERA', 'F3', 'F4'], ['ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'ERI', 'C4'], ['ELA', 'ERA', 'EOGl', 'F3', 'C3', 'O1', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'O2']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERE', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc', 'C4']], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 1.0, 5, [['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc', 'EMGl', 'C3']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 1.5, 5, [['ERA'], ['ERA'], ['ELA', 'ERA', 'ERE', 'ERI'], ['ERA', 'O1', 'O2'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 1.5, 5, [['ERA', 'F4'], ['ERA', 'F3'], ['ELA', 'ERA', 'ERE', 'ERI'], ['ERA', 'O1', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'O2'], ['ERA', 'F3', 'O1', 'F4']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 1.5, 5, [['ELA'], ['F3'], ['ELA'], [], ['F3', 'C3'], ['F3', 'C3']], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 1.5, 5, [['ELA', 'ELC', 'ERB', 'ERE', 'EMGc'], ['ELC', 'ERE'], ['EOGl', 'C4'], ['EOGl', 'M1', 'EMGc'], ['ELA', 'M1', 'C3', 'EMGc'], ['ELA', 'ERE', 'F3', 'C3']], []], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 2.0, 5, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 2.0, 5, [['ERA'], ['ERA'], ['ELA', 'ERA'], ['ERA', 'O1', 'O2'], ['ERA'], ['ERA', 'F3']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 2.0, 5, [[], ['ELC'], [], [], ['EMGc'], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 2.5, 5, [[], [], [], ['ERA'], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 2.5, 5, [['ERA'], ['ERA'], ['ERA'], ['ERA', 'O2'], ['ERA'], ['ERA', 'F3']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 3.0, 5, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 4.0, 5, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 5.0, 5, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELC', 'ERA', 'ERE', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'ERI', 'EOGl', 'EMGr', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERE', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERE', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'O2']], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 1.0, 6, [['ERA', 'F3', 'F4'], ['ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'ERI', 'C4'], ['ELA', 'ERA', 'EOGl', 'F3', 'C3', 'O1', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'F4', 'O2']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ERE', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc']], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 1.0, 6, [['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERB', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERB', 'ERE', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 1.5, 6, [['ERA'], ['ERA'], ['ELA', 'ERA', 'ERE', 'ERI'], ['ERA', 'O1', 'O2'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 1.5, 6, [['ERA', 'F4'], ['ERA', 'F3'], ['ELA', 'ERA', 'ERE', 'ERI'], ['ERA', 'O1', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'O2'], ['ERA', 'F3', 'O1', 'F4']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 1.5, 6, [['ELA'], ['F3'], ['ELA'], [], ['F3', 'C3'], ['F3', 'C3']], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 1.5, 6, [['ELA', 'ELC', 'ERB', 'ERE', 'EMGc'], ['ELC', 'ERE'], ['EOGl', 'C4'], ['EOGl', 'M1', 'EMGc'], ['ELA', 'M1', 'C3', 'EMGc'], ['ELA', 'ERE', 'F3', 'C3']], []], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 2.0, 6, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 2.0, 6, [['ERA'], ['ERA'], ['ELA', 'ERA'], ['ERA', 'O1', 'O2'], ['ERA'], ['ERA', 'F3']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 2.0, 6, [[], ['ELC'], [], [], ['EMGc'], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 2.5, 6, [[], [], [], ['ERA'], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 2.5, 6, [['ERA'], ['ERA'], ['ERA'], ['ERA', 'O2'], ['ERA'], ['ERA', 'F3']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 3.0, 6, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 4.0, 6, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'mm', 5.0, 6, [['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA'], ['ERA']], ['ERA']], ['S_02/night_3\\EEG_raw', 12210, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_02/night_3\\EEG_raw', 12210, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'O2', 'EMGc']], ['ELA', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc', 'ELC', 'M1', 'M2', 'ERE']], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELC', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc']], ['ELA', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'ELC', 'ERC', 'M2', 'O1', 'O2']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc']], ['ELA', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc', 'ELC', 'ELE', 'M2', 'ERA', 'ERB', 'ERE']], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 1.0, 1, [['ELA', 'ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2'], ['ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'O2', 'EMGc']], ['ELA', 'ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'O2', 'EMGc', 'ELE', 'F4', 'ERA', 'ERE', 'F3']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 1.5, 1, [['ELI', 'EOGr', 'EMGl', 'EMGr'], ['ELC', 'EMGl', 'EMGr'], ['EOGl', 'EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['ERC', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr']], ['ELI', 'EOGr', 'EMGl', 'EMGr', 'ELC', 'EOGl', 'ERC', 'EMGc']], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 1.5, 1, [['ELI', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'EMGl', 'EMGr', 'EMGc'], ['EMGl', 'EMGr', 'EMGc'], ['ELC', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr']], ['ELI', 'EMGl', 'EMGr', 'EMGc', 'ELC', 'ERC', 'EOGr', 'EOGl']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 1.5, 1, [['ELI', 'EOGr'], ['ELC', 'ERC'], ['ELI', 'ERC', 'M2', 'EMGc'], ['ELC', 'ERC', 'EMGl'], ['ELI', 'ERC', 'EOGl', 'EMGc'], ['EMGl', 'O1']], ['ELI', 'EOGr', 'ELC', 'ERC', 'M2', 'EMGc', 'EMGl', 'EOGl', 'O1']], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 1.5, 1, [['ELA'], ['ELE', 'ELI', 'ERC', 'M1', 'O1'], ['ELC', 'ELI', 'M1', 'M2'], ['ELC', 'ELI', 'ERC', 'EOGr'], ['ELC', 'ELI', 'M1', 'O1', 'M2', 'O2'], ['ELI', 'M1', 'O1', 'O2']], ['ELA', 'ELE', 'ELI', 'ERC', 'M1', 'O1', 'ELC', 'M2', 'EOGr', 'O2']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 2.0, 1, [[], ['EMGr'], ['EMGr'], ['EMGr'], ['EMGr'], []], ['EMGr']], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 2.0, 1, [['EMGr'], ['ELC', 'EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['ERC', 'EMGl', 'EMGr'], ['EOGl', 'EMGr']], ['EMGr', 'ELC', 'EMGl', 'ERC', 'EOGl']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 2.5, 1, [[], [], ['EMGr'], [], [], []], ['EMGr']], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 2.5, 1, [['EMGr'], ['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['EMGr']], ['EMGr', 'EMGl']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 3.0, 1, [['EMGr'], ['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['EMGr'], ['EMGl', 'EMGr'], ['EMGr']], ['EMGr', 'EMGl']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 4.0, 1, [['EMGr'], ['EMGr'], ['EMGr'], ['EMGr'], ['EMGr'], []], ['EMGr']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 5.0, 1, [['EMGr'], ['EMGr'], ['EMGr'], ['EMGr'], [], []], ['EMGr']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'O2', 'EMGc']], ['ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'EMGc', 'ELC', 'M1', 'M2']], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELC', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc']], ['ELI', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'ERC']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc']], ['ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'O2', 'EMGc', 'ELC', 'M2']], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 1.0, 2, [['ELA', 'ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2'], ['ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'O2', 'EMGc']], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc', 'ERA']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 1.5, 2, [['ELI', 'EOGr', 'EMGl', 'EMGr'], ['ELC', 'EMGl', 'EMGr'], ['EOGl', 'EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['ERC', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr']], ['EMGl', 'EMGr']], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 1.5, 2, [['ELI', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'EMGl', 'EMGr', 'EMGc'], ['EMGl', 'EMGr', 'EMGc'], ['ELC', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr']], ['EMGl', 'EMGr', 'EMGc']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 1.5, 2, [['ELI', 'EOGr'], ['ELC', 'ERC'], ['ELI', 'ERC', 'M2', 'EMGc'], ['ELC', 'ERC', 'EMGl'], ['ELI', 'ERC', 'EOGl', 'EMGc'], ['EMGl', 'O1']], ['ERC']], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 1.5, 2, [['ELA'], ['ELE', 'ELI', 'ERC', 'M1', 'O1'], ['ELC', 'ELI', 'M1', 'M2'], ['ELC', 'ELI', 'ERC', 'EOGr'], ['ELC', 'ELI', 'M1', 'O1', 'M2', 'O2'], ['ELI', 'M1', 'O1', 'O2']], ['ELI', 'M1', 'O1', 'ELC', 'O2']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 2.0, 2, [[], ['EMGr'], ['EMGr'], ['EMGr'], ['EMGr'], []], ['EMGr']], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 2.0, 2, [['EMGr'], ['ELC', 'EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['ERC', 'EMGl', 'EMGr'], ['EOGl', 'EMGr']], ['EMGr', 'EMGl']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 2.5, 2, [[], [], ['EMGr'], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 2.5, 2, [['EMGr'], ['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['EMGr']], ['EMGr', 'EMGl']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 3.0, 2, [['EMGr'], ['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['EMGr'], ['EMGl', 'EMGr'], ['EMGr']], ['EMGr', 'EMGl']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 4.0, 2, [['EMGr'], ['EMGr'], ['EMGr'], ['EMGr'], ['EMGr'], []], ['EMGr']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 5.0, 2, [['EMGr'], ['EMGr'], ['EMGr'], ['EMGr'], [], []], ['EMGr']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'O2', 'EMGc']], ['ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'EMGc', 'ELC', 'M1', 'M2']], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELC', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc']], ['EOGl', 'EMGl', 'EMGr', 'EMGc', 'ERC']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc']], ['ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'O2', 'EMGc', 'ELC', 'M2']], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 1.0, 3, [['ELA', 'ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2'], ['ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'O2', 'EMGc']], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 1.5, 3, [['ELI', 'EOGr', 'EMGl', 'EMGr'], ['ELC', 'EMGl', 'EMGr'], ['EOGl', 'EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['ERC', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr']], ['EMGl', 'EMGr']], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 1.5, 3, [['ELI', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'EMGl', 'EMGr', 'EMGc'], ['EMGl', 'EMGr', 'EMGc'], ['ELC', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr']], ['EMGl', 'EMGr', 'EMGc']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 1.5, 3, [['ELI', 'EOGr'], ['ELC', 'ERC'], ['ELI', 'ERC', 'M2', 'EMGc'], ['ELC', 'ERC', 'EMGl'], ['ELI', 'ERC', 'EOGl', 'EMGc'], ['EMGl', 'O1']], ['ERC']], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 1.5, 3, [['ELA'], ['ELE', 'ELI', 'ERC', 'M1', 'O1'], ['ELC', 'ELI', 'M1', 'M2'], ['ELC', 'ELI', 'ERC', 'EOGr'], ['ELC', 'ELI', 'M1', 'O1', 'M2', 'O2'], ['ELI', 'M1', 'O1', 'O2']], ['ELI', 'ELC']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 2.0, 3, [[], ['EMGr'], ['EMGr'], ['EMGr'], ['EMGr'], []], ['EMGr']], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 2.0, 3, [['EMGr'], ['ELC', 'EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['ERC', 'EMGl', 'EMGr'], ['EOGl', 'EMGr']], ['EMGr', 'EMGl']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 2.5, 3, [[], [], ['EMGr'], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 2.5, 3, [['EMGr'], ['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['EMGr']], ['EMGr', 'EMGl']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 3.0, 3, [['EMGr'], ['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['EMGr'], ['EMGl', 'EMGr'], ['EMGr']], ['EMGr']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 4.0, 3, [['EMGr'], ['EMGr'], ['EMGr'], ['EMGr'], ['EMGr'], []], ['EMGr']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 5.0, 3, [['EMGr'], ['EMGr'], ['EMGr'], ['EMGr'], [], []], ['EMGr']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'O2', 'EMGc']], ['ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'M2']], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELC', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc']], ['EOGl', 'EMGl', 'EMGr', 'EMGc']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc']], ['ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'EMGc', 'ELC', 'M2']], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 1.0, 4, [['ELA', 'ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2'], ['ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'O2', 'EMGc']], ['ELI', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 1.5, 4, [['ELI', 'EOGr', 'EMGl', 'EMGr'], ['ELC', 'EMGl', 'EMGr'], ['EOGl', 'EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['ERC', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr']], ['EMGl', 'EMGr']], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 1.5, 4, [['ELI', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'EMGl', 'EMGr', 'EMGc'], ['EMGl', 'EMGr', 'EMGc'], ['ELC', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr']], ['EMGl', 'EMGr', 'EMGc']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 1.5, 4, [['ELI', 'EOGr'], ['ELC', 'ERC'], ['ELI', 'ERC', 'M2', 'EMGc'], ['ELC', 'ERC', 'EMGl'], ['ELI', 'ERC', 'EOGl', 'EMGc'], ['EMGl', 'O1']], ['ERC']], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 1.5, 4, [['ELA'], ['ELE', 'ELI', 'ERC', 'M1', 'O1'], ['ELC', 'ELI', 'M1', 'M2'], ['ELC', 'ELI', 'ERC', 'EOGr'], ['ELC', 'ELI', 'M1', 'O1', 'M2', 'O2'], ['ELI', 'M1', 'O1', 'O2']], ['ELI']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 2.0, 4, [[], ['EMGr'], ['EMGr'], ['EMGr'], ['EMGr'], []], ['EMGr']], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 2.0, 4, [['EMGr'], ['ELC', 'EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['ERC', 'EMGl', 'EMGr'], ['EOGl', 'EMGr']], ['EMGr', 'EMGl']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 2.5, 4, [[], [], ['EMGr'], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 2.5, 4, [['EMGr'], ['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['EMGr']], ['EMGr', 'EMGl']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 3.0, 4, [['EMGr'], ['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['EMGr'], ['EMGl', 'EMGr'], ['EMGr']], ['EMGr']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 4.0, 4, [['EMGr'], ['EMGr'], ['EMGr'], ['EMGr'], ['EMGr'], []], ['EMGr']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 5.0, 4, [['EMGr'], ['EMGr'], ['EMGr'], ['EMGr'], [], []], ['EMGr']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'O2', 'EMGc']], ['ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELC', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc']], ['EOGl', 'EMGl', 'EMGr', 'EMGc']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc']], ['ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'EMGc']], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 1.0, 5, [['ELA', 'ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2'], ['ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'O2', 'EMGc']], ['ELI', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 1.5, 5, [['ELI', 'EOGr', 'EMGl', 'EMGr'], ['ELC', 'EMGl', 'EMGr'], ['EOGl', 'EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['ERC', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr']], ['EMGl', 'EMGr']], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 1.5, 5, [['ELI', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'EMGl', 'EMGr', 'EMGc'], ['EMGl', 'EMGr', 'EMGc'], ['ELC', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr']], ['EMGl', 'EMGr', 'EMGc']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 1.5, 5, [['ELI', 'EOGr'], ['ELC', 'ERC'], ['ELI', 'ERC', 'M2', 'EMGc'], ['ELC', 'ERC', 'EMGl'], ['ELI', 'ERC', 'EOGl', 'EMGc'], ['EMGl', 'O1']], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 1.5, 5, [['ELA'], ['ELE', 'ELI', 'ERC', 'M1', 'O1'], ['ELC', 'ELI', 'M1', 'M2'], ['ELC', 'ELI', 'ERC', 'EOGr'], ['ELC', 'ELI', 'M1', 'O1', 'M2', 'O2'], ['ELI', 'M1', 'O1', 'O2']], ['ELI']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 2.0, 5, [[], ['EMGr'], ['EMGr'], ['EMGr'], ['EMGr'], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 2.0, 5, [['EMGr'], ['ELC', 'EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['ERC', 'EMGl', 'EMGr'], ['EOGl', 'EMGr']], ['EMGr']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 2.5, 5, [[], [], ['EMGr'], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 2.5, 5, [['EMGr'], ['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['EMGr']], ['EMGr']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 3.0, 5, [['EMGr'], ['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['EMGr'], ['EMGl', 'EMGr'], ['EMGr']], ['EMGr']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 4.0, 5, [['EMGr'], ['EMGr'], ['EMGr'], ['EMGr'], ['EMGr'], []], ['EMGr']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 5.0, 5, [['EMGr'], ['EMGr'], ['EMGr'], ['EMGr'], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'EMGc'], ['ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'F4', 'O2', 'EMGc']], ['ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELC', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc']], ['EOGl', 'EMGl', 'EMGr', 'EMGc']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc']], ['ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'EMGc']], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 1.0, 6, [['ELA', 'ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2'], ['ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'O1', 'O2', 'EMGc']], ['ELI', 'ERC', 'EOGr', 'EOGl', 'M1', 'O1', 'O2']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 1.5, 6, [['ELI', 'EOGr', 'EMGl', 'EMGr'], ['ELC', 'EMGl', 'EMGr'], ['EOGl', 'EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['ERC', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr']], ['EMGl', 'EMGr']], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 1.5, 6, [['ELI', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'EMGl', 'EMGr', 'EMGc'], ['EMGl', 'EMGr', 'EMGc'], ['ELC', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr']], ['EMGl', 'EMGr']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 1.5, 6, [['ELI', 'EOGr'], ['ELC', 'ERC'], ['ELI', 'ERC', 'M2', 'EMGc'], ['ELC', 'ERC', 'EMGl'], ['ELI', 'ERC', 'EOGl', 'EMGc'], ['EMGl', 'O1']], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 1.5, 6, [['ELA'], ['ELE', 'ELI', 'ERC', 'M1', 'O1'], ['ELC', 'ELI', 'M1', 'M2'], ['ELC', 'ELI', 'ERC', 'EOGr'], ['ELC', 'ELI', 'M1', 'O1', 'M2', 'O2'], ['ELI', 'M1', 'O1', 'O2']], []], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 2.0, 6, [[], ['EMGr'], ['EMGr'], ['EMGr'], ['EMGr'], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 2.0, 6, [['EMGr'], ['ELC', 'EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['ERC', 'EMGl', 'EMGr'], ['EOGl', 'EMGr']], ['EMGr']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 2.5, 6, [[], [], ['EMGr'], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 2.5, 6, [['EMGr'], ['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['EMGr']], ['EMGr']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 3.0, 6, [['EMGr'], ['EMGl', 'EMGr'], ['EMGl', 'EMGr'], ['EMGr'], ['EMGl', 'EMGr'], ['EMGr']], ['EMGr']], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 4.0, 6, [['EMGr'], ['EMGr'], ['EMGr'], ['EMGr'], ['EMGr'], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'mm', 5.0, 6, [['EMGr'], ['EMGr'], ['EMGr'], ['EMGr'], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 1.0, 1, [['EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ERA', 'ERI', 'EOGl']], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 1.0, 1, [['ERA', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'F4'], ['EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGl']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 1.0, 1, [['ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ERB', 'EOGl', 'EMGc', 'ERC']], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 1.0, 1, [['ELC', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELC', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4'], ['ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2'], ['ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2']], ['ELC', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'O2', 'ERB', 'ERC', 'ELE', 'ERT ', 'C4', 'EMGc']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 1.5, 1, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 1.5, 1, [['F3', 'C3', 'F4', 'C4', 'O2'], ['F3'], ['F3'], ['F3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'O2'], ['F3', 'C3', 'F4']], ['F3', 'C3', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 1.5, 1, [['M1'], ['M1'], ['EMGr', 'M1'], ['EMGr', 'M1'], ['EMGr', 'M1'], ['ERA', 'EMGr', 'M1']], ['M1', 'EMGr', 'ERA']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 2.0, 1, [[], [], ['F3'], [], [], []], ['F3']], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 2.0, 1, [['F3'], ['F3'], ['F3'], ['F3', 'F4'], ['F3'], ['F3']], ['F3', 'F4']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 2.0, 1, [['M1'], ['M1'], [], ['M1'], ['M1'], []], ['M1']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 2.5, 1, [[], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 3.0, 1, [[], ['F3'], ['F3'], ['F3'], ['F3'], []], ['F3']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 4.0, 1, [[], ['F3'], ['F3'], [], [], []], ['F3']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 5.0, 1, [[], [], ['F3'], [], [], []], ['F3']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 1.0, 2, [['EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ERA']], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 1.0, 2, [['ERA', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'F4'], ['EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGl']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 1.0, 2, [['ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EOGl', 'ERC']], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 1.0, 2, [['ELC', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELC', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4'], ['ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2'], ['ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2']], ['ELC', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2', 'ERC', 'ELE', 'C4']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 1.5, 2, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 1.5, 2, [['F3', 'C3', 'F4', 'C4', 'O2'], ['F3'], ['F3'], ['F3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'O2'], ['F3', 'C3', 'F4']], ['F3', 'C3', 'F4']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 1.5, 2, [['M1'], ['M1'], ['EMGr', 'M1'], ['EMGr', 'M1'], ['EMGr', 'M1'], ['ERA', 'EMGr', 'M1']], ['M1', 'EMGr']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 2.0, 2, [[], [], ['F3'], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 2.0, 2, [['F3'], ['F3'], ['F3'], ['F3', 'F4'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 2.0, 2, [['M1'], ['M1'], [], ['M1'], ['M1'], []], ['M1']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 2.5, 2, [[], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 3.0, 2, [[], ['F3'], ['F3'], ['F3'], ['F3'], []], ['F3']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 4.0, 2, [[], ['F3'], ['F3'], [], [], []], ['F3']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 5.0, 2, [[], [], ['F3'], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 1.0, 3, [['EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ERA']], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 1.0, 3, [['ERA', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'F4'], ['EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 1.0, 3, [['ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EOGl']], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 1.0, 3, [['ELC', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELC', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4'], ['ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2'], ['ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2']], ['ELC', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2', 'ELE']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 1.5, 3, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 1.5, 3, [['F3', 'C3', 'F4', 'C4', 'O2'], ['F3'], ['F3'], ['F3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'O2'], ['F3', 'C3', 'F4']], ['F3', 'F4']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 1.5, 3, [['M1'], ['M1'], ['EMGr', 'M1'], ['EMGr', 'M1'], ['EMGr', 'M1'], ['ERA', 'EMGr', 'M1']], ['M1', 'EMGr']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 2.0, 3, [[], [], ['F3'], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 2.0, 3, [['F3'], ['F3'], ['F3'], ['F3', 'F4'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 2.0, 3, [['M1'], ['M1'], [], ['M1'], ['M1'], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 2.5, 3, [[], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 3.0, 3, [[], ['F3'], ['F3'], ['F3'], ['F3'], []], ['F3']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 4.0, 3, [[], ['F3'], ['F3'], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 5.0, 3, [[], [], ['F3'], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 1.0, 4, [['EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ERA']], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 1.0, 4, [['ERA', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'F4'], ['EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'F4']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 1.0, 4, [['ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 1.0, 4, [['ELC', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELC', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4'], ['ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2'], ['ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2']], ['ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 1.5, 4, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 1.5, 4, [['F3', 'C3', 'F4', 'C4', 'O2'], ['F3'], ['F3'], ['F3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'O2'], ['F3', 'C3', 'F4']], ['F3']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 1.5, 4, [['M1'], ['M1'], ['EMGr', 'M1'], ['EMGr', 'M1'], ['EMGr', 'M1'], ['ERA', 'EMGr', 'M1']], ['M1', 'EMGr']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 2.0, 4, [[], [], ['F3'], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 2.0, 4, [['F3'], ['F3'], ['F3'], ['F3', 'F4'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 2.0, 4, [['M1'], ['M1'], [], ['M1'], ['M1'], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 2.5, 4, [[], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 3.0, 4, [[], ['F3'], ['F3'], ['F3'], ['F3'], []], ['F3']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 4.0, 4, [[], ['F3'], ['F3'], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 5.0, 4, [[], [], ['F3'], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 1.0, 5, [['EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ERA']], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 1.0, 5, [['ERA', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'F4'], ['EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'F4']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 1.0, 5, [['ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 1.0, 5, [['ELC', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELC', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4'], ['ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2'], ['ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2']], ['ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'F4']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 1.5, 5, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 1.5, 5, [['F3', 'C3', 'F4', 'C4', 'O2'], ['F3'], ['F3'], ['F3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'O2'], ['F3', 'C3', 'F4']], ['F3']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 1.5, 5, [['M1'], ['M1'], ['EMGr', 'M1'], ['EMGr', 'M1'], ['EMGr', 'M1'], ['ERA', 'EMGr', 'M1']], ['M1']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 2.0, 5, [[], [], ['F3'], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 2.0, 5, [['F3'], ['F3'], ['F3'], ['F3', 'F4'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 2.0, 5, [['M1'], ['M1'], [], ['M1'], ['M1'], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 2.5, 5, [[], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 3.0, 5, [[], ['F3'], ['F3'], ['F3'], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 4.0, 5, [[], ['F3'], ['F3'], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 5.0, 5, [[], [], ['F3'], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 1.0, 6, [['EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 1.0, 6, [['ERA', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'F4'], ['EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'F4']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 1.0, 6, [['ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 1.0, 6, [['ELC', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELC', 'ERA', 'ERB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4'], ['ELE', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2'], ['ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2']], ['ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'C3', 'F4']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 1.5, 6, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 1.5, 6, [['F3', 'C3', 'F4', 'C4', 'O2'], ['F3'], ['F3'], ['F3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'O2'], ['F3', 'C3', 'F4']], ['F3']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 1.5, 6, [['M1'], ['M1'], ['EMGr', 'M1'], ['EMGr', 'M1'], ['EMGr', 'M1'], ['ERA', 'EMGr', 'M1']], ['M1']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 2.0, 6, [[], [], ['F3'], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 2.0, 6, [['F3'], ['F3'], ['F3'], ['F3', 'F4'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 2.0, 6, [['M1'], ['M1'], [], ['M1'], ['M1'], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 2.5, 6, [[], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 3.0, 6, [[], ['F3'], ['F3'], ['F3'], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 4.0, 6, [[], ['F3'], ['F3'], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'mm', 5.0, 6, [[], [], ['F3'], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 5854, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 1.0, 1, [['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ERA', 'ERI', 'ELC', 'ELE', 'ELI', 'EMGr']], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 1.0, 1, [['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'F4', 'C4'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELC', 'ELE', 'ELI', 'EOGl']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 1.0, 1, [['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'ERI', 'ELC', 'ELE', 'ELI', 'M1']], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 1.0, 1, [['ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'EMGc'], ['ELI', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2']], ['ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'F3', 'ELC', 'ELE', 'ELI', 'EMGc', 'ERB']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 1.5, 1, [['F3', 'F4', 'C4', 'O2'], ['F3', 'C4'], ['ELI', 'F3'], ['F3', 'F4'], ['F3'], ['F3']], ['F3', 'F4', 'C4', 'O2', 'ELI']], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 1.5, 1, [['F3', 'C3', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'F3'], ['F3', 'C3', 'F4', 'C4'], ['F3'], ['F3', 'F4', 'C4', 'O2']], ['F3', 'C3', 'F4', 'C4', 'O2', 'ELE', 'ELI']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 1.5, 1, [[], ['F3', 'C3'], ['ELI', 'F3'], ['ELI', 'F3'], [], []], ['F3', 'C3', 'ELI']], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 1.5, 1, [['EOGr', 'EOGl'], ['ERI', 'EMGr', 'C3'], ['ELC', 'EOGr', 'EOGl', 'EMGr', 'C3'], ['ELI', 'EMGr'], ['ERI', 'EMGr'], ['EOGl', 'EMGr', 'C3']], ['EOGr', 'EOGl', 'ERI', 'EMGr', 'C3', 'ELC', 'ELI']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 2.0, 1, [['F3', 'C4'], ['F3', 'F4', 'C4'], ['ELI', 'F3'], ['F3', 'F4'], ['F3'], ['F3']], ['F3', 'C4', 'F4', 'ELI']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 2.0, 1, [[], ['ERI'], [], [], [], []], ['ERI']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 2.5, 1, [[], ['C4'], ['ELI', 'F3'], [], ['F3'], ['F3']], ['C4', 'ELI', 'F3']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 3.0, 1, [[], [], ['ELI', 'F3'], [], ['F3'], []], ['ELI', 'F3']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 4.0, 1, [[], [], ['F3'], [], ['F3'], []], ['F3']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 5.0, 1, [[], [], ['F3'], [], ['F3'], []], ['F3']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 1.0, 2, [['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELI']], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 1.0, 2, [['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'F4', 'C4'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELI']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 1.0, 2, [['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ERI', 'ELI']], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 1.0, 2, [['ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'EMGc'], ['ELI', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2']], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'ELI', 'EMGc']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 1.5, 2, [['F3', 'F4', 'C4', 'O2'], ['F3', 'C4'], ['ELI', 'F3'], ['F3', 'F4'], ['F3'], ['F3']], ['F3', 'C4']], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 1.5, 2, [['F3', 'C3', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'F3'], ['F3', 'C3', 'F4', 'C4'], ['F3'], ['F3', 'F4', 'C4', 'O2']], ['F3', 'C3', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 1.5, 2, [[], ['F3', 'C3'], ['ELI', 'F3'], ['ELI', 'F3'], [], []], ['F3', 'ELI']], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 1.5, 2, [['EOGr', 'EOGl'], ['ERI', 'EMGr', 'C3'], ['ELC', 'EOGr', 'EOGl', 'EMGr', 'C3'], ['ELI', 'EMGr'], ['ERI', 'EMGr'], ['EOGl', 'EMGr', 'C3']], ['EMGr', 'C3']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 2.0, 2, [['F3', 'C4'], ['F3', 'F4', 'C4'], ['ELI', 'F3'], ['F3', 'F4'], ['F3'], ['F3']], ['F3', 'C4']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 2.0, 2, [[], ['ERI'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 2.5, 2, [[], ['C4'], ['ELI', 'F3'], [], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 3.0, 2, [[], [], ['ELI', 'F3'], [], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 4.0, 2, [[], [], ['F3'], [], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 5.0, 2, [[], [], ['F3'], [], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 1.0, 3, [['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 1.0, 3, [['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'F4', 'C4'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'C3', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 1.0, 3, [['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 1.0, 3, [['ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'EMGc'], ['ELI', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2']], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'O2']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 1.5, 3, [['F3', 'F4', 'C4', 'O2'], ['F3', 'C4'], ['ELI', 'F3'], ['F3', 'F4'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 1.5, 3, [['F3', 'C3', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'F3'], ['F3', 'C3', 'F4', 'C4'], ['F3'], ['F3', 'F4', 'C4', 'O2']], ['F3']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 1.5, 3, [[], ['F3', 'C3'], ['ELI', 'F3'], ['ELI', 'F3'], [], []], ['F3']], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 1.5, 3, [['EOGr', 'EOGl'], ['ERI', 'EMGr', 'C3'], ['ELC', 'EOGr', 'EOGl', 'EMGr', 'C3'], ['ELI', 'EMGr'], ['ERI', 'EMGr'], ['EOGl', 'EMGr', 'C3']], ['EMGr']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 2.0, 3, [['F3', 'C4'], ['F3', 'F4', 'C4'], ['ELI', 'F3'], ['F3', 'F4'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 2.0, 3, [[], ['ERI'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 2.5, 3, [[], ['C4'], ['ELI', 'F3'], [], ['F3'], ['F3']], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 3.0, 3, [[], [], ['ELI', 'F3'], [], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 4.0, 3, [[], [], ['F3'], [], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 5.0, 3, [[], [], ['F3'], [], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 1.0, 4, [['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 1.0, 4, [['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'F4', 'C4'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'C3', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 1.0, 4, [['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 1.0, 4, [['ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'EMGc'], ['ELI', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2']], ['EOGr', 'EOGl', 'EMGr', 'C3', 'F4']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 1.5, 4, [['F3', 'F4', 'C4', 'O2'], ['F3', 'C4'], ['ELI', 'F3'], ['F3', 'F4'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 1.5, 4, [['F3', 'C3', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'F3'], ['F3', 'C3', 'F4', 'C4'], ['F3'], ['F3', 'F4', 'C4', 'O2']], ['F3']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 1.5, 4, [[], ['F3', 'C3'], ['ELI', 'F3'], ['ELI', 'F3'], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 1.5, 4, [['EOGr', 'EOGl'], ['ERI', 'EMGr', 'C3'], ['ELC', 'EOGr', 'EOGl', 'EMGr', 'C3'], ['ELI', 'EMGr'], ['ERI', 'EMGr'], ['EOGl', 'EMGr', 'C3']], ['EMGr']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 2.0, 4, [['F3', 'C4'], ['F3', 'F4', 'C4'], ['ELI', 'F3'], ['F3', 'F4'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 2.0, 4, [[], ['ERI'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 2.5, 4, [[], ['C4'], ['ELI', 'F3'], [], ['F3'], ['F3']], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 3.0, 4, [[], [], ['ELI', 'F3'], [], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 4.0, 4, [[], [], ['F3'], [], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 5.0, 4, [[], [], ['F3'], [], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 1.0, 5, [['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 1.0, 5, [['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'F4', 'C4'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'F4', 'C4']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 1.0, 5, [['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGr', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 1.0, 5, [['ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'EMGc'], ['ELI', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2']], ['EOGr', 'EOGl', 'EMGr', 'C3']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 1.5, 5, [['F3', 'F4', 'C4', 'O2'], ['F3', 'C4'], ['ELI', 'F3'], ['F3', 'F4'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 1.5, 5, [['F3', 'C3', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'F3'], ['F3', 'C3', 'F4', 'C4'], ['F3'], ['F3', 'F4', 'C4', 'O2']], ['F3']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 1.5, 5, [[], ['F3', 'C3'], ['ELI', 'F3'], ['ELI', 'F3'], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 1.5, 5, [['EOGr', 'EOGl'], ['ERI', 'EMGr', 'C3'], ['ELC', 'EOGr', 'EOGl', 'EMGr', 'C3'], ['ELI', 'EMGr'], ['ERI', 'EMGr'], ['EOGl', 'EMGr', 'C3']], ['EMGr']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 2.0, 5, [['F3', 'C4'], ['F3', 'F4', 'C4'], ['ELI', 'F3'], ['F3', 'F4'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 2.0, 5, [[], ['ERI'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 2.5, 5, [[], ['C4'], ['ELI', 'F3'], [], ['F3'], ['F3']], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 3.0, 5, [[], [], ['ELI', 'F3'], [], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 4.0, 5, [[], [], ['F3'], [], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 5.0, 5, [[], [], ['F3'], [], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 1.0, 6, [['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 1.0, 6, [['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'F4', 'C4'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'F4', 'C4']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 1.0, 6, [['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGr', 'EOGl', 'EMGr', 'C3', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 1.0, 6, [['ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'EMGc'], ['ELI', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'O2']], ['EOGr', 'EOGl', 'EMGr', 'C3']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 1.5, 6, [['F3', 'F4', 'C4', 'O2'], ['F3', 'C4'], ['ELI', 'F3'], ['F3', 'F4'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 1.5, 6, [['F3', 'C3', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'F3'], ['F3', 'C3', 'F4', 'C4'], ['F3'], ['F3', 'F4', 'C4', 'O2']], ['F3']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 1.5, 6, [[], ['F3', 'C3'], ['ELI', 'F3'], ['ELI', 'F3'], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 1.5, 6, [['EOGr', 'EOGl'], ['ERI', 'EMGr', 'C3'], ['ELC', 'EOGr', 'EOGl', 'EMGr', 'C3'], ['ELI', 'EMGr'], ['ERI', 'EMGr'], ['EOGl', 'EMGr', 'C3']], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 2.0, 6, [['F3', 'C4'], ['F3', 'F4', 'C4'], ['ELI', 'F3'], ['F3', 'F4'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 2.0, 6, [[], ['ERI'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 2.5, 6, [[], ['C4'], ['ELI', 'F3'], [], ['F3'], ['F3']], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 3.0, 6, [[], [], ['ELI', 'F3'], [], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 4.0, 6, [[], [], ['F3'], [], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'mm', 5.0, 6, [[], [], ['F3'], [], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 7610, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 1.0, 1, [['ELE', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI'], ['ELE', 'ELI', 'ERE', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'C4'], ['ELE', 'ELI', 'ERA', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'M1', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELE', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'ELC']], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 1.0, 1, [['ELI', 'EMGr', 'F3', 'O1', 'O2'], ['ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI'], ['ELE', 'ELI', 'ERI', 'EOGl', 'EMGr', 'C3', 'C4'], ['ELE', 'ELI', 'ERA', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERI', 'EMGr', 'C4', 'O2'], ['ELC', 'ELE', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELI', 'EMGr', 'F3', 'O1', 'O2', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'ELE', 'EOGl', 'C3', 'C4', 'M2', 'F4', 'ELC', 'EMGl']], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 1.0, 1, [['ELE', 'ELI', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGr', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ELI', 'ERE', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERE', 'ERI', 'EMGr', 'M1', 'O1', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELE', 'ELI', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'C4', 'ELA', 'C3', 'ELB', 'ELC']], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 1.0, 1, [['ELC', 'ELE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'O2'], ['ELE', 'ELI', 'EOGr', 'EMGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELI', 'ERE', 'ERI', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2']], ['ELC', 'ELE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'O2', 'ELI', 'O1', 'F4', 'ELA', 'ERE', 'ELB', 'ERA', 'C3', 'ERC', 'EMGc']], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 1.5, 1, [['ELI'], ['ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI'], ['ELE', 'ELI'], ['ERA'], ['ELI', 'ERA', 'EMGr', 'O2'], []], ['ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'ELE', 'EMGr', 'O2']], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 1.5, 1, [['ELI'], ['ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI'], ['ELE', 'ELI', 'EMGr', 'C4'], ['ELE', 'ERA', 'O1', 'O2'], ['ELE', 'ELI', 'ERA', 'ERI', 'EMGr', 'C4', 'O2'], ['EMGr', 'O1', 'O2']], ['ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'ELE', 'EMGr', 'C4', 'O1', 'O2']], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 1.5, 1, [['ELI'], ['ERA', 'ERB', 'ERC', 'ERE', 'ERI'], [], ['ERA'], ['ELI', 'ERI', 'O2'], ['O2']], ['ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'O2']], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 1.5, 1, [['ELE', 'ERI', 'EMGr', 'M1'], ['ELE', 'ELI', 'EOGr', 'EMGr', 'O1', 'M2', 'C4', 'O2'], ['EOGl'], ['ELE', 'ELI', 'ERA', 'EMGr'], [], ['EOGl']], ['ELE', 'ERI', 'EMGr', 'M1', 'ELI', 'EOGr', 'O1', 'M2', 'C4', 'O2', 'EOGl', 'ERA']], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 2.0, 1, [[], ['ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI'], [], [], [], []], ['ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI']], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 2.0, 1, [['ELI'], ['ERA', 'ERC', 'ERT '], ['ELE', 'ELI', 'C4'], ['ERA'], ['ELI', 'O2'], ['O2']], ['ELI', 'ERA', 'ERC', 'ERT ', 'ELE', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 2.0, 1, [[], ['ELE', 'ELI', 'EMGr', 'M2'], [], [], [], []], ['ELE', 'ELI', 'EMGr', 'M2']], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 2.5, 1, [[], ['ERA', 'ERC', 'ERT '], [], [], [], []], ['ERA', 'ERC', 'ERT ']], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 2.5, 1, [['ELI'], ['ERA', 'ERT '], ['ELE', 'ELI'], [], ['ELI', 'O2'], []], ['ELI', 'ERA', 'ERT ', 'ELE', 'O2']], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 3.0, 1, [[], ['ERA', 'ERT '], [], [], [], []], ['ERA', 'ERT ']], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 3.0, 1, [['ELI'], ['ERA', 'ERT '], ['ELI'], [], ['ELI', 'O2'], []], ['ELI', 'ERA', 'ERT ', 'O2']], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 4.0, 1, [['ELI'], ['ERA', 'ERT '], [], [], [], []], ['ELI', 'ERA', 'ERT ']], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 5.0, 1, [['ELI'], ['ERA', 'ERT '], [], [], [], []], ['ELI', 'ERA', 'ERT ']], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 1.0, 2, [['ELE', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI'], ['ELE', 'ELI', 'ERE', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'C4'], ['ELE', 'ELI', 'ERA', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'M1', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELE', 'ELI', 'EMGl', 'EMGr', 'C3', 'M2', 'C4', 'O2', 'ERA', 'ERE', 'ERI']], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 1.0, 2, [['ELI', 'EMGr', 'F3', 'O1', 'O2'], ['ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI'], ['ELE', 'ELI', 'ERI', 'EOGl', 'EMGr', 'C3', 'C4'], ['ELE', 'ELI', 'ERA', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERI', 'EMGr', 'C4', 'O2'], ['ELC', 'ELE', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELI', 'EMGr', 'O2', 'ERA', 'ERI', 'ELE', 'C3', 'C4']], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 1.0, 2, [['ELE', 'ELI', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGr', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ELI', 'ERE', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERE', 'ERI', 'EMGr', 'M1', 'O1', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELE', 'ELI', 'ERI', 'EOGr', 'EMGr', 'O1', 'M2', 'F4', 'O2', 'ERA', 'ERE', 'C4', 'C3']], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 1.0, 2, [['ELC', 'ELE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'O2'], ['ELE', 'ELI', 'EOGr', 'EMGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELI', 'ERE', 'ERI', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2']], ['ELE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M2', 'C4', 'O2', 'ELI', 'O1', 'F4', 'ELA', 'ERA', 'ERC']], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 1.5, 2, [['ELI'], ['ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI'], ['ELE', 'ELI'], ['ERA'], ['ELI', 'ERA', 'EMGr', 'O2'], []], ['ERA']], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 1.5, 2, [['ELI'], ['ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI'], ['ELE', 'ELI', 'EMGr', 'C4'], ['ELE', 'ERA', 'O1', 'O2'], ['ELE', 'ELI', 'ERA', 'ERI', 'EMGr', 'C4', 'O2'], ['EMGr', 'O1', 'O2']], ['ERA', 'ELE', 'EMGr', 'O2']], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 1.5, 2, [['ELI'], ['ERA', 'ERB', 'ERC', 'ERE', 'ERI'], [], ['ERA'], ['ELI', 'ERI', 'O2'], ['O2']], ['O2']], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 1.5, 2, [['ELE', 'ERI', 'EMGr', 'M1'], ['ELE', 'ELI', 'EOGr', 'EMGr', 'O1', 'M2', 'C4', 'O2'], ['EOGl'], ['ELE', 'ELI', 'ERA', 'EMGr'], [], ['EOGl']], ['ELE', 'EMGr']], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 2.0, 2, [[], ['ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 2.0, 2, [['ELI'], ['ERA', 'ERC', 'ERT '], ['ELE', 'ELI', 'C4'], ['ERA'], ['ELI', 'O2'], ['O2']], ['O2']], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 2.0, 2, [[], ['ELE', 'ELI', 'EMGr', 'M2'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 2.5, 2, [[], ['ERA', 'ERC', 'ERT '], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 2.5, 2, [['ELI'], ['ERA', 'ERT '], ['ELE', 'ELI'], [], ['ELI', 'O2'], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 3.0, 2, [[], ['ERA', 'ERT '], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 3.0, 2, [['ELI'], ['ERA', 'ERT '], ['ELI'], [], ['ELI', 'O2'], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 4.0, 2, [['ELI'], ['ERA', 'ERT '], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 5.0, 2, [['ELI'], ['ERA', 'ERT '], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 1.0, 3, [['ELE', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI'], ['ELE', 'ELI', 'ERE', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'C4'], ['ELE', 'ELI', 'ERA', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'M1', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELE', 'ELI', 'EMGr', 'C4', 'O2', 'ERA']], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 1.0, 3, [['ELI', 'EMGr', 'F3', 'O1', 'O2'], ['ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI'], ['ELE', 'ELI', 'ERI', 'EOGl', 'EMGr', 'C3', 'C4'], ['ELE', 'ELI', 'ERA', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERI', 'EMGr', 'C4', 'O2'], ['ELC', 'ELE', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELI', 'EMGr', 'O2', 'ELE', 'C4']], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 1.0, 3, [['ELE', 'ELI', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGr', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ELI', 'ERE', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERE', 'ERI', 'EMGr', 'M1', 'O1', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELE', 'ELI', 'ERI', 'EMGr', 'O1', 'F4', 'O2', 'ERA', 'C4']], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 1.0, 3, [['ELC', 'ELE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'O2'], ['ELE', 'ELI', 'EOGr', 'EMGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELI', 'ERE', 'ERI', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2']], ['ELE', 'ERI', 'EMGl', 'EMGr', 'M2', 'C4', 'O2', 'ELI', 'O1', 'ERA']], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 1.5, 3, [['ELI'], ['ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI'], ['ELE', 'ELI'], ['ERA'], ['ELI', 'ERA', 'EMGr', 'O2'], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 1.5, 3, [['ELI'], ['ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI'], ['ELE', 'ELI', 'EMGr', 'C4'], ['ELE', 'ERA', 'O1', 'O2'], ['ELE', 'ELI', 'ERA', 'ERI', 'EMGr', 'C4', 'O2'], ['EMGr', 'O1', 'O2']], ['ELE', 'O2']], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 1.5, 3, [['ELI'], ['ERA', 'ERB', 'ERC', 'ERE', 'ERI'], [], ['ERA'], ['ELI', 'ERI', 'O2'], ['O2']], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 1.5, 3, [['ELE', 'ERI', 'EMGr', 'M1'], ['ELE', 'ELI', 'EOGr', 'EMGr', 'O1', 'M2', 'C4', 'O2'], ['EOGl'], ['ELE', 'ELI', 'ERA', 'EMGr'], [], ['EOGl']], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 2.0, 3, [[], ['ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 2.0, 3, [['ELI'], ['ERA', 'ERC', 'ERT '], ['ELE', 'ELI', 'C4'], ['ERA'], ['ELI', 'O2'], ['O2']], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 2.0, 3, [[], ['ELE', 'ELI', 'EMGr', 'M2'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 2.5, 3, [[], ['ERA', 'ERC', 'ERT '], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 2.5, 3, [['ELI'], ['ERA', 'ERT '], ['ELE', 'ELI'], [], ['ELI', 'O2'], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 3.0, 3, [[], ['ERA', 'ERT '], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 3.0, 3, [['ELI'], ['ERA', 'ERT '], ['ELI'], [], ['ELI', 'O2'], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 4.0, 3, [['ELI'], ['ERA', 'ERT '], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 5.0, 3, [['ELI'], ['ERA', 'ERT '], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 1.0, 4, [['ELE', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI'], ['ELE', 'ELI', 'ERE', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'C4'], ['ELE', 'ELI', 'ERA', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'M1', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELE', 'ELI', 'EMGr', 'C4']], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 1.0, 4, [['ELI', 'EMGr', 'F3', 'O1', 'O2'], ['ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI'], ['ELE', 'ELI', 'ERI', 'EOGl', 'EMGr', 'C3', 'C4'], ['ELE', 'ELI', 'ERA', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERI', 'EMGr', 'C4', 'O2'], ['ELC', 'ELE', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EMGr', 'ELE', 'C4']], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 1.0, 4, [['ELE', 'ELI', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGr', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ELI', 'ERE', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERE', 'ERI', 'EMGr', 'M1', 'O1', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELE', 'ELI', 'ERI', 'EMGr', 'O1', 'O2', 'C4']], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 1.0, 4, [['ELC', 'ELE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'O2'], ['ELE', 'ELI', 'EOGr', 'EMGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELI', 'ERE', 'ERI', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2']], ['ERI', 'EMGr', 'M2', 'O2', 'ELI', 'O1']], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 1.5, 4, [['ELI'], ['ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI'], ['ELE', 'ELI'], ['ERA'], ['ELI', 'ERA', 'EMGr', 'O2'], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 1.5, 4, [['ELI'], ['ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI'], ['ELE', 'ELI', 'EMGr', 'C4'], ['ELE', 'ERA', 'O1', 'O2'], ['ELE', 'ELI', 'ERA', 'ERI', 'EMGr', 'C4', 'O2'], ['EMGr', 'O1', 'O2']], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 1.5, 4, [['ELI'], ['ERA', 'ERB', 'ERC', 'ERE', 'ERI'], [], ['ERA'], ['ELI', 'ERI', 'O2'], ['O2']], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 1.5, 4, [['ELE', 'ERI', 'EMGr', 'M1'], ['ELE', 'ELI', 'EOGr', 'EMGr', 'O1', 'M2', 'C4', 'O2'], ['EOGl'], ['ELE', 'ELI', 'ERA', 'EMGr'], [], ['EOGl']], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 2.0, 4, [[], ['ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 2.0, 4, [['ELI'], ['ERA', 'ERC', 'ERT '], ['ELE', 'ELI', 'C4'], ['ERA'], ['ELI', 'O2'], ['O2']], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 2.0, 4, [[], ['ELE', 'ELI', 'EMGr', 'M2'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 2.5, 4, [[], ['ERA', 'ERC', 'ERT '], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 2.5, 4, [['ELI'], ['ERA', 'ERT '], ['ELE', 'ELI'], [], ['ELI', 'O2'], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 3.0, 4, [[], ['ERA', 'ERT '], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 3.0, 4, [['ELI'], ['ERA', 'ERT '], ['ELI'], [], ['ELI', 'O2'], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 4.0, 4, [['ELI'], ['ERA', 'ERT '], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 5.0, 4, [['ELI'], ['ERA', 'ERT '], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 1.0, 5, [['ELE', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI'], ['ELE', 'ELI', 'ERE', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'C4'], ['ELE', 'ELI', 'ERA', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'M1', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 1.0, 5, [['ELI', 'EMGr', 'F3', 'O1', 'O2'], ['ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI'], ['ELE', 'ELI', 'ERI', 'EOGl', 'EMGr', 'C3', 'C4'], ['ELE', 'ELI', 'ERA', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERI', 'EMGr', 'C4', 'O2'], ['ELC', 'ELE', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 1.0, 5, [['ELE', 'ELI', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGr', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ELI', 'ERE', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERE', 'ERI', 'EMGr', 'M1', 'O1', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELE', 'ELI', 'ERI', 'EMGr', 'O1', 'O2', 'C4']], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 1.0, 5, [['ELC', 'ELE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'O2'], ['ELE', 'ELI', 'EOGr', 'EMGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELI', 'ERE', 'ERI', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2']], ['EMGr', 'O2', 'ELI']], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 1.5, 5, [['ELI'], ['ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI'], ['ELE', 'ELI'], ['ERA'], ['ELI', 'ERA', 'EMGr', 'O2'], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 1.5, 5, [['ELI'], ['ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI'], ['ELE', 'ELI', 'EMGr', 'C4'], ['ELE', 'ERA', 'O1', 'O2'], ['ELE', 'ELI', 'ERA', 'ERI', 'EMGr', 'C4', 'O2'], ['EMGr', 'O1', 'O2']], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 1.5, 5, [['ELI'], ['ERA', 'ERB', 'ERC', 'ERE', 'ERI'], [], ['ERA'], ['ELI', 'ERI', 'O2'], ['O2']], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 1.5, 5, [['ELE', 'ERI', 'EMGr', 'M1'], ['ELE', 'ELI', 'EOGr', 'EMGr', 'O1', 'M2', 'C4', 'O2'], ['EOGl'], ['ELE', 'ELI', 'ERA', 'EMGr'], [], ['EOGl']], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 2.0, 5, [[], ['ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 2.0, 5, [['ELI'], ['ERA', 'ERC', 'ERT '], ['ELE', 'ELI', 'C4'], ['ERA'], ['ELI', 'O2'], ['O2']], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 2.0, 5, [[], ['ELE', 'ELI', 'EMGr', 'M2'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 2.5, 5, [[], ['ERA', 'ERC', 'ERT '], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 2.5, 5, [['ELI'], ['ERA', 'ERT '], ['ELE', 'ELI'], [], ['ELI', 'O2'], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 3.0, 5, [[], ['ERA', 'ERT '], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 3.0, 5, [['ELI'], ['ERA', 'ERT '], ['ELI'], [], ['ELI', 'O2'], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 4.0, 5, [['ELI'], ['ERA', 'ERT '], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 5.0, 5, [['ELI'], ['ERA', 'ERT '], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 1.0, 6, [['ELE', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI'], ['ELE', 'ELI', 'ERE', 'ERI', 'EOGl', 'EMGr', 'C3', 'M2', 'C4'], ['ELE', 'ELI', 'ERA', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'M1', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 1.0, 6, [['ELI', 'EMGr', 'F3', 'O1', 'O2'], ['ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI'], ['ELE', 'ELI', 'ERI', 'EOGl', 'EMGr', 'C3', 'C4'], ['ELE', 'ELI', 'ERA', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERI', 'EMGr', 'C4', 'O2'], ['ELC', 'ELE', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 1.0, 6, [['ELE', 'ELI', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EMGr', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ELI', 'ERE', 'ERI', 'EOGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERE', 'ERI', 'EMGr', 'M1', 'O1', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELE', 'ELI', 'ERI', 'EMGr', 'O1', 'O2']], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 1.0, 6, [['ELC', 'ELE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'C4', 'O2'], ['ELE', 'ELI', 'EOGr', 'EMGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELI', 'ERE', 'ERI', 'EOGl', 'EMGr', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2']], ['EMGr', 'O2']], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 1.5, 6, [['ELI'], ['ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI'], ['ELE', 'ELI'], ['ERA'], ['ELI', 'ERA', 'EMGr', 'O2'], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 1.5, 6, [['ELI'], ['ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI'], ['ELE', 'ELI', 'EMGr', 'C4'], ['ELE', 'ERA', 'O1', 'O2'], ['ELE', 'ELI', 'ERA', 'ERI', 'EMGr', 'C4', 'O2'], ['EMGr', 'O1', 'O2']], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 1.5, 6, [['ELI'], ['ERA', 'ERB', 'ERC', 'ERE', 'ERI'], [], ['ERA'], ['ELI', 'ERI', 'O2'], ['O2']], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 1.5, 6, [['ELE', 'ERI', 'EMGr', 'M1'], ['ELE', 'ELI', 'EOGr', 'EMGr', 'O1', 'M2', 'C4', 'O2'], ['EOGl'], ['ELE', 'ELI', 'ERA', 'EMGr'], [], ['EOGl']], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 2.0, 6, [[], ['ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 2.0, 6, [['ELI'], ['ERA', 'ERC', 'ERT '], ['ELE', 'ELI', 'C4'], ['ERA'], ['ELI', 'O2'], ['O2']], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 2.0, 6, [[], ['ELE', 'ELI', 'EMGr', 'M2'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 2.5, 6, [[], ['ERA', 'ERC', 'ERT '], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 2.5, 6, [['ELI'], ['ERA', 'ERT '], ['ELE', 'ELI'], [], ['ELI', 'O2'], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 3.0, 6, [[], ['ERA', 'ERT '], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 3.0, 6, [['ELI'], ['ERA', 'ERT '], ['ELI'], [], ['ELI', 'O2'], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 4.0, 6, [['ELI'], ['ERA', 'ERT '], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'mm', 5.0, 6, [['ELI'], ['ERA', 'ERT '], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 10580, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 1.0, 1, [['ELE', 'ELI', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ELI', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2', 'EMGc']], ['ELE', 'ELI', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ERA', 'EOGl', 'M1', 'F3', 'ELA', 'ELC', 'ERE', 'ERI', 'M2', 'ELB']], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 1.0, 1, [['ELE', 'ELI', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELE', 'ELI', 'EMGl', 'EMGr', 'O1', 'O2'], ['ELC', 'EOGl', 'EMGr', 'M1', 'M2', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'EMGr'], ['ELB', 'ELC', 'ELE', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc']], ['ELE', 'ELI', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'EOGr', 'EOGl', 'EMGl', 'O1', 'O2', 'EMGc', 'ELC', 'M1', 'M2', 'ELB']], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 1.0, 1, [['ELE', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERE', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERE', 'ERI', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2', 'EMGc']], ['ELE', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'M1', 'ERE', 'ELC', 'ERI', 'M2', 'ELB', 'ERC']], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 1.0, 1, [['ELE', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELE', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELE', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ERC', 'EMGr', 'M1', 'ERE', 'ELC', 'ERI', 'ELB', 'M2']], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 1.5, 1, [['ELI', 'EMGr'], [], ['ELI'], ['EMGr'], ['ELC', 'ELE', 'ELI'], ['ELC', 'ELE']], ['ELI', 'EMGr', 'ELC', 'ELE']], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 1.5, 1, [['ELE', 'ELI', 'EMGr', 'C3', 'C4'], ['F3', 'F4', 'C4', 'O2'], ['ELE', 'ELI'], ['EMGr'], ['ELC', 'ELE', 'ELI', 'EMGr'], ['ELC', 'ELE', 'EMGl', 'EMGr', 'O1', 'O2']], ['ELE', 'ELI', 'EMGr', 'C3', 'C4', 'F3', 'F4', 'O2', 'ELC', 'EMGl', 'O1']], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 1.5, 1, [['ELI', 'EMGr'], [], [], [], ['ELB', 'ELC', 'ERE'], ['ELC', 'O1']], ['ELI', 'EMGr', 'ELB', 'ELC', 'ERE', 'O1']], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 1.5, 1, [['ELE', 'EOGl', 'EMGl', 'EMGc'], ['ELE', 'EMGr', 'M1'], ['ELE', 'EMGl', 'EMGr', 'M1', 'EMGc'], ['ELC', 'ERI'], ['ELB', 'ERE', 'EOGl', 'EMGl', 'O2', 'EMGc'], ['O1', 'O2']], ['ELE', 'EOGl', 'EMGl', 'EMGc', 'EMGr', 'M1', 'ELC', 'ERI', 'ELB', 'ERE', 'O2', 'O1']], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 2.0, 1, [['ELI', 'EMGr'], ['O2'], ['ELI'], ['EMGr'], ['ELC', 'ELE', 'ELI'], ['ELC', 'ELE']], ['ELI', 'EMGr', 'O2', 'ELC', 'ELE']], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 2.5, 1, [['ELI', 'EMGr'], [], ['ELI'], ['EMGr'], ['ELC', 'ELE', 'ELI'], []], ['ELI', 'EMGr', 'ELC', 'ELE']], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 3.0, 1, [['EMGr'], [], ['ELI'], ['EMGr'], ['ELC', 'ELE', 'ELI'], []], ['EMGr', 'ELI', 'ELC', 'ELE']], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 4.0, 1, [['EMGr'], [], ['ELI'], ['EMGr'], ['ELI'], []], ['EMGr', 'ELI']], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 1.0, 2, [['ELE', 'ELI', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ELI', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2', 'EMGc']], ['ELE', 'ELI', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'EOGl', 'ELC', 'ERE', 'M2', 'ELB']], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 1.0, 2, [['ELE', 'ELI', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELE', 'ELI', 'EMGl', 'EMGr', 'O1', 'O2'], ['ELC', 'EOGl', 'EMGr', 'M1', 'M2', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'EMGr'], ['ELB', 'ELC', 'ELE', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc']], ['ELE', 'ELI', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'EMGl', 'O1', 'O2', 'ELC', 'ELB']], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 1.0, 2, [['ELE', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERE', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERE', 'ERI', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2', 'EMGc']], ['ELE', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'M1', 'ERE', 'ELC', 'M2', 'ELB']], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 1.0, 2, [['ELE', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELE', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'EMGr', 'M1', 'ERE', 'ELB']], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 1.5, 2, [['ELI', 'EMGr'], [], ['ELI'], ['EMGr'], ['ELC', 'ELE', 'ELI'], ['ELC', 'ELE']], ['ELC', 'ELE']], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 1.5, 2, [['ELE', 'ELI', 'EMGr', 'C3', 'C4'], ['F3', 'F4', 'C4', 'O2'], ['ELE', 'ELI'], ['EMGr'], ['ELC', 'ELE', 'ELI', 'EMGr'], ['ELC', 'ELE', 'EMGl', 'EMGr', 'O1', 'O2']], ['ELE', 'EMGr', 'C4', 'ELC']], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 1.5, 2, [['ELI', 'EMGr'], [], [], [], ['ELB', 'ELC', 'ERE'], ['ELC', 'O1']], ['ELC']], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 1.5, 2, [['ELE', 'EOGl', 'EMGl', 'EMGc'], ['ELE', 'EMGr', 'M1'], ['ELE', 'EMGl', 'EMGr', 'M1', 'EMGc'], ['ELC', 'ERI'], ['ELB', 'ERE', 'EOGl', 'EMGl', 'O2', 'EMGc'], ['O1', 'O2']], ['ELE', 'EMGr', 'M1', 'O2']], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 2.0, 2, [['ELI', 'EMGr'], ['O2'], ['ELI'], ['EMGr'], ['ELC', 'ELE', 'ELI'], ['ELC', 'ELE']], ['ELC', 'ELE']], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 2.5, 2, [['ELI', 'EMGr'], [], ['ELI'], ['EMGr'], ['ELC', 'ELE', 'ELI'], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 3.0, 2, [['EMGr'], [], ['ELI'], ['EMGr'], ['ELC', 'ELE', 'ELI'], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 4.0, 2, [['EMGr'], [], ['ELI'], ['EMGr'], ['ELI'], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 1.0, 3, [['ELE', 'ELI', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ELI', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2', 'EMGc']], ['ELE', 'ELI', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EOGl', 'ELC']], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 1.0, 3, [['ELE', 'ELI', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELE', 'ELI', 'EMGl', 'EMGr', 'O1', 'O2'], ['ELC', 'EOGl', 'EMGr', 'M1', 'M2', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'EMGr'], ['ELB', 'ELC', 'ELE', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc']], ['EMGr', 'O2', 'ELC']], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 1.0, 3, [['ELE', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERE', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERE', 'ERI', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2', 'EMGc']], ['ELE', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'M1', 'ERE', 'ELC']], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 1.0, 3, [['ELE', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELE', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELE', 'EOGr', 'EOGl', 'EMGl', 'C3', 'O1', 'F4', 'C4', 'O2', 'M1', 'ERE']], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 1.5, 3, [['ELI', 'EMGr'], [], ['ELI'], ['EMGr'], ['ELC', 'ELE', 'ELI'], ['ELC', 'ELE']], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 1.5, 3, [['ELE', 'ELI', 'EMGr', 'C3', 'C4'], ['F3', 'F4', 'C4', 'O2'], ['ELE', 'ELI'], ['EMGr'], ['ELC', 'ELE', 'ELI', 'EMGr'], ['ELC', 'ELE', 'EMGl', 'EMGr', 'O1', 'O2']], ['EMGr']], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 1.5, 3, [['ELI', 'EMGr'], [], [], [], ['ELB', 'ELC', 'ERE'], ['ELC', 'O1']], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 1.5, 3, [['ELE', 'EOGl', 'EMGl', 'EMGc'], ['ELE', 'EMGr', 'M1'], ['ELE', 'EMGl', 'EMGr', 'M1', 'EMGc'], ['ELC', 'ERI'], ['ELB', 'ERE', 'EOGl', 'EMGl', 'O2', 'EMGc'], ['O1', 'O2']], ['ELE']], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 2.0, 3, [['ELI', 'EMGr'], ['O2'], ['ELI'], ['EMGr'], ['ELC', 'ELE', 'ELI'], ['ELC', 'ELE']], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 2.5, 3, [['ELI', 'EMGr'], [], ['ELI'], ['EMGr'], ['ELC', 'ELE', 'ELI'], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 3.0, 3, [['EMGr'], [], ['ELI'], ['EMGr'], ['ELC', 'ELE', 'ELI'], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 4.0, 3, [['EMGr'], [], ['ELI'], ['EMGr'], ['ELI'], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 1.0, 4, [['ELE', 'ELI', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ELI', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2', 'EMGc']], ['ELE', 'ELI', 'EMGr', 'F4', 'O2']], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 1.0, 4, [['ELE', 'ELI', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELE', 'ELI', 'EMGl', 'EMGr', 'O1', 'O2'], ['ELC', 'EOGl', 'EMGr', 'M1', 'M2', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'EMGr'], ['ELB', 'ELC', 'ELE', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc']], ['EMGr']], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 1.0, 4, [['ELE', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERE', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERE', 'ERI', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2', 'EMGc']], ['ELE', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 1.0, 4, [['ELE', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELE', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELE', 'EOGr', 'EOGl', 'EMGl', 'C3', 'F4', 'O2']], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 1.5, 4, [['ELI', 'EMGr'], [], ['ELI'], ['EMGr'], ['ELC', 'ELE', 'ELI'], ['ELC', 'ELE']], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 1.5, 4, [['ELE', 'ELI', 'EMGr', 'C3', 'C4'], ['F3', 'F4', 'C4', 'O2'], ['ELE', 'ELI'], ['EMGr'], ['ELC', 'ELE', 'ELI', 'EMGr'], ['ELC', 'ELE', 'EMGl', 'EMGr', 'O1', 'O2']], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 1.5, 4, [['ELI', 'EMGr'], [], [], [], ['ELB', 'ELC', 'ERE'], ['ELC', 'O1']], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 1.5, 4, [['ELE', 'EOGl', 'EMGl', 'EMGc'], ['ELE', 'EMGr', 'M1'], ['ELE', 'EMGl', 'EMGr', 'M1', 'EMGc'], ['ELC', 'ERI'], ['ELB', 'ERE', 'EOGl', 'EMGl', 'O2', 'EMGc'], ['O1', 'O2']], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 2.0, 4, [['ELI', 'EMGr'], ['O2'], ['ELI'], ['EMGr'], ['ELC', 'ELE', 'ELI'], ['ELC', 'ELE']], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 2.5, 4, [['ELI', 'EMGr'], [], ['ELI'], ['EMGr'], ['ELC', 'ELE', 'ELI'], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 3.0, 4, [['EMGr'], [], ['ELI'], ['EMGr'], ['ELC', 'ELE', 'ELI'], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 4.0, 4, [['EMGr'], [], ['ELI'], ['EMGr'], ['ELI'], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 1.0, 5, [['ELE', 'ELI', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ELI', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2', 'EMGc']], ['EMGr']], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 1.0, 5, [['ELE', 'ELI', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELE', 'ELI', 'EMGl', 'EMGr', 'O1', 'O2'], ['ELC', 'EOGl', 'EMGr', 'M1', 'M2', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'EMGr'], ['ELB', 'ELC', 'ELE', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc']], ['EMGr']], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 1.0, 5, [['ELE', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERE', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERE', 'ERI', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2', 'EMGc']], ['ELE']], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 1.0, 5, [['ELE', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELE', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'EMGl', 'C3', 'O2']], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 1.5, 5, [['ELI', 'EMGr'], [], ['ELI'], ['EMGr'], ['ELC', 'ELE', 'ELI'], ['ELC', 'ELE']], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 1.5, 5, [['ELE', 'ELI', 'EMGr', 'C3', 'C4'], ['F3', 'F4', 'C4', 'O2'], ['ELE', 'ELI'], ['EMGr'], ['ELC', 'ELE', 'ELI', 'EMGr'], ['ELC', 'ELE', 'EMGl', 'EMGr', 'O1', 'O2']], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 1.5, 5, [['ELI', 'EMGr'], [], [], [], ['ELB', 'ELC', 'ERE'], ['ELC', 'O1']], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 1.5, 5, [['ELE', 'EOGl', 'EMGl', 'EMGc'], ['ELE', 'EMGr', 'M1'], ['ELE', 'EMGl', 'EMGr', 'M1', 'EMGc'], ['ELC', 'ERI'], ['ELB', 'ERE', 'EOGl', 'EMGl', 'O2', 'EMGc'], ['O1', 'O2']], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 2.0, 5, [['ELI', 'EMGr'], ['O2'], ['ELI'], ['EMGr'], ['ELC', 'ELE', 'ELI'], ['ELC', 'ELE']], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 2.5, 5, [['ELI', 'EMGr'], [], ['ELI'], ['EMGr'], ['ELC', 'ELE', 'ELI'], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 3.0, 5, [['EMGr'], [], ['ELI'], ['EMGr'], ['ELC', 'ELE', 'ELI'], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 4.0, 5, [['EMGr'], [], ['ELI'], ['EMGr'], ['ELI'], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 1.0, 6, [['ELE', 'ELI', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELE', 'ELI', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2', 'EMGc']], ['EMGr']], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 1.0, 6, [['ELE', 'ELI', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELE', 'ELI', 'EMGl', 'EMGr', 'O1', 'O2'], ['ELC', 'EOGl', 'EMGr', 'M1', 'M2', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'EMGr'], ['ELB', 'ELC', 'ELE', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc']], ['EMGr']], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 1.0, 6, [['ELE', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERE', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERE', 'ERI', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ELE', 'ELI', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2', 'EMGc']], ['ELE']], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 1.0, 6, [['ELE', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELE', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELE', 'ELI', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['EOGl', 'EMGl', 'C3', 'O2']], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 1.5, 6, [['ELI', 'EMGr'], [], ['ELI'], ['EMGr'], ['ELC', 'ELE', 'ELI'], ['ELC', 'ELE']], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 1.5, 6, [['ELE', 'ELI', 'EMGr', 'C3', 'C4'], ['F3', 'F4', 'C4', 'O2'], ['ELE', 'ELI'], ['EMGr'], ['ELC', 'ELE', 'ELI', 'EMGr'], ['ELC', 'ELE', 'EMGl', 'EMGr', 'O1', 'O2']], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 1.5, 6, [['ELI', 'EMGr'], [], [], [], ['ELB', 'ELC', 'ERE'], ['ELC', 'O1']], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 1.5, 6, [['ELE', 'EOGl', 'EMGl', 'EMGc'], ['ELE', 'EMGr', 'M1'], ['ELE', 'EMGl', 'EMGr', 'M1', 'EMGc'], ['ELC', 'ERI'], ['ELB', 'ERE', 'EOGl', 'EMGl', 'O2', 'EMGc'], ['O1', 'O2']], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 2.0, 6, [['ELI', 'EMGr'], ['O2'], ['ELI'], ['EMGr'], ['ELC', 'ELE', 'ELI'], ['ELC', 'ELE']], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 2.5, 6, [['ELI', 'EMGr'], [], ['ELI'], ['EMGr'], ['ELC', 'ELE', 'ELI'], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 3.0, 6, [['EMGr'], [], ['ELI'], ['EMGr'], ['ELC', 'ELE', 'ELI'], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 4.0, 6, [['EMGr'], [], ['ELI'], ['EMGr'], ['ELI'], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 11920, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 1.0, 1, [['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELC', 'ELE', 'ELI', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EOGr', 'EOGl']], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 1.0, 1, [['ELE', 'ELI', 'F3', 'C3', 'F4', 'O2'], ['ELE', 'ELI', 'F3', 'C3', 'F4'], ['ELE', 'ELI', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELE', 'F3', 'C3', 'F4', 'C4'], ['ELE', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELE', 'ELI', 'F3', 'C3', 'F4', 'O2', 'C4', 'O1']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 1.0, 1, [['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EOGl', 'ERC']], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 1.0, 1, [['ELC', 'ELI', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'O2'], ['ELI', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ELC', 'ELI', 'ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2']], ['ELC', 'ELI', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'O2', 'EOGl', 'O1', 'C4']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 1.5, 1, [['ELE'], ['ELE', 'F3'], ['ELE', 'F4'], ['ELE'], ['ELE'], ['ELE']], ['ELE', 'F3', 'F4']], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 1.5, 1, [['ELE'], ['ELE'], ['ELE', 'F3', 'F4'], ['ELE', 'F3', 'C3'], ['ELE', 'F3', 'C3', 'C4'], ['ELE']], ['ELE', 'F3', 'F4', 'C3', 'C4']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 1.5, 1, [['ELE'], ['ELI'], [], [], [], []], ['ELE', 'ELI']], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 1.5, 1, [['ELI'], ['ERI', 'EMGr'], ['ELI', 'ERI', 'EMGr'], ['ELI', 'EMGr', 'F3'], ['EMGl', 'EMGr', 'F3', 'F4'], ['ELC', 'ELI', 'EMGl', 'EMGr']], ['ELI', 'ERI', 'EMGr', 'F3', 'EMGl', 'F4', 'ELC']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 2.0, 1, [[], [], ['ELE'], ['ELE'], ['ELE'], []], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 2.0, 1, [['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE', 'F3', 'C3'], ['ELE']], ['ELE', 'F3', 'C3']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 2.5, 1, [['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 3.0, 1, [['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 4.0, 1, [['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 5.0, 1, [['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 1.0, 2, [['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELC', 'ELE', 'ELI', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELC', 'ELE', 'ELI', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EOGr']], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 1.0, 2, [['ELE', 'ELI', 'F3', 'C3', 'F4', 'O2'], ['ELE', 'ELI', 'F3', 'C3', 'F4'], ['ELE', 'ELI', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELE', 'F3', 'C3', 'F4', 'C4'], ['ELE', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELE', 'ELI', 'F3', 'C3', 'F4', 'O2', 'C4', 'O1']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 1.0, 2, [['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELC', 'ELE', 'ELI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EOGl']], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 1.0, 2, [['ELC', 'ELI', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'O2'], ['ELI', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ELC', 'ELI', 'ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2']], ['ELC', 'ELI', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'O2', 'EOGl', 'O1', 'C4']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 1.5, 2, [['ELE'], ['ELE', 'F3'], ['ELE', 'F4'], ['ELE'], ['ELE'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 1.5, 2, [['ELE'], ['ELE'], ['ELE', 'F3', 'F4'], ['ELE', 'F3', 'C3'], ['ELE', 'F3', 'C3', 'C4'], ['ELE']], ['ELE', 'F3', 'C3']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 1.5, 2, [['ELE'], ['ELI'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 1.5, 2, [['ELI'], ['ERI', 'EMGr'], ['ELI', 'ERI', 'EMGr'], ['ELI', 'EMGr', 'F3'], ['EMGl', 'EMGr', 'F3', 'F4'], ['ELC', 'ELI', 'EMGl', 'EMGr']], ['ELI', 'ERI', 'EMGr', 'F3', 'EMGl']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 2.0, 2, [[], [], ['ELE'], ['ELE'], ['ELE'], []], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 2.0, 2, [['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE', 'F3', 'C3'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 2.5, 2, [['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 3.0, 2, [['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 4.0, 2, [['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 5.0, 2, [['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 1.0, 3, [['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELC', 'ELE', 'ELI', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELE', 'ELI', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 1.0, 3, [['ELE', 'ELI', 'F3', 'C3', 'F4', 'O2'], ['ELE', 'ELI', 'F3', 'C3', 'F4'], ['ELE', 'ELI', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELE', 'F3', 'C3', 'F4', 'C4'], ['ELE', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELE', 'ELI', 'F3', 'C3', 'F4', 'C4']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 1.0, 3, [['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EOGl']], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 1.0, 3, [['ELC', 'ELI', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'O2'], ['ELI', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ELC', 'ELI', 'ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2']], ['ELC', 'ELI', 'ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'O2', 'EOGl', 'C4']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 1.5, 3, [['ELE'], ['ELE', 'F3'], ['ELE', 'F4'], ['ELE'], ['ELE'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 1.5, 3, [['ELE'], ['ELE'], ['ELE', 'F3', 'F4'], ['ELE', 'F3', 'C3'], ['ELE', 'F3', 'C3', 'C4'], ['ELE']], ['ELE', 'F3']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 1.5, 3, [['ELE'], ['ELI'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 1.5, 3, [['ELI'], ['ERI', 'EMGr'], ['ELI', 'ERI', 'EMGr'], ['ELI', 'EMGr', 'F3'], ['EMGl', 'EMGr', 'F3', 'F4'], ['ELC', 'ELI', 'EMGl', 'EMGr']], ['EMGr']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 2.0, 3, [[], [], ['ELE'], ['ELE'], ['ELE'], []], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 2.0, 3, [['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE', 'F3', 'C3'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 2.5, 3, [['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 3.0, 3, [['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 4.0, 3, [['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 5.0, 3, [['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 1.0, 4, [['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELC', 'ELE', 'ELI', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELE', 'ELI', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 1.0, 4, [['ELE', 'ELI', 'F3', 'C3', 'F4', 'O2'], ['ELE', 'ELI', 'F3', 'C3', 'F4'], ['ELE', 'ELI', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELE', 'F3', 'C3', 'F4', 'C4'], ['ELE', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELE', 'F3', 'C3', 'F4', 'C4']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 1.0, 4, [['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 1.0, 4, [['ELC', 'ELI', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'O2'], ['ELI', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ELC', 'ELI', 'ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2']], ['ELC', 'ELI', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 1.5, 4, [['ELE'], ['ELE', 'F3'], ['ELE', 'F4'], ['ELE'], ['ELE'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 1.5, 4, [['ELE'], ['ELE'], ['ELE', 'F3', 'F4'], ['ELE', 'F3', 'C3'], ['ELE', 'F3', 'C3', 'C4'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 1.5, 4, [['ELE'], ['ELI'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 1.5, 4, [['ELI'], ['ERI', 'EMGr'], ['ELI', 'ERI', 'EMGr'], ['ELI', 'EMGr', 'F3'], ['EMGl', 'EMGr', 'F3', 'F4'], ['ELC', 'ELI', 'EMGl', 'EMGr']], ['EMGr']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 2.0, 4, [[], [], ['ELE'], ['ELE'], ['ELE'], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 2.0, 4, [['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE', 'F3', 'C3'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 2.5, 4, [['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 3.0, 4, [['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 4.0, 4, [['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 5.0, 4, [['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 1.0, 5, [['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELC', 'ELE', 'ELI', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELE', 'ELI', 'EMGr', 'F3', 'C3', 'F4', 'O2']], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 1.0, 5, [['ELE', 'ELI', 'F3', 'C3', 'F4', 'O2'], ['ELE', 'ELI', 'F3', 'C3', 'F4'], ['ELE', 'ELI', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELE', 'F3', 'C3', 'F4', 'C4'], ['ELE', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELE', 'F3', 'C3', 'F4']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 1.0, 5, [['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELI', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 1.0, 5, [['ELC', 'ELI', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'O2'], ['ELI', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ELC', 'ELI', 'ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2']], ['ELI', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 1.5, 5, [['ELE'], ['ELE', 'F3'], ['ELE', 'F4'], ['ELE'], ['ELE'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 1.5, 5, [['ELE'], ['ELE'], ['ELE', 'F3', 'F4'], ['ELE', 'F3', 'C3'], ['ELE', 'F3', 'C3', 'C4'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 1.5, 5, [['ELE'], ['ELI'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 1.5, 5, [['ELI'], ['ERI', 'EMGr'], ['ELI', 'ERI', 'EMGr'], ['ELI', 'EMGr', 'F3'], ['EMGl', 'EMGr', 'F3', 'F4'], ['ELC', 'ELI', 'EMGl', 'EMGr']], ['EMGr']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 2.0, 5, [[], [], ['ELE'], ['ELE'], ['ELE'], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 2.0, 5, [['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE', 'F3', 'C3'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 2.5, 5, [['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 3.0, 5, [['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 4.0, 5, [['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 5.0, 5, [['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 1.0, 6, [['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELC', 'ELE', 'ELI', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELE', 'ELI', 'EMGr', 'F3', 'C3', 'F4', 'O2']], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 1.0, 6, [['ELE', 'ELI', 'F3', 'C3', 'F4', 'O2'], ['ELE', 'ELI', 'F3', 'C3', 'F4'], ['ELE', 'ELI', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELE', 'F3', 'C3', 'F4', 'C4'], ['ELE', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELE', 'F3', 'C3', 'F4']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 1.0, 6, [['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELI', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 1.0, 6, [['ELC', 'ELI', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'O2'], ['ELI', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4'], ['ELC', 'ELI', 'ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELI', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2']], ['ELI', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'F4']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 1.5, 6, [['ELE'], ['ELE', 'F3'], ['ELE', 'F4'], ['ELE'], ['ELE'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 1.5, 6, [['ELE'], ['ELE'], ['ELE', 'F3', 'F4'], ['ELE', 'F3', 'C3'], ['ELE', 'F3', 'C3', 'C4'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 1.5, 6, [['ELE'], ['ELI'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 1.5, 6, [['ELI'], ['ERI', 'EMGr'], ['ELI', 'ERI', 'EMGr'], ['ELI', 'EMGr', 'F3'], ['EMGl', 'EMGr', 'F3', 'F4'], ['ELC', 'ELI', 'EMGl', 'EMGr']], []], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 2.0, 6, [[], [], ['ELE'], ['ELE'], ['ELE'], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 2.0, 6, [['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE', 'F3', 'C3'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 2.5, 6, [['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 3.0, 6, [['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 4.0, 6, [['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'mm', 5.0, 6, [['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE'], ['ELE']], ['ELE']], ['S_01/night_2\\EEG_raw', 12640, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 12640, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 1.0, 1, [['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ERA', 'ERE', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERE', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELE', 'ELI', 'EMGl', 'ELC', 'ERE', 'ERC']], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 1.0, 1, [['EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4', 'C4'], ['ELE', 'ERE', 'F3', 'C3', 'F4', 'C4']], ['EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELE', 'ERE']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 1.0, 1, [['ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELE', 'ELI', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERE', 'ERI', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ELE', 'ELI', 'ELC', 'ERE']], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 1.0, 1, [['ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'O2'], ['ELE', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'O2']], ['ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'O2', 'ELC', 'ELE', 'C4', 'ELI', 'ERE', 'ERB']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 1.5, 1, [['F3'], ['F3', 'C3', 'F4', 'C4'], ['F3'], ['F3'], ['F3'], ['ELE', 'F3']], ['F3', 'C3', 'F4', 'C4', 'ELE']], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 1.5, 1, [['F3', 'O2'], ['F3'], ['F3'], ['F3'], ['F3', 'C3', 'F4', 'C4'], ['ELE', 'F3', 'F4']], ['F3', 'O2', 'C3', 'F4', 'C4', 'ELE']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 1.5, 1, [[], ['C3'], ['ELE'], [], [], []], ['C3', 'ELE']], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 1.5, 1, [['ERA', 'ERI', 'EMGr', 'C3'], ['EMGr'], ['ELE', 'ERI', 'EMGr'], ['ELC', 'ERA', 'EMGr'], ['EMGr'], ['ELI', 'EMGr']], ['ERA', 'ERI', 'EMGr', 'C3', 'ELE', 'ELC', 'ELI']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 2.0, 1, [[], ['F3', 'C3'], [], [], [], []], ['F3', 'C3']], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 2.0, 1, [['F3'], ['F3'], ['F3'], ['F3'], ['F3', 'F4'], ['F3']], ['F3', 'F4']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 2.0, 1, [[], [], ['ELE'], [], [], []], ['ELE']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 2.5, 1, [[], ['F3'], [], [], [], []], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 2.5, 1, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 3.0, 1, [[], ['F3'], [], [], [], []], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 3.0, 1, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 4.0, 1, [[], ['F3'], [], [], [], []], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 4.0, 1, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 5.0, 1, [['F3'], ['F3'], ['F3'], ['F3'], [], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 1.0, 2, [['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ERA', 'ERE', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERE', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 1.0, 2, [['EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4', 'C4'], ['ELE', 'ERE', 'F3', 'C3', 'F4', 'C4']], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 1.0, 2, [['ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELE', 'ELI', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERE', 'ERI', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELE', 'ELC']], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 1.0, 2, [['ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'O2'], ['ELE', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'O2']], ['ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'O2', 'ELC', 'ELE', 'C4']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 1.5, 2, [['F3'], ['F3', 'C3', 'F4', 'C4'], ['F3'], ['F3'], ['F3'], ['ELE', 'F3']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 1.5, 2, [['F3', 'O2'], ['F3'], ['F3'], ['F3'], ['F3', 'C3', 'F4', 'C4'], ['ELE', 'F3', 'F4']], ['F3', 'F4']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 1.5, 2, [[], ['C3'], ['ELE'], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 1.5, 2, [['ERA', 'ERI', 'EMGr', 'C3'], ['EMGr'], ['ELE', 'ERI', 'EMGr'], ['ELC', 'ERA', 'EMGr'], ['EMGr'], ['ELI', 'EMGr']], ['EMGr']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 2.0, 2, [[], ['F3', 'C3'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 2.0, 2, [['F3'], ['F3'], ['F3'], ['F3'], ['F3', 'F4'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 2.0, 2, [[], [], ['ELE'], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 2.5, 2, [[], ['F3'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 2.5, 2, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 3.0, 2, [[], ['F3'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 3.0, 2, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 4.0, 2, [[], ['F3'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 4.0, 2, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 5.0, 2, [['F3'], ['F3'], ['F3'], ['F3'], [], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 1.0, 3, [['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ERA', 'ERE', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERE', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 1.0, 3, [['EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4', 'C4'], ['ELE', 'ERE', 'F3', 'C3', 'F4', 'C4']], ['F3', 'C3', 'F4', 'C4']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 1.0, 3, [['ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELE', 'ELI', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERE', 'ERI', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 1.0, 3, [['ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'O2'], ['ELE', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'O2']], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'O2', 'ELC', 'ELE']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 1.5, 3, [['F3'], ['F3', 'C3', 'F4', 'C4'], ['F3'], ['F3'], ['F3'], ['ELE', 'F3']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 1.5, 3, [['F3', 'O2'], ['F3'], ['F3'], ['F3'], ['F3', 'C3', 'F4', 'C4'], ['ELE', 'F3', 'F4']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 1.5, 3, [[], ['C3'], ['ELE'], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 1.5, 3, [['ERA', 'ERI', 'EMGr', 'C3'], ['EMGr'], ['ELE', 'ERI', 'EMGr'], ['ELC', 'ERA', 'EMGr'], ['EMGr'], ['ELI', 'EMGr']], ['EMGr']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 2.0, 3, [[], ['F3', 'C3'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 2.0, 3, [['F3'], ['F3'], ['F3'], ['F3'], ['F3', 'F4'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 2.0, 3, [[], [], ['ELE'], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 2.5, 3, [[], ['F3'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 2.5, 3, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 3.0, 3, [[], ['F3'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 3.0, 3, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 4.0, 3, [[], ['F3'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 4.0, 3, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 5.0, 3, [['F3'], ['F3'], ['F3'], ['F3'], [], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 1.0, 4, [['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ERA', 'ERE', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERE', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 1.0, 4, [['EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4', 'C4'], ['ELE', 'ERE', 'F3', 'C3', 'F4', 'C4']], ['F3', 'C3', 'F4', 'C4']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 1.0, 4, [['ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELE', 'ELI', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERE', 'ERI', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 1.0, 4, [['ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'O2'], ['ELE', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'O2']], ['ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'ELC', 'ELE']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 1.5, 4, [['F3'], ['F3', 'C3', 'F4', 'C4'], ['F3'], ['F3'], ['F3'], ['ELE', 'F3']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 1.5, 4, [['F3', 'O2'], ['F3'], ['F3'], ['F3'], ['F3', 'C3', 'F4', 'C4'], ['ELE', 'F3', 'F4']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 1.5, 4, [[], ['C3'], ['ELE'], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 1.5, 4, [['ERA', 'ERI', 'EMGr', 'C3'], ['EMGr'], ['ELE', 'ERI', 'EMGr'], ['ELC', 'ERA', 'EMGr'], ['EMGr'], ['ELI', 'EMGr']], ['EMGr']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 2.0, 4, [[], ['F3', 'C3'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 2.0, 4, [['F3'], ['F3'], ['F3'], ['F3'], ['F3', 'F4'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 2.0, 4, [[], [], ['ELE'], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 2.5, 4, [[], ['F3'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 2.5, 4, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 3.0, 4, [[], ['F3'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 3.0, 4, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 4.0, 4, [[], ['F3'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 4.0, 4, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 5.0, 4, [['F3'], ['F3'], ['F3'], ['F3'], [], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 1.0, 5, [['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ERA', 'ERE', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERE', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 1.0, 5, [['EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4', 'C4'], ['ELE', 'ERE', 'F3', 'C3', 'F4', 'C4']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 1.0, 5, [['ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELE', 'ELI', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERE', 'ERI', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERI', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 1.0, 5, [['ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'O2'], ['ELE', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'O2']], ['ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'ELE']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 1.5, 5, [['F3'], ['F3', 'C3', 'F4', 'C4'], ['F3'], ['F3'], ['F3'], ['ELE', 'F3']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 1.5, 5, [['F3', 'O2'], ['F3'], ['F3'], ['F3'], ['F3', 'C3', 'F4', 'C4'], ['ELE', 'F3', 'F4']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 1.5, 5, [[], ['C3'], ['ELE'], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 1.5, 5, [['ERA', 'ERI', 'EMGr', 'C3'], ['EMGr'], ['ELE', 'ERI', 'EMGr'], ['ELC', 'ERA', 'EMGr'], ['EMGr'], ['ELI', 'EMGr']], ['EMGr']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 2.0, 5, [[], ['F3', 'C3'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 2.0, 5, [['F3'], ['F3'], ['F3'], ['F3'], ['F3', 'F4'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 2.0, 5, [[], [], ['ELE'], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 2.5, 5, [[], ['F3'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 2.5, 5, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 3.0, 5, [[], ['F3'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 3.0, 5, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 4.0, 5, [[], ['F3'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 4.0, 5, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 5.0, 5, [['F3'], ['F3'], ['F3'], ['F3'], [], ['F3']], []], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 1.0, 6, [['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ERA', 'ERE', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERE', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 1.0, 6, [['EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4', 'C4'], ['ELE', 'ERE', 'F3', 'C3', 'F4', 'C4']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 1.0, 6, [['ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELE', 'ELI', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERE', 'ERI', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERI', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 1.0, 6, [['ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'O2'], ['ELE', 'ELI', 'ERB', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'O2']], ['ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 1.5, 6, [['F3'], ['F3', 'C3', 'F4', 'C4'], ['F3'], ['F3'], ['F3'], ['ELE', 'F3']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 1.5, 6, [['F3', 'O2'], ['F3'], ['F3'], ['F3'], ['F3', 'C3', 'F4', 'C4'], ['ELE', 'F3', 'F4']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 1.5, 6, [[], ['C3'], ['ELE'], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 1.5, 6, [['ERA', 'ERI', 'EMGr', 'C3'], ['EMGr'], ['ELE', 'ERI', 'EMGr'], ['ELC', 'ERA', 'EMGr'], ['EMGr'], ['ELI', 'EMGr']], ['EMGr']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 2.0, 6, [[], ['F3', 'C3'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 2.0, 6, [['F3'], ['F3'], ['F3'], ['F3'], ['F3', 'F4'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 2.0, 6, [[], [], ['ELE'], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 2.5, 6, [[], ['F3'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 2.5, 6, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 3.0, 6, [[], ['F3'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 3.0, 6, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 4.0, 6, [[], ['F3'], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 4.0, 6, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'mm', 5.0, 6, [['F3'], ['F3'], ['F3'], ['F3'], [], ['F3']], []], ['S_01/night_2\\EEG_raw', 13000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 13000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 1.0, 1, [['ELI', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'F3', 'C4', 'O2'], ['ELE', 'ERA', 'ERC', 'ERI', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERI', 'F3', 'O1', 'C4'], ['ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'ERI', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELE', 'ERA', 'ERB', 'ERI', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELI', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'ERA', 'ERC', 'ERT ', 'C4', 'ELE', 'ERB', 'EMGc']], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 1.0, 1, [['ELI', 'ERI', 'EMGr', 'F3'], ['ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'F3'], ['ERA', 'ERE', 'ERI', 'F3', 'O1', 'O2'], ['ELI', 'ERI', 'F3', 'C4'], ['ELE', 'ELI', 'ERE', 'ERI', 'F3'], ['ERB', 'ERE', 'ERI', 'F3', 'C4']], ['ELI', 'ERI', 'EMGr', 'F3', 'ERA', 'ERC', 'ERT ', 'ERE', 'O1', 'O2', 'C4', 'ELE', 'ERB']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 1.0, 1, [['ELI', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'C4', 'O2'], ['ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EMGr', 'O1', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'ERI', 'EMGr', 'F3', 'C3', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'O2']], ['ELI', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2', 'ELE', 'ERA', 'ERB', 'ERC', 'ERT ', 'C4', 'EOGl', 'O1', 'ELA']], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 1.0, 1, [['ELI', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C3', 'M2', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'O1', 'M2'], ['ELE', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F4', 'C4', 'O2']], ['ELI', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc', 'ELE', 'ERB', 'ERC', 'EMGr', 'C4', 'ERA', 'ERT ']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 1.5, 1, [['ELI', 'EMGr', 'F3'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'F3'], ['ERI', 'F3'], ['ELE', 'ELI', 'ERI', 'F3', 'C4'], ['ELI', 'F3'], ['F3']], ['ELI', 'EMGr', 'F3', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'ELE', 'C4']], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 1.5, 1, [['EMGr', 'F3'], ['ERC', 'ERT ', 'F3'], ['ERI', 'F3'], ['ELI', 'ERI', 'F3', 'C4'], ['ELI', 'F3'], ['ERB', 'ERI', 'F3']], ['EMGr', 'F3', 'ERC', 'ERT ', 'ERI', 'ELI', 'C4', 'ERB']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 1.5, 1, [['ERI', 'EMGr'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI'], [], ['ELI', 'ERA', 'ERI', 'O1', 'C4'], [], []], ['ERI', 'EMGr', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'O1', 'C4']], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 1.5, 1, [['EOGr', 'EMGl'], ['ERB', 'C3', 'O2'], ['C3'], ['ERA', 'ERC', 'ERE'], ['ERA'], ['ERI']], ['EOGr', 'EMGl', 'ERB', 'C3', 'O2', 'ERA', 'ERC', 'ERE', 'ERI']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 2.0, 1, [['EMGr', 'F3'], ['ELI', 'F3'], [], ['ERI', 'F3', 'C4'], [], []], ['EMGr', 'F3', 'ELI', 'ERI', 'C4']], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 2.0, 1, [['EMGr', 'F3'], ['F3'], ['ERI', 'F3'], ['ELI', 'ERI', 'F3', 'C4'], ['ELI', 'F3'], ['F3']], ['EMGr', 'F3', 'ERI', 'ELI', 'C4']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 2.0, 1, [[], [], [], ['C4'], [], []], ['C4']], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 2.0, 1, [[], [], [], ['ERA', 'ERE'], [], []], ['ERA', 'ERE']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 2.5, 1, [[], ['F3'], [], ['F3'], [], []], ['F3']], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 2.5, 1, [['EMGr', 'F3'], ['F3'], ['ERI', 'F3'], ['F3', 'C4'], ['ELI', 'F3'], ['F3']], ['EMGr', 'F3', 'ERI', 'C4', 'ELI']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 3.0, 1, [['EMGr', 'F3'], ['F3'], ['ERI', 'F3'], ['F3', 'C4'], ['ELI', 'F3'], ['F3']], ['EMGr', 'F3', 'ERI', 'C4', 'ELI']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 4.0, 1, [['EMGr', 'F3'], ['F3'], ['ERI'], ['F3', 'C4'], ['ELI', 'F3'], ['F3']], ['EMGr', 'F3', 'ERI', 'C4', 'ELI']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 5.0, 1, [['EMGr', 'F3'], ['F3'], [], ['F3'], ['ELI'], ['F3']], ['EMGr', 'F3', 'ELI']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 1.0, 2, [['ELI', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'F3', 'C4', 'O2'], ['ELE', 'ERA', 'ERC', 'ERI', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERI', 'F3', 'O1', 'C4'], ['ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'ERI', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELE', 'ERA', 'ERB', 'ERI', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ELI', 'ERE', 'ERI', 'F3', 'O1', 'O2', 'ERA', 'ERC', 'C4', 'ELE', 'ERB']], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 1.0, 2, [['ELI', 'ERI', 'EMGr', 'F3'], ['ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'F3'], ['ERA', 'ERE', 'ERI', 'F3', 'O1', 'O2'], ['ELI', 'ERI', 'F3', 'C4'], ['ELE', 'ELI', 'ERE', 'ERI', 'F3'], ['ERB', 'ERE', 'ERI', 'F3', 'C4']], ['ELI', 'ERI', 'F3', 'ERA', 'ERE']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 1.0, 2, [['ELI', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'C4', 'O2'], ['ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EMGr', 'O1', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'ERI', 'EMGr', 'F3', 'C3', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'O2']], ['ELI', 'ERE', 'ERI', 'EMGl', 'EMGr', 'F3', 'O2', 'ELE', 'ERA', 'ERB', 'ERC', 'C4', 'O1']], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 1.0, 2, [['ELI', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C3', 'M2', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'O1', 'M2'], ['ELE', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F4', 'C4', 'O2']], ['ELI', 'ERE', 'ERI', 'EOGr', 'EMGl', 'C3', 'O1', 'M2', 'O2', 'ELE', 'ERB', 'ERC', 'EMGr', 'C4', 'ERA']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 1.5, 2, [['ELI', 'EMGr', 'F3'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'F3'], ['ERI', 'F3'], ['ELE', 'ELI', 'ERI', 'F3', 'C4'], ['ELI', 'F3'], ['F3']], ['ELI', 'F3', 'ERI']], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 1.5, 2, [['EMGr', 'F3'], ['ERC', 'ERT ', 'F3'], ['ERI', 'F3'], ['ELI', 'ERI', 'F3', 'C4'], ['ELI', 'F3'], ['ERB', 'ERI', 'F3']], ['F3', 'ERI', 'ELI']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 1.5, 2, [['ERI', 'EMGr'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI'], [], ['ELI', 'ERA', 'ERI', 'O1', 'C4'], [], []], ['ERI']], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 1.5, 2, [['EOGr', 'EMGl'], ['ERB', 'C3', 'O2'], ['C3'], ['ERA', 'ERC', 'ERE'], ['ERA'], ['ERI']], ['C3', 'ERA']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 2.0, 2, [['EMGr', 'F3'], ['ELI', 'F3'], [], ['ERI', 'F3', 'C4'], [], []], ['F3']], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 2.0, 2, [['EMGr', 'F3'], ['F3'], ['ERI', 'F3'], ['ELI', 'ERI', 'F3', 'C4'], ['ELI', 'F3'], ['F3']], ['F3', 'ERI', 'ELI']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 2.0, 2, [[], [], [], ['C4'], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 2.0, 2, [[], [], [], ['ERA', 'ERE'], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 2.5, 2, [[], ['F3'], [], ['F3'], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 2.5, 2, [['EMGr', 'F3'], ['F3'], ['ERI', 'F3'], ['F3', 'C4'], ['ELI', 'F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 3.0, 2, [['EMGr', 'F3'], ['F3'], ['ERI', 'F3'], ['F3', 'C4'], ['ELI', 'F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 4.0, 2, [['EMGr', 'F3'], ['F3'], ['ERI'], ['F3', 'C4'], ['ELI', 'F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 5.0, 2, [['EMGr', 'F3'], ['F3'], [], ['F3'], ['ELI'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 1.0, 3, [['ELI', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'F3', 'C4', 'O2'], ['ELE', 'ERA', 'ERC', 'ERI', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERI', 'F3', 'O1', 'C4'], ['ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'ERI', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELE', 'ERA', 'ERB', 'ERI', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ERI', 'F3', 'O1', 'O2', 'ERA', 'ERC', 'C4', 'ELE']], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 1.0, 3, [['ELI', 'ERI', 'EMGr', 'F3'], ['ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'F3'], ['ERA', 'ERE', 'ERI', 'F3', 'O1', 'O2'], ['ELI', 'ERI', 'F3', 'C4'], ['ELE', 'ELI', 'ERE', 'ERI', 'F3'], ['ERB', 'ERE', 'ERI', 'F3', 'C4']], ['ERI', 'F3']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 1.0, 3, [['ELI', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'C4', 'O2'], ['ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EMGr', 'O1', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'ERI', 'EMGr', 'F3', 'C3', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'O2']], ['ELI', 'ERE', 'ERI', 'EMGl', 'EMGr', 'O2', 'ELE', 'ERA', 'ERC', 'C4']], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 1.0, 3, [['ELI', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C3', 'M2', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'O1', 'M2'], ['ELE', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F4', 'C4', 'O2']], ['ELI', 'ERE', 'EMGl', 'C3', 'O2', 'ELE', 'ERC', 'EMGr', 'ERA']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 1.5, 3, [['ELI', 'EMGr', 'F3'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'F3'], ['ERI', 'F3'], ['ELE', 'ELI', 'ERI', 'F3', 'C4'], ['ELI', 'F3'], ['F3']], ['F3', 'ERI']], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 1.5, 3, [['EMGr', 'F3'], ['ERC', 'ERT ', 'F3'], ['ERI', 'F3'], ['ELI', 'ERI', 'F3', 'C4'], ['ELI', 'F3'], ['ERB', 'ERI', 'F3']], ['F3']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 1.5, 3, [['ERI', 'EMGr'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI'], [], ['ELI', 'ERA', 'ERI', 'O1', 'C4'], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 1.5, 3, [['EOGr', 'EMGl'], ['ERB', 'C3', 'O2'], ['C3'], ['ERA', 'ERC', 'ERE'], ['ERA'], ['ERI']], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 2.0, 3, [['EMGr', 'F3'], ['ELI', 'F3'], [], ['ERI', 'F3', 'C4'], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 2.0, 3, [['EMGr', 'F3'], ['F3'], ['ERI', 'F3'], ['ELI', 'ERI', 'F3', 'C4'], ['ELI', 'F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 2.0, 3, [[], [], [], ['C4'], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 2.0, 3, [[], [], [], ['ERA', 'ERE'], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 2.5, 3, [[], ['F3'], [], ['F3'], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 2.5, 3, [['EMGr', 'F3'], ['F3'], ['ERI', 'F3'], ['F3', 'C4'], ['ELI', 'F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 3.0, 3, [['EMGr', 'F3'], ['F3'], ['ERI', 'F3'], ['F3', 'C4'], ['ELI', 'F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 4.0, 3, [['EMGr', 'F3'], ['F3'], ['ERI'], ['F3', 'C4'], ['ELI', 'F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 5.0, 3, [['EMGr', 'F3'], ['F3'], [], ['F3'], ['ELI'], ['F3']], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 1.0, 4, [['ELI', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'F3', 'C4', 'O2'], ['ELE', 'ERA', 'ERC', 'ERI', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERI', 'F3', 'O1', 'C4'], ['ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'ERI', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELE', 'ERA', 'ERB', 'ERI', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ERI', 'F3', 'O1', 'ERA', 'C4', 'ELE']], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 1.0, 4, [['ELI', 'ERI', 'EMGr', 'F3'], ['ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'F3'], ['ERA', 'ERE', 'ERI', 'F3', 'O1', 'O2'], ['ELI', 'ERI', 'F3', 'C4'], ['ELE', 'ELI', 'ERE', 'ERI', 'F3'], ['ERB', 'ERE', 'ERI', 'F3', 'C4']], ['ERI', 'F3']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 1.0, 4, [['ELI', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'C4', 'O2'], ['ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EMGr', 'O1', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'ERI', 'EMGr', 'F3', 'C3', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'O2']], ['ERE', 'ERI', 'ELE', 'ERA', 'C4']], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 1.0, 4, [['ELI', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C3', 'M2', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'O1', 'M2'], ['ELE', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F4', 'C4', 'O2']], ['ELI', 'ERE', 'ELE', 'EMGr', 'ERA']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 1.5, 4, [['ELI', 'EMGr', 'F3'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'F3'], ['ERI', 'F3'], ['ELE', 'ELI', 'ERI', 'F3', 'C4'], ['ELI', 'F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 1.5, 4, [['EMGr', 'F3'], ['ERC', 'ERT ', 'F3'], ['ERI', 'F3'], ['ELI', 'ERI', 'F3', 'C4'], ['ELI', 'F3'], ['ERB', 'ERI', 'F3']], ['F3']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 1.5, 4, [['ERI', 'EMGr'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI'], [], ['ELI', 'ERA', 'ERI', 'O1', 'C4'], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 1.5, 4, [['EOGr', 'EMGl'], ['ERB', 'C3', 'O2'], ['C3'], ['ERA', 'ERC', 'ERE'], ['ERA'], ['ERI']], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 2.0, 4, [['EMGr', 'F3'], ['ELI', 'F3'], [], ['ERI', 'F3', 'C4'], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 2.0, 4, [['EMGr', 'F3'], ['F3'], ['ERI', 'F3'], ['ELI', 'ERI', 'F3', 'C4'], ['ELI', 'F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 2.0, 4, [[], [], [], ['C4'], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 2.0, 4, [[], [], [], ['ERA', 'ERE'], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 2.5, 4, [[], ['F3'], [], ['F3'], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 2.5, 4, [['EMGr', 'F3'], ['F3'], ['ERI', 'F3'], ['F3', 'C4'], ['ELI', 'F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 3.0, 4, [['EMGr', 'F3'], ['F3'], ['ERI', 'F3'], ['F3', 'C4'], ['ELI', 'F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 4.0, 4, [['EMGr', 'F3'], ['F3'], ['ERI'], ['F3', 'C4'], ['ELI', 'F3'], ['F3']], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 5.0, 4, [['EMGr', 'F3'], ['F3'], [], ['F3'], ['ELI'], ['F3']], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 1.0, 5, [['ELI', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'F3', 'C4', 'O2'], ['ELE', 'ERA', 'ERC', 'ERI', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERI', 'F3', 'O1', 'C4'], ['ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'ERI', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELE', 'ERA', 'ERB', 'ERI', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ERI', 'F3', 'ERA', 'C4']], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 1.0, 5, [['ELI', 'ERI', 'EMGr', 'F3'], ['ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'F3'], ['ERA', 'ERE', 'ERI', 'F3', 'O1', 'O2'], ['ELI', 'ERI', 'F3', 'C4'], ['ELE', 'ELI', 'ERE', 'ERI', 'F3'], ['ERB', 'ERE', 'ERI', 'F3', 'C4']], ['ERI', 'F3']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 1.0, 5, [['ELI', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'C4', 'O2'], ['ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EMGr', 'O1', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'ERI', 'EMGr', 'F3', 'C3', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'O2']], ['ERE', 'ERI', 'ELE', 'ERA', 'C4']], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 1.0, 5, [['ELI', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C3', 'M2', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'O1', 'M2'], ['ELE', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F4', 'C4', 'O2']], ['ERE', 'ELE', 'EMGr']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 1.5, 5, [['ELI', 'EMGr', 'F3'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'F3'], ['ERI', 'F3'], ['ELE', 'ELI', 'ERI', 'F3', 'C4'], ['ELI', 'F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 1.5, 5, [['EMGr', 'F3'], ['ERC', 'ERT ', 'F3'], ['ERI', 'F3'], ['ELI', 'ERI', 'F3', 'C4'], ['ELI', 'F3'], ['ERB', 'ERI', 'F3']], ['F3']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 1.5, 5, [['ERI', 'EMGr'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI'], [], ['ELI', 'ERA', 'ERI', 'O1', 'C4'], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 1.5, 5, [['EOGr', 'EMGl'], ['ERB', 'C3', 'O2'], ['C3'], ['ERA', 'ERC', 'ERE'], ['ERA'], ['ERI']], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 2.0, 5, [['EMGr', 'F3'], ['ELI', 'F3'], [], ['ERI', 'F3', 'C4'], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 2.0, 5, [['EMGr', 'F3'], ['F3'], ['ERI', 'F3'], ['ELI', 'ERI', 'F3', 'C4'], ['ELI', 'F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 2.0, 5, [[], [], [], ['C4'], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 2.0, 5, [[], [], [], ['ERA', 'ERE'], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 2.5, 5, [[], ['F3'], [], ['F3'], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 2.5, 5, [['EMGr', 'F3'], ['F3'], ['ERI', 'F3'], ['F3', 'C4'], ['ELI', 'F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 3.0, 5, [['EMGr', 'F3'], ['F3'], ['ERI', 'F3'], ['F3', 'C4'], ['ELI', 'F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 4.0, 5, [['EMGr', 'F3'], ['F3'], ['ERI'], ['F3', 'C4'], ['ELI', 'F3'], ['F3']], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 5.0, 5, [['EMGr', 'F3'], ['F3'], [], ['F3'], ['ELI'], ['F3']], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 1.0, 6, [['ELI', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'F3', 'C4', 'O2'], ['ELE', 'ERA', 'ERC', 'ERI', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERI', 'F3', 'O1', 'C4'], ['ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'ERI', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELE', 'ERA', 'ERB', 'ERI', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2']], ['ERI', 'F3']], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 1.0, 6, [['ELI', 'ERI', 'EMGr', 'F3'], ['ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'F3'], ['ERA', 'ERE', 'ERI', 'F3', 'O1', 'O2'], ['ELI', 'ERI', 'F3', 'C4'], ['ELE', 'ELI', 'ERE', 'ERI', 'F3'], ['ERB', 'ERE', 'ERI', 'F3', 'C4']], ['ERI', 'F3']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 1.0, 6, [['ELI', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'C4', 'O2'], ['ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EMGr', 'O1', 'C4'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'ERI', 'EMGr', 'F3', 'C3', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERB', 'ERI', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'O2']], ['ERI']], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 1.0, 6, [['ELI', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'C3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELE', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C3', 'M2', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'EMGr', 'M1', 'O1', 'M2'], ['ELE', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'C4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERB', 'ERE', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F4', 'C4', 'O2']], ['ERE']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 1.5, 6, [['ELI', 'EMGr', 'F3'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'F3'], ['ERI', 'F3'], ['ELE', 'ELI', 'ERI', 'F3', 'C4'], ['ELI', 'F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 1.5, 6, [['EMGr', 'F3'], ['ERC', 'ERT ', 'F3'], ['ERI', 'F3'], ['ELI', 'ERI', 'F3', 'C4'], ['ELI', 'F3'], ['ERB', 'ERI', 'F3']], ['F3']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 1.5, 6, [['ERI', 'EMGr'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI'], [], ['ELI', 'ERA', 'ERI', 'O1', 'C4'], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 1.5, 6, [['EOGr', 'EMGl'], ['ERB', 'C3', 'O2'], ['C3'], ['ERA', 'ERC', 'ERE'], ['ERA'], ['ERI']], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 2.0, 6, [['EMGr', 'F3'], ['ELI', 'F3'], [], ['ERI', 'F3', 'C4'], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 2.0, 6, [['EMGr', 'F3'], ['F3'], ['ERI', 'F3'], ['ELI', 'ERI', 'F3', 'C4'], ['ELI', 'F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 2.0, 6, [[], [], [], ['C4'], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 2.0, 6, [[], [], [], ['ERA', 'ERE'], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 2.5, 6, [[], ['F3'], [], ['F3'], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 2.5, 6, [['EMGr', 'F3'], ['F3'], ['ERI', 'F3'], ['F3', 'C4'], ['ELI', 'F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 3.0, 6, [['EMGr', 'F3'], ['F3'], ['ERI', 'F3'], ['F3', 'C4'], ['ELI', 'F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 4.0, 6, [['EMGr', 'F3'], ['F3'], ['ERI'], ['F3', 'C4'], ['ELI', 'F3'], ['F3']], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'mm', 5.0, 6, [['EMGr', 'F3'], ['F3'], [], ['F3'], ['ELI'], ['F3']], []], ['S_01/night_2\\EEG_raw', 16210, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 16210, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 1.0, 1, [['ERB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERI', 'EMGr', 'F3', 'F4'], ['ELI', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ELA', 'ELE', 'ELI', 'ERC', 'EMGl', 'ERA', 'ERT ']], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 1.0, 1, [['ERB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELI', 'EMGr', 'F3', 'F4', 'O2'], ['F3'], ['F3', 'O1', 'O2'], ['ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELI', 'ERC', 'EMGl', 'EMGc', 'ELA']], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 1.0, 1, [['ELC', 'ELI', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELC', 'ELI', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc', 'ELA', 'ELE', 'ERC', 'C4', 'ELB', 'ERA', 'ERT ']], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 1.0, 1, [['ELC', 'ELI', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2'], ['ELC', 'ELE', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4'], ['ELC', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1']], ['ELC', 'ELI', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'EMGc', 'ELA', 'ELE', 'O1', 'O2', 'ERC', 'C3', 'C4', 'ERA', 'F4', 'ERT ']], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 1.5, 1, [['F3'], ['ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELI'], ['F3'], ['F3'], []], ['F3', 'ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc']], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 1.5, 1, [['EOGr', 'F3'], ['ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELI', 'F3'], ['F3'], ['F3'], ['EOGr', 'F3', 'O1', 'O2']], ['EOGr', 'F3', 'ELI', 'ERC', 'EMGl', 'EMGr', 'EMGc', 'ELA', 'O1', 'O2']], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 1.5, 1, [['F3'], ['ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'EMGc'], [], [], ['F3'], ['F3']], ['F3', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc']], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 1.5, 1, [['ERB', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELE', 'EOGl'], ['ERC', 'ERI'], ['ELI', 'ERA', 'ERI', 'EMGr'], ['ERA', 'ERI', 'EMGr'], ['ERT ', 'ERI', 'EMGr', 'F3']], ['ERB', 'EMGr', 'F3', 'EMGc', 'ELA', 'ELE', 'EOGl', 'ERC', 'ERI', 'ELI', 'ERA', 'ERT ']], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 2.0, 1, [[], ['EMGc'], [], ['F3'], [], []], ['EMGc', 'F3']], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 2.0, 1, [['F3'], ['ERC', 'EOGr', 'EMGr', 'EMGc'], ['ELI'], ['F3'], ['F3'], []], ['F3', 'ERC', 'EOGr', 'EMGr', 'EMGc', 'ELI']], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 2.5, 1, [[], [], [], ['F3'], [], []], ['F3']], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 2.5, 1, [['F3'], ['ERC', 'EMGc'], ['ELI'], ['F3'], ['F3'], []], ['F3', 'ERC', 'EMGc', 'ELI']], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 3.0, 1, [[], [], [], ['F3'], [], []], ['F3']], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 3.0, 1, [[], ['EMGc'], ['ELI'], ['F3'], ['F3'], []], ['EMGc', 'ELI', 'F3']], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 4.0, 1, [[], [], [], ['F3'], [], []], ['F3']], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 4.0, 1, [[], [], ['ELI'], ['F3'], ['F3'], []], ['ELI', 'F3']], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 5.0, 1, [[], [], [], ['F3'], [], []], ['F3']], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 5.0, 1, [[], [], ['ELI'], ['F3'], ['F3'], []], ['ELI', 'F3']], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 1.0, 2, [['ERB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERI', 'EMGr', 'F3', 'F4'], ['ELI', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ELI', 'ERC', 'EMGl', 'ERA', 'ERT ']], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 1.0, 2, [['ERB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELI', 'EMGr', 'F3', 'F4', 'O2'], ['F3'], ['F3', 'O1', 'O2'], ['ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGr', 'EMGr', 'F3', 'O1', 'O2', 'ELI']], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 1.0, 2, [['ELC', 'ELI', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELI', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc', 'ELA', 'ERC', 'C4', 'ERA', 'ERT ']], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 1.0, 2, [['ELC', 'ELI', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2'], ['ELC', 'ELE', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4'], ['ELC', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1']], ['ELC', 'ELI', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'ELE', 'O1', 'O2', 'ERC', 'C3', 'C4', 'ERA', 'F4', 'ERT ']], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 1.5, 2, [['F3'], ['ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELI'], ['F3'], ['F3'], []], ['F3', 'ELI']], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 1.5, 2, [['EOGr', 'F3'], ['ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELI', 'F3'], ['F3'], ['F3'], ['EOGr', 'F3', 'O1', 'O2']], ['EOGr', 'F3', 'ELI']], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 1.5, 2, [['F3'], ['ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'EMGc'], [], [], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 1.5, 2, [['ERB', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELE', 'EOGl'], ['ERC', 'ERI'], ['ELI', 'ERA', 'ERI', 'EMGr'], ['ERA', 'ERI', 'EMGr'], ['ERT ', 'ERI', 'EMGr', 'F3']], ['EMGr', 'ERI', 'ERA']], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 2.0, 2, [[], ['EMGc'], [], ['F3'], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 2.0, 2, [['F3'], ['ERC', 'EOGr', 'EMGr', 'EMGc'], ['ELI'], ['F3'], ['F3'], []], ['F3']], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 2.5, 2, [[], [], [], ['F3'], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 2.5, 2, [['F3'], ['ERC', 'EMGc'], ['ELI'], ['F3'], ['F3'], []], ['F3']], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 3.0, 2, [[], [], [], ['F3'], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 3.0, 2, [[], ['EMGc'], ['ELI'], ['F3'], ['F3'], []], ['F3']], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 4.0, 2, [[], [], [], ['F3'], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 4.0, 2, [[], [], ['ELI'], ['F3'], ['F3'], []], ['F3']], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 5.0, 2, [[], [], [], ['F3'], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 5.0, 2, [[], [], ['ELI'], ['F3'], ['F3'], []], ['F3']], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 1.0, 3, [['ERB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERI', 'EMGr', 'F3', 'F4'], ['ELI', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'F4', 'EMGc']], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 1.0, 3, [['ERB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELI', 'EMGr', 'F3', 'F4', 'O2'], ['F3'], ['F3', 'O1', 'O2'], ['ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EMGr', 'F3']], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 1.0, 3, [['ELC', 'ELI', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELI', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc', 'C4', 'ERA']], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 1.0, 3, [['ELC', 'ELI', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2'], ['ELC', 'ELE', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4'], ['ELC', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1']], ['ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'ELE', 'O1', 'O2', 'ERA']], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 1.5, 3, [['F3'], ['ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELI'], ['F3'], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 1.5, 3, [['EOGr', 'F3'], ['ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELI', 'F3'], ['F3'], ['F3'], ['EOGr', 'F3', 'O1', 'O2']], ['F3']], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 1.5, 3, [['F3'], ['ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'EMGc'], [], [], ['F3'], ['F3']], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 1.5, 3, [['ERB', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELE', 'EOGl'], ['ERC', 'ERI'], ['ELI', 'ERA', 'ERI', 'EMGr'], ['ERA', 'ERI', 'EMGr'], ['ERT ', 'ERI', 'EMGr', 'F3']], ['EMGr', 'ERI']], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 2.0, 3, [[], ['EMGc'], [], ['F3'], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 2.0, 3, [['F3'], ['ERC', 'EOGr', 'EMGr', 'EMGc'], ['ELI'], ['F3'], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 2.5, 3, [[], [], [], ['F3'], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 2.5, 3, [['F3'], ['ERC', 'EMGc'], ['ELI'], ['F3'], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 3.0, 3, [[], [], [], ['F3'], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 3.0, 3, [[], ['EMGc'], ['ELI'], ['F3'], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 4.0, 3, [[], [], [], ['F3'], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 4.0, 3, [[], [], ['ELI'], ['F3'], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 5.0, 3, [[], [], [], ['F3'], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 5.0, 3, [[], [], ['ELI'], ['F3'], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 1.0, 4, [['ERB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERI', 'EMGr', 'F3', 'F4'], ['ELI', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERI', 'EMGr', 'F3', 'F4']], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 1.0, 4, [['ERB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELI', 'EMGr', 'F3', 'F4', 'O2'], ['F3'], ['F3', 'O1', 'O2'], ['ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3']], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 1.0, 4, [['ELC', 'ELI', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELI', 'ERI', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'O2', 'C4']], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 1.0, 4, [['ELC', 'ELI', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2'], ['ELC', 'ELE', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4'], ['ELC', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1']], ['ERI', 'EOGl', 'EMGl', 'EMGr']], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 1.5, 4, [['F3'], ['ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELI'], ['F3'], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 1.5, 4, [['EOGr', 'F3'], ['ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELI', 'F3'], ['F3'], ['F3'], ['EOGr', 'F3', 'O1', 'O2']], ['F3']], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 1.5, 4, [['F3'], ['ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'EMGc'], [], [], ['F3'], ['F3']], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 1.5, 4, [['ERB', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELE', 'EOGl'], ['ERC', 'ERI'], ['ELI', 'ERA', 'ERI', 'EMGr'], ['ERA', 'ERI', 'EMGr'], ['ERT ', 'ERI', 'EMGr', 'F3']], ['ERI']], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 2.0, 4, [[], ['EMGc'], [], ['F3'], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 2.0, 4, [['F3'], ['ERC', 'EOGr', 'EMGr', 'EMGc'], ['ELI'], ['F3'], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 2.5, 4, [[], [], [], ['F3'], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 2.5, 4, [['F3'], ['ERC', 'EMGc'], ['ELI'], ['F3'], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 3.0, 4, [[], [], [], ['F3'], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 3.0, 4, [[], ['EMGc'], ['ELI'], ['F3'], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 4.0, 4, [[], [], [], ['F3'], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 4.0, 4, [[], [], ['ELI'], ['F3'], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 5.0, 4, [[], [], [], ['F3'], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 5.0, 4, [[], [], ['ELI'], ['F3'], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 1.0, 5, [['ERB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERI', 'EMGr', 'F3', 'F4'], ['ELI', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EMGr', 'F3']], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 1.0, 5, [['ERB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELI', 'EMGr', 'F3', 'F4', 'O2'], ['F3'], ['F3', 'O1', 'O2'], ['ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3']], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 1.0, 5, [['ELC', 'ELI', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELI', 'EOGl', 'EMGr', 'F3']], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 1.0, 5, [['ELC', 'ELI', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2'], ['ELC', 'ELE', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4'], ['ELC', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1']], ['EOGl', 'EMGl', 'EMGr']], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 1.5, 5, [['F3'], ['ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELI'], ['F3'], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 1.5, 5, [['EOGr', 'F3'], ['ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELI', 'F3'], ['F3'], ['F3'], ['EOGr', 'F3', 'O1', 'O2']], ['F3']], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 1.5, 5, [['F3'], ['ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'EMGc'], [], [], ['F3'], ['F3']], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 1.5, 5, [['ERB', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELE', 'EOGl'], ['ERC', 'ERI'], ['ELI', 'ERA', 'ERI', 'EMGr'], ['ERA', 'ERI', 'EMGr'], ['ERT ', 'ERI', 'EMGr', 'F3']], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 2.0, 5, [[], ['EMGc'], [], ['F3'], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 2.0, 5, [['F3'], ['ERC', 'EOGr', 'EMGr', 'EMGc'], ['ELI'], ['F3'], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 2.5, 5, [[], [], [], ['F3'], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 2.5, 5, [['F3'], ['ERC', 'EMGc'], ['ELI'], ['F3'], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 3.0, 5, [[], [], [], ['F3'], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 3.0, 5, [[], ['EMGc'], ['ELI'], ['F3'], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 4.0, 5, [[], [], [], ['F3'], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 4.0, 5, [[], [], ['ELI'], ['F3'], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 5.0, 5, [[], [], [], ['F3'], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 5.0, 5, [[], [], ['ELI'], ['F3'], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 1.0, 6, [['ERB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERI', 'EMGr', 'F3', 'F4'], ['ELI', 'ERA', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EMGr', 'F3']], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 1.0, 6, [['ERB', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELI', 'EMGr', 'F3', 'F4', 'O2'], ['F3'], ['F3', 'O1', 'O2'], ['ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3']], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 1.0, 6, [['ELC', 'ELI', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGl', 'EMGr', 'F3']], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 1.0, 6, [['ELC', 'ELI', 'ERB', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2'], ['ELC', 'ELE', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'O1', 'F4', 'C4', 'O2'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4'], ['ELC', 'ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1']], ['EOGl', 'EMGl', 'EMGr']], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 1.5, 6, [['F3'], ['ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELI'], ['F3'], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 1.5, 6, [['EOGr', 'F3'], ['ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELI', 'F3'], ['F3'], ['F3'], ['EOGr', 'F3', 'O1', 'O2']], ['F3']], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 1.5, 6, [['F3'], ['ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'EMGc'], [], [], ['F3'], ['F3']], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 1.5, 6, [['ERB', 'EMGr', 'F3', 'EMGc'], ['ELA', 'ELE', 'EOGl'], ['ERC', 'ERI'], ['ELI', 'ERA', 'ERI', 'EMGr'], ['ERA', 'ERI', 'EMGr'], ['ERT ', 'ERI', 'EMGr', 'F3']], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 2.0, 6, [[], ['EMGc'], [], ['F3'], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 2.0, 6, [['F3'], ['ERC', 'EOGr', 'EMGr', 'EMGc'], ['ELI'], ['F3'], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 2.5, 6, [[], [], [], ['F3'], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 2.5, 6, [['F3'], ['ERC', 'EMGc'], ['ELI'], ['F3'], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 3.0, 6, [[], [], [], ['F3'], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 3.0, 6, [[], ['EMGc'], ['ELI'], ['F3'], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 4.0, 6, [[], [], [], ['F3'], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 4.0, 6, [[], [], ['ELI'], ['F3'], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'mm', 5.0, 6, [[], [], [], ['F3'], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'mm', 5.0, 6, [[], [], ['ELI'], ['F3'], ['F3'], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 19760, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 1.0, 1, [['ELE', 'ELI', 'ERI', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELE', 'ERI', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ERI', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERE', 'ERI', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERI', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELE', 'ELI', 'ERI', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'F4', 'M1', 'ERE', 'ELC', 'EOGr', 'EOGl']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 1.0, 1, [['ELE', 'F3'], ['ELE', 'F3', 'O1'], ['ELE', 'F3', 'C3', 'O1', 'F4', 'C4'], ['F3', 'O1', 'C4', 'O2'], ['F3', 'O1'], ['F3', 'O1']], ['ELE', 'F3', 'O1', 'C3', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 1.0, 1, [['ELE', 'ELI', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELE', 'ELI', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'ERC', 'M1', 'ELC', 'ERE']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 1.0, 1, [['ELI', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'M2', 'F4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2'], ['ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'O2']], ['ELI', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'O2', 'ELC', 'ERC', 'ELE', 'M1', 'ERE', 'C4']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 1.5, 1, [['ELE', 'F3', 'O2'], ['ELE', 'F3', 'C3', 'O1'], ['F3', 'O1'], ['F3', 'O1', 'O2'], ['F3', 'O1', 'O2'], ['EOGl', 'F3', 'C3', 'F4']], ['ELE', 'F3', 'O2', 'C3', 'O1', 'EOGl', 'F4']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 1.5, 1, [['ELE', 'F3'], ['F3', 'O1'], ['ELE', 'F3', 'O1'], ['F3', 'O1'], ['F3', 'O1'], ['F3', 'O1']], ['ELE', 'F3', 'O1']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 1.5, 1, [['ELE', 'M2'], ['ELE', 'O1'], ['ELE', 'M1'], [], ['C3', 'O2'], ['O2']], ['ELE', 'M2', 'O1', 'M1', 'C3', 'O2']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 1.5, 1, [['ERI', 'EMGl', 'EMGr', 'M2'], ['ERI', 'EMGl', 'EMGr', 'M2', 'O2'], ['ERI', 'EMGr', 'M1', 'M2', 'O2'], ['ERI', 'EOGl', 'EMGl', 'EMGr'], ['ERI', 'EOGl', 'EMGr', 'C3', 'C4', 'O2'], ['ERI', 'EOGl', 'EMGl', 'EMGr', 'O2']], ['ERI', 'EMGl', 'EMGr', 'M2', 'O2', 'M1', 'EOGl', 'C3', 'C4']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 2.0, 1, [['F3'], ['F3', 'O1'], ['F3', 'O1'], ['F3'], ['F3'], ['F3']], ['F3', 'O1']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 2.0, 1, [['F3'], ['F3'], ['F3', 'O1'], ['F3', 'O1'], ['F3'], ['F3', 'O1']], ['F3', 'O1']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 2.0, 1, [[], [], ['M2'], [], [], []], ['M2']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 2.5, 1, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 2.5, 1, [['F3'], ['F3'], ['F3', 'O1'], ['F3', 'O1'], ['F3'], ['F3']], ['F3', 'O1']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 3.0, 1, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 3.0, 1, [['F3'], ['F3'], ['F3', 'O1'], ['F3'], ['F3'], ['F3']], ['F3', 'O1']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 4.0, 1, [['F3'], [], [], [], [], []], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 4.0, 1, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 5.0, 1, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 1.0, 2, [['ELE', 'ELI', 'ERI', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELE', 'ERI', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ERI', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERE', 'ERI', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERI', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELE', 'ERI', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'F4']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 1.0, 2, [['ELE', 'F3'], ['ELE', 'F3', 'O1'], ['ELE', 'F3', 'C3', 'O1', 'F4', 'C4'], ['F3', 'O1', 'C4', 'O2'], ['F3', 'O1'], ['F3', 'O1']], ['ELE', 'F3', 'O1', 'C4']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 1.0, 2, [['ELE', 'ELI', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELE', 'ELI', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'ELC']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 1.0, 2, [['ELI', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'M2', 'F4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2'], ['ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'O2']], ['ELI', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'O2', 'ELC', 'ELE']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 1.5, 2, [['ELE', 'F3', 'O2'], ['ELE', 'F3', 'C3', 'O1'], ['F3', 'O1'], ['F3', 'O1', 'O2'], ['F3', 'O1', 'O2'], ['EOGl', 'F3', 'C3', 'F4']], ['ELE', 'F3', 'O2', 'O1']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 1.5, 2, [['ELE', 'F3'], ['F3', 'O1'], ['ELE', 'F3', 'O1'], ['F3', 'O1'], ['F3', 'O1'], ['F3', 'O1']], ['F3', 'O1']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 1.5, 2, [['ELE', 'M2'], ['ELE', 'O1'], ['ELE', 'M1'], [], ['C3', 'O2'], ['O2']], ['ELE', 'O2']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 1.5, 2, [['ERI', 'EMGl', 'EMGr', 'M2'], ['ERI', 'EMGl', 'EMGr', 'M2', 'O2'], ['ERI', 'EMGr', 'M1', 'M2', 'O2'], ['ERI', 'EOGl', 'EMGl', 'EMGr'], ['ERI', 'EOGl', 'EMGr', 'C3', 'C4', 'O2'], ['ERI', 'EOGl', 'EMGl', 'EMGr', 'O2']], ['ERI', 'EMGl', 'EMGr', 'M2', 'O2', 'EOGl']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 2.0, 2, [['F3'], ['F3', 'O1'], ['F3', 'O1'], ['F3'], ['F3'], ['F3']], ['F3', 'O1']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 2.0, 2, [['F3'], ['F3'], ['F3', 'O1'], ['F3', 'O1'], ['F3'], ['F3', 'O1']], ['F3', 'O1']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 2.0, 2, [[], [], ['M2'], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 2.5, 2, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 2.5, 2, [['F3'], ['F3'], ['F3', 'O1'], ['F3', 'O1'], ['F3'], ['F3']], ['F3', 'O1']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 3.0, 2, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 3.0, 2, [['F3'], ['F3'], ['F3', 'O1'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 4.0, 2, [['F3'], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 4.0, 2, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 5.0, 2, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 1.0, 3, [['ELE', 'ELI', 'ERI', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELE', 'ERI', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ERI', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERE', 'ERI', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERI', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELE', 'ERI', 'F3', 'C3', 'O1', 'C4', 'O2', 'F4']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 1.0, 3, [['ELE', 'F3'], ['ELE', 'F3', 'O1'], ['ELE', 'F3', 'C3', 'O1', 'F4', 'C4'], ['F3', 'O1', 'C4', 'O2'], ['F3', 'O1'], ['F3', 'O1']], ['ELE', 'F3', 'O1']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 1.0, 3, [['ELE', 'ELI', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELE', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2', 'ELC']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 1.0, 3, [['ELI', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'M2', 'F4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2'], ['ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'O2']], ['ELI', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'O2', 'ELE']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 1.5, 3, [['ELE', 'F3', 'O2'], ['ELE', 'F3', 'C3', 'O1'], ['F3', 'O1'], ['F3', 'O1', 'O2'], ['F3', 'O1', 'O2'], ['EOGl', 'F3', 'C3', 'F4']], ['F3', 'O1']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 1.5, 3, [['ELE', 'F3'], ['F3', 'O1'], ['ELE', 'F3', 'O1'], ['F3', 'O1'], ['F3', 'O1'], ['F3', 'O1']], ['F3', 'O1']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 1.5, 3, [['ELE', 'M2'], ['ELE', 'O1'], ['ELE', 'M1'], [], ['C3', 'O2'], ['O2']], ['ELE']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 1.5, 3, [['ERI', 'EMGl', 'EMGr', 'M2'], ['ERI', 'EMGl', 'EMGr', 'M2', 'O2'], ['ERI', 'EMGr', 'M1', 'M2', 'O2'], ['ERI', 'EOGl', 'EMGl', 'EMGr'], ['ERI', 'EOGl', 'EMGr', 'C3', 'C4', 'O2'], ['ERI', 'EOGl', 'EMGl', 'EMGr', 'O2']], ['ERI', 'EMGr', 'M2', 'EOGl']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 2.0, 3, [['F3'], ['F3', 'O1'], ['F3', 'O1'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 2.0, 3, [['F3'], ['F3'], ['F3', 'O1'], ['F3', 'O1'], ['F3'], ['F3', 'O1']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 2.0, 3, [[], [], ['M2'], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 2.5, 3, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 2.5, 3, [['F3'], ['F3'], ['F3', 'O1'], ['F3', 'O1'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 3.0, 3, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 3.0, 3, [['F3'], ['F3'], ['F3', 'O1'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 4.0, 3, [['F3'], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 4.0, 3, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 5.0, 3, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 1.0, 4, [['ELE', 'ELI', 'ERI', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELE', 'ERI', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ERI', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERE', 'ERI', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERI', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERI', 'F3', 'C3', 'O1', 'C4', 'O2', 'F4']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 1.0, 4, [['ELE', 'F3'], ['ELE', 'F3', 'O1'], ['ELE', 'F3', 'C3', 'O1', 'F4', 'C4'], ['F3', 'O1', 'C4', 'O2'], ['F3', 'O1'], ['F3', 'O1']], ['F3', 'O1']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 1.0, 4, [['ELE', 'ELI', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELE', 'ERI', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 1.0, 4, [['ELI', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'M2', 'F4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2'], ['ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'O2']], ['ERA', 'ERI', 'EMGl', 'EMGr', 'C3', 'F4', 'O2', 'ELE']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 1.5, 4, [['ELE', 'F3', 'O2'], ['ELE', 'F3', 'C3', 'O1'], ['F3', 'O1'], ['F3', 'O1', 'O2'], ['F3', 'O1', 'O2'], ['EOGl', 'F3', 'C3', 'F4']], ['F3', 'O1']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 1.5, 4, [['ELE', 'F3'], ['F3', 'O1'], ['ELE', 'F3', 'O1'], ['F3', 'O1'], ['F3', 'O1'], ['F3', 'O1']], ['F3', 'O1']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 1.5, 4, [['ELE', 'M2'], ['ELE', 'O1'], ['ELE', 'M1'], [], ['C3', 'O2'], ['O2']], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 1.5, 4, [['ERI', 'EMGl', 'EMGr', 'M2'], ['ERI', 'EMGl', 'EMGr', 'M2', 'O2'], ['ERI', 'EMGr', 'M1', 'M2', 'O2'], ['ERI', 'EOGl', 'EMGl', 'EMGr'], ['ERI', 'EOGl', 'EMGr', 'C3', 'C4', 'O2'], ['ERI', 'EOGl', 'EMGl', 'EMGr', 'O2']], ['ERI', 'EMGr']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 2.0, 4, [['F3'], ['F3', 'O1'], ['F3', 'O1'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 2.0, 4, [['F3'], ['F3'], ['F3', 'O1'], ['F3', 'O1'], ['F3'], ['F3', 'O1']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 2.0, 4, [[], [], ['M2'], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 2.5, 4, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 2.5, 4, [['F3'], ['F3'], ['F3', 'O1'], ['F3', 'O1'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 3.0, 4, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 3.0, 4, [['F3'], ['F3'], ['F3', 'O1'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 4.0, 4, [['F3'], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 4.0, 4, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 5.0, 4, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 1.0, 5, [['ELE', 'ELI', 'ERI', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELE', 'ERI', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ERI', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERE', 'ERI', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERI', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERI', 'F3', 'C3', 'O1', 'C4', 'O2', 'F4']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 1.0, 5, [['ELE', 'F3'], ['ELE', 'F3', 'O1'], ['ELE', 'F3', 'C3', 'O1', 'F4', 'C4'], ['F3', 'O1', 'C4', 'O2'], ['F3', 'O1'], ['F3', 'O1']], ['F3', 'O1']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 1.0, 5, [['ELE', 'ELI', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERI', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 1.0, 5, [['ELI', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'M2', 'F4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2'], ['ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'O2']], ['ERA', 'ERI', 'EMGl', 'EMGr', 'F4', 'O2']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 1.5, 5, [['ELE', 'F3', 'O2'], ['ELE', 'F3', 'C3', 'O1'], ['F3', 'O1'], ['F3', 'O1', 'O2'], ['F3', 'O1', 'O2'], ['EOGl', 'F3', 'C3', 'F4']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 1.5, 5, [['ELE', 'F3'], ['F3', 'O1'], ['ELE', 'F3', 'O1'], ['F3', 'O1'], ['F3', 'O1'], ['F3', 'O1']], ['F3', 'O1']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 1.5, 5, [['ELE', 'M2'], ['ELE', 'O1'], ['ELE', 'M1'], [], ['C3', 'O2'], ['O2']], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 1.5, 5, [['ERI', 'EMGl', 'EMGr', 'M2'], ['ERI', 'EMGl', 'EMGr', 'M2', 'O2'], ['ERI', 'EMGr', 'M1', 'M2', 'O2'], ['ERI', 'EOGl', 'EMGl', 'EMGr'], ['ERI', 'EOGl', 'EMGr', 'C3', 'C4', 'O2'], ['ERI', 'EOGl', 'EMGl', 'EMGr', 'O2']], ['ERI', 'EMGr']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 2.0, 5, [['F3'], ['F3', 'O1'], ['F3', 'O1'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 2.0, 5, [['F3'], ['F3'], ['F3', 'O1'], ['F3', 'O1'], ['F3'], ['F3', 'O1']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 2.0, 5, [[], [], ['M2'], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 2.5, 5, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 2.5, 5, [['F3'], ['F3'], ['F3', 'O1'], ['F3', 'O1'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 3.0, 5, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 3.0, 5, [['F3'], ['F3'], ['F3', 'O1'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 4.0, 5, [['F3'], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 4.0, 5, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 5.0, 5, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 1.0, 6, [['ELE', 'ELI', 'ERI', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELE', 'ERI', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELE', 'ERI', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERE', 'ERI', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERI', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERI', 'F3', 'C3', 'O1', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 1.0, 6, [['ELE', 'F3'], ['ELE', 'F3', 'O1'], ['ELE', 'F3', 'C3', 'O1', 'F4', 'C4'], ['F3', 'O1', 'C4', 'O2'], ['F3', 'O1'], ['F3', 'O1']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 1.0, 6, [['ELE', 'ELI', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELE', 'ELI', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ELI', 'ERA', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'M2', 'F4', 'C4', 'O2'], ['ELC', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERI', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 1.0, 6, [['ELI', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'F4', 'O2'], ['ELC', 'ELI', 'ERA', 'ERC', 'ERI', 'EMGl', 'EMGr', 'M2', 'F4', 'O2'], ['ELE', 'ELI', 'ERA', 'ERI', 'EMGl', 'EMGr', 'M1', 'C3', 'M2', 'F4', 'O2'], ['ELE', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'O2'], ['ELC', 'ELE', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'F4', 'O2']], ['ERA', 'ERI', 'EMGl', 'EMGr', 'F4', 'O2']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 1.5, 6, [['ELE', 'F3', 'O2'], ['ELE', 'F3', 'C3', 'O1'], ['F3', 'O1'], ['F3', 'O1', 'O2'], ['F3', 'O1', 'O2'], ['EOGl', 'F3', 'C3', 'F4']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 1.5, 6, [['ELE', 'F3'], ['F3', 'O1'], ['ELE', 'F3', 'O1'], ['F3', 'O1'], ['F3', 'O1'], ['F3', 'O1']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 1.5, 6, [['ELE', 'M2'], ['ELE', 'O1'], ['ELE', 'M1'], [], ['C3', 'O2'], ['O2']], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 1.5, 6, [['ERI', 'EMGl', 'EMGr', 'M2'], ['ERI', 'EMGl', 'EMGr', 'M2', 'O2'], ['ERI', 'EMGr', 'M1', 'M2', 'O2'], ['ERI', 'EOGl', 'EMGl', 'EMGr'], ['ERI', 'EOGl', 'EMGr', 'C3', 'C4', 'O2'], ['ERI', 'EOGl', 'EMGl', 'EMGr', 'O2']], ['ERI', 'EMGr']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 2.0, 6, [['F3'], ['F3', 'O1'], ['F3', 'O1'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 2.0, 6, [['F3'], ['F3'], ['F3', 'O1'], ['F3', 'O1'], ['F3'], ['F3', 'O1']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 2.0, 6, [[], [], ['M2'], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 2.5, 6, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 2.5, 6, [['F3'], ['F3'], ['F3', 'O1'], ['F3', 'O1'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 3.0, 6, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 3.0, 6, [['F3'], ['F3'], ['F3', 'O1'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 4.0, 6, [['F3'], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 4.0, 6, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'mm', 5.0, 6, [['F3'], ['F3'], ['F3'], ['F3'], ['F3'], ['F3']], ['F3']], ['S_01/night_2\\EEG_raw', 21070, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_01/night_2\\EEG_raw', 21070, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 1.0, 1, [['EOGr', 'EOGl', 'EMGl', 'F3', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'F4', 'EMGc']], ['EOGr', 'EOGl', 'EMGl', 'F3', 'F4', 'EMGc', 'ERC', 'EMGr', 'ELC', 'ELI', 'O1', 'O2']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 1.0, 1, [['EOGr', 'EOGl', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['ELC', 'ERC', 'EOGl', 'EMGl', 'F4'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['EOGr', 'EOGl', 'EMGr', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'F4', 'EMGc']], ['EOGr', 'EOGl', 'F4', 'EMGc', 'ERC', 'EMGl', 'ELC', 'EMGr']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc', 'ELI', 'C3', 'ERI', 'F4']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc', 'C3', 'C4', 'ELE', 'ELI', 'ERI']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 1.5, 1, [['EOGr', 'EOGl', 'F4', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['ELC', 'ERC', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['EOGl'], ['ERC', 'EOGr', 'EOGl', 'F4', 'EMGc']], ['EOGr', 'EOGl', 'F4', 'EMGc', 'EMGl', 'ELC', 'ERC']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 1.5, 1, [['EOGr', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['ELC', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['EOGl'], ['ERC', 'EOGr', 'EOGl', 'F4', 'EMGc']], ['EOGr', 'EOGl', 'F4', 'ELC', 'ERC', 'EMGc']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 1.5, 1, [['EOGr', 'EMGr', 'EMGc'], ['ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc']], ['EOGr', 'EMGr', 'EMGc', 'ERC', 'EMGl', 'ELA', 'ELC']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 1.5, 1, [['ELA', 'ERC', 'EMGl', 'EMGr', 'O2', 'EMGc'], ['M1', 'M2', 'O2'], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELC', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELC', 'EOGr', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERI', 'M1', 'M2']], ['ELA', 'ERC', 'EMGl', 'EMGr', 'O2', 'EMGc', 'M1', 'M2', 'ERA', 'EOGr', 'ELC', 'F3', 'O1', 'ERI']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 2.0, 1, [['EOGr', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['ELC', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4', 'EMGc']], ['EOGr', 'EOGl', 'F4', 'ELC', 'EMGc']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 2.0, 1, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4', 'EMGc']], ['EOGl', 'F4', 'EOGr', 'EMGc']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 2.0, 1, [[], ['EOGr'], ['EOGr'], [], [], ['EOGr']], ['EOGr']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 2.0, 1, [['ERC'], ['M2'], [], ['ELC'], ['EMGc'], ['ERI', 'M1']], ['ERC', 'M2', 'ELC', 'EMGc', 'ERI', 'M1']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 2.5, 1, [['EOGl', 'F4'], ['EOGl', 'F4'], ['ELC', 'EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4', 'EMGc']], ['EOGl', 'F4', 'ELC', 'EMGc']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 2.5, 1, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4', 'EMGc']], ['EOGl', 'F4', 'EOGr', 'EMGc']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 3.0, 1, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 3.0, 1, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 4.0, 1, [['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4']], ['F4', 'EOGl']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 4.0, 1, [['F4'], ['F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4']], ['F4', 'EOGl']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], ['EOGl'], ['EOGl'], []], ['EOGl']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 5.0, 1, [['F4'], ['F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl']], ['F4', 'EOGl']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 1.0, 2, [['EOGr', 'EOGl', 'EMGl', 'F3', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'F4', 'EMGc']], ['EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc', 'ERC', 'EMGr']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 1.0, 2, [['EOGr', 'EOGl', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['ELC', 'ERC', 'EOGl', 'EMGl', 'F4'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['EOGr', 'EOGl', 'EMGr', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'F4', 'EMGc']], ['EOGr', 'EOGl', 'F4', 'EMGc', 'ERC', 'EMGl']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc', 'ELI']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 1.5, 2, [['EOGr', 'EOGl', 'F4', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['ELC', 'ERC', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['EOGl'], ['ERC', 'EOGr', 'EOGl', 'F4', 'EMGc']], ['EOGr', 'EOGl', 'F4', 'EMGc']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 1.5, 2, [['EOGr', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['ELC', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['EOGl'], ['ERC', 'EOGr', 'EOGl', 'F4', 'EMGc']], ['EOGr', 'EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 1.5, 2, [['EOGr', 'EMGr', 'EMGc'], ['ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc']], ['EOGr', 'EMGr', 'EMGc', 'ERC', 'EMGl']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 1.5, 2, [['ELA', 'ERC', 'EMGl', 'EMGr', 'O2', 'EMGc'], ['M1', 'M2', 'O2'], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELC', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELC', 'EOGr', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERI', 'M1', 'M2']], ['O2', 'EMGc', 'M1', 'M2', 'ELC']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 2.0, 2, [['EOGr', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['ELC', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4', 'EMGc']], ['EOGr', 'EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 2.0, 2, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4', 'EMGc']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 2.0, 2, [[], ['EOGr'], ['EOGr'], [], [], ['EOGr']], ['EOGr']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 2.0, 2, [['ERC'], ['M2'], [], ['ELC'], ['EMGc'], ['ERI', 'M1']], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 2.5, 2, [['EOGl', 'F4'], ['EOGl', 'F4'], ['ELC', 'EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4', 'EMGc']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 2.5, 2, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4', 'EMGc']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 3.0, 2, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 3.0, 2, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 4.0, 2, [['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4']], ['F4', 'EOGl']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 4.0, 2, [['F4'], ['F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4']], ['F4', 'EOGl']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], ['EOGl'], ['EOGl'], []], ['EOGl']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 5.0, 2, [['F4'], ['F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl']], ['F4', 'EOGl']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 1.0, 3, [['EOGr', 'EOGl', 'EMGl', 'F3', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'F4', 'EMGc']], ['EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc', 'ERC']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 1.0, 3, [['EOGr', 'EOGl', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['ELC', 'ERC', 'EOGl', 'EMGl', 'F4'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['EOGr', 'EOGl', 'EMGr', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'F4', 'EMGc']], ['EOGr', 'EOGl', 'F4', 'EMGc', 'ERC', 'EMGl']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'M2', 'F4', 'EMGc']], ['ELA', 'ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'M2', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 1.5, 3, [['EOGr', 'EOGl', 'F4', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['ELC', 'ERC', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['EOGl'], ['ERC', 'EOGr', 'EOGl', 'F4', 'EMGc']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 1.5, 3, [['EOGr', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['ELC', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['EOGl'], ['ERC', 'EOGr', 'EOGl', 'F4', 'EMGc']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 1.5, 3, [['EOGr', 'EMGr', 'EMGc'], ['ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc']], ['EOGr', 'EMGr', 'EMGc', 'ERC', 'EMGl']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 1.5, 3, [['ELA', 'ERC', 'EMGl', 'EMGr', 'O2', 'EMGc'], ['M1', 'M2', 'O2'], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELC', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELC', 'EOGr', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERI', 'M1', 'M2']], ['EMGc', 'M1', 'M2', 'ELC']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 2.0, 3, [['EOGr', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['ELC', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4', 'EMGc']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 2.0, 3, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4', 'EMGc']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 2.0, 3, [[], ['EOGr'], ['EOGr'], [], [], ['EOGr']], []], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 2.0, 3, [['ERC'], ['M2'], [], ['ELC'], ['EMGc'], ['ERI', 'M1']], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 2.5, 3, [['EOGl', 'F4'], ['EOGl', 'F4'], ['ELC', 'EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4', 'EMGc']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 2.5, 3, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4', 'EMGc']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 3.0, 3, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 3.0, 3, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 4.0, 3, [['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4']], ['F4', 'EOGl']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 4.0, 3, [['F4'], ['F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4']], ['F4', 'EOGl']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], ['EOGl'], ['EOGl'], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 5.0, 3, [['F4'], ['F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl']], ['F4', 'EOGl']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 1.0, 4, [['EOGr', 'EOGl', 'EMGl', 'F3', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'F4', 'EMGc']], ['EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 1.0, 4, [['EOGr', 'EOGl', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['ELC', 'ERC', 'EOGl', 'EMGl', 'F4'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['EOGr', 'EOGl', 'EMGr', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'F4', 'EMGc']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'M2', 'F4', 'EMGc']], ['ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'M2', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc']], ['ELA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 1.5, 4, [['EOGr', 'EOGl', 'F4', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['ELC', 'ERC', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['EOGl'], ['ERC', 'EOGr', 'EOGl', 'F4', 'EMGc']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 1.5, 4, [['EOGr', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['ELC', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['EOGl'], ['ERC', 'EOGr', 'EOGl', 'F4', 'EMGc']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 1.5, 4, [['EOGr', 'EMGr', 'EMGc'], ['ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc']], ['EOGr', 'EMGr', 'EMGc']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 1.5, 4, [['ELA', 'ERC', 'EMGl', 'EMGr', 'O2', 'EMGc'], ['M1', 'M2', 'O2'], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELC', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELC', 'EOGr', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERI', 'M1', 'M2']], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 2.0, 4, [['EOGr', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['ELC', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4', 'EMGc']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 2.0, 4, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4', 'EMGc']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 2.0, 4, [[], ['EOGr'], ['EOGr'], [], [], ['EOGr']], []], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 2.0, 4, [['ERC'], ['M2'], [], ['ELC'], ['EMGc'], ['ERI', 'M1']], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 2.5, 4, [['EOGl', 'F4'], ['EOGl', 'F4'], ['ELC', 'EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4', 'EMGc']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 2.5, 4, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4', 'EMGc']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 3.0, 4, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 3.0, 4, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 4.0, 4, [['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 4.0, 4, [['F4'], ['F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4']], ['F4', 'EOGl']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], ['EOGl'], ['EOGl'], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 5.0, 4, [['F4'], ['F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl']], ['F4', 'EOGl']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 1.0, 5, [['EOGr', 'EOGl', 'EMGl', 'F3', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'F4', 'EMGc']], ['EOGr', 'EOGl', 'EMGl', 'EMGc']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 1.0, 5, [['EOGr', 'EOGl', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['ELC', 'ERC', 'EOGl', 'EMGl', 'F4'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['EOGr', 'EOGl', 'EMGr', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'F4', 'EMGc']], ['EOGl']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'M2', 'F4', 'EMGc']], ['ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc']], ['ELA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 1.5, 5, [['EOGr', 'EOGl', 'F4', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['ELC', 'ERC', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['EOGl'], ['ERC', 'EOGr', 'EOGl', 'F4', 'EMGc']], ['EOGl']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 1.5, 5, [['EOGr', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['ELC', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['EOGl'], ['ERC', 'EOGr', 'EOGl', 'F4', 'EMGc']], ['EOGl']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 1.5, 5, [['EOGr', 'EMGr', 'EMGc'], ['ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc']], ['EOGr', 'EMGc']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 1.5, 5, [['ELA', 'ERC', 'EMGl', 'EMGr', 'O2', 'EMGc'], ['M1', 'M2', 'O2'], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELC', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELC', 'EOGr', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERI', 'M1', 'M2']], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 2.0, 5, [['EOGr', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['ELC', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4', 'EMGc']], ['EOGl']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 2.0, 5, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4', 'EMGc']], ['EOGl']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 2.0, 5, [[], ['EOGr'], ['EOGr'], [], [], ['EOGr']], []], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 2.0, 5, [['ERC'], ['M2'], [], ['ELC'], ['EMGc'], ['ERI', 'M1']], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 2.5, 5, [['EOGl', 'F4'], ['EOGl', 'F4'], ['ELC', 'EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4', 'EMGc']], ['EOGl']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 2.5, 5, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4', 'EMGc']], ['EOGl']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 3.0, 5, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4']], ['EOGl']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 3.0, 5, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4']], ['EOGl']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 4.0, 5, [['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4']], []], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 4.0, 5, [['F4'], ['F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4']], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], ['EOGl'], ['EOGl'], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 5.0, 5, [['F4'], ['F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl']], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 1.0, 6, [['EOGr', 'EOGl', 'EMGl', 'F3', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'F4', 'EMGc']], ['EOGr', 'EOGl', 'EMGc']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 1.0, 6, [['EOGr', 'EOGl', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['ELC', 'ERC', 'EOGl', 'EMGl', 'F4'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['EOGr', 'EOGl', 'EMGr', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EOGl', 'F4', 'EMGc']], ['EOGl']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'M2', 'O2', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'O2', 'EMGc'], ['ELC', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'M2', 'F4', 'EMGc']], ['ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc']], ['ELA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 1.5, 6, [['EOGr', 'EOGl', 'F4', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['ELC', 'ERC', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['EOGl'], ['ERC', 'EOGr', 'EOGl', 'F4', 'EMGc']], ['EOGl']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 1.5, 6, [['EOGr', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['ELC', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['EOGl'], ['ERC', 'EOGr', 'EOGl', 'F4', 'EMGc']], ['EOGl']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 1.5, 6, [['EOGr', 'EMGr', 'EMGc'], ['ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EMGc'], ['ELC', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc']], ['EOGr', 'EMGc']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 1.5, 6, [['ELA', 'ERC', 'EMGl', 'EMGr', 'O2', 'EMGc'], ['M1', 'M2', 'O2'], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELC', 'M1', 'F3', 'M2', 'O2', 'EMGc'], ['ELC', 'EOGr', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERI', 'M1', 'M2']], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 2.0, 6, [['EOGr', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['ELC', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4', 'EMGc']], ['EOGl']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 2.0, 6, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4', 'EMGc']], ['EOGl']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 2.0, 6, [[], ['EOGr'], ['EOGr'], [], [], ['EOGr']], []], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 2.0, 6, [['ERC'], ['M2'], [], ['ELC'], ['EMGc'], ['ERI', 'M1']], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 2.5, 6, [['EOGl', 'F4'], ['EOGl', 'F4'], ['ELC', 'EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4', 'EMGc']], ['EOGl']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 2.5, 6, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4', 'EMGc']], ['EOGl']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 3.0, 6, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4']], ['EOGl']], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 3.0, 6, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4']], ['EOGl']], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 4.0, 6, [['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4']], []], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 4.0, 6, [['F4'], ['F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl', 'F4']], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], ['EOGl'], ['EOGl'], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'mm', 5.0, 6, [['F4'], ['F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl']], []], ['S_11/night_1\\EEG_raw', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 1.0, 1, [['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['EOGr', 'EOGl', 'O1', 'O2', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'O2', 'EMGc']], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc', 'ELA', 'C3', 'O1', 'O2', 'ELC', 'F3']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 1.0, 1, [['EOGl', 'EMGl', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'O1', 'O2', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'F3', 'O1', 'O2', 'EMGc']], ['EOGl', 'EMGl', 'F4', 'EMGc', 'ERC', 'EOGr', 'EMGr', 'O1', 'O2', 'ELA', 'ELC', 'F3']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'M2', 'F4', 'O2', 'EMGc', 'M1', 'F3', 'C3', 'C4', 'ELC', 'ERA']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc', 'ELC', 'M1', 'F3', 'C3']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 1.5, 1, [['EOGl', 'EMGl', 'F4', 'EMGc'], ['ERC', 'EOGl'], ['EOGl', 'EMGr'], ['EOGr', 'EOGl', 'O1', 'O2'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl', 'EMGc']], ['EOGl', 'EMGl', 'F4', 'EMGc', 'ERC', 'EMGr', 'EOGr', 'O1', 'O2']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 1.5, 1, [['EOGl', 'EMGl', 'F4', 'EMGc'], ['ERC', 'EOGl', 'EMGr'], ['EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl', 'EMGc']], ['EOGl', 'EMGl', 'F4', 'EMGc', 'ERC', 'EMGr', 'EOGr']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 1.5, 1, [['ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'EOGr', 'EMGr', 'M2', 'EMGc'], ['ERC', 'EOGr', 'EMGr', 'F3', 'EMGc'], ['EOGr', 'EMGc'], ['ERC', 'EOGr', 'C3', 'O2', 'EMGc'], ['EOGr', 'O2']], ['ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc', 'M2', 'F3', 'C3', 'O2']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 1.5, 1, [['ELA', 'ELB', 'M2', 'O2'], ['ELA', 'EMGr', 'M2'], ['ELA', 'ERC', 'EMGl', 'EMGr', 'F3', 'C4', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EMGl', 'O1', 'EMGc'], ['ELA', 'EMGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc']], ['ELA', 'ELB', 'M2', 'O2', 'EMGr', 'ERC', 'EMGl', 'F3', 'C4', 'EMGc', 'ELC', 'EOGr', 'O1', 'C3', 'ERA']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 2.0, 1, [['EOGl', 'F4'], ['ERC', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGr', 'EOGl'], ['EOGl']], ['EOGl', 'F4', 'ERC', 'EOGr']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 2.0, 1, [['EOGl', 'F4', 'EMGc'], ['ERC', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGr', 'EOGl'], ['EOGl']], ['EOGl', 'F4', 'EMGc', 'ERC', 'EOGr']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 2.0, 1, [['ELA'], [], ['ERC'], ['EMGl'], [], []], ['ELA', 'ERC', 'EMGl']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 2.5, 1, [['EOGl', 'F4'], ['ERC', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl', 'F4', 'ERC']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 2.5, 1, [['EOGl', 'F4'], ['ERC', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGr', 'EOGl'], ['EOGl']], ['EOGl', 'F4', 'ERC', 'EOGr']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 3.0, 1, [['EOGl', 'F4'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 3.0, 1, [['EOGl', 'F4'], ['ERC', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl', 'F4', 'ERC']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 4.0, 1, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], [], []], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 4.0, 1, [['EOGl', 'F4'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 5.0, 1, [['EOGl'], [], ['EOGl'], ['EOGl'], [], []], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 5.0, 1, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 1.0, 2, [['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['EOGr', 'EOGl', 'O1', 'O2', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'O2', 'EMGc']], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'C3', 'O1', 'O2']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 1.0, 2, [['EOGl', 'EMGl', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'O1', 'O2', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'F3', 'O1', 'O2', 'EMGc']], ['EOGl', 'EMGl', 'EMGc', 'EOGr', 'EMGr']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc', 'M1', 'F3', 'C3', 'C4', 'ELC', 'ERA']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc', 'ELC', 'F3', 'C3']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 1.5, 2, [['EOGl', 'EMGl', 'F4', 'EMGc'], ['ERC', 'EOGl'], ['EOGl', 'EMGr'], ['EOGr', 'EOGl', 'O1', 'O2'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl', 'EMGc']], ['EOGl', 'EOGr']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 1.5, 2, [['EOGl', 'EMGl', 'F4', 'EMGc'], ['ERC', 'EOGl', 'EMGr'], ['EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl', 'EMGc']], ['EOGl', 'EOGr']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 1.5, 2, [['ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'EOGr', 'EMGr', 'M2', 'EMGc'], ['ERC', 'EOGr', 'EMGr', 'F3', 'EMGc'], ['EOGr', 'EMGc'], ['ERC', 'EOGr', 'C3', 'O2', 'EMGc'], ['EOGr', 'O2']], ['ERC', 'EOGr', 'EMGr', 'EMGc', 'O2']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 1.5, 2, [['ELA', 'ELB', 'M2', 'O2'], ['ELA', 'EMGr', 'M2'], ['ELA', 'ERC', 'EMGl', 'EMGr', 'F3', 'C4', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EMGl', 'O1', 'EMGc'], ['ELA', 'EMGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc']], ['ELA', 'M2', 'EMGr', 'EMGl', 'EMGc']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 2.0, 2, [['EOGl', 'F4'], ['ERC', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGr', 'EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 2.0, 2, [['EOGl', 'F4', 'EMGc'], ['ERC', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGr', 'EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 2.0, 2, [['ELA'], [], ['ERC'], ['EMGl'], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 2.5, 2, [['EOGl', 'F4'], ['ERC', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 2.5, 2, [['EOGl', 'F4'], ['ERC', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGr', 'EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 3.0, 2, [['EOGl', 'F4'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 3.0, 2, [['EOGl', 'F4'], ['ERC', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 4.0, 2, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], [], []], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 4.0, 2, [['EOGl', 'F4'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 5.0, 2, [['EOGl'], [], ['EOGl'], ['EOGl'], [], []], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 5.0, 2, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 1.0, 3, [['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['EOGr', 'EOGl', 'O1', 'O2', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'O1', 'O2']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 1.0, 3, [['EOGl', 'EMGl', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'O1', 'O2', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'F3', 'O1', 'O2', 'EMGc']], ['EOGl', 'EMGc', 'EOGr']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc', 'M1', 'F3', 'C3', 'C4', 'ELC', 'ERA']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc', 'ELC', 'F3', 'C3']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 1.5, 3, [['EOGl', 'EMGl', 'F4', 'EMGc'], ['ERC', 'EOGl'], ['EOGl', 'EMGr'], ['EOGr', 'EOGl', 'O1', 'O2'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl', 'EMGc']], ['EOGl', 'EOGr']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 1.5, 3, [['EOGl', 'EMGl', 'F4', 'EMGc'], ['ERC', 'EOGl', 'EMGr'], ['EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl', 'EMGc']], ['EOGl', 'EOGr']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 1.5, 3, [['ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'EOGr', 'EMGr', 'M2', 'EMGc'], ['ERC', 'EOGr', 'EMGr', 'F3', 'EMGc'], ['EOGr', 'EMGc'], ['ERC', 'EOGr', 'C3', 'O2', 'EMGc'], ['EOGr', 'O2']], ['ERC', 'EOGr', 'EMGr', 'EMGc']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 1.5, 3, [['ELA', 'ELB', 'M2', 'O2'], ['ELA', 'EMGr', 'M2'], ['ELA', 'ERC', 'EMGl', 'EMGr', 'F3', 'C4', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EMGl', 'O1', 'EMGc'], ['ELA', 'EMGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc']], ['ELA', 'EMGl', 'EMGc']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 2.0, 3, [['EOGl', 'F4'], ['ERC', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGr', 'EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 2.0, 3, [['EOGl', 'F4', 'EMGc'], ['ERC', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGr', 'EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 2.0, 3, [['ELA'], [], ['ERC'], ['EMGl'], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 2.5, 3, [['EOGl', 'F4'], ['ERC', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 2.5, 3, [['EOGl', 'F4'], ['ERC', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGr', 'EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 3.0, 3, [['EOGl', 'F4'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 3.0, 3, [['EOGl', 'F4'], ['ERC', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 4.0, 3, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], [], []], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 4.0, 3, [['EOGl', 'F4'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 5.0, 3, [['EOGl'], [], ['EOGl'], ['EOGl'], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 5.0, 3, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 1.0, 4, [['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['EOGr', 'EOGl', 'O1', 'O2', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'EMGc']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 1.0, 4, [['EOGl', 'EMGl', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'O1', 'O2', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'F3', 'O1', 'O2', 'EMGc']], ['EOGl', 'EMGc', 'EOGr']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc', 'F3', 'C3', 'C4']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc', 'F3', 'C3']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 1.5, 4, [['EOGl', 'EMGl', 'F4', 'EMGc'], ['ERC', 'EOGl'], ['EOGl', 'EMGr'], ['EOGr', 'EOGl', 'O1', 'O2'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl', 'EMGc']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 1.5, 4, [['EOGl', 'EMGl', 'F4', 'EMGc'], ['ERC', 'EOGl', 'EMGr'], ['EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl', 'EMGc']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 1.5, 4, [['ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'EOGr', 'EMGr', 'M2', 'EMGc'], ['ERC', 'EOGr', 'EMGr', 'F3', 'EMGc'], ['EOGr', 'EMGc'], ['ERC', 'EOGr', 'C3', 'O2', 'EMGc'], ['EOGr', 'O2']], ['EOGr', 'EMGc']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 1.5, 4, [['ELA', 'ELB', 'M2', 'O2'], ['ELA', 'EMGr', 'M2'], ['ELA', 'ERC', 'EMGl', 'EMGr', 'F3', 'C4', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EMGl', 'O1', 'EMGc'], ['ELA', 'EMGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc']], ['ELA', 'EMGl', 'EMGc']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 2.0, 4, [['EOGl', 'F4'], ['ERC', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGr', 'EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 2.0, 4, [['EOGl', 'F4', 'EMGc'], ['ERC', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGr', 'EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 2.0, 4, [['ELA'], [], ['ERC'], ['EMGl'], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 2.5, 4, [['EOGl', 'F4'], ['ERC', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 2.5, 4, [['EOGl', 'F4'], ['ERC', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGr', 'EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 3.0, 4, [['EOGl', 'F4'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 3.0, 4, [['EOGl', 'F4'], ['ERC', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 4.0, 4, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], [], []], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 4.0, 4, [['EOGl', 'F4'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 5.0, 4, [['EOGl'], [], ['EOGl'], ['EOGl'], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 5.0, 4, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 1.0, 5, [['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['EOGr', 'EOGl', 'O1', 'O2', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'EMGc']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 1.0, 5, [['EOGl', 'EMGl', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'O1', 'O2', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'F3', 'O1', 'O2', 'EMGc']], ['EOGl', 'EMGc', 'EOGr']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc', 'F3']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc', 'F3']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 1.5, 5, [['EOGl', 'EMGl', 'F4', 'EMGc'], ['ERC', 'EOGl'], ['EOGl', 'EMGr'], ['EOGr', 'EOGl', 'O1', 'O2'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl', 'EMGc']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 1.5, 5, [['EOGl', 'EMGl', 'F4', 'EMGc'], ['ERC', 'EOGl', 'EMGr'], ['EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl', 'EMGc']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 1.5, 5, [['ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'EOGr', 'EMGr', 'M2', 'EMGc'], ['ERC', 'EOGr', 'EMGr', 'F3', 'EMGc'], ['EOGr', 'EMGc'], ['ERC', 'EOGr', 'C3', 'O2', 'EMGc'], ['EOGr', 'O2']], ['EOGr', 'EMGc']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 1.5, 5, [['ELA', 'ELB', 'M2', 'O2'], ['ELA', 'EMGr', 'M2'], ['ELA', 'ERC', 'EMGl', 'EMGr', 'F3', 'C4', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EMGl', 'O1', 'EMGc'], ['ELA', 'EMGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc']], ['ELA']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 2.0, 5, [['EOGl', 'F4'], ['ERC', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGr', 'EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 2.0, 5, [['EOGl', 'F4', 'EMGc'], ['ERC', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGr', 'EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 2.0, 5, [['ELA'], [], ['ERC'], ['EMGl'], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 2.5, 5, [['EOGl', 'F4'], ['ERC', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 2.5, 5, [['EOGl', 'F4'], ['ERC', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGr', 'EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 3.0, 5, [['EOGl', 'F4'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 3.0, 5, [['EOGl', 'F4'], ['ERC', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 4.0, 5, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 4.0, 5, [['EOGl', 'F4'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 5.0, 5, [['EOGl'], [], ['EOGl'], ['EOGl'], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 5.0, 5, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 1.0, 6, [['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['EOGr', 'EOGl', 'O1', 'O2', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'EMGc']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 1.0, 6, [['EOGl', 'EMGl', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'O1', 'O2', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'F3', 'O1', 'O2', 'EMGc']], ['EOGl', 'EMGc']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'EOGr', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 1.5, 6, [['EOGl', 'EMGl', 'F4', 'EMGc'], ['ERC', 'EOGl'], ['EOGl', 'EMGr'], ['EOGr', 'EOGl', 'O1', 'O2'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl', 'EMGc']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 1.5, 6, [['EOGl', 'EMGl', 'F4', 'EMGc'], ['ERC', 'EOGl', 'EMGr'], ['EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl'], ['EOGr', 'EOGl', 'EMGc']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 1.5, 6, [['ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'EOGr', 'EMGr', 'M2', 'EMGc'], ['ERC', 'EOGr', 'EMGr', 'F3', 'EMGc'], ['EOGr', 'EMGc'], ['ERC', 'EOGr', 'C3', 'O2', 'EMGc'], ['EOGr', 'O2']], ['EOGr']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 1.5, 6, [['ELA', 'ELB', 'M2', 'O2'], ['ELA', 'EMGr', 'M2'], ['ELA', 'ERC', 'EMGl', 'EMGr', 'F3', 'C4', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EMGl', 'O1', 'EMGc'], ['ELA', 'EMGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc']], ['ELA']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 2.0, 6, [['EOGl', 'F4'], ['ERC', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGr', 'EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 2.0, 6, [['EOGl', 'F4', 'EMGc'], ['ERC', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGr', 'EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 2.0, 6, [['ELA'], [], ['ERC'], ['EMGl'], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 2.5, 6, [['EOGl', 'F4'], ['ERC', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 2.5, 6, [['EOGl', 'F4'], ['ERC', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGr', 'EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 3.0, 6, [['EOGl', 'F4'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 3.0, 6, [['EOGl', 'F4'], ['ERC', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 4.0, 6, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 4.0, 6, [['EOGl', 'F4'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'mm', 5.0, 6, [['EOGl'], [], ['EOGl'], ['EOGl'], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'mm', 5.0, 6, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 100, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 100, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 1.0, 1, [['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERC', 'EOGl', 'EMGr', 'F4'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'EMGc']], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'F4', 'ELC', 'ELI', 'ERC', 'ELA', 'M2', 'O1', 'C4', 'O2', 'F3']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 1.0, 1, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGl', 'F4'], ['ELC', 'ELI', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGl', 'EMGr', 'EMGc']], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'F4', 'ELC', 'ELI', 'ERC']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 1.0, 1, [['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc', 'ELC', 'ELI', 'C3', 'F4']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 1.0, 1, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc', 'ELI', 'EMGr', 'C3']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 1.5, 1, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGr', 'F4', 'EMGc'], ['ELC', 'EOGl', 'F4'], ['ELC', 'EOGl', 'EMGr'], ['EOGl', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr']], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'F4', 'ELC']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 1.5, 1, [['EOGr', 'EOGl', 'EMGr', 'EMGc'], ['EOGl', 'F4'], ['ELC', 'EOGl', 'F4'], ['ELC', 'EOGl', 'EMGr'], ['EOGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGr']], ['EOGr', 'EOGl', 'EMGr', 'EMGc', 'F4', 'ELC']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 1.5, 1, [['ERA', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['EMGr', 'F4'], ['ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGc'], ['EOGr', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EMGl', 'EMGr', 'EMGc']], ['ERA', 'EOGr', 'EMGl', 'EMGr', 'EMGc', 'ERC', 'F4', 'ELC']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 1.5, 1, [['ELA', 'ERA', 'M2', 'C4'], ['ERC', 'M2', 'O2'], ['EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERC', 'EOGr', 'C4'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'C3', 'C4'], ['ELA', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M2']], ['ELA', 'ERA', 'M2', 'C4', 'ERC', 'O2', 'EOGr', 'EMGl', 'EMGr', 'EMGc', 'ELI', 'ELC', 'C3']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 2.0, 1, [['EOGl', 'EMGc'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl', 'EMGc', 'F4']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 2.0, 1, [['EOGr', 'EOGl', 'EMGc'], ['EOGl', 'F4'], ['ELC', 'EOGl', 'F4'], ['ELC', 'EOGl', 'EMGr'], ['EOGl', 'EMGr', 'EMGc'], ['EOGl']], ['EOGr', 'EOGl', 'EMGc', 'F4', 'ELC', 'EMGr']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 2.0, 1, [['EOGr', 'EMGc'], [], [], [], [], []], ['EOGr', 'EMGc']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 2.0, 1, [[], ['M2'], [], [], ['C4'], []], ['M2', 'C4']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 2.5, 1, [['EOGl', 'EMGc'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl', 'EMGc', 'F4']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 2.5, 1, [['EOGl', 'EMGc'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['ELC', 'EOGl', 'EMGr'], ['EOGl', 'EMGc'], ['EOGl']], ['EOGl', 'EMGc', 'F4', 'ELC', 'EMGr']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 3.0, 1, [['EOGl'], ['EOGl', 'F4'], ['EOGl', 'F4'], [], ['EOGl'], ['EOGl']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 3.0, 1, [['EOGl', 'EMGc'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['ELC', 'EOGl'], ['EOGl'], ['EOGl']], ['EOGl', 'EMGc', 'F4', 'ELC']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 4.0, 1, [['EOGl'], ['EOGl', 'F4'], [], [], [], ['EOGl']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 4.0, 1, [['EOGl'], ['F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 5.0, 1, [[], ['EOGl', 'F4'], [], [], [], []], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 5.0, 1, [['EOGl'], ['F4'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 1.0, 2, [['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERC', 'EOGl', 'EMGr', 'F4'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'EMGc']], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'F4', 'ELC', 'ERC', 'ELA']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 1.0, 2, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGl', 'F4'], ['ELC', 'ELI', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGl', 'EMGr', 'EMGc']], ['EOGl', 'EMGl', 'EMGr', 'EMGc', 'F4', 'ELC', 'ELI', 'ERC']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 1.0, 2, [['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc', 'ELC', 'ELI', 'C3']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 1.0, 2, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc', 'ELI', 'EMGr', 'C3']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 1.5, 2, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGr', 'F4', 'EMGc'], ['ELC', 'EOGl', 'F4'], ['ELC', 'EOGl', 'EMGr'], ['EOGl', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr']], ['EOGl', 'EMGr', 'EMGc', 'F4', 'ELC']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 1.5, 2, [['EOGr', 'EOGl', 'EMGr', 'EMGc'], ['EOGl', 'F4'], ['ELC', 'EOGl', 'F4'], ['ELC', 'EOGl', 'EMGr'], ['EOGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGr']], ['EOGl', 'EMGr', 'F4', 'ELC']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 1.5, 2, [['ERA', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['EMGr', 'F4'], ['ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGc'], ['EOGr', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EMGl', 'EMGr', 'EMGc']], ['EOGr', 'EMGl', 'EMGr', 'EMGc']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 1.5, 2, [['ELA', 'ERA', 'M2', 'C4'], ['ERC', 'M2', 'O2'], ['EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERC', 'EOGr', 'C4'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'C3', 'C4'], ['ELA', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M2']], ['ELA', 'ERA', 'M2', 'C4', 'ERC', 'EOGr']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 2.0, 2, [['EOGl', 'EMGc'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 2.0, 2, [['EOGr', 'EOGl', 'EMGc'], ['EOGl', 'F4'], ['ELC', 'EOGl', 'F4'], ['ELC', 'EOGl', 'EMGr'], ['EOGl', 'EMGr', 'EMGc'], ['EOGl']], ['EOGl', 'F4', 'ELC', 'EMGr']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 2.0, 2, [['EOGr', 'EMGc'], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 2.0, 2, [[], ['M2'], [], [], ['C4'], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 2.5, 2, [['EOGl', 'EMGc'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 2.5, 2, [['EOGl', 'EMGc'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['ELC', 'EOGl', 'EMGr'], ['EOGl', 'EMGc'], ['EOGl']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 3.0, 2, [['EOGl'], ['EOGl', 'F4'], ['EOGl', 'F4'], [], ['EOGl'], ['EOGl']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 3.0, 2, [['EOGl', 'EMGc'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['ELC', 'EOGl'], ['EOGl'], ['EOGl']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 4.0, 2, [['EOGl'], ['EOGl', 'F4'], [], [], [], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 4.0, 2, [['EOGl'], ['F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 5.0, 2, [[], ['EOGl', 'F4'], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 5.0, 2, [['EOGl'], ['F4'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 1.0, 3, [['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERC', 'EOGl', 'EMGr', 'F4'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'EMGc']], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 1.0, 3, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGl', 'F4'], ['ELC', 'ELI', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGl', 'EMGr', 'EMGc']], ['EOGl', 'EMGr', 'EMGc']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 1.0, 3, [['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc', 'ELC', 'ELI']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 1.0, 3, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'O1', 'M2', 'C4', 'O2', 'EMGc', 'ELI', 'EMGr', 'C3']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 1.5, 3, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGr', 'F4', 'EMGc'], ['ELC', 'EOGl', 'F4'], ['ELC', 'EOGl', 'EMGr'], ['EOGl', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr']], ['EOGl', 'EMGr']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 1.5, 3, [['EOGr', 'EOGl', 'EMGr', 'EMGc'], ['EOGl', 'F4'], ['ELC', 'EOGl', 'F4'], ['ELC', 'EOGl', 'EMGr'], ['EOGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGr']], ['EOGl', 'EMGr']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 1.5, 3, [['ERA', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['EMGr', 'F4'], ['ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGc'], ['EOGr', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EMGl', 'EMGr', 'EMGc']], ['EOGr', 'EMGl', 'EMGr', 'EMGc']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 1.5, 3, [['ELA', 'ERA', 'M2', 'C4'], ['ERC', 'M2', 'O2'], ['EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERC', 'EOGr', 'C4'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'C3', 'C4'], ['ELA', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M2']], ['EOGr']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 2.0, 3, [['EOGl', 'EMGc'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 2.0, 3, [['EOGr', 'EOGl', 'EMGc'], ['EOGl', 'F4'], ['ELC', 'EOGl', 'F4'], ['ELC', 'EOGl', 'EMGr'], ['EOGl', 'EMGr', 'EMGc'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 2.0, 3, [['EOGr', 'EMGc'], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 2.0, 3, [[], ['M2'], [], [], ['C4'], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 2.5, 3, [['EOGl', 'EMGc'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 2.5, 3, [['EOGl', 'EMGc'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['ELC', 'EOGl', 'EMGr'], ['EOGl', 'EMGc'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 3.0, 3, [['EOGl'], ['EOGl', 'F4'], ['EOGl', 'F4'], [], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 3.0, 3, [['EOGl', 'EMGc'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['ELC', 'EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 4.0, 3, [['EOGl'], ['EOGl', 'F4'], [], [], [], ['EOGl']], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 4.0, 3, [['EOGl'], ['F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 5.0, 3, [[], ['EOGl', 'F4'], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 5.0, 3, [['EOGl'], ['F4'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 1.0, 4, [['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERC', 'EOGl', 'EMGr', 'F4'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'EMGc']], ['EOGl', 'EMGr']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 1.0, 4, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGl', 'F4'], ['ELC', 'ELI', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGl', 'EMGr', 'EMGc']], ['EOGl', 'EMGr', 'EMGc']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 1.0, 4, [['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'C4', 'EMGc', 'ELC']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 1.0, 4, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'O1', 'C4', 'O2', 'EMGc', 'C3']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 1.5, 4, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGr', 'F4', 'EMGc'], ['ELC', 'EOGl', 'F4'], ['ELC', 'EOGl', 'EMGr'], ['EOGl', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 1.5, 4, [['EOGr', 'EOGl', 'EMGr', 'EMGc'], ['EOGl', 'F4'], ['ELC', 'EOGl', 'F4'], ['ELC', 'EOGl', 'EMGr'], ['EOGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGr']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 1.5, 4, [['ERA', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['EMGr', 'F4'], ['ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGc'], ['EOGr', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EMGl', 'EMGr', 'EMGc']], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 1.5, 4, [['ELA', 'ERA', 'M2', 'C4'], ['ERC', 'M2', 'O2'], ['EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERC', 'EOGr', 'C4'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'C3', 'C4'], ['ELA', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M2']], ['EOGr']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 2.0, 4, [['EOGl', 'EMGc'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 2.0, 4, [['EOGr', 'EOGl', 'EMGc'], ['EOGl', 'F4'], ['ELC', 'EOGl', 'F4'], ['ELC', 'EOGl', 'EMGr'], ['EOGl', 'EMGr', 'EMGc'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 2.0, 4, [['EOGr', 'EMGc'], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 2.0, 4, [[], ['M2'], [], [], ['C4'], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 2.5, 4, [['EOGl', 'EMGc'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 2.5, 4, [['EOGl', 'EMGc'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['ELC', 'EOGl', 'EMGr'], ['EOGl', 'EMGc'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 3.0, 4, [['EOGl'], ['EOGl', 'F4'], ['EOGl', 'F4'], [], ['EOGl'], ['EOGl']], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 3.0, 4, [['EOGl', 'EMGc'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['ELC', 'EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 4.0, 4, [['EOGl'], ['EOGl', 'F4'], [], [], [], ['EOGl']], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 4.0, 4, [['EOGl'], ['F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 5.0, 4, [[], ['EOGl', 'F4'], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 5.0, 4, [['EOGl'], ['F4'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 1.0, 5, [['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERC', 'EOGl', 'EMGr', 'F4'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'EMGc']], ['EOGl', 'EMGr']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 1.0, 5, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGl', 'F4'], ['ELC', 'ELI', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGl', 'EMGr', 'EMGc']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 1.0, 5, [['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc', 'ELC']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 1.0, 5, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 1.5, 5, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGr', 'F4', 'EMGc'], ['ELC', 'EOGl', 'F4'], ['ELC', 'EOGl', 'EMGr'], ['EOGl', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 1.5, 5, [['EOGr', 'EOGl', 'EMGr', 'EMGc'], ['EOGl', 'F4'], ['ELC', 'EOGl', 'F4'], ['ELC', 'EOGl', 'EMGr'], ['EOGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGr']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 1.5, 5, [['ERA', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['EMGr', 'F4'], ['ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGc'], ['EOGr', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EMGl', 'EMGr', 'EMGc']], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 1.5, 5, [['ELA', 'ERA', 'M2', 'C4'], ['ERC', 'M2', 'O2'], ['EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERC', 'EOGr', 'C4'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'C3', 'C4'], ['ELA', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M2']], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 2.0, 5, [['EOGl', 'EMGc'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 2.0, 5, [['EOGr', 'EOGl', 'EMGc'], ['EOGl', 'F4'], ['ELC', 'EOGl', 'F4'], ['ELC', 'EOGl', 'EMGr'], ['EOGl', 'EMGr', 'EMGc'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 2.0, 5, [['EOGr', 'EMGc'], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 2.0, 5, [[], ['M2'], [], [], ['C4'], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 2.5, 5, [['EOGl', 'EMGc'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 2.5, 5, [['EOGl', 'EMGc'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['ELC', 'EOGl', 'EMGr'], ['EOGl', 'EMGc'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 3.0, 5, [['EOGl'], ['EOGl', 'F4'], ['EOGl', 'F4'], [], ['EOGl'], ['EOGl']], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 3.0, 5, [['EOGl', 'EMGc'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['ELC', 'EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 4.0, 5, [['EOGl'], ['EOGl', 'F4'], [], [], [], ['EOGl']], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 4.0, 5, [['EOGl'], ['F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl'], ['EOGl']], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 5.0, 5, [[], ['EOGl', 'F4'], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 5.0, 5, [['EOGl'], ['F4'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 1.0, 6, [['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ELC', 'ELI', 'ERA', 'ERC', 'EOGl', 'EMGr', 'F4'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'EMGc']], ['EOGl', 'EMGr']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 1.0, 6, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGl', 'F4'], ['ELC', 'ELI', 'ERC', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGl', 'EMGr', 'EMGc']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 1.0, 6, [['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELC', 'ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['EOGr', 'EMGl', 'EMGr', 'EMGc']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 1.0, 6, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'EOGr', 'EMGl', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 1.5, 6, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGr', 'F4', 'EMGc'], ['ELC', 'EOGl', 'F4'], ['ELC', 'EOGl', 'EMGr'], ['EOGl', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'EMGr']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 1.5, 6, [['EOGr', 'EOGl', 'EMGr', 'EMGc'], ['EOGl', 'F4'], ['ELC', 'EOGl', 'F4'], ['ELC', 'EOGl', 'EMGr'], ['EOGl', 'EMGr', 'EMGc'], ['EOGl', 'EMGr']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 1.5, 6, [['ERA', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['EMGr', 'F4'], ['ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGc'], ['EOGr', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EMGl', 'EMGr', 'EMGc']], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 1.5, 6, [['ELA', 'ERA', 'M2', 'C4'], ['ERC', 'M2', 'O2'], ['EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERC', 'EOGr', 'C4'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'C3', 'C4'], ['ELA', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M2']], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 2.0, 6, [['EOGl', 'EMGc'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 2.0, 6, [['EOGr', 'EOGl', 'EMGc'], ['EOGl', 'F4'], ['ELC', 'EOGl', 'F4'], ['ELC', 'EOGl', 'EMGr'], ['EOGl', 'EMGr', 'EMGc'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 2.0, 6, [['EOGr', 'EMGc'], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 2.0, 6, [[], ['M2'], [], [], ['C4'], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 2.5, 6, [['EOGl', 'EMGc'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 2.5, 6, [['EOGl', 'EMGc'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['ELC', 'EOGl', 'EMGr'], ['EOGl', 'EMGc'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 3.0, 6, [['EOGl'], ['EOGl', 'F4'], ['EOGl', 'F4'], [], ['EOGl'], ['EOGl']], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 3.0, 6, [['EOGl', 'EMGc'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['ELC', 'EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 4.0, 6, [['EOGl'], ['EOGl', 'F4'], [], [], [], ['EOGl']], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 4.0, 6, [['EOGl'], ['F4'], ['EOGl', 'F4'], ['EOGl'], ['EOGl'], ['EOGl']], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'mm', 5.0, 6, [[], ['EOGl', 'F4'], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'mm', 5.0, 6, [['EOGl'], ['F4'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], []], ['S_11/night_1\\EEG_raw', 570, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 570, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 1.0, 1, [['ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ERI', 'EOGr', 'EOGl', 'EMGr', 'F4', 'EMGc'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ERI', 'EOGr', 'EOGl', 'F4'], ['ERI', 'EOGr', 'EOGl', 'F4'], ['ERI', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc']], ['ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc', 'ERA']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 1.0, 1, [['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'EMGr', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4']], ['ERI', 'EOGl', 'F4', 'EMGr']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 1.0, 1, [['ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'EMGc'], ['ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'EMGc'], ['ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'EMGc']], ['ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'EMGc', 'O1', 'ELA', 'O2', 'M2']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 1.0, 1, [['ELA', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc', 'ELC', 'M2']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 1.5, 1, [['ERI', 'EOGl', 'F4'], ['ERI', 'EOGr', 'EOGl', 'EMGr', 'F4'], ['EOGl', 'F4', 'EMGc'], ['ERI', 'EOGr', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4']], ['ERI', 'EOGl', 'F4', 'EOGr', 'EMGr', 'EMGc']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 1.5, 1, [['F4'], ['ERI', 'EOGl', 'EMGr', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['ERI', 'EOGl', 'F4'], ['EOGl', 'F4']], ['F4', 'ERI', 'EOGl', 'EMGr']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 1.5, 1, [['ERI', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ERI', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'ERI', 'EOGr', 'EMGr', 'F3'], ['ERA', 'ERI', 'EOGr', 'EMGr', 'EMGc'], ['ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'EMGc']], ['ERI', 'EOGr', 'EMGl', 'EMGr', 'EMGc', 'ERA', 'ERC', 'F3']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 1.5, 1, [['ERA', 'ERC', 'EMGl', 'C4'], ['ERA', 'ERC', 'EMGl', 'C4', 'EMGc'], ['ELA', 'ERA', 'EMGl', 'C4'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ERA', 'ERC', 'EMGl', 'EMGr', 'C4', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc']], ['ERA', 'ERC', 'EMGl', 'C4', 'EMGc', 'ELA', 'EOGr', 'EMGr', 'F3', 'C3']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 2.0, 1, [['ERI', 'EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4', 'EMGc'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4']], ['ERI', 'EOGl', 'F4', 'EMGc']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 2.0, 1, [['F4'], ['ERI', 'EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4', 'ERI', 'EOGl']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 2.0, 1, [['ERI'], ['ERI'], ['ERA', 'EMGc'], [], [], ['ERI', 'EMGc']], ['ERI', 'ERA', 'EMGc']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 2.0, 1, [['ERA'], ['C4'], ['C4'], ['EMGl', 'EMGr'], ['EMGc'], []], ['ERA', 'C4', 'EMGl', 'EMGr', 'EMGc']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 2.5, 1, [['ERI', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['ERI', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'F4']], ['ERI', 'F4', 'EOGl']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 2.5, 1, [['F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4', 'EOGl']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 3.0, 1, [['ERI', 'F4'], ['F4'], ['EOGl', 'F4'], ['ERI', 'F4'], ['ERI', 'F4'], ['ERI', 'F4']], ['ERI', 'F4', 'EOGl']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 3.0, 1, [['F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4', 'EOGl']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 4.0, 1, [['F4'], ['F4'], ['F4'], ['ERI', 'F4'], ['ERI', 'F4'], ['ERI', 'F4']], ['F4', 'ERI']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 4.0, 1, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 5.0, 1, [['F4'], [], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 5.0, 1, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 1.0, 2, [['ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ERI', 'EOGr', 'EOGl', 'EMGr', 'F4', 'EMGc'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ERI', 'EOGr', 'EOGl', 'F4'], ['ERI', 'EOGr', 'EOGl', 'F4'], ['ERI', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc']], ['ERI', 'EOGr', 'EOGl', 'EMGr', 'F4', 'EMGc']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 1.0, 2, [['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'EMGr', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4']], ['ERI', 'EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 1.0, 2, [['ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'EMGc'], ['ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'EMGc'], ['ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'EMGc']], ['ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'EMGc', 'ELA']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 1.0, 2, [['ELA', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 1.5, 2, [['ERI', 'EOGl', 'F4'], ['ERI', 'EOGr', 'EOGl', 'EMGr', 'F4'], ['EOGl', 'F4', 'EMGc'], ['ERI', 'EOGr', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4']], ['ERI', 'EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 1.5, 2, [['F4'], ['ERI', 'EOGl', 'EMGr', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['ERI', 'EOGl', 'F4'], ['EOGl', 'F4']], ['F4', 'EOGl']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 1.5, 2, [['ERI', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ERI', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'ERI', 'EOGr', 'EMGr', 'F3'], ['ERA', 'ERI', 'EOGr', 'EMGr', 'EMGc'], ['ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'EMGc']], ['ERI', 'EOGr', 'EMGl', 'EMGr', 'EMGc']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 1.5, 2, [['ERA', 'ERC', 'EMGl', 'C4'], ['ERA', 'ERC', 'EMGl', 'C4', 'EMGc'], ['ELA', 'ERA', 'EMGl', 'C4'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ERA', 'ERC', 'EMGl', 'EMGr', 'C4', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc']], ['ERA', 'ERC', 'EMGl', 'C4', 'EMGc', 'ELA', 'EMGr']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 2.0, 2, [['ERI', 'EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4', 'EMGc'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4']], ['ERI', 'EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 2.0, 2, [['F4'], ['ERI', 'EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 2.0, 2, [['ERI'], ['ERI'], ['ERA', 'EMGc'], [], [], ['ERI', 'EMGc']], ['ERI']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 2.0, 2, [['ERA'], ['C4'], ['C4'], ['EMGl', 'EMGr'], ['EMGc'], []], ['C4']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 2.5, 2, [['ERI', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['ERI', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'F4']], ['ERI', 'F4', 'EOGl']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 2.5, 2, [['F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 3.0, 2, [['ERI', 'F4'], ['F4'], ['EOGl', 'F4'], ['ERI', 'F4'], ['ERI', 'F4'], ['ERI', 'F4']], ['ERI', 'F4']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 3.0, 2, [['F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 4.0, 2, [['F4'], ['F4'], ['F4'], ['ERI', 'F4'], ['ERI', 'F4'], ['ERI', 'F4']], ['F4', 'ERI']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 4.0, 2, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 5.0, 2, [['F4'], [], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 5.0, 2, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 1.0, 3, [['ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ERI', 'EOGr', 'EOGl', 'EMGr', 'F4', 'EMGc'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ERI', 'EOGr', 'EOGl', 'F4'], ['ERI', 'EOGr', 'EOGl', 'F4'], ['ERI', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc']], ['ERI', 'EOGr', 'EOGl', 'EMGr', 'F4', 'EMGc']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 1.0, 3, [['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'EMGr', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4']], ['ERI', 'EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 1.0, 3, [['ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'EMGc'], ['ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'EMGc'], ['ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'EMGc']], ['ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'EMGc']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 1.0, 3, [['ELA', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 1.5, 3, [['ERI', 'EOGl', 'F4'], ['ERI', 'EOGr', 'EOGl', 'EMGr', 'F4'], ['EOGl', 'F4', 'EMGc'], ['ERI', 'EOGr', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4']], ['ERI', 'EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 1.5, 3, [['F4'], ['ERI', 'EOGl', 'EMGr', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['ERI', 'EOGl', 'F4'], ['EOGl', 'F4']], ['F4', 'EOGl']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 1.5, 3, [['ERI', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ERI', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'ERI', 'EOGr', 'EMGr', 'F3'], ['ERA', 'ERI', 'EOGr', 'EMGr', 'EMGc'], ['ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'EMGc']], ['ERI', 'EOGr', 'EMGl', 'EMGr', 'EMGc']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 1.5, 3, [['ERA', 'ERC', 'EMGl', 'C4'], ['ERA', 'ERC', 'EMGl', 'C4', 'EMGc'], ['ELA', 'ERA', 'EMGl', 'C4'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ERA', 'ERC', 'EMGl', 'EMGr', 'C4', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc']], ['ERA', 'ERC', 'EMGl', 'C4', 'EMGc']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 2.0, 3, [['ERI', 'EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4', 'EMGc'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4']], ['ERI', 'EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 2.0, 3, [['F4'], ['ERI', 'EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 2.0, 3, [['ERI'], ['ERI'], ['ERA', 'EMGc'], [], [], ['ERI', 'EMGc']], []], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 2.0, 3, [['ERA'], ['C4'], ['C4'], ['EMGl', 'EMGr'], ['EMGc'], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 2.5, 3, [['ERI', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['ERI', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'F4']], ['ERI', 'F4']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 2.5, 3, [['F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 3.0, 3, [['ERI', 'F4'], ['F4'], ['EOGl', 'F4'], ['ERI', 'F4'], ['ERI', 'F4'], ['ERI', 'F4']], ['ERI', 'F4']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 3.0, 3, [['F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 4.0, 3, [['F4'], ['F4'], ['F4'], ['ERI', 'F4'], ['ERI', 'F4'], ['ERI', 'F4']], ['F4', 'ERI']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 4.0, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 5.0, 3, [['F4'], [], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 5.0, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 1.0, 4, [['ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ERI', 'EOGr', 'EOGl', 'EMGr', 'F4', 'EMGc'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ERI', 'EOGr', 'EOGl', 'F4'], ['ERI', 'EOGr', 'EOGl', 'F4'], ['ERI', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc']], ['ERI', 'EOGr', 'EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 1.0, 4, [['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'EMGr', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4']], ['ERI', 'EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 1.0, 4, [['ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'EMGc'], ['ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'EMGc'], ['ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'EMGc']], ['ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'EMGc']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 1.0, 4, [['ELA', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 1.5, 4, [['ERI', 'EOGl', 'F4'], ['ERI', 'EOGr', 'EOGl', 'EMGr', 'F4'], ['EOGl', 'F4', 'EMGc'], ['ERI', 'EOGr', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 1.5, 4, [['F4'], ['ERI', 'EOGl', 'EMGr', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['ERI', 'EOGl', 'F4'], ['EOGl', 'F4']], ['F4', 'EOGl']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 1.5, 4, [['ERI', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ERI', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'ERI', 'EOGr', 'EMGr', 'F3'], ['ERA', 'ERI', 'EOGr', 'EMGr', 'EMGc'], ['ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'EMGc']], ['ERI', 'EOGr', 'EMGr']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 1.5, 4, [['ERA', 'ERC', 'EMGl', 'C4'], ['ERA', 'ERC', 'EMGl', 'C4', 'EMGc'], ['ELA', 'ERA', 'EMGl', 'C4'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ERA', 'ERC', 'EMGl', 'EMGr', 'C4', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc']], ['ERA', 'EMGl']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 2.0, 4, [['ERI', 'EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4', 'EMGc'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 2.0, 4, [['F4'], ['ERI', 'EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 2.0, 4, [['ERI'], ['ERI'], ['ERA', 'EMGc'], [], [], ['ERI', 'EMGc']], []], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 2.0, 4, [['ERA'], ['C4'], ['C4'], ['EMGl', 'EMGr'], ['EMGc'], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 2.5, 4, [['ERI', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['ERI', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 2.5, 4, [['F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 3.0, 4, [['ERI', 'F4'], ['F4'], ['EOGl', 'F4'], ['ERI', 'F4'], ['ERI', 'F4'], ['ERI', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 3.0, 4, [['F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 4.0, 4, [['F4'], ['F4'], ['F4'], ['ERI', 'F4'], ['ERI', 'F4'], ['ERI', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 4.0, 4, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 5.0, 4, [['F4'], [], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 5.0, 4, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 1.0, 5, [['ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ERI', 'EOGr', 'EOGl', 'EMGr', 'F4', 'EMGc'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ERI', 'EOGr', 'EOGl', 'F4'], ['ERI', 'EOGr', 'EOGl', 'F4'], ['ERI', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc']], ['ERI', 'EOGr', 'EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 1.0, 5, [['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'EMGr', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4']], ['ERI', 'EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 1.0, 5, [['ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'EMGc'], ['ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'EMGc'], ['ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'EMGc']], ['ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'EMGc']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 1.0, 5, [['ELA', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 1.5, 5, [['ERI', 'EOGl', 'F4'], ['ERI', 'EOGr', 'EOGl', 'EMGr', 'F4'], ['EOGl', 'F4', 'EMGc'], ['ERI', 'EOGr', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 1.5, 5, [['F4'], ['ERI', 'EOGl', 'EMGr', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['ERI', 'EOGl', 'F4'], ['EOGl', 'F4']], ['F4', 'EOGl']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 1.5, 5, [['ERI', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ERI', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'ERI', 'EOGr', 'EMGr', 'F3'], ['ERA', 'ERI', 'EOGr', 'EMGr', 'EMGc'], ['ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'EMGc']], ['ERI', 'EOGr', 'EMGr']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 1.5, 5, [['ERA', 'ERC', 'EMGl', 'C4'], ['ERA', 'ERC', 'EMGl', 'C4', 'EMGc'], ['ELA', 'ERA', 'EMGl', 'C4'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ERA', 'ERC', 'EMGl', 'EMGr', 'C4', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc']], ['ERA', 'EMGl']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 2.0, 5, [['ERI', 'EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4', 'EMGc'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 2.0, 5, [['F4'], ['ERI', 'EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 2.0, 5, [['ERI'], ['ERI'], ['ERA', 'EMGc'], [], [], ['ERI', 'EMGc']], []], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 2.0, 5, [['ERA'], ['C4'], ['C4'], ['EMGl', 'EMGr'], ['EMGc'], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 2.5, 5, [['ERI', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['ERI', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 2.5, 5, [['F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 3.0, 5, [['ERI', 'F4'], ['F4'], ['EOGl', 'F4'], ['ERI', 'F4'], ['ERI', 'F4'], ['ERI', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 3.0, 5, [['F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 4.0, 5, [['F4'], ['F4'], ['F4'], ['ERI', 'F4'], ['ERI', 'F4'], ['ERI', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 4.0, 5, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 5.0, 5, [['F4'], [], ['F4'], ['F4'], ['F4'], ['F4']], []], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 5.0, 5, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 1.0, 6, [['ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ERI', 'EOGr', 'EOGl', 'EMGr', 'F4', 'EMGc'], ['ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ERI', 'EOGr', 'EOGl', 'F4'], ['ERI', 'EOGr', 'EOGl', 'F4'], ['ERI', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc']], ['ERI', 'EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 1.0, 6, [['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'EMGr', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4']], ['ERI', 'EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 1.0, 6, [['ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'EMGc'], ['ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'EMGc'], ['ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'C4', 'EMGc']], ['ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'C4', 'EMGc']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 1.0, 6, [['ELA', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 1.5, 6, [['ERI', 'EOGl', 'F4'], ['ERI', 'EOGr', 'EOGl', 'EMGr', 'F4'], ['EOGl', 'F4', 'EMGc'], ['ERI', 'EOGr', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 1.5, 6, [['F4'], ['ERI', 'EOGl', 'EMGr', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['ERI', 'EOGl', 'F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 1.5, 6, [['ERI', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ERI', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ERA', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ERC', 'ERI', 'EOGr', 'EMGr', 'F3'], ['ERA', 'ERI', 'EOGr', 'EMGr', 'EMGc'], ['ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'EMGc']], ['ERI', 'EOGr', 'EMGr']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 1.5, 6, [['ERA', 'ERC', 'EMGl', 'C4'], ['ERA', 'ERC', 'EMGl', 'C4', 'EMGc'], ['ELA', 'ERA', 'EMGl', 'C4'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'EMGc'], ['ERA', 'ERC', 'EMGl', 'EMGr', 'C4', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'EMGl', 'C3', 'C4', 'EMGc']], ['ERA', 'EMGl']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 2.0, 6, [['ERI', 'EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4', 'EMGc'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 2.0, 6, [['F4'], ['ERI', 'EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 2.0, 6, [['ERI'], ['ERI'], ['ERA', 'EMGc'], [], [], ['ERI', 'EMGc']], []], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 2.0, 6, [['ERA'], ['C4'], ['C4'], ['EMGl', 'EMGr'], ['EMGc'], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 2.5, 6, [['ERI', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['ERI', 'F4'], ['ERI', 'EOGl', 'F4'], ['ERI', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 2.5, 6, [['F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 3.0, 6, [['ERI', 'F4'], ['F4'], ['EOGl', 'F4'], ['ERI', 'F4'], ['ERI', 'F4'], ['ERI', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 3.0, 6, [['F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 4.0, 6, [['F4'], ['F4'], ['F4'], ['ERI', 'F4'], ['ERI', 'F4'], ['ERI', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 4.0, 6, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'mm', 5.0, 6, [['F4'], [], ['F4'], ['F4'], ['F4'], ['F4']], []], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'mm', 5.0, 6, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 900, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 900, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 1.0, 1, [['ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M2', 'F4', 'EMGc'], ['ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F4'], ['ELT', 'ELE', 'ERI', 'EOGr', 'EOGl', 'F4'], ['ELB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'F4', 'O2']], ['ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M2', 'F4', 'EMGc', 'M1', 'ELT', 'ELE', 'ELB', 'EMGr', 'ERA', 'ERE', 'O2']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 1.0, 1, [['ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'F4'], ['ERC', 'ERI', 'EOGr', 'EOGl', 'F4'], ['F4'], ['ERI', 'EOGl', 'F4'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['ERA', 'ERC', 'EOGl', 'F4']], ['ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc', 'ERA']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 1.0, 1, [['ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'EMGc'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M2', 'O2', 'EMGc']], ['ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'O2', 'EMGc', 'M1', 'O1', 'ELB', 'ELC', 'ELT', 'ELE', 'ELA', 'ELI', 'ERA', 'ERT ', 'ERE']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 1.0, 1, [['ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELI', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'M2', 'O2', 'EMGc']], ['ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc', 'ELI', 'M1', 'ELB', 'ELC', 'ELT', 'ERI', 'ELA']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 1.5, 1, [['ERI', 'EOGl', 'F4', 'EMGc'], ['ERI', 'EOGr', 'EOGl', 'F4'], ['ELT', 'ELE', 'ERI', 'EOGr', 'EOGl', 'F4'], ['ELB', 'ERC', 'EOGr', 'F4', 'EMGc'], ['EOGr', 'EOGl', 'F4', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F4', 'O2']], ['ERI', 'EOGl', 'F4', 'EMGc', 'EOGr', 'ELT', 'ELE', 'ELB', 'ERC', 'ERA', 'O2']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 1.5, 1, [['ERI', 'EOGr', 'EOGl', 'F4'], ['ERI', 'EOGr', 'EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['ERA', 'EOGl', 'F4']], ['ERI', 'EOGr', 'EOGl', 'F4', 'ERA']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 1.5, 1, [['ERC', 'ERI', 'EOGr', 'EMGl', 'M2', 'EMGc'], ['ERC', 'ERI', 'EOGr', 'EMGl', 'M1', 'EMGc'], ['ELB', 'ELT', 'ERI', 'EOGr', 'EOGl', 'EMGr'], ['ELB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGc'], ['ERC', 'EOGr', 'EMGr', 'EMGc'], ['ERA', 'ERC', 'ERE', 'O2']], ['ERC', 'ERI', 'EOGr', 'EMGl', 'M2', 'EMGc', 'M1', 'ELB', 'ELT', 'EOGl', 'EMGr', 'ERA', 'ERE', 'O2']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 1.5, 1, [['ERC', 'EMGl', 'EMGr', 'M2', 'C4', 'O2'], ['ERC', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ELB', 'ERC', 'M2'], ['EMGr', 'M2', 'EMGc'], ['ELI', 'EOGr', 'EMGl', 'M2', 'EMGc']], ['ERC', 'EMGl', 'EMGr', 'M2', 'C4', 'O2', 'EMGc', 'ELB', 'ELC', 'ELT', 'EOGr', 'ELI']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 2.0, 1, [['EOGl', 'F4'], ['EOGl', 'F4'], ['ELE', 'F4'], ['F4', 'EMGc'], ['EOGl', 'F4', 'EMGc'], ['F4', 'O2']], ['EOGl', 'F4', 'ELE', 'EMGc', 'O2']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 2.0, 1, [['EOGr', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['F4'], ['F4'], ['EOGr', 'EOGl', 'F4'], ['ERA', 'F4']], ['EOGr', 'EOGl', 'F4', 'ERA']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 2.0, 1, [[], [], ['ERI'], ['ERI'], [], ['O2']], ['ERI', 'O2']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 2.0, 1, [['M2'], [], ['ERC', 'EMGr', 'EMGc'], ['M2'], [], ['ELI', 'M2']], ['M2', 'ERC', 'EMGr', 'EMGc', 'ELI']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 2.5, 1, [['F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4', 'EOGl']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 2.5, 1, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 3.0, 1, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 3.0, 1, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 4.0, 1, [[], [], ['F4'], ['F4'], [], []], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 4.0, 1, [['F4'], ['F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4', 'EOGl']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 5.0, 1, [[], [], [], ['F4'], [], []], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 5.0, 1, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 1.0, 2, [['ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M2', 'F4', 'EMGc'], ['ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F4'], ['ELT', 'ELE', 'ERI', 'EOGr', 'EOGl', 'F4'], ['ELB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'F4', 'O2']], ['ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 1.0, 2, [['ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'F4'], ['ERC', 'ERI', 'EOGr', 'EOGl', 'F4'], ['F4'], ['ERI', 'EOGl', 'F4'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['ERA', 'ERC', 'EOGl', 'F4']], ['ERC', 'ERI', 'EOGr', 'EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 1.0, 2, [['ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'EMGc'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M2', 'O2', 'EMGc']], ['ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'EMGc', 'ELB', 'ELI']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 1.0, 2, [['ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELI', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'M2', 'O2', 'EMGc']], ['ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc', 'ELI', 'ELB', 'ELA']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 1.5, 2, [['ERI', 'EOGl', 'F4', 'EMGc'], ['ERI', 'EOGr', 'EOGl', 'F4'], ['ELT', 'ELE', 'ERI', 'EOGr', 'EOGl', 'F4'], ['ELB', 'ERC', 'EOGr', 'F4', 'EMGc'], ['EOGr', 'EOGl', 'F4', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F4', 'O2']], ['ERI', 'EOGl', 'F4', 'EMGc', 'EOGr']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 1.5, 2, [['ERI', 'EOGr', 'EOGl', 'F4'], ['ERI', 'EOGr', 'EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['ERA', 'EOGl', 'F4']], ['ERI', 'EOGr', 'EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 1.5, 2, [['ERC', 'ERI', 'EOGr', 'EMGl', 'M2', 'EMGc'], ['ERC', 'ERI', 'EOGr', 'EMGl', 'M1', 'EMGc'], ['ELB', 'ELT', 'ERI', 'EOGr', 'EOGl', 'EMGr'], ['ELB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGc'], ['ERC', 'EOGr', 'EMGr', 'EMGc'], ['ERA', 'ERC', 'ERE', 'O2']], ['ERC', 'ERI', 'EOGr', 'EMGl', 'EMGc', 'ELB', 'EOGl']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 1.5, 2, [['ERC', 'EMGl', 'EMGr', 'M2', 'C4', 'O2'], ['ERC', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ELB', 'ERC', 'M2'], ['EMGr', 'M2', 'EMGc'], ['ELI', 'EOGr', 'EMGl', 'M2', 'EMGc']], ['ERC', 'M2', 'EMGc', 'ELB']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 2.0, 2, [['EOGl', 'F4'], ['EOGl', 'F4'], ['ELE', 'F4'], ['F4', 'EMGc'], ['EOGl', 'F4', 'EMGc'], ['F4', 'O2']], ['EOGl', 'F4', 'EMGc']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 2.0, 2, [['EOGr', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['F4'], ['F4'], ['EOGr', 'EOGl', 'F4'], ['ERA', 'F4']], ['EOGr', 'EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 2.0, 2, [[], [], ['ERI'], ['ERI'], [], ['O2']], ['ERI']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 2.0, 2, [['M2'], [], ['ERC', 'EMGr', 'EMGc'], ['M2'], [], ['ELI', 'M2']], []], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 2.5, 2, [['F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 2.5, 2, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 3.0, 2, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 3.0, 2, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 4.0, 2, [[], [], ['F4'], ['F4'], [], []], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 4.0, 2, [['F4'], ['F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 5.0, 2, [[], [], [], ['F4'], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 5.0, 2, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 1.0, 3, [['ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M2', 'F4', 'EMGc'], ['ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F4'], ['ELT', 'ELE', 'ERI', 'EOGr', 'EOGl', 'F4'], ['ELB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'F4', 'O2']], ['ERC', 'ERI', 'EOGr', 'EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 1.0, 3, [['ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'F4'], ['ERC', 'ERI', 'EOGr', 'EOGl', 'F4'], ['F4'], ['ERI', 'EOGl', 'F4'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['ERA', 'ERC', 'EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 1.0, 3, [['ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'EMGc'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M2', 'O2', 'EMGc']], ['ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 1.0, 3, [['ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELI', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'M2', 'O2', 'EMGc']], ['ERC', 'EOGr', 'EMGl', 'EMGr', 'M2', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 1.5, 3, [['ERI', 'EOGl', 'F4', 'EMGc'], ['ERI', 'EOGr', 'EOGl', 'F4'], ['ELT', 'ELE', 'ERI', 'EOGr', 'EOGl', 'F4'], ['ELB', 'ERC', 'EOGr', 'F4', 'EMGc'], ['EOGr', 'EOGl', 'F4', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F4', 'O2']], ['ERI', 'EOGl', 'F4', 'EOGr']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 1.5, 3, [['ERI', 'EOGr', 'EOGl', 'F4'], ['ERI', 'EOGr', 'EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['ERA', 'EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 1.5, 3, [['ERC', 'ERI', 'EOGr', 'EMGl', 'M2', 'EMGc'], ['ERC', 'ERI', 'EOGr', 'EMGl', 'M1', 'EMGc'], ['ELB', 'ELT', 'ERI', 'EOGr', 'EOGl', 'EMGr'], ['ELB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGc'], ['ERC', 'EOGr', 'EMGr', 'EMGc'], ['ERA', 'ERC', 'ERE', 'O2']], ['ERC', 'ERI', 'EOGr']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 1.5, 3, [['ERC', 'EMGl', 'EMGr', 'M2', 'C4', 'O2'], ['ERC', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ELB', 'ERC', 'M2'], ['EMGr', 'M2', 'EMGc'], ['ELI', 'EOGr', 'EMGl', 'M2', 'EMGc']], ['ERC', 'M2']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 2.0, 3, [['EOGl', 'F4'], ['EOGl', 'F4'], ['ELE', 'F4'], ['F4', 'EMGc'], ['EOGl', 'F4', 'EMGc'], ['F4', 'O2']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 2.0, 3, [['EOGr', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['F4'], ['F4'], ['EOGr', 'EOGl', 'F4'], ['ERA', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 2.0, 3, [[], [], ['ERI'], ['ERI'], [], ['O2']], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 2.0, 3, [['M2'], [], ['ERC', 'EMGr', 'EMGc'], ['M2'], [], ['ELI', 'M2']], []], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 2.5, 3, [['F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 2.5, 3, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 3.0, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 3.0, 3, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 4.0, 3, [[], [], ['F4'], ['F4'], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 4.0, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 5.0, 3, [[], [], [], ['F4'], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 5.0, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 1.0, 4, [['ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M2', 'F4', 'EMGc'], ['ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F4'], ['ELT', 'ELE', 'ERI', 'EOGr', 'EOGl', 'F4'], ['ELB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'F4', 'O2']], ['ERI', 'EOGr', 'EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 1.0, 4, [['ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'F4'], ['ERC', 'ERI', 'EOGr', 'EOGl', 'F4'], ['F4'], ['ERI', 'EOGl', 'F4'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['ERA', 'ERC', 'EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 1.0, 4, [['ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'EMGc'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M2', 'O2', 'EMGc']], ['ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 1.0, 4, [['ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELI', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'M2', 'O2', 'EMGc']], ['ERC', 'EOGr', 'EMGl', 'EMGr', 'M2', 'EMGc']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 1.5, 4, [['ERI', 'EOGl', 'F4', 'EMGc'], ['ERI', 'EOGr', 'EOGl', 'F4'], ['ELT', 'ELE', 'ERI', 'EOGr', 'EOGl', 'F4'], ['ELB', 'ERC', 'EOGr', 'F4', 'EMGc'], ['EOGr', 'EOGl', 'F4', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F4', 'O2']], ['F4', 'EOGr']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 1.5, 4, [['ERI', 'EOGr', 'EOGl', 'F4'], ['ERI', 'EOGr', 'EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['ERA', 'EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 1.5, 4, [['ERC', 'ERI', 'EOGr', 'EMGl', 'M2', 'EMGc'], ['ERC', 'ERI', 'EOGr', 'EMGl', 'M1', 'EMGc'], ['ELB', 'ELT', 'ERI', 'EOGr', 'EOGl', 'EMGr'], ['ELB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGc'], ['ERC', 'EOGr', 'EMGr', 'EMGc'], ['ERA', 'ERC', 'ERE', 'O2']], ['ERI', 'EOGr']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 1.5, 4, [['ERC', 'EMGl', 'EMGr', 'M2', 'C4', 'O2'], ['ERC', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ELB', 'ERC', 'M2'], ['EMGr', 'M2', 'EMGc'], ['ELI', 'EOGr', 'EMGl', 'M2', 'EMGc']], ['ERC']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 2.0, 4, [['EOGl', 'F4'], ['EOGl', 'F4'], ['ELE', 'F4'], ['F4', 'EMGc'], ['EOGl', 'F4', 'EMGc'], ['F4', 'O2']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 2.0, 4, [['EOGr', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['F4'], ['F4'], ['EOGr', 'EOGl', 'F4'], ['ERA', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 2.0, 4, [[], [], ['ERI'], ['ERI'], [], ['O2']], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 2.0, 4, [['M2'], [], ['ERC', 'EMGr', 'EMGc'], ['M2'], [], ['ELI', 'M2']], []], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 2.5, 4, [['F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 2.5, 4, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 3.0, 4, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 3.0, 4, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 4.0, 4, [[], [], ['F4'], ['F4'], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 4.0, 4, [['F4'], ['F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 5.0, 4, [[], [], [], ['F4'], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 5.0, 4, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 1.0, 5, [['ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M2', 'F4', 'EMGc'], ['ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F4'], ['ELT', 'ELE', 'ERI', 'EOGr', 'EOGl', 'F4'], ['ELB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'F4', 'O2']], ['EOGr', 'EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 1.0, 5, [['ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'F4'], ['ERC', 'ERI', 'EOGr', 'EOGl', 'F4'], ['F4'], ['ERI', 'EOGl', 'F4'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['ERA', 'ERC', 'EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 1.0, 5, [['ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'EMGc'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M2', 'O2', 'EMGc']], ['ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 1.0, 5, [['ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELI', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'M2', 'O2', 'EMGc']], ['ERC', 'EOGr', 'EMGl', 'EMGr', 'M2', 'EMGc']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 1.5, 5, [['ERI', 'EOGl', 'F4', 'EMGc'], ['ERI', 'EOGr', 'EOGl', 'F4'], ['ELT', 'ELE', 'ERI', 'EOGr', 'EOGl', 'F4'], ['ELB', 'ERC', 'EOGr', 'F4', 'EMGc'], ['EOGr', 'EOGl', 'F4', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F4', 'O2']], ['F4', 'EOGr']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 1.5, 5, [['ERI', 'EOGr', 'EOGl', 'F4'], ['ERI', 'EOGr', 'EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['ERA', 'EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 1.5, 5, [['ERC', 'ERI', 'EOGr', 'EMGl', 'M2', 'EMGc'], ['ERC', 'ERI', 'EOGr', 'EMGl', 'M1', 'EMGc'], ['ELB', 'ELT', 'ERI', 'EOGr', 'EOGl', 'EMGr'], ['ELB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGc'], ['ERC', 'EOGr', 'EMGr', 'EMGc'], ['ERA', 'ERC', 'ERE', 'O2']], ['EOGr']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 1.5, 5, [['ERC', 'EMGl', 'EMGr', 'M2', 'C4', 'O2'], ['ERC', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ELB', 'ERC', 'M2'], ['EMGr', 'M2', 'EMGc'], ['ELI', 'EOGr', 'EMGl', 'M2', 'EMGc']], []], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 2.0, 5, [['EOGl', 'F4'], ['EOGl', 'F4'], ['ELE', 'F4'], ['F4', 'EMGc'], ['EOGl', 'F4', 'EMGc'], ['F4', 'O2']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 2.0, 5, [['EOGr', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['F4'], ['F4'], ['EOGr', 'EOGl', 'F4'], ['ERA', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 2.0, 5, [[], [], ['ERI'], ['ERI'], [], ['O2']], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 2.0, 5, [['M2'], [], ['ERC', 'EMGr', 'EMGc'], ['M2'], [], ['ELI', 'M2']], []], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 2.5, 5, [['F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 2.5, 5, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 3.0, 5, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 3.0, 5, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 4.0, 5, [[], [], ['F4'], ['F4'], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 4.0, 5, [['F4'], ['F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 5.0, 5, [[], [], [], ['F4'], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 5.0, 5, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 1.0, 6, [['ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M2', 'F4', 'EMGc'], ['ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F4'], ['ELT', 'ELE', 'ERI', 'EOGr', 'EOGl', 'F4'], ['ELB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EOGl', 'F4', 'O2']], ['EOGr', 'EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 1.0, 6, [['ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'F4'], ['ERC', 'ERI', 'EOGr', 'EOGl', 'F4'], ['F4'], ['ERI', 'EOGl', 'F4'], ['ERC', 'EOGr', 'EOGl', 'EMGl', 'F4', 'EMGc'], ['ERA', 'ERC', 'EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 1.0, 6, [['ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'O2', 'EMGc'], ['ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ELE', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'EMGc'], ['ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'M2', 'O2', 'EMGc']], ['ERC', 'EOGr', 'EMGl', 'EMGc']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 1.0, 6, [['ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELI', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELI', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'M2', 'O2', 'EMGc']], ['ERC', 'EOGr', 'EMGl', 'EMGr', 'M2', 'EMGc']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 1.5, 6, [['ERI', 'EOGl', 'F4', 'EMGc'], ['ERI', 'EOGr', 'EOGl', 'F4'], ['ELT', 'ELE', 'ERI', 'EOGr', 'EOGl', 'F4'], ['ELB', 'ERC', 'EOGr', 'F4', 'EMGc'], ['EOGr', 'EOGl', 'F4', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F4', 'O2']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 1.5, 6, [['ERI', 'EOGr', 'EOGl', 'F4'], ['ERI', 'EOGr', 'EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['ERA', 'EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 1.5, 6, [['ERC', 'ERI', 'EOGr', 'EMGl', 'M2', 'EMGc'], ['ERC', 'ERI', 'EOGr', 'EMGl', 'M1', 'EMGc'], ['ELB', 'ELT', 'ERI', 'EOGr', 'EOGl', 'EMGr'], ['ELB', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGc'], ['ERC', 'EOGr', 'EMGr', 'EMGc'], ['ERA', 'ERC', 'ERE', 'O2']], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 1.5, 6, [['ERC', 'EMGl', 'EMGr', 'M2', 'C4', 'O2'], ['ERC', 'M2', 'EMGc'], ['ELB', 'ELC', 'ELT', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'EMGc'], ['ELB', 'ERC', 'M2'], ['EMGr', 'M2', 'EMGc'], ['ELI', 'EOGr', 'EMGl', 'M2', 'EMGc']], []], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 2.0, 6, [['EOGl', 'F4'], ['EOGl', 'F4'], ['ELE', 'F4'], ['F4', 'EMGc'], ['EOGl', 'F4', 'EMGc'], ['F4', 'O2']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 2.0, 6, [['EOGr', 'EOGl', 'F4'], ['EOGr', 'EOGl', 'F4'], ['F4'], ['F4'], ['EOGr', 'EOGl', 'F4'], ['ERA', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 2.0, 6, [[], [], ['ERI'], ['ERI'], [], ['O2']], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 2.0, 6, [['M2'], [], ['ERC', 'EMGr', 'EMGc'], ['M2'], [], ['ELI', 'M2']], []], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 2.5, 6, [['F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 2.5, 6, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 3.0, 6, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 3.0, 6, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 4.0, 6, [[], [], ['F4'], ['F4'], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 4.0, 6, [['F4'], ['F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'mm', 5.0, 6, [[], [], [], ['F4'], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'mm', 5.0, 6, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 3720, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 3720, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 1.0, 1, [['EOGr', 'EOGl', 'F4', 'C4'], ['ERA', 'EOGr', 'EOGl', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'C3', 'F4', 'C4', 'EMGc'], ['EOGl', 'F3', 'C3', 'F4', 'C4'], ['ERC', 'EOGr', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'F4', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'F4', 'C4', 'ERA', 'EMGc', 'ELA', 'C3', 'F3', 'ERC', 'EMGr', 'O2']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 1.0, 1, [['EOGr', 'EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F3', 'C3', 'F4', 'C4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGr', 'EOGl', 'F4', 'F3', 'C3', 'C4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 1.0, 1, [['ELA', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc', 'ELC', 'EMGr']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 1.5, 1, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4', 'C4'], ['EOGl', 'C3', 'F4', 'C4'], ['EOGr', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4', 'C4', 'C3', 'EOGr']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 1.5, 1, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'C3', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4', 'C3']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 1.5, 1, [['EOGr', 'C4'], ['EOGr', 'C4', 'EMGc'], ['ERA', 'EOGr', 'C4'], ['C3', 'C4', 'O2'], ['ERC', 'EOGr'], ['C4']], ['EOGr', 'C4', 'EMGc', 'ERA', 'C3', 'O2', 'ERC']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 1.5, 1, [['ELA', 'ERA', 'ERI', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'C4', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C4', 'EMGc']], ['ELA', 'ERA', 'ERI', 'EOGr', 'C4', 'EMGc', 'ERC']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 2.0, 1, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 2.0, 1, [['EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 2.0, 1, [['ERI', 'C4'], [], ['C4'], ['C4'], ['ERC'], ['C4']], ['ERI', 'C4', 'ERC']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 2.5, 1, [['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4']], ['F4', 'EOGl']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 2.5, 1, [['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 3.0, 1, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 3.0, 1, [['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 4.0, 1, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 4.0, 1, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 5.0, 1, [['F4'], ['F4'], [], [], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 5.0, 1, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 1.0, 2, [['EOGr', 'EOGl', 'F4', 'C4'], ['ERA', 'EOGr', 'EOGl', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'C3', 'F4', 'C4', 'EMGc'], ['EOGl', 'F3', 'C3', 'F4', 'C4'], ['ERC', 'EOGr', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'F4', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'F4', 'C4', 'ERA', 'EMGc', 'C3', 'O2']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 1.0, 2, [['EOGr', 'EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F3', 'C3', 'F4', 'C4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 1.0, 2, [['ELA', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc', 'ELC', 'EMGr']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 1.5, 2, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4', 'C4'], ['EOGl', 'C3', 'F4', 'C4'], ['EOGr', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4', 'C4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 1.5, 2, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'C3', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 1.5, 2, [['EOGr', 'C4'], ['EOGr', 'C4', 'EMGc'], ['ERA', 'EOGr', 'C4'], ['C3', 'C4', 'O2'], ['ERC', 'EOGr'], ['C4']], ['EOGr', 'C4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 1.5, 2, [['ELA', 'ERA', 'ERI', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'C4', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C4', 'EMGc']], ['ELA', 'ERA', 'EOGr', 'C4', 'EMGc', 'ERC']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 2.0, 2, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 2.0, 2, [['EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 2.0, 2, [['ERI', 'C4'], [], ['C4'], ['C4'], ['ERC'], ['C4']], ['C4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 2.5, 2, [['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 2.5, 2, [['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 3.0, 2, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 3.0, 2, [['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 4.0, 2, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 4.0, 2, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 5.0, 2, [['F4'], ['F4'], [], [], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 5.0, 2, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 1.0, 3, [['EOGr', 'EOGl', 'F4', 'C4'], ['ERA', 'EOGr', 'EOGl', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'C3', 'F4', 'C4', 'EMGc'], ['EOGl', 'F3', 'C3', 'F4', 'C4'], ['ERC', 'EOGr', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'F4', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'F4', 'C4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 1.0, 3, [['EOGr', 'EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F3', 'C3', 'F4', 'C4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 1.0, 3, [['ELA', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc', 'EMGr']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 1.5, 3, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4', 'C4'], ['EOGl', 'C3', 'F4', 'C4'], ['EOGr', 'F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 1.5, 3, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'C3', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 1.5, 3, [['EOGr', 'C4'], ['EOGr', 'C4', 'EMGc'], ['ERA', 'EOGr', 'C4'], ['C3', 'C4', 'O2'], ['ERC', 'EOGr'], ['C4']], ['EOGr', 'C4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 1.5, 3, [['ELA', 'ERA', 'ERI', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'C4', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C4', 'EMGc']], ['ELA', 'ERA', 'EOGr', 'C4', 'EMGc']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 2.0, 3, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 2.0, 3, [['EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 2.0, 3, [['ERI', 'C4'], [], ['C4'], ['C4'], ['ERC'], ['C4']], []], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 2.5, 3, [['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 2.5, 3, [['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 3.0, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 3.0, 3, [['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 4.0, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 4.0, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 5.0, 3, [['F4'], ['F4'], [], [], ['F4'], ['F4']], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 5.0, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 1.0, 4, [['EOGr', 'EOGl', 'F4', 'C4'], ['ERA', 'EOGr', 'EOGl', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'C3', 'F4', 'C4', 'EMGc'], ['EOGl', 'F3', 'C3', 'F4', 'C4'], ['ERC', 'EOGr', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'F4', 'O2', 'EMGc']], ['EOGl', 'F4', 'C4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 1.0, 4, [['EOGr', 'EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F3', 'C3', 'F4', 'C4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 1.0, 4, [['ELA', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc', 'EMGr']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 1.5, 4, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4', 'C4'], ['EOGl', 'C3', 'F4', 'C4'], ['EOGr', 'F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 1.5, 4, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'C3', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 1.5, 4, [['EOGr', 'C4'], ['EOGr', 'C4', 'EMGc'], ['ERA', 'EOGr', 'C4'], ['C3', 'C4', 'O2'], ['ERC', 'EOGr'], ['C4']], ['C4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 1.5, 4, [['ELA', 'ERA', 'ERI', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'C4', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C4', 'EMGc']], ['ELA', 'ERA', 'EOGr', 'C4', 'EMGc']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 2.0, 4, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 2.0, 4, [['EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 2.0, 4, [['ERI', 'C4'], [], ['C4'], ['C4'], ['ERC'], ['C4']], []], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 2.5, 4, [['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 2.5, 4, [['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 3.0, 4, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 3.0, 4, [['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 4.0, 4, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 4.0, 4, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 5.0, 4, [['F4'], ['F4'], [], [], ['F4'], ['F4']], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 5.0, 4, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 1.0, 5, [['EOGr', 'EOGl', 'F4', 'C4'], ['ERA', 'EOGr', 'EOGl', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'C3', 'F4', 'C4', 'EMGc'], ['EOGl', 'F3', 'C3', 'F4', 'C4'], ['ERC', 'EOGr', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'F4', 'O2', 'EMGc']], ['EOGl', 'F4', 'C4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 1.0, 5, [['EOGr', 'EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F3', 'C3', 'F4', 'C4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 1.0, 5, [['ELA', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 1.5, 5, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4', 'C4'], ['EOGl', 'C3', 'F4', 'C4'], ['EOGr', 'F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 1.5, 5, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'C3', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 1.5, 5, [['EOGr', 'C4'], ['EOGr', 'C4', 'EMGc'], ['ERA', 'EOGr', 'C4'], ['C3', 'C4', 'O2'], ['ERC', 'EOGr'], ['C4']], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 1.5, 5, [['ELA', 'ERA', 'ERI', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'C4', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C4', 'EMGc']], ['ERA', 'EOGr', 'C4', 'EMGc']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 2.0, 5, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 2.0, 5, [['EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 2.0, 5, [['ERI', 'C4'], [], ['C4'], ['C4'], ['ERC'], ['C4']], []], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 2.5, 5, [['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 2.5, 5, [['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 3.0, 5, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 3.0, 5, [['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 4.0, 5, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 4.0, 5, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 5.0, 5, [['F4'], ['F4'], [], [], ['F4'], ['F4']], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 5.0, 5, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 1.0, 6, [['EOGr', 'EOGl', 'F4', 'C4'], ['ERA', 'EOGr', 'EOGl', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'C3', 'F4', 'C4', 'EMGc'], ['EOGl', 'F3', 'C3', 'F4', 'C4'], ['ERC', 'EOGr', 'EOGl', 'EMGr', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'F4', 'O2', 'EMGc']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 1.0, 6, [['EOGr', 'EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F3', 'C3', 'F4', 'C4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 1.0, 6, [['ELA', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 1.5, 6, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4', 'C4'], ['EOGl', 'C3', 'F4', 'C4'], ['EOGr', 'F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 1.5, 6, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'C3', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 1.5, 6, [['EOGr', 'C4'], ['EOGr', 'C4', 'EMGc'], ['ERA', 'EOGr', 'C4'], ['C3', 'C4', 'O2'], ['ERC', 'EOGr'], ['C4']], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 1.5, 6, [['ELA', 'ERA', 'ERI', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'C4', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C4', 'EMGc']], ['ERA', 'EOGr', 'C4', 'EMGc']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 2.0, 6, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 2.0, 6, [['EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 2.0, 6, [['ERI', 'C4'], [], ['C4'], ['C4'], ['ERC'], ['C4']], []], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 2.5, 6, [['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 2.5, 6, [['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 3.0, 6, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 3.0, 6, [['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 4.0, 6, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 4.0, 6, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'mm', 5.0, 6, [['F4'], ['F4'], [], [], ['F4'], ['F4']], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'mm', 5.0, 6, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4170, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4170, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 1.0, 1, [['EOGl', 'C3', 'F4'], ['EOGl', 'F4', 'C4', 'O2'], ['EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGl', 'F3', 'C3', 'F4']], ['EOGl', 'C3', 'F4', 'C4', 'O2', 'F3', 'O1', 'ELA', 'ERA', 'ERC', 'EOGr', 'EMGc']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 1.0, 1, [['EOGl', 'F3', 'C3', 'O1', 'F4', 'C4'], ['EOGl', 'C3', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F3', 'C3', 'F4', 'C4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F3', 'C3', 'O1', 'F4', 'C4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 1.0, 1, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 1.5, 1, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'C3', 'F4', 'C4'], ['EOGl', 'F4', 'C4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4', 'C3', 'C4']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 1.5, 1, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 1.5, 1, [['C3', 'C4'], ['ERA', 'F3', 'C4', 'EMGc'], ['F3', 'C4', 'EMGc'], ['F3', 'C4', 'EMGc'], ['ERA', 'F3'], ['EMGl', 'F3', 'C4', 'EMGc']], ['C3', 'C4', 'ERA', 'F3', 'EMGc', 'EMGl']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 1.5, 1, [['ERA', 'EOGr', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'C4', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EMGl', 'F3', 'C4', 'EMGc']], ['ERA', 'EOGr', 'C4', 'O2', 'EMGc', 'ELA', 'F3', 'C3', 'ERC', 'ELC', 'EMGl']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 2.0, 1, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 2.0, 1, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 2.0, 1, [[], ['ERA', 'EMGc'], ['EMGc'], ['EMGc'], ['ERA'], ['EMGc']], ['ERA', 'EMGc']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 2.5, 1, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 2.5, 1, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 3.0, 1, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 3.0, 1, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 4.0, 1, [['EOGl', 'F4'], ['EOGl', 'F4'], [], [], ['F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 4.0, 1, [['F4'], ['F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['F4']], ['F4', 'EOGl']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], ['F4'], ['EOGl', 'F4']], ['F4', 'EOGl']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 5.0, 1, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 1.0, 2, [['EOGl', 'C3', 'F4'], ['EOGl', 'F4', 'C4', 'O2'], ['EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGl', 'F3', 'C3', 'F4']], ['EOGl', 'C3', 'F4', 'C4', 'O2', 'F3', 'O1']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 1.0, 2, [['EOGl', 'F3', 'C3', 'O1', 'F4', 'C4'], ['EOGl', 'C3', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F3', 'C3', 'F4', 'C4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'C3', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 1.0, 2, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 1.5, 2, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'C3', 'F4', 'C4'], ['EOGl', 'F4', 'C4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4', 'C4']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 1.5, 2, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 1.5, 2, [['C3', 'C4'], ['ERA', 'F3', 'C4', 'EMGc'], ['F3', 'C4', 'EMGc'], ['F3', 'C4', 'EMGc'], ['ERA', 'F3'], ['EMGl', 'F3', 'C4', 'EMGc']], ['C4', 'F3', 'EMGc']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 1.5, 2, [['ERA', 'EOGr', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'C4', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EMGl', 'F3', 'C4', 'EMGc']], ['ERA', 'EOGr', 'C4', 'EMGc', 'ELA', 'F3', 'ERC']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 2.0, 2, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 2.0, 2, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 2.0, 2, [[], ['ERA', 'EMGc'], ['EMGc'], ['EMGc'], ['ERA'], ['EMGc']], ['EMGc']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 2.5, 2, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 2.5, 2, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 3.0, 2, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 3.0, 2, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 4.0, 2, [['EOGl', 'F4'], ['EOGl', 'F4'], [], [], ['F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 4.0, 2, [['F4'], ['F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], ['F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 5.0, 2, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 1.0, 3, [['EOGl', 'C3', 'F4'], ['EOGl', 'F4', 'C4', 'O2'], ['EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGl', 'F3', 'C3', 'F4']], ['EOGl', 'C3', 'F4', 'C4', 'F3', 'O1']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 1.0, 3, [['EOGl', 'F3', 'C3', 'O1', 'F4', 'C4'], ['EOGl', 'C3', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F3', 'C3', 'F4', 'C4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 1.0, 3, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 1.5, 3, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'C3', 'F4', 'C4'], ['EOGl', 'F4', 'C4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 1.5, 3, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 1.5, 3, [['C3', 'C4'], ['ERA', 'F3', 'C4', 'EMGc'], ['F3', 'C4', 'EMGc'], ['F3', 'C4', 'EMGc'], ['ERA', 'F3'], ['EMGl', 'F3', 'C4', 'EMGc']], ['C4', 'F3', 'EMGc']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 1.5, 3, [['ERA', 'EOGr', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'C4', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EMGl', 'F3', 'C4', 'EMGc']], ['ERA', 'EOGr', 'C4', 'EMGc', 'ELA']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 2.0, 3, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 2.0, 3, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 2.0, 3, [[], ['ERA', 'EMGc'], ['EMGc'], ['EMGc'], ['ERA'], ['EMGc']], ['EMGc']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 2.5, 3, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 2.5, 3, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 3.0, 3, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 3.0, 3, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 4.0, 3, [['EOGl', 'F4'], ['EOGl', 'F4'], [], [], ['F4'], ['EOGl', 'F4']], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 4.0, 3, [['F4'], ['F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], ['F4'], ['EOGl', 'F4']], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 5.0, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 1.0, 4, [['EOGl', 'C3', 'F4'], ['EOGl', 'F4', 'C4', 'O2'], ['EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGl', 'F3', 'C3', 'F4']], ['EOGl', 'C3', 'F4', 'C4', 'F3']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 1.0, 4, [['EOGl', 'F3', 'C3', 'O1', 'F4', 'C4'], ['EOGl', 'C3', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F3', 'C3', 'F4', 'C4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 1.0, 4, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 1.5, 4, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'C3', 'F4', 'C4'], ['EOGl', 'F4', 'C4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 1.5, 4, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 1.5, 4, [['C3', 'C4'], ['ERA', 'F3', 'C4', 'EMGc'], ['F3', 'C4', 'EMGc'], ['F3', 'C4', 'EMGc'], ['ERA', 'F3'], ['EMGl', 'F3', 'C4', 'EMGc']], ['C4', 'F3']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 1.5, 4, [['ERA', 'EOGr', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'C4', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EMGl', 'F3', 'C4', 'EMGc']], ['ERA', 'C4', 'EMGc', 'ELA']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 2.0, 4, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 2.0, 4, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 2.0, 4, [[], ['ERA', 'EMGc'], ['EMGc'], ['EMGc'], ['ERA'], ['EMGc']], []], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 2.5, 4, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 2.5, 4, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 3.0, 4, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 3.0, 4, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 4.0, 4, [['EOGl', 'F4'], ['EOGl', 'F4'], [], [], ['F4'], ['EOGl', 'F4']], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 4.0, 4, [['F4'], ['F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], ['F4'], ['EOGl', 'F4']], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 5.0, 4, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 1.0, 5, [['EOGl', 'C3', 'F4'], ['EOGl', 'F4', 'C4', 'O2'], ['EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGl', 'F3', 'C3', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 1.0, 5, [['EOGl', 'F3', 'C3', 'O1', 'F4', 'C4'], ['EOGl', 'C3', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F3', 'C3', 'F4', 'C4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 1.0, 5, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 1.5, 5, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'C3', 'F4', 'C4'], ['EOGl', 'F4', 'C4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 1.5, 5, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 1.5, 5, [['C3', 'C4'], ['ERA', 'F3', 'C4', 'EMGc'], ['F3', 'C4', 'EMGc'], ['F3', 'C4', 'EMGc'], ['ERA', 'F3'], ['EMGl', 'F3', 'C4', 'EMGc']], ['F3']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 1.5, 5, [['ERA', 'EOGr', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'C4', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EMGl', 'F3', 'C4', 'EMGc']], ['ERA', 'EMGc', 'ELA']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 2.0, 5, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 2.0, 5, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 2.0, 5, [[], ['ERA', 'EMGc'], ['EMGc'], ['EMGc'], ['ERA'], ['EMGc']], []], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 2.5, 5, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 2.5, 5, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 3.0, 5, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 3.0, 5, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 4.0, 5, [['EOGl', 'F4'], ['EOGl', 'F4'], [], [], ['F4'], ['EOGl', 'F4']], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 4.0, 5, [['F4'], ['F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], ['F4'], ['EOGl', 'F4']], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 5.0, 5, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 1.0, 6, [['EOGl', 'C3', 'F4'], ['EOGl', 'F4', 'C4', 'O2'], ['EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGl', 'F3', 'C3', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 1.0, 6, [['EOGl', 'F3', 'C3', 'O1', 'F4', 'C4'], ['EOGl', 'C3', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F3', 'C3', 'F4', 'C4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 1.0, 6, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 1.5, 6, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'C3', 'F4', 'C4'], ['EOGl', 'F4', 'C4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 1.5, 6, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 1.5, 6, [['C3', 'C4'], ['ERA', 'F3', 'C4', 'EMGc'], ['F3', 'C4', 'EMGc'], ['F3', 'C4', 'EMGc'], ['ERA', 'F3'], ['EMGl', 'F3', 'C4', 'EMGc']], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 1.5, 6, [['ERA', 'EOGr', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'C4', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EMGl', 'F3', 'C4', 'EMGc']], ['ERA', 'EMGc']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 2.0, 6, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 2.0, 6, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 2.0, 6, [[], ['ERA', 'EMGc'], ['EMGc'], ['EMGc'], ['ERA'], ['EMGc']], []], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 2.5, 6, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 2.5, 6, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 3.0, 6, [['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 3.0, 6, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 4.0, 6, [['EOGl', 'F4'], ['EOGl', 'F4'], [], [], ['F4'], ['EOGl', 'F4']], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 4.0, 6, [['F4'], ['F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], ['F4'], ['EOGl', 'F4']], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'mm', 5.0, 6, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4560, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4560, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 1.0, 1, [['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ELA']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 1.0, 1, [['EOGr', 'EOGl', 'C3', 'F4', 'C4'], ['EOGl', 'F4'], ['EOGl', 'C3', 'F4', 'C4'], ['EOGl', 'F3', 'C3', 'F4'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGl', 'O1', 'F4', 'O2']], ['EOGr', 'EOGl', 'C3', 'F4', 'C4', 'F3', 'O1', 'O2']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc', 'EOGl']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 1.0, 1, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 1.5, 1, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F3', 'F4', 'C4'], ['EOGl', 'O1', 'F4'], ['EOGl', 'F3', 'C3', 'F4', 'C4'], ['EOGr', 'EOGl', 'F3', 'F4']], ['EOGl', 'F4', 'F3', 'C4', 'O1', 'C3', 'EOGr']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 1.5, 1, [['EOGl', 'F4', 'C4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'C3', 'F4'], ['EOGl', 'C3', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4', 'C4', 'C3']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 1.5, 1, [['F3', 'C4'], ['F3', 'C4', 'EMGc'], ['EOGr', 'F3', 'C4', 'EMGc'], ['F3', 'O1', 'EMGc'], ['F3', 'C4', 'O2'], ['EOGr', 'F3', 'O1', 'EMGc']], ['F3', 'C4', 'EMGc', 'EOGr', 'O1', 'O2']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 1.5, 1, [['ERA', 'ERC', 'EOGr', 'F3', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'F3', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'F3', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'O1', 'C4', 'O2', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'F3', 'O1', 'EMGc']], ['ERA', 'ERC', 'EOGr', 'F3', 'C4', 'EMGc', 'ELA', 'O1', 'O2']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 2.0, 1, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 2.0, 1, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 2.0, 1, [[], ['EMGc'], ['EMGc'], ['EMGc'], ['ELA', 'ERC'], ['EMGc']], ['EMGc', 'ELA', 'ERC']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 2.5, 1, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['EOGl', 'F4']], ['F4', 'EOGl']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 2.5, 1, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 3.0, 1, [['F4'], ['F4'], ['F4'], ['F4'], [], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 3.0, 1, [['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 4.0, 1, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 5.0, 1, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 1.0, 2, [['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 1.0, 2, [['EOGr', 'EOGl', 'C3', 'F4', 'C4'], ['EOGl', 'F4'], ['EOGl', 'C3', 'F4', 'C4'], ['EOGl', 'F3', 'C3', 'F4'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGl', 'O1', 'F4', 'O2']], ['EOGl', 'C3', 'F4', 'F3', 'O1', 'O2']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 1.0, 2, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 1.5, 2, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F3', 'F4', 'C4'], ['EOGl', 'O1', 'F4'], ['EOGl', 'F3', 'C3', 'F4', 'C4'], ['EOGr', 'EOGl', 'F3', 'F4']], ['EOGl', 'F4', 'F3']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 1.5, 2, [['EOGl', 'F4', 'C4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'C3', 'F4'], ['EOGl', 'C3', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4', 'C3']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 1.5, 2, [['F3', 'C4'], ['F3', 'C4', 'EMGc'], ['EOGr', 'F3', 'C4', 'EMGc'], ['F3', 'O1', 'EMGc'], ['F3', 'C4', 'O2'], ['EOGr', 'F3', 'O1', 'EMGc']], ['F3', 'C4', 'EMGc']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 1.5, 2, [['ERA', 'ERC', 'EOGr', 'F3', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'F3', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'F3', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'O1', 'C4', 'O2', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'F3', 'O1', 'EMGc']], ['ERA', 'ERC', 'EOGr', 'F3', 'C4', 'EMGc', 'ELA', 'O1']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 2.0, 2, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 2.0, 2, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 2.0, 2, [[], ['EMGc'], ['EMGc'], ['EMGc'], ['ELA', 'ERC'], ['EMGc']], ['EMGc']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 2.5, 2, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 2.5, 2, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 3.0, 2, [['F4'], ['F4'], ['F4'], ['F4'], [], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 3.0, 2, [['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 4.0, 2, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 5.0, 2, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 1.0, 3, [['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 1.0, 3, [['EOGr', 'EOGl', 'C3', 'F4', 'C4'], ['EOGl', 'F4'], ['EOGl', 'C3', 'F4', 'C4'], ['EOGl', 'F3', 'C3', 'F4'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGl', 'O1', 'F4', 'O2']], ['EOGl', 'C3', 'F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 1.0, 3, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 1.5, 3, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F3', 'F4', 'C4'], ['EOGl', 'O1', 'F4'], ['EOGl', 'F3', 'C3', 'F4', 'C4'], ['EOGr', 'EOGl', 'F3', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 1.5, 3, [['EOGl', 'F4', 'C4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'C3', 'F4'], ['EOGl', 'C3', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 1.5, 3, [['F3', 'C4'], ['F3', 'C4', 'EMGc'], ['EOGr', 'F3', 'C4', 'EMGc'], ['F3', 'O1', 'EMGc'], ['F3', 'C4', 'O2'], ['EOGr', 'F3', 'O1', 'EMGc']], ['F3', 'C4', 'EMGc']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 1.5, 3, [['ERA', 'ERC', 'EOGr', 'F3', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'F3', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'F3', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'O1', 'C4', 'O2', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'F3', 'O1', 'EMGc']], ['ERA', 'ERC', 'EOGr', 'F3', 'C4', 'EMGc', 'ELA']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 2.0, 3, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 2.0, 3, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 2.0, 3, [[], ['EMGc'], ['EMGc'], ['EMGc'], ['ELA', 'ERC'], ['EMGc']], ['EMGc']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 2.5, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 2.5, 3, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 3.0, 3, [['F4'], ['F4'], ['F4'], ['F4'], [], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 3.0, 3, [['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 4.0, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 5.0, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 1.0, 4, [['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGr', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 1.0, 4, [['EOGr', 'EOGl', 'C3', 'F4', 'C4'], ['EOGl', 'F4'], ['EOGl', 'C3', 'F4', 'C4'], ['EOGl', 'F3', 'C3', 'F4'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGl', 'O1', 'F4', 'O2']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 1.0, 4, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 1.5, 4, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F3', 'F4', 'C4'], ['EOGl', 'O1', 'F4'], ['EOGl', 'F3', 'C3', 'F4', 'C4'], ['EOGr', 'EOGl', 'F3', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 1.5, 4, [['EOGl', 'F4', 'C4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'C3', 'F4'], ['EOGl', 'C3', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 1.5, 4, [['F3', 'C4'], ['F3', 'C4', 'EMGc'], ['EOGr', 'F3', 'C4', 'EMGc'], ['F3', 'O1', 'EMGc'], ['F3', 'C4', 'O2'], ['EOGr', 'F3', 'O1', 'EMGc']], ['F3']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 1.5, 4, [['ERA', 'ERC', 'EOGr', 'F3', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'F3', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'F3', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'O1', 'C4', 'O2', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'F3', 'O1', 'EMGc']], ['ERA', 'EOGr', 'EMGc', 'ELA']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 2.0, 4, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 2.0, 4, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 2.0, 4, [[], ['EMGc'], ['EMGc'], ['EMGc'], ['ELA', 'ERC'], ['EMGc']], []], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 2.5, 4, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 2.5, 4, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 3.0, 4, [['F4'], ['F4'], ['F4'], ['F4'], [], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 3.0, 4, [['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 4.0, 4, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 5.0, 4, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 1.0, 5, [['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGr', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'O2']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 1.0, 5, [['EOGr', 'EOGl', 'C3', 'F4', 'C4'], ['EOGl', 'F4'], ['EOGl', 'C3', 'F4', 'C4'], ['EOGl', 'F3', 'C3', 'F4'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGl', 'O1', 'F4', 'O2']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 1.0, 5, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 1.5, 5, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F3', 'F4', 'C4'], ['EOGl', 'O1', 'F4'], ['EOGl', 'F3', 'C3', 'F4', 'C4'], ['EOGr', 'EOGl', 'F3', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 1.5, 5, [['EOGl', 'F4', 'C4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'C3', 'F4'], ['EOGl', 'C3', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 1.5, 5, [['F3', 'C4'], ['F3', 'C4', 'EMGc'], ['EOGr', 'F3', 'C4', 'EMGc'], ['F3', 'O1', 'EMGc'], ['F3', 'C4', 'O2'], ['EOGr', 'F3', 'O1', 'EMGc']], ['F3']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 1.5, 5, [['ERA', 'ERC', 'EOGr', 'F3', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'F3', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'F3', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'O1', 'C4', 'O2', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'F3', 'O1', 'EMGc']], ['ERA', 'EOGr', 'EMGc']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 2.0, 5, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 2.0, 5, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 2.0, 5, [[], ['EMGc'], ['EMGc'], ['EMGc'], ['ELA', 'ERC'], ['EMGc']], []], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 2.5, 5, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 2.5, 5, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 3.0, 5, [['F4'], ['F4'], ['F4'], ['F4'], [], ['F4']], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 3.0, 5, [['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 4.0, 5, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 5.0, 5, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 1.0, 6, [['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGr', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'O2']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 1.0, 6, [['EOGr', 'EOGl', 'C3', 'F4', 'C4'], ['EOGl', 'F4'], ['EOGl', 'C3', 'F4', 'C4'], ['EOGl', 'F3', 'C3', 'F4'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGl', 'O1', 'F4', 'O2']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 1.0, 6, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 1.5, 6, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F3', 'F4', 'C4'], ['EOGl', 'O1', 'F4'], ['EOGl', 'F3', 'C3', 'F4', 'C4'], ['EOGr', 'EOGl', 'F3', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 1.5, 6, [['EOGl', 'F4', 'C4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'C3', 'F4'], ['EOGl', 'C3', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 1.5, 6, [['F3', 'C4'], ['F3', 'C4', 'EMGc'], ['EOGr', 'F3', 'C4', 'EMGc'], ['F3', 'O1', 'EMGc'], ['F3', 'C4', 'O2'], ['EOGr', 'F3', 'O1', 'EMGc']], ['F3']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 1.5, 6, [['ERA', 'ERC', 'EOGr', 'F3', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'F3', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'F3', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'O1', 'C4', 'O2', 'EMGc'], ['ERA', 'ERC', 'EOGr', 'F3', 'O1', 'EMGc']], ['ERA', 'EOGr', 'EMGc']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 2.0, 6, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 2.0, 6, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 2.0, 6, [[], ['EMGc'], ['EMGc'], ['EMGc'], ['ELA', 'ERC'], ['EMGc']], []], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 2.5, 6, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['EOGl', 'F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 2.5, 6, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 3.0, 6, [['F4'], ['F4'], ['F4'], ['F4'], [], ['F4']], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 3.0, 6, [['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 4.0, 6, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'mm', 5.0, 6, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 4840, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 4840, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELC', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc', 'ERA', 'ERT ', 'ERI', 'EMGl', 'ELB']], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 1.0, 1, [['EOGl', 'C3', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'F3', 'O1', 'O2'], ['EOGl', 'F3', 'C3', 'C4', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'C4'], ['EOGl', 'C4'], ['ELA', 'EOGr', 'EOGl', 'F3', 'C4']], ['EOGl', 'C3', 'C4', 'O2', 'ERA', 'EOGr', 'F3', 'O1', 'EMGc', 'ELA']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc', 'EMGl', 'ELB']], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 1.0, 1, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc', 'EMGl', 'ELB']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 1.5, 1, [['EOGl'], ['EOGr', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGl', 'F3', 'C4'], ['EOGl', 'F3']], ['EOGl', 'EOGr', 'F3', 'C4']], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 1.5, 1, [['EOGl'], ['EOGr', 'EOGl'], ['EOGl'], ['EOGl', 'F3'], ['EOGl'], ['EOGl', 'C4']], ['EOGl', 'EOGr', 'F3', 'C4']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 1.5, 1, [['F3', 'C4'], ['O1', 'C4', 'O2'], ['ELB', 'F3'], ['F3', 'C3', 'C4'], [], ['EMGl', 'F3']], ['F3', 'C4', 'O1', 'O2', 'ELB', 'C3', 'EMGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 1.5, 1, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'O1', 'C4', 'EMGc'], ['ELB', 'ERA', 'EMGr', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'O1', 'O2', 'EMGc'], ['ERA', 'ERC', 'EMGl', 'EMGr']], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'C4', 'EMGc', 'O1', 'ELB', 'EMGr', 'C3', 'F3', 'O2', 'EMGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 2.0, 1, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 2.0, 1, [['EOGl'], ['EOGr', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl', 'C4']], ['EOGl', 'EOGr', 'C4']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], ['EMGl']], ['EMGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 2.5, 1, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 2.5, 1, [['EOGl'], ['EOGr', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl', 'C4']], ['EOGl', 'EOGr', 'C4']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 3.0, 1, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 4.0, 1, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 5.0, 1, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELC', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc', 'ERA']], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 1.0, 2, [['EOGl', 'C3', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'F3', 'O1', 'O2'], ['EOGl', 'F3', 'C3', 'C4', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'C4'], ['EOGl', 'C4'], ['ELA', 'EOGr', 'EOGl', 'F3', 'C4']], ['EOGl', 'C3', 'C4', 'O2', 'F3']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc', 'EMGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 1.0, 2, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc', 'EMGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 1.5, 2, [['EOGl'], ['EOGr', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGl', 'F3', 'C4'], ['EOGl', 'F3']], ['EOGl', 'F3']], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 1.5, 2, [['EOGl'], ['EOGr', 'EOGl'], ['EOGl'], ['EOGl', 'F3'], ['EOGl'], ['EOGl', 'C4']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 1.5, 2, [['F3', 'C4'], ['O1', 'C4', 'O2'], ['ELB', 'F3'], ['F3', 'C3', 'C4'], [], ['EMGl', 'F3']], ['F3', 'C4']], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 1.5, 2, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'O1', 'C4', 'EMGc'], ['ELB', 'ERA', 'EMGr', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'O1', 'O2', 'EMGc'], ['ERA', 'ERC', 'EMGl', 'EMGr']], ['ELA', 'ERA', 'EOGr', 'C4', 'EMGc']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 2.0, 2, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 2.0, 2, [['EOGl'], ['EOGr', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl', 'C4']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], ['EMGl']], []], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 2.5, 2, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 2.5, 2, [['EOGl'], ['EOGr', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl', 'C4']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 3.0, 2, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 4.0, 2, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 5.0, 2, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELC', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc', 'ERA']], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 1.0, 3, [['EOGl', 'C3', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'F3', 'O1', 'O2'], ['EOGl', 'F3', 'C3', 'C4', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'C4'], ['EOGl', 'C4'], ['ELA', 'EOGr', 'EOGl', 'F3', 'C4']], ['EOGl', 'C4', 'F3']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 1.0, 3, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc', 'EMGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 1.5, 3, [['EOGl'], ['EOGr', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGl', 'F3', 'C4'], ['EOGl', 'F3']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 1.5, 3, [['EOGl'], ['EOGr', 'EOGl'], ['EOGl'], ['EOGl', 'F3'], ['EOGl'], ['EOGl', 'C4']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 1.5, 3, [['F3', 'C4'], ['O1', 'C4', 'O2'], ['ELB', 'F3'], ['F3', 'C3', 'C4'], [], ['EMGl', 'F3']], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 1.5, 3, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'O1', 'C4', 'EMGc'], ['ELB', 'ERA', 'EMGr', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'O1', 'O2', 'EMGc'], ['ERA', 'ERC', 'EMGl', 'EMGr']], ['ERA', 'EMGc']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 2.0, 3, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 2.0, 3, [['EOGl'], ['EOGr', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl', 'C4']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], ['EMGl']], []], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 2.5, 3, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 2.5, 3, [['EOGl'], ['EOGr', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl', 'C4']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 3.0, 3, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 4.0, 3, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 5.0, 3, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELC', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc', 'ERA']], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 1.0, 4, [['EOGl', 'C3', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'F3', 'O1', 'O2'], ['EOGl', 'F3', 'C3', 'C4', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'C4'], ['EOGl', 'C4'], ['ELA', 'EOGr', 'EOGl', 'F3', 'C4']], ['EOGl', 'C4']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 1.0, 4, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 1.5, 4, [['EOGl'], ['EOGr', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGl', 'F3', 'C4'], ['EOGl', 'F3']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 1.5, 4, [['EOGl'], ['EOGr', 'EOGl'], ['EOGl'], ['EOGl', 'F3'], ['EOGl'], ['EOGl', 'C4']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 1.5, 4, [['F3', 'C4'], ['O1', 'C4', 'O2'], ['ELB', 'F3'], ['F3', 'C3', 'C4'], [], ['EMGl', 'F3']], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 1.5, 4, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'O1', 'C4', 'EMGc'], ['ELB', 'ERA', 'EMGr', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'O1', 'O2', 'EMGc'], ['ERA', 'ERC', 'EMGl', 'EMGr']], ['ERA', 'EMGc']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 2.0, 4, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 2.0, 4, [['EOGl'], ['EOGr', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl', 'C4']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], ['EMGl']], []], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 2.5, 4, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 2.5, 4, [['EOGl'], ['EOGr', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl', 'C4']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 3.0, 4, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 4.0, 4, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 5.0, 4, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELC', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc', 'ERA']], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 1.0, 5, [['EOGl', 'C3', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'F3', 'O1', 'O2'], ['EOGl', 'F3', 'C3', 'C4', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'C4'], ['EOGl', 'C4'], ['ELA', 'EOGr', 'EOGl', 'F3', 'C4']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 1.0, 5, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 1.5, 5, [['EOGl'], ['EOGr', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGl', 'F3', 'C4'], ['EOGl', 'F3']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 1.5, 5, [['EOGl'], ['EOGr', 'EOGl'], ['EOGl'], ['EOGl', 'F3'], ['EOGl'], ['EOGl', 'C4']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 1.5, 5, [['F3', 'C4'], ['O1', 'C4', 'O2'], ['ELB', 'F3'], ['F3', 'C3', 'C4'], [], ['EMGl', 'F3']], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 1.5, 5, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'O1', 'C4', 'EMGc'], ['ELB', 'ERA', 'EMGr', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'O1', 'O2', 'EMGc'], ['ERA', 'ERC', 'EMGl', 'EMGr']], ['ERA', 'EMGc']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 2.0, 5, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 2.0, 5, [['EOGl'], ['EOGr', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl', 'C4']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], ['EMGl']], []], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 2.5, 5, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 2.5, 5, [['EOGl'], ['EOGr', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl', 'C4']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 3.0, 5, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 4.0, 5, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 5.0, 5, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELC', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ERA', 'ERC', 'ERT ', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 1.0, 6, [['EOGl', 'C3', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'F3', 'O1', 'O2'], ['EOGl', 'F3', 'C3', 'C4', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'C4'], ['EOGl', 'C4'], ['ELA', 'EOGr', 'EOGl', 'F3', 'C4']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 1.0, 6, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 1.5, 6, [['EOGl'], ['EOGr', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGl', 'F3', 'C4'], ['EOGl', 'F3']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 1.5, 6, [['EOGl'], ['EOGr', 'EOGl'], ['EOGl'], ['EOGl', 'F3'], ['EOGl'], ['EOGl', 'C4']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 1.5, 6, [['F3', 'C4'], ['O1', 'C4', 'O2'], ['ELB', 'F3'], ['F3', 'C3', 'C4'], [], ['EMGl', 'F3']], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 1.5, 6, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'C4', 'EMGc'], ['ELA', 'ERA', 'O1', 'C4', 'EMGc'], ['ELB', 'ERA', 'EMGr', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'O1', 'O2', 'EMGc'], ['ERA', 'ERC', 'EMGl', 'EMGr']], ['ERA']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 2.0, 6, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 2.0, 6, [['EOGl'], ['EOGr', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl', 'C4']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], ['EMGl']], []], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 2.5, 6, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 2.5, 6, [['EOGl'], ['EOGr', 'EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl', 'C4']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 3.0, 6, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 4.0, 6, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'mm', 5.0, 6, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_11/night_1\\EEG_raw', 6400, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6400, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 1.0, 1, [['ERC', 'EOGl', 'F3', 'F4', 'C4'], ['ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ERC', 'EOGl', 'F3', 'F4', 'C4', 'ERA', 'C3', 'O1', 'O2', 'EOGr']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 1.0, 1, [['EOGl', 'F4'], ['EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGl', 'C3', 'F4', 'C4'], ['EOGl', 'F3', 'C3', 'O1', 'F4', 'O2'], ['EOGl', 'F3', 'C3', 'F4', 'C4'], ['EOGl', 'F4']], ['EOGl', 'F4', 'F3', 'C3', 'O1', 'C4', 'O2']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc', 'EMGl', 'M1', 'ELB']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 1.0, 1, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGl', 'EMGc', 'EMGr', 'ELB']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 1.5, 1, [['EOGl', 'F3', 'F4'], ['EOGl', 'F3', 'C3', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F3', 'F4', 'C3']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 1.5, 1, [['EOGl', 'F4'], ['EOGl', 'F3', 'C3', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4', 'F3', 'C3']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 1.5, 1, [['EOGr', 'C4'], ['C3', 'C4'], ['EOGr', 'F3', 'C4'], ['EOGr', 'F3', 'C4'], ['F3', 'C4'], ['EOGr']], ['EOGr', 'C4', 'C3', 'F3']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 1.5, 1, [['ELA', 'ERA', 'ERC', 'EOGr', 'C3', 'C4'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C3', 'O1', 'C4'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C4'], ['ELA', 'ERA', 'ERC', 'EOGr', 'F3', 'C4'], ['ERA', 'ERC', 'EOGr', 'C3', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'C3']], ['ELA', 'ERA', 'ERC', 'EOGr', 'C3', 'C4', 'O1', 'F3', 'ELB', 'ELC']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 2.0, 1, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 2.0, 1, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 2.0, 1, [['ERA'], [], ['ERC'], [], [], []], ['ERA', 'ERC']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 2.5, 1, [['F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4', 'EOGl']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 2.5, 1, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 3.0, 1, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 3.0, 1, [['F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4', 'EOGl']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 4.0, 1, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 4.0, 1, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 5.0, 1, [['F4'], [], ['F4'], [], [], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 5.0, 1, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 1.0, 2, [['ERC', 'EOGl', 'F3', 'F4', 'C4'], ['ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGl', 'F3', 'F4', 'C4', 'C3', 'O1', 'O2', 'EOGr']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 1.0, 2, [['EOGl', 'F4'], ['EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGl', 'C3', 'F4', 'C4'], ['EOGl', 'F3', 'C3', 'O1', 'F4', 'O2'], ['EOGl', 'F3', 'C3', 'F4', 'C4'], ['EOGl', 'F4']], ['EOGl', 'F4', 'F3', 'C3', 'C4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc', 'EMGl']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 1.0, 2, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc', 'EMGr']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 1.5, 2, [['EOGl', 'F3', 'F4'], ['EOGl', 'F3', 'C3', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F3', 'F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 1.5, 2, [['EOGl', 'F4'], ['EOGl', 'F3', 'C3', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 1.5, 2, [['EOGr', 'C4'], ['C3', 'C4'], ['EOGr', 'F3', 'C4'], ['EOGr', 'F3', 'C4'], ['F3', 'C4'], ['EOGr']], ['EOGr', 'C4', 'F3']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 1.5, 2, [['ELA', 'ERA', 'ERC', 'EOGr', 'C3', 'C4'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C3', 'O1', 'C4'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C4'], ['ELA', 'ERA', 'ERC', 'EOGr', 'F3', 'C4'], ['ERA', 'ERC', 'EOGr', 'C3', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'C3']], ['ELA', 'ERA', 'ERC', 'EOGr', 'C3', 'C4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 2.0, 2, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 2.0, 2, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 2.0, 2, [['ERA'], [], ['ERC'], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 2.5, 2, [['F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4', 'EOGl']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 2.5, 2, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 3.0, 2, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 3.0, 2, [['F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 4.0, 2, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 4.0, 2, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 5.0, 2, [['F4'], [], ['F4'], [], [], ['F4']], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 5.0, 2, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 1.0, 3, [['ERC', 'EOGl', 'F3', 'F4', 'C4'], ['ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGl', 'F3', 'F4', 'C4', 'O1', 'O2']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 1.0, 3, [['EOGl', 'F4'], ['EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGl', 'C3', 'F4', 'C4'], ['EOGl', 'F3', 'C3', 'O1', 'F4', 'O2'], ['EOGl', 'F3', 'C3', 'F4', 'C4'], ['EOGl', 'F4']], ['EOGl', 'F4', 'C3']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 1.0, 3, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGr']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 1.5, 3, [['EOGl', 'F3', 'F4'], ['EOGl', 'F3', 'C3', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 1.5, 3, [['EOGl', 'F4'], ['EOGl', 'F3', 'C3', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 1.5, 3, [['EOGr', 'C4'], ['C3', 'C4'], ['EOGr', 'F3', 'C4'], ['EOGr', 'F3', 'C4'], ['F3', 'C4'], ['EOGr']], ['C4', 'F3']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 1.5, 3, [['ELA', 'ERA', 'ERC', 'EOGr', 'C3', 'C4'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C3', 'O1', 'C4'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C4'], ['ELA', 'ERA', 'ERC', 'EOGr', 'F3', 'C4'], ['ERA', 'ERC', 'EOGr', 'C3', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'C3']], ['ELA', 'ERA', 'ERC', 'EOGr', 'C4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 2.0, 3, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 2.0, 3, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 2.0, 3, [['ERA'], [], ['ERC'], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 2.5, 3, [['F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 2.5, 3, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 3.0, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 3.0, 3, [['F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 4.0, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 4.0, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 5.0, 3, [['F4'], [], ['F4'], [], [], ['F4']], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 5.0, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 1.0, 4, [['ERC', 'EOGl', 'F3', 'F4', 'C4'], ['ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGl', 'F3', 'F4', 'C4']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 1.0, 4, [['EOGl', 'F4'], ['EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGl', 'C3', 'F4', 'C4'], ['EOGl', 'F3', 'C3', 'O1', 'F4', 'O2'], ['EOGl', 'F3', 'C3', 'F4', 'C4'], ['EOGl', 'F4']], ['EOGl', 'F4', 'C3']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 1.0, 4, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 1.5, 4, [['EOGl', 'F3', 'F4'], ['EOGl', 'F3', 'C3', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 1.5, 4, [['EOGl', 'F4'], ['EOGl', 'F3', 'C3', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 1.5, 4, [['EOGr', 'C4'], ['C3', 'C4'], ['EOGr', 'F3', 'C4'], ['EOGr', 'F3', 'C4'], ['F3', 'C4'], ['EOGr']], ['C4']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 1.5, 4, [['ELA', 'ERA', 'ERC', 'EOGr', 'C3', 'C4'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C3', 'O1', 'C4'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C4'], ['ELA', 'ERA', 'ERC', 'EOGr', 'F3', 'C4'], ['ERA', 'ERC', 'EOGr', 'C3', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'C3']], ['ELA', 'ERA', 'ERC', 'EOGr', 'C4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 2.0, 4, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 2.0, 4, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 2.0, 4, [['ERA'], [], ['ERC'], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 2.5, 4, [['F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 2.5, 4, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 3.0, 4, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 3.0, 4, [['F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 4.0, 4, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 4.0, 4, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 5.0, 4, [['F4'], [], ['F4'], [], [], ['F4']], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 5.0, 4, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 1.0, 5, [['ERC', 'EOGl', 'F3', 'F4', 'C4'], ['ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGl', 'F3', 'F4', 'C4']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 1.0, 5, [['EOGl', 'F4'], ['EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGl', 'C3', 'F4', 'C4'], ['EOGl', 'F3', 'C3', 'O1', 'F4', 'O2'], ['EOGl', 'F3', 'C3', 'F4', 'C4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 1.0, 5, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 1.5, 5, [['EOGl', 'F3', 'F4'], ['EOGl', 'F3', 'C3', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 1.5, 5, [['EOGl', 'F4'], ['EOGl', 'F3', 'C3', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 1.5, 5, [['EOGr', 'C4'], ['C3', 'C4'], ['EOGr', 'F3', 'C4'], ['EOGr', 'F3', 'C4'], ['F3', 'C4'], ['EOGr']], ['C4']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 1.5, 5, [['ELA', 'ERA', 'ERC', 'EOGr', 'C3', 'C4'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C3', 'O1', 'C4'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C4'], ['ELA', 'ERA', 'ERC', 'EOGr', 'F3', 'C4'], ['ERA', 'ERC', 'EOGr', 'C3', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'C3']], ['ERA', 'ERC', 'EOGr', 'C4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 2.0, 5, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 2.0, 5, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 2.0, 5, [['ERA'], [], ['ERC'], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 2.5, 5, [['F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 2.5, 5, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 3.0, 5, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 3.0, 5, [['F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 4.0, 5, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 4.0, 5, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 5.0, 5, [['F4'], [], ['F4'], [], [], ['F4']], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 5.0, 5, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 1.0, 6, [['ERC', 'EOGl', 'F3', 'F4', 'C4'], ['ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGl', 'F3', 'F4', 'C4']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 1.0, 6, [['EOGl', 'F4'], ['EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGl', 'C3', 'F4', 'C4'], ['EOGl', 'F3', 'C3', 'O1', 'F4', 'O2'], ['EOGl', 'F3', 'C3', 'F4', 'C4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 1.0, 6, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ERA', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 1.5, 6, [['EOGl', 'F3', 'F4'], ['EOGl', 'F3', 'C3', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 1.5, 6, [['EOGl', 'F4'], ['EOGl', 'F3', 'C3', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4']], ['EOGl', 'F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 1.5, 6, [['EOGr', 'C4'], ['C3', 'C4'], ['EOGr', 'F3', 'C4'], ['EOGr', 'F3', 'C4'], ['F3', 'C4'], ['EOGr']], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 1.5, 6, [['ELA', 'ERA', 'ERC', 'EOGr', 'C3', 'C4'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C3', 'O1', 'C4'], ['ELA', 'ERA', 'ERC', 'EOGr', 'C4'], ['ELA', 'ERA', 'ERC', 'EOGr', 'F3', 'C4'], ['ERA', 'ERC', 'EOGr', 'C3', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'C3']], ['ERA', 'ERC', 'EOGr']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 2.0, 6, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 2.0, 6, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 2.0, 6, [['ERA'], [], ['ERC'], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 2.5, 6, [['F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 2.5, 6, [['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 3.0, 6, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 3.0, 6, [['F4'], ['EOGl', 'F4'], ['F4'], ['F4'], ['EOGl', 'F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 4.0, 6, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 4.0, 6, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'mm', 5.0, 6, [['F4'], [], ['F4'], [], [], ['F4']], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'mm', 5.0, 6, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_11/night_1\\EEG_raw', 6660, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_11/night_1\\EEG_raw', 6660, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 1.0, 1, [['ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'F4', 'EMGc'], ['ELT', 'ELE', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc']], ['ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc', 'F3', 'F4', 'ELT', 'ELE', 'ELI', 'M1', 'ELA', 'O1', 'O2']], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 1.0, 1, [['ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'F4', 'O2'], ['ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'EMGc'], ['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'EOGr', 'ELI', 'F3', 'M2', 'F4', 'O2', 'ERB', 'M1']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 1.0, 1, [['ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'O2', 'EMGc'], ['ELB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'O2', 'EMGc']], ['ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'O2', 'EMGc', 'F3', 'O1', 'ELT', 'ELE', 'ELI', 'ERA', 'M1', 'ELA', 'F4']], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2']], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc', 'F3', 'ERE', 'F4']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 1.5, 1, [['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE'], ['EMGr', 'F3', 'F4'], ['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['ERE', 'EMGl', 'EMGr', 'EMGc', 'F3', 'F4', 'EOGl']], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 1.5, 1, [['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'EMGr', 'F3', 'F4'], ['ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGl', 'EMGr', 'EMGc']], ['ERE', 'EMGl', 'EMGr', 'EMGc', 'EOGr', 'EOGl', 'F3', 'F4']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 1.5, 1, [['ERE', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EMGl', 'EMGr', 'EMGc'], ['EMGc'], ['EMGr', 'F3', 'F4'], ['ERE', 'EOGl', 'EMGl', 'EMGr'], ['ERE', 'EOGr', 'EMGl', 'EMGr', 'EMGc']], ['ERE', 'EMGl', 'EMGr', 'EMGc', 'EOGr', 'F3', 'F4', 'EOGl']], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 1.5, 1, [['ERC', 'M2', 'EMGc'], ['EOGr', 'M2'], ['ERC', 'EOGl', 'M1', 'M2'], ['ERE', 'EMGr', 'M2'], ['ERB', 'EOGr', 'EMGr', 'M1', 'M2'], ['M1', 'M2']], ['ERC', 'M2', 'EMGc', 'EOGr', 'EOGl', 'M1', 'ERE', 'EMGr', 'ERB']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 2.0, 1, [[], [], ['ERE'], [], [], []], ['ERE']], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 2.0, 1, [['ERE', 'EMGl', 'EMGr'], ['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGc'], ['F3', 'F4'], ['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGr', 'EMGc']], ['ERE', 'EMGl', 'EMGr', 'EMGc', 'F3', 'F4']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 2.0, 1, [[], ['M2'], [], [], [], []], ['M2']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 2.5, 1, [['ERE', 'EMGl'], ['ERE', 'EMGr'], ['ERE', 'EMGc'], ['F3', 'F4'], ['ERE'], ['ERE', 'EMGc']], ['ERE', 'EMGl', 'EMGr', 'EMGc', 'F3', 'F4']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 3.0, 1, [['ERE'], ['ERE'], ['ERE'], [], ['ERE'], ['ERE', 'EMGc']], ['ERE', 'EMGc']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 4.0, 1, [['ERE'], ['ERE'], ['ERE'], [], [], ['ERE', 'EMGc']], ['ERE', 'EMGc']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 5.0, 1, [['ERE'], [], ['ERE'], [], [], []], ['ERE']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 1.0, 2, [['ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'F4', 'EMGc'], ['ELT', 'ELE', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc']], ['ELB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc', 'M1']], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 1.0, 2, [['ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'F4', 'O2'], ['ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'EMGc'], ['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'EOGr']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 1.0, 2, [['ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'O2', 'EMGc'], ['ELB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'O2', 'EMGc']], ['ELB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'O2', 'EMGc', 'M1']], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2']], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc', 'ERE']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 1.5, 2, [['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE'], ['EMGr', 'F3', 'F4'], ['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['ERE', 'EMGl', 'EMGr', 'EMGc']], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 1.5, 2, [['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'EMGr', 'F3', 'F4'], ['ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGl', 'EMGr', 'EMGc']], ['ERE', 'EMGl', 'EMGr', 'EMGc', 'EOGl']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 1.5, 2, [['ERE', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EMGl', 'EMGr', 'EMGc'], ['EMGc'], ['EMGr', 'F3', 'F4'], ['ERE', 'EOGl', 'EMGl', 'EMGr'], ['ERE', 'EOGr', 'EMGl', 'EMGr', 'EMGc']], ['ERE', 'EMGl', 'EMGr', 'EMGc']], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 1.5, 2, [['ERC', 'M2', 'EMGc'], ['EOGr', 'M2'], ['ERC', 'EOGl', 'M1', 'M2'], ['ERE', 'EMGr', 'M2'], ['ERB', 'EOGr', 'EMGr', 'M1', 'M2'], ['M1', 'M2']], ['M2', 'M1', 'EMGr']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 2.0, 2, [[], [], ['ERE'], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 2.0, 2, [['ERE', 'EMGl', 'EMGr'], ['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGc'], ['F3', 'F4'], ['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGr', 'EMGc']], ['ERE', 'EMGl', 'EMGr', 'EMGc']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 2.0, 2, [[], ['M2'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 2.5, 2, [['ERE', 'EMGl'], ['ERE', 'EMGr'], ['ERE', 'EMGc'], ['F3', 'F4'], ['ERE'], ['ERE', 'EMGc']], ['ERE']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 3.0, 2, [['ERE'], ['ERE'], ['ERE'], [], ['ERE'], ['ERE', 'EMGc']], ['ERE']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 4.0, 2, [['ERE'], ['ERE'], ['ERE'], [], [], ['ERE', 'EMGc']], ['ERE']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 5.0, 2, [['ERE'], [], ['ERE'], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 1.0, 3, [['ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'F4', 'EMGc'], ['ELT', 'ELE', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc']], ['ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc', 'M1']], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 1.0, 3, [['ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'F4', 'O2'], ['ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'EMGc'], ['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'EOGr']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 1.0, 3, [['ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'O2', 'EMGc'], ['ELB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'O2', 'EMGc']], ['ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc', 'M1']], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2']], ['ELA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc', 'ERE']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 1.5, 3, [['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE'], ['EMGr', 'F3', 'F4'], ['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['ERE', 'EMGr']], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 1.5, 3, [['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'EMGr', 'F3', 'F4'], ['ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGl', 'EMGr', 'EMGc']], ['ERE', 'EMGr', 'EMGc']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 1.5, 3, [['ERE', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EMGl', 'EMGr', 'EMGc'], ['EMGc'], ['EMGr', 'F3', 'F4'], ['ERE', 'EOGl', 'EMGl', 'EMGr'], ['ERE', 'EOGr', 'EMGl', 'EMGr', 'EMGc']], ['EMGr', 'EMGc']], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 1.5, 3, [['ERC', 'M2', 'EMGc'], ['EOGr', 'M2'], ['ERC', 'EOGl', 'M1', 'M2'], ['ERE', 'EMGr', 'M2'], ['ERB', 'EOGr', 'EMGr', 'M1', 'M2'], ['M1', 'M2']], ['M2']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 2.0, 3, [[], [], ['ERE'], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 2.0, 3, [['ERE', 'EMGl', 'EMGr'], ['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGc'], ['F3', 'F4'], ['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGr', 'EMGc']], ['ERE']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 2.0, 3, [[], ['M2'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 2.5, 3, [['ERE', 'EMGl'], ['ERE', 'EMGr'], ['ERE', 'EMGc'], ['F3', 'F4'], ['ERE'], ['ERE', 'EMGc']], ['ERE']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 3.0, 3, [['ERE'], ['ERE'], ['ERE'], [], ['ERE'], ['ERE', 'EMGc']], ['ERE']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 4.0, 3, [['ERE'], ['ERE'], ['ERE'], [], [], ['ERE', 'EMGc']], ['ERE']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 5.0, 3, [['ERE'], [], ['ERE'], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 1.0, 4, [['ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'F4', 'EMGc'], ['ELT', 'ELE', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc']], ['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc']], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 1.0, 4, [['ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'F4', 'O2'], ['ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'EMGc'], ['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['ERE', 'EMGl', 'EMGr']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 1.0, 4, [['ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'O2', 'EMGc'], ['ELB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc']], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2']], ['ELA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 1.5, 4, [['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE'], ['EMGr', 'F3', 'F4'], ['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 1.5, 4, [['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'EMGr', 'F3', 'F4'], ['ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGl', 'EMGr', 'EMGc']], ['EMGr']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 1.5, 4, [['ERE', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EMGl', 'EMGr', 'EMGc'], ['EMGc'], ['EMGr', 'F3', 'F4'], ['ERE', 'EOGl', 'EMGl', 'EMGr'], ['ERE', 'EOGr', 'EMGl', 'EMGr', 'EMGc']], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 1.5, 4, [['ERC', 'M2', 'EMGc'], ['EOGr', 'M2'], ['ERC', 'EOGl', 'M1', 'M2'], ['ERE', 'EMGr', 'M2'], ['ERB', 'EOGr', 'EMGr', 'M1', 'M2'], ['M1', 'M2']], ['M2']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 2.0, 4, [[], [], ['ERE'], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 2.0, 4, [['ERE', 'EMGl', 'EMGr'], ['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGc'], ['F3', 'F4'], ['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGr', 'EMGc']], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 2.0, 4, [[], ['M2'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 2.5, 4, [['ERE', 'EMGl'], ['ERE', 'EMGr'], ['ERE', 'EMGc'], ['F3', 'F4'], ['ERE'], ['ERE', 'EMGc']], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 3.0, 4, [['ERE'], ['ERE'], ['ERE'], [], ['ERE'], ['ERE', 'EMGc']], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 4.0, 4, [['ERE'], ['ERE'], ['ERE'], [], [], ['ERE', 'EMGc']], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 5.0, 4, [['ERE'], [], ['ERE'], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 1.0, 5, [['ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'F4', 'EMGc'], ['ELT', 'ELE', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc']], ['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc']], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 1.0, 5, [['ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'F4', 'O2'], ['ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'EMGc'], ['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['ERE', 'EMGl', 'EMGr']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 1.0, 5, [['ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'O2', 'EMGc'], ['ELB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc']], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2']], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'O2', 'EMGc']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 1.5, 5, [['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE'], ['EMGr', 'F3', 'F4'], ['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 1.5, 5, [['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'EMGr', 'F3', 'F4'], ['ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGl', 'EMGr', 'EMGc']], ['EMGr']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 1.5, 5, [['ERE', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EMGl', 'EMGr', 'EMGc'], ['EMGc'], ['EMGr', 'F3', 'F4'], ['ERE', 'EOGl', 'EMGl', 'EMGr'], ['ERE', 'EOGr', 'EMGl', 'EMGr', 'EMGc']], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 1.5, 5, [['ERC', 'M2', 'EMGc'], ['EOGr', 'M2'], ['ERC', 'EOGl', 'M1', 'M2'], ['ERE', 'EMGr', 'M2'], ['ERB', 'EOGr', 'EMGr', 'M1', 'M2'], ['M1', 'M2']], ['M2']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 2.0, 5, [[], [], ['ERE'], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 2.0, 5, [['ERE', 'EMGl', 'EMGr'], ['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGc'], ['F3', 'F4'], ['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGr', 'EMGc']], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 2.0, 5, [[], ['M2'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 2.5, 5, [['ERE', 'EMGl'], ['ERE', 'EMGr'], ['ERE', 'EMGc'], ['F3', 'F4'], ['ERE'], ['ERE', 'EMGc']], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 3.0, 5, [['ERE'], ['ERE'], ['ERE'], [], ['ERE'], ['ERE', 'EMGc']], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 4.0, 5, [['ERE'], ['ERE'], ['ERE'], [], [], ['ERE', 'EMGc']], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 5.0, 5, [['ERE'], [], ['ERE'], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 1.0, 6, [['ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'F4', 'EMGc'], ['ELT', 'ELE', 'ELI', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc']], ['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc']], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 1.0, 6, [['ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELI', 'ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'F4', 'O2'], ['ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'EMGc'], ['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['ERE', 'EMGl', 'EMGr']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 1.0, 6, [['ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'O2', 'EMGc'], ['ELB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELT', 'ELE', 'ELI', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'EMGc'], ['ELA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'M2', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc']], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2']], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'M2', 'O2']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 1.5, 6, [['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE'], ['EMGr', 'F3', 'F4'], ['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 1.5, 6, [['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGr', 'EMGc'], ['EOGr', 'EOGl', 'EMGr', 'F3', 'F4'], ['ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGl', 'EMGr', 'EMGc']], ['EMGr']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 1.5, 6, [['ERE', 'EMGl', 'EMGr', 'EMGc'], ['EOGr', 'EMGl', 'EMGr', 'EMGc'], ['EMGc'], ['EMGr', 'F3', 'F4'], ['ERE', 'EOGl', 'EMGl', 'EMGr'], ['ERE', 'EOGr', 'EMGl', 'EMGr', 'EMGc']], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 1.5, 6, [['ERC', 'M2', 'EMGc'], ['EOGr', 'M2'], ['ERC', 'EOGl', 'M1', 'M2'], ['ERE', 'EMGr', 'M2'], ['ERB', 'EOGr', 'EMGr', 'M1', 'M2'], ['M1', 'M2']], ['M2']], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 2.0, 6, [[], [], ['ERE'], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 2.0, 6, [['ERE', 'EMGl', 'EMGr'], ['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGc'], ['F3', 'F4'], ['ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGr', 'EMGc']], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 2.0, 6, [[], ['M2'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 2.5, 6, [['ERE', 'EMGl'], ['ERE', 'EMGr'], ['ERE', 'EMGc'], ['F3', 'F4'], ['ERE'], ['ERE', 'EMGc']], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 3.0, 6, [['ERE'], ['ERE'], ['ERE'], [], ['ERE'], ['ERE', 'EMGc']], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 4.0, 6, [['ERE'], ['ERE'], ['ERE'], [], [], ['ERE', 'EMGc']], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'mm', 5.0, 6, [['ERE'], [], ['ERE'], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'F4', 'O2', 'EMGc']], ['ELA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc', 'ERA', 'M1', 'F3', 'ELB', 'ERC', 'ELC', 'ELI', 'C4', 'ELT', 'F4']], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 1.0, 1, [['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'ELA', 'O1', 'M2', 'O2', 'ELB', 'ERA', 'ERB', 'ERC', 'ELC', 'ELI', 'F4']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc']], ['ELA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'F4', 'O2', 'EMGc', 'ERA', 'M1', 'ELB', 'ELC', 'ERC', 'ELT', 'ELI', 'C3', 'C4', 'F3']], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 1.0, 1, [['ELA', 'ELT', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O2', 'EMGc']], ['ELA', 'ELT', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'O2', 'EMGc', 'ELB', 'ERA', 'ERC', 'ERE', 'M1', 'O1', 'ELC', 'ELI', 'C3']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 1.5, 1, [['ERE', 'EMGc'], ['ERE'], ['ELB', 'ERB', 'ERE'], ['ELA', 'ERE'], ['ERE'], ['ERE']], ['ERE', 'EMGc', 'ELB', 'ERB', 'ELA']], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 1.5, 1, [['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGc'], ['ERE'], ['ELA', 'ELB', 'ERB', 'ERE'], ['ELA', 'ELB', 'ERE'], ['ERE', 'EMGr'], ['ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGc', 'ELA', 'ELB', 'ERB', 'EMGr']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 1.5, 1, [['EOGl'], ['ERE'], ['EMGl'], ['ELB'], [], []], ['EOGl', 'ERE', 'EMGl', 'ELB']], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 1.5, 1, [['ELA', 'ERB', 'EOGr', 'EMGl', 'M2'], ['ELA', 'ERB', 'EOGr', 'M2'], ['EMGl', 'M2'], ['ELB', 'EMGl'], ['ELB', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EMGl']], ['ELA', 'ERB', 'EOGr', 'EMGl', 'M2', 'ELB', 'EMGc', 'ERA']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 2.0, 1, [['ERE'], ['ERE'], ['ELB', 'ERB', 'ERE'], ['ERE'], ['ERE'], ['ERE', 'EMGl', 'EMGr', 'EMGc']], ['ERE', 'ELB', 'ERB', 'EMGl', 'EMGr', 'EMGc']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 2.5, 1, [['ERE'], ['ERE'], ['ELB', 'ERB', 'ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE', 'ELB', 'ERB']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 3.0, 1, [['ERE'], ['ERE'], [], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 4.0, 1, [['ERE'], [], [], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 5.0, 1, [['ERE'], [], [], [], ['ERE'], ['ERE']], ['ERE']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'F4', 'O2', 'EMGc']], ['ELA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc', 'F3', 'ELB', 'ERC', 'F4']], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 1.0, 2, [['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'ELA', 'ELB', 'ERB']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc']], ['ELA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'F4', 'O2', 'EMGc', 'ELB', 'ELC', 'ERC', 'ELT', 'F3']], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 1.0, 2, [['ELA', 'ELT', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O2', 'EMGc']], ['ELA', 'ELT', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'O2', 'EMGc', 'ELB', 'ERA', 'ERC', 'M1']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 1.5, 2, [['ERE', 'EMGc'], ['ERE'], ['ELB', 'ERB', 'ERE'], ['ELA', 'ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 1.5, 2, [['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGc'], ['ERE'], ['ELA', 'ELB', 'ERB', 'ERE'], ['ELA', 'ELB', 'ERE'], ['ERE', 'EMGr'], ['ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['ERE', 'ELA', 'ELB', 'EMGr']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 1.5, 2, [['EOGl'], ['ERE'], ['EMGl'], ['ELB'], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 1.5, 2, [['ELA', 'ERB', 'EOGr', 'EMGl', 'M2'], ['ELA', 'ERB', 'EOGr', 'M2'], ['EMGl', 'M2'], ['ELB', 'EMGl'], ['ELB', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EMGl']], ['ELA', 'ERB', 'EOGr', 'EMGl', 'M2', 'ELB']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 2.0, 2, [['ERE'], ['ERE'], ['ELB', 'ERB', 'ERE'], ['ERE'], ['ERE'], ['ERE', 'EMGl', 'EMGr', 'EMGc']], ['ERE']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 2.5, 2, [['ERE'], ['ERE'], ['ELB', 'ERB', 'ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 3.0, 2, [['ERE'], ['ERE'], [], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 4.0, 2, [['ERE'], [], [], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 5.0, 2, [['ERE'], [], [], [], ['ERE'], ['ERE']], ['ERE']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'F4', 'O2', 'EMGc']], ['ELA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc', 'ELB', 'ERC']], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 1.0, 3, [['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['ERE', 'EMGl', 'EMGr', 'EMGc', 'ELA']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc']], ['ELA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc', 'ELB', 'ERC']], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 1.0, 3, [['ELA', 'ELT', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O2', 'EMGc']], ['ELA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'O2', 'EMGc', 'ELB', 'ERA', 'ERC']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 1.5, 3, [['ERE', 'EMGc'], ['ERE'], ['ELB', 'ERB', 'ERE'], ['ELA', 'ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 1.5, 3, [['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGc'], ['ERE'], ['ELA', 'ELB', 'ERB', 'ERE'], ['ELA', 'ELB', 'ERE'], ['ERE', 'EMGr'], ['ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['ERE']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 1.5, 3, [['EOGl'], ['ERE'], ['EMGl'], ['ELB'], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 1.5, 3, [['ELA', 'ERB', 'EOGr', 'EMGl', 'M2'], ['ELA', 'ERB', 'EOGr', 'M2'], ['EMGl', 'M2'], ['ELB', 'EMGl'], ['ELB', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EMGl']], ['M2']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 2.0, 3, [['ERE'], ['ERE'], ['ELB', 'ERB', 'ERE'], ['ERE'], ['ERE'], ['ERE', 'EMGl', 'EMGr', 'EMGc']], ['ERE']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 2.5, 3, [['ERE'], ['ERE'], ['ELB', 'ERB', 'ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 3.0, 3, [['ERE'], ['ERE'], [], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 4.0, 3, [['ERE'], [], [], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 5.0, 3, [['ERE'], [], [], [], ['ERE'], ['ERE']], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'F4', 'O2', 'EMGc']], ['ELA', 'ERB', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'EMGc', 'ELB', 'ERC']], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 1.0, 4, [['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['ERE', 'EMGl', 'EMGr', 'EMGc']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc']], ['ELA', 'ERB', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'EMGc', 'ELB', 'ERC']], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 1.0, 4, [['ELA', 'ELT', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O2', 'EMGc']], ['ELA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc', 'ERA', 'ERC']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 1.5, 4, [['ERE', 'EMGc'], ['ERE'], ['ELB', 'ERB', 'ERE'], ['ELA', 'ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 1.5, 4, [['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGc'], ['ERE'], ['ELA', 'ELB', 'ERB', 'ERE'], ['ELA', 'ELB', 'ERE'], ['ERE', 'EMGr'], ['ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['ERE']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 1.5, 4, [['EOGl'], ['ERE'], ['EMGl'], ['ELB'], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 1.5, 4, [['ELA', 'ERB', 'EOGr', 'EMGl', 'M2'], ['ELA', 'ERB', 'EOGr', 'M2'], ['EMGl', 'M2'], ['ELB', 'EMGl'], ['ELB', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EMGl']], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 2.0, 4, [['ERE'], ['ERE'], ['ELB', 'ERB', 'ERE'], ['ERE'], ['ERE'], ['ERE', 'EMGl', 'EMGr', 'EMGc']], ['ERE']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 2.5, 4, [['ERE'], ['ERE'], ['ELB', 'ERB', 'ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 3.0, 4, [['ERE'], ['ERE'], [], ['ERE'], ['ERE'], ['ERE']], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 4.0, 4, [['ERE'], [], [], ['ERE'], ['ERE'], ['ERE']], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 5.0, 4, [['ERE'], [], [], [], ['ERE'], ['ERE']], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'F4', 'O2', 'EMGc']], ['ELA', 'ERE', 'EMGl', 'EMGr', 'EMGc']], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 1.0, 5, [['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['ERE', 'EMGl', 'EMGr', 'EMGc']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc']], ['ELA', 'ERB', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'EMGc']], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 1.0, 5, [['ELA', 'ELT', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O2', 'EMGc']], ['ELA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'EMGc', 'ERA', 'ERC']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 1.5, 5, [['ERE', 'EMGc'], ['ERE'], ['ELB', 'ERB', 'ERE'], ['ELA', 'ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 1.5, 5, [['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGc'], ['ERE'], ['ELA', 'ELB', 'ERB', 'ERE'], ['ELA', 'ELB', 'ERE'], ['ERE', 'EMGr'], ['ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['ERE']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 1.5, 5, [['EOGl'], ['ERE'], ['EMGl'], ['ELB'], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 1.5, 5, [['ELA', 'ERB', 'EOGr', 'EMGl', 'M2'], ['ELA', 'ERB', 'EOGr', 'M2'], ['EMGl', 'M2'], ['ELB', 'EMGl'], ['ELB', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EMGl']], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 2.0, 5, [['ERE'], ['ERE'], ['ELB', 'ERB', 'ERE'], ['ERE'], ['ERE'], ['ERE', 'EMGl', 'EMGr', 'EMGc']], ['ERE']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 2.5, 5, [['ERE'], ['ERE'], ['ELB', 'ERB', 'ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 3.0, 5, [['ERE'], ['ERE'], [], ['ERE'], ['ERE'], ['ERE']], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 4.0, 5, [['ERE'], [], [], ['ERE'], ['ERE'], ['ERE']], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 5.0, 5, [['ERE'], [], [], [], ['ERE'], ['ERE']], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'F4', 'O2', 'EMGc']], ['ELA', 'ERE', 'EMGl', 'EMGc']], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 1.0, 6, [['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EMGl', 'EMGr', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERB', 'ERE', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc'], ['ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['ERE', 'EMGl', 'EMGr', 'EMGc']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc']], ['ELA', 'ERB', 'EOGr', 'EMGl', 'EMGr', 'EMGc']], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 1.0, 6, [['ELA', 'ELT', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O2', 'EMGc']], ['ELA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'EMGc']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 1.5, 6, [['ERE', 'EMGc'], ['ERE'], ['ELB', 'ERB', 'ERE'], ['ELA', 'ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 1.5, 6, [['ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGc'], ['ERE'], ['ELA', 'ELB', 'ERB', 'ERE'], ['ELA', 'ELB', 'ERE'], ['ERE', 'EMGr'], ['ERE', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['ERE']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 1.5, 6, [['EOGl'], ['ERE'], ['EMGl'], ['ELB'], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 1.5, 6, [['ELA', 'ERB', 'EOGr', 'EMGl', 'M2'], ['ELA', 'ERB', 'EOGr', 'M2'], ['EMGl', 'M2'], ['ELB', 'EMGl'], ['ELB', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EMGl']], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 2.0, 6, [['ERE'], ['ERE'], ['ELB', 'ERB', 'ERE'], ['ERE'], ['ERE'], ['ERE', 'EMGl', 'EMGr', 'EMGc']], ['ERE']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 2.5, 6, [['ERE'], ['ERE'], ['ELB', 'ERB', 'ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 3.0, 6, [['ERE'], ['ERE'], [], ['ERE'], ['ERE'], ['ERE']], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 4.0, 6, [['ERE'], [], [], ['ERE'], ['ERE'], ['ERE']], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'mm', 5.0, 6, [['ERE'], [], [], [], ['ERE'], ['ERE']], []], ['S_13/night_1\\EEG_raw', 1000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 1000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ERB', 'ERC', 'ERT ', 'ERE', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'F4', 'O2'], ['ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'F4', 'O2', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELB', 'ELC', 'ERT ', 'ERE', 'ERI', 'EMGl', 'EMGr', 'EMGc', 'M1']], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 1.0, 1, [['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ERB', 'ERC', 'O2'], ['ELB', 'ELC', 'ERB', 'ERC', 'ERE', 'ERI', 'O2'], ['ERA', 'ERB', 'EMGl', 'EMGr', 'F3', 'F4', 'EMGc'], ['ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELB', 'ELC', 'ERB', 'ERC', 'ERE', 'ERI', 'ERA', 'EMGl', 'EMGr', 'EMGc']], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 1.0, 1, [['ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'F3', 'F4', 'O2'], ['ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELA', 'ELB', 'ELC', 'ERC', 'ERT ', 'ERE', 'ERI', 'EMGc', 'ELI', 'EMGl', 'M1', 'M2']], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 1.0, 1, [['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ERA', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELC', 'ERT ', 'ERE', 'EMGl', 'M2', 'EMGc', 'ELI', 'ELB', 'M1']], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 1.5, 1, [[], ['ELB', 'ERC'], ['ELB', 'ELC', 'ERC'], ['ERB'], ['C4'], ['EMGc']], ['ELB', 'ERC', 'ELC', 'ERB', 'C4', 'EMGc']], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 1.5, 1, [['EOGr', 'F3', 'C3', 'O1', 'F4'], ['ELB', 'ELC', 'ERB', 'ERC', 'O2'], ['ELB', 'ELC', 'ERB', 'ERC', 'ERE'], ['ERB', 'EMGl', 'EMGr', 'F3', 'F4', 'EMGc'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['F3', 'O1', 'F4', 'O2', 'EMGc']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'ELB', 'ELC', 'ERB', 'ERC', 'O2', 'ERE', 'EMGl', 'EMGr', 'EMGc', 'C4']], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 1.5, 1, [[], [], ['ELB', 'ELC', 'ERB'], [], [], ['EMGc']], ['ELB', 'ELC', 'ERB', 'EMGc']], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 1.5, 1, [['EOGr'], ['ERT ', 'EMGc'], ['ERA', 'ERB', 'EOGr'], ['M1', 'M2'], ['M2'], ['ERA', 'M2']], ['EOGr', 'ERT ', 'EMGc', 'ERA', 'ERB', 'M1', 'M2']], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 2.0, 1, [[], ['ELB', 'ERC', 'O2'], ['ELB', 'ELC', 'ERC'], ['ERB', 'F4'], ['F3', 'F4', 'C4'], ['F3', 'F4', 'EMGc']], ['ELB', 'ERC', 'O2', 'ELC', 'ERB', 'F4', 'F3', 'C4', 'EMGc']], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 2.5, 1, [[], ['ELB', 'ERC'], ['ELB', 'ELC', 'ERC'], [], [], ['EMGc']], ['ELB', 'ERC', 'ELC', 'EMGc']], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 3.0, 1, [[], ['ERC'], ['ELB', 'ERC'], [], [], []], ['ERC', 'ELB']], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 4.0, 1, [[], ['ERC'], ['ELB'], [], [], []], ['ERC', 'ELB']], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ERB', 'ERC', 'ERT ', 'ERE', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'F4', 'O2'], ['ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'F4', 'O2', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'ERB', 'ERC', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'ELB', 'ELC', 'ERE']], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 1.0, 2, [['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ERB', 'ERC', 'O2'], ['ELB', 'ELC', 'ERB', 'ERC', 'ERE', 'ERI', 'O2'], ['ERA', 'ERB', 'EMGl', 'EMGr', 'F3', 'F4', 'EMGc'], ['ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELB', 'ELC', 'ERB', 'ERC', 'ERA']], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 1.0, 2, [['ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'F3', 'F4', 'O2'], ['ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'ERB', 'EOGr', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'ELA', 'ELB', 'ELC', 'ERC', 'ERE', 'ERI', 'EMGc', 'M1', 'M2']], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 1.0, 2, [['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ERA', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGr', 'F3', 'C3', 'F4', 'O2', 'ERE', 'M2', 'EMGc', 'M1']], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 1.5, 2, [[], ['ELB', 'ERC'], ['ELB', 'ELC', 'ERC'], ['ERB'], ['C4'], ['EMGc']], ['ELB', 'ERC']], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 1.5, 2, [['EOGr', 'F3', 'C3', 'O1', 'F4'], ['ELB', 'ELC', 'ERB', 'ERC', 'O2'], ['ELB', 'ELC', 'ERB', 'ERC', 'ERE'], ['ERB', 'EMGl', 'EMGr', 'F3', 'F4', 'EMGc'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['F3', 'O1', 'F4', 'O2', 'EMGc']], ['F3', 'F4', 'ELB', 'ELC', 'ERB', 'ERC', 'O2']], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 1.5, 2, [[], [], ['ELB', 'ELC', 'ERB'], [], [], ['EMGc']], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 1.5, 2, [['EOGr'], ['ERT ', 'EMGc'], ['ERA', 'ERB', 'EOGr'], ['M1', 'M2'], ['M2'], ['ERA', 'M2']], ['M2']], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 2.0, 2, [[], ['ELB', 'ERC', 'O2'], ['ELB', 'ELC', 'ERC'], ['ERB', 'F4'], ['F3', 'F4', 'C4'], ['F3', 'F4', 'EMGc']], ['ELB', 'ERC', 'F4', 'F3']], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 2.5, 2, [[], ['ELB', 'ERC'], ['ELB', 'ELC', 'ERC'], [], [], ['EMGc']], ['ELB', 'ERC']], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 3.0, 2, [[], ['ERC'], ['ELB', 'ERC'], [], [], []], ['ERC']], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 4.0, 2, [[], ['ERC'], ['ELB'], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ERB', 'ERC', 'ERT ', 'ERE', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'F4', 'O2'], ['ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'F4', 'O2', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'ERB', 'ERC', 'EOGr', 'F3', 'F4', 'O2', 'ELB']], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 1.0, 3, [['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ERB', 'ERC', 'O2'], ['ELB', 'ELC', 'ERB', 'ERC', 'ERE', 'ERI', 'O2'], ['ERA', 'ERB', 'EMGl', 'EMGr', 'F3', 'F4', 'EMGc'], ['ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['F3', 'F4', 'O2', 'ERB']], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 1.0, 3, [['ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'F3', 'F4', 'O2'], ['ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'ERB', 'EOGr', 'EMGr', 'F3', 'F4', 'O2', 'ELB', 'ERC', 'ERE', 'EMGc', 'M2']], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 1.0, 3, [['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ERA', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERB', 'EOGr', 'EMGr', 'F3', 'C3', 'F4', 'ERE', 'M2', 'M1']], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 1.5, 3, [[], ['ELB', 'ERC'], ['ELB', 'ELC', 'ERC'], ['ERB'], ['C4'], ['EMGc']], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 1.5, 3, [['EOGr', 'F3', 'C3', 'O1', 'F4'], ['ELB', 'ELC', 'ERB', 'ERC', 'O2'], ['ELB', 'ELC', 'ERB', 'ERC', 'ERE'], ['ERB', 'EMGl', 'EMGr', 'F3', 'F4', 'EMGc'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['F3', 'O1', 'F4', 'O2', 'EMGc']], ['F3', 'F4', 'ERB']], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 1.5, 3, [[], [], ['ELB', 'ELC', 'ERB'], [], [], ['EMGc']], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 1.5, 3, [['EOGr'], ['ERT ', 'EMGc'], ['ERA', 'ERB', 'EOGr'], ['M1', 'M2'], ['M2'], ['ERA', 'M2']], ['M2']], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 2.0, 3, [[], ['ELB', 'ERC', 'O2'], ['ELB', 'ELC', 'ERC'], ['ERB', 'F4'], ['F3', 'F4', 'C4'], ['F3', 'F4', 'EMGc']], ['F4']], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 2.5, 3, [[], ['ELB', 'ERC'], ['ELB', 'ELC', 'ERC'], [], [], ['EMGc']], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 3.0, 3, [[], ['ERC'], ['ELB', 'ERC'], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 4.0, 3, [[], ['ERC'], ['ELB'], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ERB', 'ERC', 'ERT ', 'ERE', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'F4', 'O2'], ['ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'F4', 'O2', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'ERB', 'ERC', 'EOGr', 'F4', 'O2']], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 1.0, 4, [['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ERB', 'ERC', 'O2'], ['ELB', 'ELC', 'ERB', 'ERC', 'ERE', 'ERI', 'O2'], ['ERA', 'ERB', 'EMGl', 'EMGr', 'F3', 'F4', 'EMGc'], ['ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 1.0, 4, [['ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'F3', 'F4', 'O2'], ['ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'ERB', 'EOGr', 'F3', 'F4', 'O2']], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 1.0, 4, [['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ERA', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'ERB', 'EOGr', 'F3', 'C3']], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 1.5, 4, [[], ['ELB', 'ERC'], ['ELB', 'ELC', 'ERC'], ['ERB'], ['C4'], ['EMGc']], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 1.5, 4, [['EOGr', 'F3', 'C3', 'O1', 'F4'], ['ELB', 'ELC', 'ERB', 'ERC', 'O2'], ['ELB', 'ELC', 'ERB', 'ERC', 'ERE'], ['ERB', 'EMGl', 'EMGr', 'F3', 'F4', 'EMGc'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['F3', 'O1', 'F4', 'O2', 'EMGc']], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 1.5, 4, [[], [], ['ELB', 'ELC', 'ERB'], [], [], ['EMGc']], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 1.5, 4, [['EOGr'], ['ERT ', 'EMGc'], ['ERA', 'ERB', 'EOGr'], ['M1', 'M2'], ['M2'], ['ERA', 'M2']], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 2.0, 4, [[], ['ELB', 'ERC', 'O2'], ['ELB', 'ELC', 'ERC'], ['ERB', 'F4'], ['F3', 'F4', 'C4'], ['F3', 'F4', 'EMGc']], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 2.5, 4, [[], ['ELB', 'ERC'], ['ELB', 'ELC', 'ERC'], [], [], ['EMGc']], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 3.0, 4, [[], ['ERC'], ['ELB', 'ERC'], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 4.0, 4, [[], ['ERC'], ['ELB'], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ERB', 'ERC', 'ERT ', 'ERE', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'F4', 'O2'], ['ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'F4', 'O2', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ERB', 'F4', 'O2']], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 1.0, 5, [['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ERB', 'ERC', 'O2'], ['ELB', 'ELC', 'ERB', 'ERC', 'ERE', 'ERI', 'O2'], ['ERA', 'ERB', 'EMGl', 'EMGr', 'F3', 'F4', 'EMGc'], ['ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 1.0, 5, [['ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'F3', 'F4', 'O2'], ['ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERB', 'F3', 'O2']], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 1.0, 5, [['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ERA', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ERA', 'ERB', 'EOGr', 'C3']], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 1.5, 5, [[], ['ELB', 'ERC'], ['ELB', 'ELC', 'ERC'], ['ERB'], ['C4'], ['EMGc']], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 1.5, 5, [['EOGr', 'F3', 'C3', 'O1', 'F4'], ['ELB', 'ELC', 'ERB', 'ERC', 'O2'], ['ELB', 'ELC', 'ERB', 'ERC', 'ERE'], ['ERB', 'EMGl', 'EMGr', 'F3', 'F4', 'EMGc'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['F3', 'O1', 'F4', 'O2', 'EMGc']], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 1.5, 5, [[], [], ['ELB', 'ELC', 'ERB'], [], [], ['EMGc']], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 1.5, 5, [['EOGr'], ['ERT ', 'EMGc'], ['ERA', 'ERB', 'EOGr'], ['M1', 'M2'], ['M2'], ['ERA', 'M2']], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 2.0, 5, [[], ['ELB', 'ERC', 'O2'], ['ELB', 'ELC', 'ERC'], ['ERB', 'F4'], ['F3', 'F4', 'C4'], ['F3', 'F4', 'EMGc']], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 2.5, 5, [[], ['ELB', 'ERC'], ['ELB', 'ELC', 'ERC'], [], [], ['EMGc']], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 3.0, 5, [[], ['ERC'], ['ELB', 'ERC'], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 4.0, 5, [[], ['ERC'], ['ELB'], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ERB', 'ERC', 'ERT ', 'ERE', 'F4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'F4', 'O2'], ['ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'F4', 'O2', 'EMGc'], ['ERA', 'ERB', 'EOGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['F4', 'O2']], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 1.0, 6, [['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELC', 'ERB', 'ERC', 'O2'], ['ELB', 'ELC', 'ERB', 'ERC', 'ERE', 'ERI', 'O2'], ['ERA', 'ERB', 'EMGl', 'EMGr', 'F3', 'F4', 'EMGc'], ['ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 1.0, 6, [['ERA', 'ERB', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERB', 'ERC', 'ERT ', 'ERE', 'ERI', 'F3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'F3', 'F4', 'O2'], ['ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['F3', 'O2']], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 1.0, 6, [['ELA', 'ERA', 'ERB', 'ERC', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELC', 'ERA', 'ERB', 'ERT ', 'ERE', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'M2', 'F4', 'EMGc'], ['ELA', 'ELI', 'ERA', 'ERB', 'ERE', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELI', 'ERA', 'ERB', 'ERC', 'EOGr', 'EMGr', 'M1', 'C3', 'M2', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ERA', 'ERB', 'EOGr', 'C3']], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 1.5, 6, [[], ['ELB', 'ERC'], ['ELB', 'ELC', 'ERC'], ['ERB'], ['C4'], ['EMGc']], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 1.5, 6, [['EOGr', 'F3', 'C3', 'O1', 'F4'], ['ELB', 'ELC', 'ERB', 'ERC', 'O2'], ['ELB', 'ELC', 'ERB', 'ERC', 'ERE'], ['ERB', 'EMGl', 'EMGr', 'F3', 'F4', 'EMGc'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['F3', 'O1', 'F4', 'O2', 'EMGc']], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 1.5, 6, [[], [], ['ELB', 'ELC', 'ERB'], [], [], ['EMGc']], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 1.5, 6, [['EOGr'], ['ERT ', 'EMGc'], ['ERA', 'ERB', 'EOGr'], ['M1', 'M2'], ['M2'], ['ERA', 'M2']], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 2.0, 6, [[], ['ELB', 'ERC', 'O2'], ['ELB', 'ELC', 'ERC'], ['ERB', 'F4'], ['F3', 'F4', 'C4'], ['F3', 'F4', 'EMGc']], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 2.5, 6, [[], ['ELB', 'ERC'], ['ELB', 'ELC', 'ERC'], [], [], ['EMGc']], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 3.0, 6, [[], ['ERC'], ['ELB', 'ERC'], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 4.0, 6, [[], ['ERC'], ['ELB'], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 6000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 1.0, 1, [['ELB', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELB', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc', 'ERE', 'ERI', 'C3', 'ELA', 'ERA', 'C4']], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 1.0, 1, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['EMGl', 'EMGr', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc', 'ELA', 'ELB', 'ELC', 'ERA', 'C3', 'C4']], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc', 'ELI', 'ERE', 'C4', 'ERC', 'ERI', 'M1']], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERE', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc', 'ELE', 'ELI', 'ERE', 'ERI', 'O1', 'ERC']], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 1.5, 1, [['EMGc'], ['EMGl', 'EMGr', 'EMGc'], [], [], [], []], ['EMGc', 'EMGl', 'EMGr']], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 1.5, 1, [['EMGl', 'EMGr', 'F3', 'O2', 'EMGc'], ['EMGl', 'EMGr', 'EMGc'], ['EMGl', 'EMGr', 'EMGc'], ['ELC', 'O1', 'O2'], ['O1', 'O2'], ['ELB', 'EMGr', 'O1', 'O2']], ['EMGl', 'EMGr', 'F3', 'O2', 'EMGc', 'ELC', 'O1', 'ELB']], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 1.5, 1, [['EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGr'], [], [], [], []], ['EOGl', 'EMGl', 'EMGr', 'EMGc', 'ERE']], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 1.5, 1, [['ELC', 'ERA'], ['ELB', 'ELI', 'C3'], ['ELB', 'ELC', 'EOGl', 'EMGl'], ['ELB', 'ELC', 'EOGl'], ['ELA', 'ELC', 'ERA'], ['ELA', 'ELB']], ['ELC', 'ERA', 'ELB', 'ELI', 'C3', 'EOGl', 'EMGl', 'ELA']], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 2.0, 1, [[], ['EMGl'], [], [], [], []], ['EMGl']], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 2.0, 1, [['EMGl', 'O2', 'EMGc'], ['EMGl', 'EMGr', 'EMGc'], ['EMGl', 'EMGr'], [], [], []], ['EMGl', 'O2', 'EMGc', 'EMGr']], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 2.5, 1, [[], ['EMGl'], [], [], [], []], ['EMGl']], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 2.5, 1, [['EMGc'], ['EMGl', 'EMGr', 'EMGc'], [], [], [], []], ['EMGc', 'EMGl', 'EMGr']], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 3.0, 1, [['EMGc'], ['EMGl', 'EMGr', 'EMGc'], [], [], [], []], ['EMGc', 'EMGl', 'EMGr']], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 4.0, 1, [[], ['EMGl', 'EMGr'], [], [], [], []], ['EMGl', 'EMGr']], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 5.0, 1, [[], ['EMGl'], [], [], [], []], ['EMGl']], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 1.0, 2, [['ELB', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELB', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc', 'C3', 'ELA', 'ERA', 'C4']], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 1.0, 2, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['EMGl', 'EMGr', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc', 'ELA', 'ELB', 'ELC', 'C3', 'C4']], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'O2', 'EMGc', 'C4']], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERE', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc', 'O1']], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 1.5, 2, [['EMGc'], ['EMGl', 'EMGr', 'EMGc'], [], [], [], []], ['EMGc']], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 1.5, 2, [['EMGl', 'EMGr', 'F3', 'O2', 'EMGc'], ['EMGl', 'EMGr', 'EMGc'], ['EMGl', 'EMGr', 'EMGc'], ['ELC', 'O1', 'O2'], ['O1', 'O2'], ['ELB', 'EMGr', 'O1', 'O2']], ['EMGl', 'EMGr', 'O2', 'EMGc', 'O1']], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 1.5, 2, [['EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGr'], [], [], [], []], ['EMGr']], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 1.5, 2, [['ELC', 'ERA'], ['ELB', 'ELI', 'C3'], ['ELB', 'ELC', 'EOGl', 'EMGl'], ['ELB', 'ELC', 'EOGl'], ['ELA', 'ELC', 'ERA'], ['ELA', 'ELB']], ['ELC', 'ELB', 'EOGl', 'ELA']], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 2.0, 2, [[], ['EMGl'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 2.0, 2, [['EMGl', 'O2', 'EMGc'], ['EMGl', 'EMGr', 'EMGc'], ['EMGl', 'EMGr'], [], [], []], ['EMGl', 'EMGc', 'EMGr']], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 2.5, 2, [[], ['EMGl'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 2.5, 2, [['EMGc'], ['EMGl', 'EMGr', 'EMGc'], [], [], [], []], ['EMGc']], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 3.0, 2, [['EMGc'], ['EMGl', 'EMGr', 'EMGc'], [], [], [], []], ['EMGc']], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 4.0, 2, [[], ['EMGl', 'EMGr'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 5.0, 2, [[], ['EMGl'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 1.0, 3, [['ELB', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELB', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc', 'C3', 'ELA', 'ERA', 'C4']], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 1.0, 3, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['EMGl', 'EMGr', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['EOGr', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc', 'ELB', 'ELC', 'C3', 'C4']], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc']], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERE', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'C4', 'O2', 'EMGc', 'O1']], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 1.5, 3, [['EMGc'], ['EMGl', 'EMGr', 'EMGc'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 1.5, 3, [['EMGl', 'EMGr', 'F3', 'O2', 'EMGc'], ['EMGl', 'EMGr', 'EMGc'], ['EMGl', 'EMGr', 'EMGc'], ['ELC', 'O1', 'O2'], ['O1', 'O2'], ['ELB', 'EMGr', 'O1', 'O2']], ['EMGl', 'EMGr', 'O2', 'EMGc', 'O1']], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 1.5, 3, [['EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGr'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 1.5, 3, [['ELC', 'ERA'], ['ELB', 'ELI', 'C3'], ['ELB', 'ELC', 'EOGl', 'EMGl'], ['ELB', 'ELC', 'EOGl'], ['ELA', 'ELC', 'ERA'], ['ELA', 'ELB']], ['ELC', 'ELB']], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 2.0, 3, [[], ['EMGl'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 2.0, 3, [['EMGl', 'O2', 'EMGc'], ['EMGl', 'EMGr', 'EMGc'], ['EMGl', 'EMGr'], [], [], []], ['EMGl']], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 2.5, 3, [[], ['EMGl'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 2.5, 3, [['EMGc'], ['EMGl', 'EMGr', 'EMGc'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 3.0, 3, [['EMGc'], ['EMGl', 'EMGr', 'EMGc'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 4.0, 3, [[], ['EMGl', 'EMGr'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 5.0, 3, [[], ['EMGl'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 1.0, 4, [['ELB', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELB', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc', 'C3', 'ELA', 'ERA', 'C4']], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 1.0, 4, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['EMGl', 'EMGr', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['EOGr', 'EMGl', 'EMGr', 'F3', 'F4', 'O2', 'EMGc', 'ELB', 'ELC']], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc']], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERE', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O2']], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 1.5, 4, [['EMGc'], ['EMGl', 'EMGr', 'EMGc'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 1.5, 4, [['EMGl', 'EMGr', 'F3', 'O2', 'EMGc'], ['EMGl', 'EMGr', 'EMGc'], ['EMGl', 'EMGr', 'EMGc'], ['ELC', 'O1', 'O2'], ['O1', 'O2'], ['ELB', 'EMGr', 'O1', 'O2']], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 1.5, 4, [['EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGr'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 1.5, 4, [['ELC', 'ERA'], ['ELB', 'ELI', 'C3'], ['ELB', 'ELC', 'EOGl', 'EMGl'], ['ELB', 'ELC', 'EOGl'], ['ELA', 'ELC', 'ERA'], ['ELA', 'ELB']], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 2.0, 4, [[], ['EMGl'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 2.0, 4, [['EMGl', 'O2', 'EMGc'], ['EMGl', 'EMGr', 'EMGc'], ['EMGl', 'EMGr'], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 2.5, 4, [[], ['EMGl'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 2.5, 4, [['EMGc'], ['EMGl', 'EMGr', 'EMGc'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 3.0, 4, [['EMGc'], ['EMGl', 'EMGr', 'EMGc'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 4.0, 4, [[], ['EMGl', 'EMGr'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 5.0, 4, [[], ['EMGl'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 1.0, 5, [['ELB', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['EOGr', 'EMGr', 'EMGc']], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 1.0, 5, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['EMGl', 'EMGr', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['EMGl', 'EMGr', 'EMGc']], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc']], ['ELB', 'EOGr', 'EOGl', 'EMGr', 'C3', 'O1', 'EMGc']], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERE', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELC', 'EOGr', 'EOGl', 'O2']], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 1.5, 5, [['EMGc'], ['EMGl', 'EMGr', 'EMGc'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 1.5, 5, [['EMGl', 'EMGr', 'F3', 'O2', 'EMGc'], ['EMGl', 'EMGr', 'EMGc'], ['EMGl', 'EMGr', 'EMGc'], ['ELC', 'O1', 'O2'], ['O1', 'O2'], ['ELB', 'EMGr', 'O1', 'O2']], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 1.5, 5, [['EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGr'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 1.5, 5, [['ELC', 'ERA'], ['ELB', 'ELI', 'C3'], ['ELB', 'ELC', 'EOGl', 'EMGl'], ['ELB', 'ELC', 'EOGl'], ['ELA', 'ELC', 'ERA'], ['ELA', 'ELB']], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 2.0, 5, [[], ['EMGl'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 2.0, 5, [['EMGl', 'O2', 'EMGc'], ['EMGl', 'EMGr', 'EMGc'], ['EMGl', 'EMGr'], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 2.5, 5, [[], ['EMGl'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 2.5, 5, [['EMGc'], ['EMGl', 'EMGr', 'EMGc'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 3.0, 5, [['EMGc'], ['EMGl', 'EMGr', 'EMGc'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 4.0, 5, [[], ['EMGl', 'EMGr'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 5.0, 5, [[], ['EMGl'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 1.0, 6, [['ELB', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['ERE', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['EOGr', 'EMGr', 'EMGc']], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 1.0, 6, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc'], ['EMGl', 'EMGr', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'F4', 'O2', 'EMGc'], ['ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['EMGl', 'EMGr', 'EMGc']], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELB', 'ELI', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'F4', 'O2', 'EMGc']], ['ELB', 'EOGr', 'EOGl', 'EMGr', 'O1', 'EMGc']], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERE', 'ERI', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'ERI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELC', 'EOGr', 'EOGl', 'O2']], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 1.5, 6, [['EMGc'], ['EMGl', 'EMGr', 'EMGc'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 1.5, 6, [['EMGl', 'EMGr', 'F3', 'O2', 'EMGc'], ['EMGl', 'EMGr', 'EMGc'], ['EMGl', 'EMGr', 'EMGc'], ['ELC', 'O1', 'O2'], ['O1', 'O2'], ['ELB', 'EMGr', 'O1', 'O2']], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 1.5, 6, [['EOGl', 'EMGl', 'EMGr', 'EMGc'], ['ERE', 'EMGr'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 1.5, 6, [['ELC', 'ERA'], ['ELB', 'ELI', 'C3'], ['ELB', 'ELC', 'EOGl', 'EMGl'], ['ELB', 'ELC', 'EOGl'], ['ELA', 'ELC', 'ERA'], ['ELA', 'ELB']], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 2.0, 6, [[], ['EMGl'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 2.0, 6, [['EMGl', 'O2', 'EMGc'], ['EMGl', 'EMGr', 'EMGc'], ['EMGl', 'EMGr'], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 2.5, 6, [[], ['EMGl'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 2.5, 6, [['EMGc'], ['EMGl', 'EMGr', 'EMGc'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 3.0, 6, [['EMGc'], ['EMGl', 'EMGr', 'EMGc'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 4.0, 6, [[], ['EMGl', 'EMGr'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'mm', 5.0, 6, [[], ['EMGl'], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 12000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F4', 'O2'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'O2', 'EMGc', 'EMGl', 'O1', 'C4']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 1.0, 1, [['EOGr', 'EOGl', 'F4', 'O2'], ['ELA', 'EOGr', 'EOGl', 'F4', 'O2'], ['ELA', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc'], ['ERE', 'F4', 'O2', 'EMGc'], ['ELA', 'ERE', 'EOGr', 'F4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'F4', 'O2', 'ELA', 'EMGc', 'ERE']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc', 'ELT', 'C3']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'C4', 'EMGc']], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'C4', 'EMGc', 'ERT ', 'EMGr']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 1.5, 1, [['F4', 'O2', 'EMGc'], ['F4', 'O2'], ['EOGl', 'F4', 'O2'], ['ERA', 'ERE', 'F4', 'O2'], ['F4', 'O2'], ['ELA', 'F4', 'O2']], ['F4', 'O2', 'EMGc', 'EOGl', 'ERA', 'ERE', 'ELA']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 1.5, 1, [['F4', 'O2'], ['ELA', 'F4', 'O2'], ['F4', 'O2'], ['ERE', 'F4', 'O2'], ['F4', 'O2'], ['F4', 'O2']], ['F4', 'O2', 'ELA', 'ERE']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 1.5, 1, [['ELA', 'ERE', 'EOGr', 'EOGl', 'EMGc'], ['ERE', 'EOGr', 'EOGl', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGc'], ['ERE', 'EOGr', 'EOGl', 'O2', 'EMGc'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'EMGc'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'O2', 'EMGc']], ['ELA', 'ERE', 'EOGr', 'EOGl', 'EMGc', 'O2']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 1.5, 1, [['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EMGc'], ['ELB', 'ERE', 'EOGr', 'EOGl', 'O1', 'EMGc'], ['ELB', 'ERA', 'ERE', 'EOGr', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'O1', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'EOGl', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EMGc', 'EOGl', 'O1']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 2.0, 1, [['F4', 'O2'], ['F4'], ['F4', 'O2'], ['F4', 'O2'], ['F4'], ['F4']], ['F4', 'O2']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 2.0, 1, [['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2']], ['F4', 'O2']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 2.0, 1, [[], [], [], ['ERE'], [], ['ERE']], ['ERE']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 2.0, 1, [['ERE'], ['ERE'], [], [], ['ERE'], []], ['ERE']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 2.5, 1, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 2.5, 1, [['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2']], ['F4', 'O2']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 3.0, 1, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 3.0, 1, [['F4', 'O2'], ['F4'], ['F4'], ['F4', 'O2'], ['F4'], ['F4', 'O2']], ['F4', 'O2']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], ['F4'], []], ['F4']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 4.0, 1, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 5.0, 1, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F4', 'O2'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc', 'O1']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 1.0, 2, [['EOGr', 'EOGl', 'F4', 'O2'], ['ELA', 'EOGr', 'EOGl', 'F4', 'O2'], ['ELA', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc'], ['ERE', 'F4', 'O2', 'EMGc'], ['ELA', 'ERE', 'EOGr', 'F4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'F4', 'O2', 'ELA', 'EMGc', 'ERE']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc', 'ELT', 'C3']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'C4', 'EMGc']], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'EMGc']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 1.5, 2, [['F4', 'O2', 'EMGc'], ['F4', 'O2'], ['EOGl', 'F4', 'O2'], ['ERA', 'ERE', 'F4', 'O2'], ['F4', 'O2'], ['ELA', 'F4', 'O2']], ['F4', 'O2']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 1.5, 2, [['F4', 'O2'], ['ELA', 'F4', 'O2'], ['F4', 'O2'], ['ERE', 'F4', 'O2'], ['F4', 'O2'], ['F4', 'O2']], ['F4', 'O2']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 1.5, 2, [['ELA', 'ERE', 'EOGr', 'EOGl', 'EMGc'], ['ERE', 'EOGr', 'EOGl', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGc'], ['ERE', 'EOGr', 'EOGl', 'O2', 'EMGc'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'EMGc'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'O2', 'EMGc']], ['ELA', 'ERE', 'EOGr', 'EOGl', 'EMGc']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 1.5, 2, [['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EMGc'], ['ELB', 'ERE', 'EOGr', 'EOGl', 'O1', 'EMGc'], ['ELB', 'ERA', 'ERE', 'EOGr', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'O1', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'EOGl', 'EMGc']], ['ELA', 'ELB', 'ERE', 'EOGr', 'EMGc']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 2.0, 2, [['F4', 'O2'], ['F4'], ['F4', 'O2'], ['F4', 'O2'], ['F4'], ['F4']], ['F4', 'O2']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 2.0, 2, [['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2']], ['F4', 'O2']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 2.0, 2, [[], [], [], ['ERE'], [], ['ERE']], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 2.0, 2, [['ERE'], ['ERE'], [], [], ['ERE'], []], ['ERE']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 2.5, 2, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 2.5, 2, [['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2']], ['F4', 'O2']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 3.0, 2, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 3.0, 2, [['F4', 'O2'], ['F4'], ['F4'], ['F4', 'O2'], ['F4'], ['F4', 'O2']], ['F4']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], ['F4'], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 4.0, 2, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 5.0, 2, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F4', 'O2'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERE', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 1.0, 3, [['EOGr', 'EOGl', 'F4', 'O2'], ['ELA', 'EOGr', 'EOGl', 'F4', 'O2'], ['ELA', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc'], ['ERE', 'F4', 'O2', 'EMGc'], ['ELA', 'ERE', 'EOGr', 'F4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'F4', 'O2', 'EMGc']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'C4', 'O2', 'EMGc', 'ELT']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'C4', 'EMGc']], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'EMGc']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 1.5, 3, [['F4', 'O2', 'EMGc'], ['F4', 'O2'], ['EOGl', 'F4', 'O2'], ['ERA', 'ERE', 'F4', 'O2'], ['F4', 'O2'], ['ELA', 'F4', 'O2']], ['F4', 'O2']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 1.5, 3, [['F4', 'O2'], ['ELA', 'F4', 'O2'], ['F4', 'O2'], ['ERE', 'F4', 'O2'], ['F4', 'O2'], ['F4', 'O2']], ['F4', 'O2']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 1.5, 3, [['ELA', 'ERE', 'EOGr', 'EOGl', 'EMGc'], ['ERE', 'EOGr', 'EOGl', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGc'], ['ERE', 'EOGr', 'EOGl', 'O2', 'EMGc'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'EMGc'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'O2', 'EMGc']], ['ERE', 'EOGr', 'EOGl', 'EMGc']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 1.5, 3, [['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EMGc'], ['ELB', 'ERE', 'EOGr', 'EOGl', 'O1', 'EMGc'], ['ELB', 'ERA', 'ERE', 'EOGr', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'O1', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'EOGl', 'EMGc']], ['ELA', 'ELB', 'ERE', 'EOGr', 'EMGc']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 2.0, 3, [['F4', 'O2'], ['F4'], ['F4', 'O2'], ['F4', 'O2'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 2.0, 3, [['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2']], ['F4', 'O2']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 2.0, 3, [[], [], [], ['ERE'], [], ['ERE']], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 2.0, 3, [['ERE'], ['ERE'], [], [], ['ERE'], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 2.5, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 2.5, 3, [['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2']], ['F4', 'O2']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 3.0, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 3.0, 3, [['F4', 'O2'], ['F4'], ['F4'], ['F4', 'O2'], ['F4'], ['F4', 'O2']], ['F4']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], ['F4'], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 4.0, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 5.0, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F4', 'O2'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ERE', 'EOGr', 'EOGl', 'F4', 'O2']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 1.0, 4, [['EOGr', 'EOGl', 'F4', 'O2'], ['ELA', 'EOGr', 'EOGl', 'F4', 'O2'], ['ELA', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc'], ['ERE', 'F4', 'O2', 'EMGc'], ['ELA', 'ERE', 'EOGr', 'F4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc']], ['F4', 'O2', 'EMGc']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'O1', 'EMGc']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'C4', 'EMGc']], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'EMGc']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 1.5, 4, [['F4', 'O2', 'EMGc'], ['F4', 'O2'], ['EOGl', 'F4', 'O2'], ['ERA', 'ERE', 'F4', 'O2'], ['F4', 'O2'], ['ELA', 'F4', 'O2']], ['F4', 'O2']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 1.5, 4, [['F4', 'O2'], ['ELA', 'F4', 'O2'], ['F4', 'O2'], ['ERE', 'F4', 'O2'], ['F4', 'O2'], ['F4', 'O2']], ['F4', 'O2']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 1.5, 4, [['ELA', 'ERE', 'EOGr', 'EOGl', 'EMGc'], ['ERE', 'EOGr', 'EOGl', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGc'], ['ERE', 'EOGr', 'EOGl', 'O2', 'EMGc'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'EMGc'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'EMGc']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 1.5, 4, [['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EMGc'], ['ELB', 'ERE', 'EOGr', 'EOGl', 'O1', 'EMGc'], ['ELB', 'ERA', 'ERE', 'EOGr', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'O1', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'EOGl', 'EMGc']], ['ELB', 'ERE', 'EOGr', 'EMGc']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 2.0, 4, [['F4', 'O2'], ['F4'], ['F4', 'O2'], ['F4', 'O2'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 2.0, 4, [['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2']], ['F4', 'O2']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 2.0, 4, [[], [], [], ['ERE'], [], ['ERE']], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 2.0, 4, [['ERE'], ['ERE'], [], [], ['ERE'], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 2.5, 4, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 2.5, 4, [['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2']], ['F4', 'O2']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 3.0, 4, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 3.0, 4, [['F4', 'O2'], ['F4'], ['F4'], ['F4', 'O2'], ['F4'], ['F4', 'O2']], ['F4']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], ['F4'], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 4.0, 4, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 5.0, 4, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F4', 'O2'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ERE', 'EOGr', 'EOGl', 'F4', 'O2']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 1.0, 5, [['EOGr', 'EOGl', 'F4', 'O2'], ['ELA', 'EOGr', 'EOGl', 'F4', 'O2'], ['ELA', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc'], ['ERE', 'F4', 'O2', 'EMGc'], ['ELA', 'ERE', 'EOGr', 'F4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc']], ['F4', 'O2']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'O1', 'EMGc']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'C4', 'EMGc']], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'EMGc']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 1.5, 5, [['F4', 'O2', 'EMGc'], ['F4', 'O2'], ['EOGl', 'F4', 'O2'], ['ERA', 'ERE', 'F4', 'O2'], ['F4', 'O2'], ['ELA', 'F4', 'O2']], ['F4', 'O2']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 1.5, 5, [['F4', 'O2'], ['ELA', 'F4', 'O2'], ['F4', 'O2'], ['ERE', 'F4', 'O2'], ['F4', 'O2'], ['F4', 'O2']], ['F4', 'O2']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 1.5, 5, [['ELA', 'ERE', 'EOGr', 'EOGl', 'EMGc'], ['ERE', 'EOGr', 'EOGl', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGc'], ['ERE', 'EOGr', 'EOGl', 'O2', 'EMGc'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'EMGc'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'EMGc']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 1.5, 5, [['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EMGc'], ['ELB', 'ERE', 'EOGr', 'EOGl', 'O1', 'EMGc'], ['ELB', 'ERA', 'ERE', 'EOGr', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'O1', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'EOGl', 'EMGc']], ['ELB', 'ERE', 'EMGc']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 2.0, 5, [['F4', 'O2'], ['F4'], ['F4', 'O2'], ['F4', 'O2'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 2.0, 5, [['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2']], ['F4', 'O2']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 2.0, 5, [[], [], [], ['ERE'], [], ['ERE']], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 2.0, 5, [['ERE'], ['ERE'], [], [], ['ERE'], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 2.5, 5, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 2.5, 5, [['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2']], ['F4', 'O2']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 3.0, 5, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 3.0, 5, [['F4', 'O2'], ['F4'], ['F4'], ['F4', 'O2'], ['F4'], ['F4', 'O2']], ['F4']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], ['F4'], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 4.0, 5, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 5.0, 5, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F4', 'O2'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ERE', 'EOGr', 'F4', 'O2']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 1.0, 6, [['EOGr', 'EOGl', 'F4', 'O2'], ['ELA', 'EOGr', 'EOGl', 'F4', 'O2'], ['ELA', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc'], ['ERE', 'F4', 'O2', 'EMGc'], ['ELA', 'ERE', 'EOGr', 'F4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'F4', 'O2', 'EMGc']], ['F4', 'O2']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'O1', 'EMGc']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'EMGc'], ['ELA', 'ELB', 'ELT', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'C4', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'O1', 'EMGc']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 1.5, 6, [['F4', 'O2', 'EMGc'], ['F4', 'O2'], ['EOGl', 'F4', 'O2'], ['ERA', 'ERE', 'F4', 'O2'], ['F4', 'O2'], ['ELA', 'F4', 'O2']], ['F4', 'O2']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 1.5, 6, [['F4', 'O2'], ['ELA', 'F4', 'O2'], ['F4', 'O2'], ['ERE', 'F4', 'O2'], ['F4', 'O2'], ['F4', 'O2']], ['F4', 'O2']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 1.5, 6, [['ELA', 'ERE', 'EOGr', 'EOGl', 'EMGc'], ['ERE', 'EOGr', 'EOGl', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGc'], ['ERE', 'EOGr', 'EOGl', 'O2', 'EMGc'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'EMGc'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'EMGc']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 1.5, 6, [['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EMGc'], ['ELB', 'ERE', 'EOGr', 'EOGl', 'O1', 'EMGc'], ['ELB', 'ERA', 'ERE', 'EOGr', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'O1', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'EOGl', 'EMGc']], ['ELB', 'ERE', 'EMGc']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 2.0, 6, [['F4', 'O2'], ['F4'], ['F4', 'O2'], ['F4', 'O2'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 2.0, 6, [['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2']], ['F4', 'O2']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 2.0, 6, [[], [], [], ['ERE'], [], ['ERE']], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 2.0, 6, [['ERE'], ['ERE'], [], [], ['ERE'], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 2.5, 6, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 2.5, 6, [['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2'], ['F4', 'O2']], ['F4', 'O2']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 3.0, 6, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 3.0, 6, [['F4', 'O2'], ['F4'], ['F4'], ['F4', 'O2'], ['F4'], ['F4', 'O2']], ['F4']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], ['F4'], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 4.0, 6, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'mm', 5.0, 6, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 1500, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 1500, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ERB', 'EOGl']], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 1.0, 1, [['ELA', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'ERB', 'F4', 'C4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ERE', 'F4', 'O2']], ['ELA', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ERB', 'F3', 'C3', 'ERE']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc', 'ERB', 'EOGl', 'F4', 'EMGl']], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 1.0, 1, [['ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ERA', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc']], ['ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'O1', 'C4', 'O2', 'EMGc', 'EMGl', 'F3', 'ERB', 'EOGl', 'ELT']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 1.5, 1, [['ELA', 'F4', 'O2'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'ERE', 'F4', 'O2']], ['ELA', 'F4', 'O2', 'ERB', 'ERE']], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 1.5, 1, [['ELA', 'F4', 'O2'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F4', 'O2']], ['ELA', 'F4', 'O2', 'ERB']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 1.5, 1, [[], ['ERB', 'O2'], ['ERB', 'O2'], ['ELA', 'ERE'], [], ['ERE', 'O2']], ['ERB', 'O2', 'ELA', 'ERE']], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 1.5, 1, [['ELB', 'ERE', 'EOGr', 'O1', 'EMGc'], ['ELB', 'ERE', 'EOGr', 'O1', 'EMGc'], ['ELB', 'EOGr', 'EMGc'], ['ELB', 'ERE', 'EOGr', 'O1', 'EMGc'], ['ELB', 'ERE', 'EOGr', 'EMGc'], ['ELB', 'EOGr', 'O1', 'EMGc']], ['ELB', 'ERE', 'EOGr', 'O1', 'EMGc']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 2.0, 1, [[], ['ERB', 'F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['ERB', 'F4']], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 2.0, 1, [['ELA', 'F4', 'O2'], ['ELA', 'ERB', 'F4'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F4', 'O2']], ['ELA', 'F4', 'O2', 'ERB']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 2.0, 1, [[], ['ERB'], ['ERB'], [], [], []], ['ERB']], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 2.0, 1, [['EMGc'], [], ['EMGc'], [], ['ERE', 'EMGc'], []], ['EMGc', 'ERE']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 2.5, 1, [['ELA', 'F4'], ['ELA', 'ERB', 'F4'], ['ELA', 'ERB', 'F4'], ['F4'], ['ELA', 'F4'], ['ELA', 'F4']], ['ELA', 'F4', 'ERB']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 3.0, 1, [['ELA', 'F4'], ['ERB', 'F4'], ['ELA', 'F4'], ['F4'], ['ELA', 'F4'], ['ELA', 'F4']], ['ELA', 'F4', 'ERB']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 4.0, 1, [['F4'], ['ERB', 'F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4', 'ERB']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 5.0, 1, [[], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ERB']], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 1.0, 2, [['ELA', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'ERB', 'F4', 'C4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ERE', 'F4', 'O2']], ['ELA', 'F4', 'O2', 'ERB']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc', 'ERB']], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 1.0, 2, [['ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ERA', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc']], ['ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'O1', 'C4', 'O2', 'EMGc', 'EMGl', 'F3', 'EOGl']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 1.5, 2, [['ELA', 'F4', 'O2'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'ERE', 'F4', 'O2']], ['ELA', 'F4', 'O2', 'ERB']], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 1.5, 2, [['ELA', 'F4', 'O2'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F4', 'O2']], ['ELA', 'F4', 'O2', 'ERB']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 1.5, 2, [[], ['ERB', 'O2'], ['ERB', 'O2'], ['ELA', 'ERE'], [], ['ERE', 'O2']], ['ERB', 'O2']], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 1.5, 2, [['ELB', 'ERE', 'EOGr', 'O1', 'EMGc'], ['ELB', 'ERE', 'EOGr', 'O1', 'EMGc'], ['ELB', 'EOGr', 'EMGc'], ['ELB', 'ERE', 'EOGr', 'O1', 'EMGc'], ['ELB', 'ERE', 'EOGr', 'EMGc'], ['ELB', 'EOGr', 'O1', 'EMGc']], ['ELB', 'ERE', 'EOGr', 'O1', 'EMGc']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 2.0, 2, [[], ['ERB', 'F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 2.0, 2, [['ELA', 'F4', 'O2'], ['ELA', 'ERB', 'F4'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F4', 'O2']], ['ELA', 'F4', 'O2', 'ERB']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 2.0, 2, [[], ['ERB'], ['ERB'], [], [], []], ['ERB']], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 2.0, 2, [['EMGc'], [], ['EMGc'], [], ['ERE', 'EMGc'], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 2.5, 2, [['ELA', 'F4'], ['ELA', 'ERB', 'F4'], ['ELA', 'ERB', 'F4'], ['F4'], ['ELA', 'F4'], ['ELA', 'F4']], ['ELA', 'F4', 'ERB']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 3.0, 2, [['ELA', 'F4'], ['ERB', 'F4'], ['ELA', 'F4'], ['F4'], ['ELA', 'F4'], ['ELA', 'F4']], ['ELA', 'F4']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 4.0, 2, [['F4'], ['ERB', 'F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 5.0, 2, [[], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 1.0, 3, [['ELA', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'ERB', 'F4', 'C4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ERE', 'F4', 'O2']], ['ELA', 'F4', 'O2']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 1.0, 3, [['ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ERA', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc']], ['ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'O1', 'C4', 'O2', 'EMGc', 'EMGl', 'F3']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 1.5, 3, [['ELA', 'F4', 'O2'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'ERE', 'F4', 'O2']], ['ELA', 'F4', 'O2']], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 1.5, 3, [['ELA', 'F4', 'O2'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F4', 'O2']], ['ELA', 'F4', 'O2']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 1.5, 3, [[], ['ERB', 'O2'], ['ERB', 'O2'], ['ELA', 'ERE'], [], ['ERE', 'O2']], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 1.5, 3, [['ELB', 'ERE', 'EOGr', 'O1', 'EMGc'], ['ELB', 'ERE', 'EOGr', 'O1', 'EMGc'], ['ELB', 'EOGr', 'EMGc'], ['ELB', 'ERE', 'EOGr', 'O1', 'EMGc'], ['ELB', 'ERE', 'EOGr', 'EMGc'], ['ELB', 'EOGr', 'O1', 'EMGc']], ['ELB', 'EOGr', 'EMGc']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 2.0, 3, [[], ['ERB', 'F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 2.0, 3, [['ELA', 'F4', 'O2'], ['ELA', 'ERB', 'F4'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F4', 'O2']], ['ELA', 'F4', 'O2']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 2.0, 3, [[], ['ERB'], ['ERB'], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 2.0, 3, [['EMGc'], [], ['EMGc'], [], ['ERE', 'EMGc'], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 2.5, 3, [['ELA', 'F4'], ['ELA', 'ERB', 'F4'], ['ELA', 'ERB', 'F4'], ['F4'], ['ELA', 'F4'], ['ELA', 'F4']], ['ELA', 'F4']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 3.0, 3, [['ELA', 'F4'], ['ERB', 'F4'], ['ELA', 'F4'], ['F4'], ['ELA', 'F4'], ['ELA', 'F4']], ['F4']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 4.0, 3, [['F4'], ['ERB', 'F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 5.0, 3, [[], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERE', 'EOGr', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 1.0, 4, [['ELA', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'ERB', 'F4', 'C4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ERE', 'F4', 'O2']], ['ELA', 'F4', 'O2']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERE', 'EOGr', 'F3', 'O1', 'C4', 'O2', 'EMGc']], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 1.0, 4, [['ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ERA', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc']], ['ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'O1', 'C4', 'O2', 'EMGc']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 1.5, 4, [['ELA', 'F4', 'O2'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'ERE', 'F4', 'O2']], ['ELA', 'F4', 'O2']], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 1.5, 4, [['ELA', 'F4', 'O2'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F4', 'O2']], ['ELA', 'F4', 'O2']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 1.5, 4, [[], ['ERB', 'O2'], ['ERB', 'O2'], ['ELA', 'ERE'], [], ['ERE', 'O2']], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 1.5, 4, [['ELB', 'ERE', 'EOGr', 'O1', 'EMGc'], ['ELB', 'ERE', 'EOGr', 'O1', 'EMGc'], ['ELB', 'EOGr', 'EMGc'], ['ELB', 'ERE', 'EOGr', 'O1', 'EMGc'], ['ELB', 'ERE', 'EOGr', 'EMGc'], ['ELB', 'EOGr', 'O1', 'EMGc']], ['ELB', 'EOGr', 'EMGc']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 2.0, 4, [[], ['ERB', 'F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 2.0, 4, [['ELA', 'F4', 'O2'], ['ELA', 'ERB', 'F4'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F4', 'O2']], ['ELA', 'F4', 'O2']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 2.0, 4, [[], ['ERB'], ['ERB'], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 2.0, 4, [['EMGc'], [], ['EMGc'], [], ['ERE', 'EMGc'], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 2.5, 4, [['ELA', 'F4'], ['ELA', 'ERB', 'F4'], ['ELA', 'ERB', 'F4'], ['F4'], ['ELA', 'F4'], ['ELA', 'F4']], ['F4']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 3.0, 4, [['ELA', 'F4'], ['ERB', 'F4'], ['ELA', 'F4'], ['F4'], ['ELA', 'F4'], ['ELA', 'F4']], ['F4']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 4.0, 4, [['F4'], ['ERB', 'F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 5.0, 4, [[], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERE', 'EOGr', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 1.0, 5, [['ELA', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'ERB', 'F4', 'C4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ERE', 'F4', 'O2']], ['ELA', 'F4', 'O2']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERE', 'EOGr', 'F3', 'O1', 'C4', 'O2', 'EMGc']], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 1.0, 5, [['ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ERA', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc']], ['ELB', 'ERA', 'ERE', 'EOGr', 'O1', 'C4', 'O2', 'EMGc']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 1.5, 5, [['ELA', 'F4', 'O2'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'ERE', 'F4', 'O2']], ['ELA', 'F4', 'O2']], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 1.5, 5, [['ELA', 'F4', 'O2'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F4', 'O2']], ['ELA', 'F4', 'O2']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 1.5, 5, [[], ['ERB', 'O2'], ['ERB', 'O2'], ['ELA', 'ERE'], [], ['ERE', 'O2']], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 1.5, 5, [['ELB', 'ERE', 'EOGr', 'O1', 'EMGc'], ['ELB', 'ERE', 'EOGr', 'O1', 'EMGc'], ['ELB', 'EOGr', 'EMGc'], ['ELB', 'ERE', 'EOGr', 'O1', 'EMGc'], ['ELB', 'ERE', 'EOGr', 'EMGc'], ['ELB', 'EOGr', 'O1', 'EMGc']], ['ELB', 'EOGr', 'EMGc']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 2.0, 5, [[], ['ERB', 'F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 2.0, 5, [['ELA', 'F4', 'O2'], ['ELA', 'ERB', 'F4'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F4', 'O2']], ['ELA', 'F4']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 2.0, 5, [[], ['ERB'], ['ERB'], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 2.0, 5, [['EMGc'], [], ['EMGc'], [], ['ERE', 'EMGc'], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 2.5, 5, [['ELA', 'F4'], ['ELA', 'ERB', 'F4'], ['ELA', 'ERB', 'F4'], ['F4'], ['ELA', 'F4'], ['ELA', 'F4']], ['F4']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 3.0, 5, [['ELA', 'F4'], ['ERB', 'F4'], ['ELA', 'F4'], ['F4'], ['ELA', 'F4'], ['ELA', 'F4']], ['F4']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 4.0, 5, [['F4'], ['ERB', 'F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 5.0, 5, [[], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERE', 'EOGr', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 1.0, 6, [['ELA', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'ERB', 'F4', 'C4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F3', 'C3', 'F4', 'O2'], ['ELA', 'ERE', 'F4', 'O2']], ['ELA', 'F4', 'O2']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERE', 'EOGr', 'F3', 'O1', 'C4', 'O2', 'EMGc']], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 1.0, 6, [['ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ELT', 'ERA', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc']], ['ELB', 'ERA', 'ERE', 'EOGr', 'O1', 'C4', 'O2', 'EMGc']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 1.5, 6, [['ELA', 'F4', 'O2'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'ERE', 'F4', 'O2']], ['ELA', 'F4', 'O2']], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 1.5, 6, [['ELA', 'F4', 'O2'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F4', 'O2']], ['ELA', 'F4', 'O2']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 1.5, 6, [[], ['ERB', 'O2'], ['ERB', 'O2'], ['ELA', 'ERE'], [], ['ERE', 'O2']], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 1.5, 6, [['ELB', 'ERE', 'EOGr', 'O1', 'EMGc'], ['ELB', 'ERE', 'EOGr', 'O1', 'EMGc'], ['ELB', 'EOGr', 'EMGc'], ['ELB', 'ERE', 'EOGr', 'O1', 'EMGc'], ['ELB', 'ERE', 'EOGr', 'EMGc'], ['ELB', 'EOGr', 'O1', 'EMGc']], ['ELB', 'EOGr', 'EMGc']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 2.0, 6, [[], ['ERB', 'F4'], ['F4'], ['F4'], ['F4'], ['F4']], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 2.0, 6, [['ELA', 'F4', 'O2'], ['ELA', 'ERB', 'F4'], ['ELA', 'ERB', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F4', 'O2'], ['ELA', 'F4', 'O2']], ['ELA', 'F4']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 2.0, 6, [[], ['ERB'], ['ERB'], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 2.0, 6, [['EMGc'], [], ['EMGc'], [], ['ERE', 'EMGc'], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 2.5, 6, [['ELA', 'F4'], ['ELA', 'ERB', 'F4'], ['ELA', 'ERB', 'F4'], ['F4'], ['ELA', 'F4'], ['ELA', 'F4']], ['F4']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 3.0, 6, [['ELA', 'F4'], ['ERB', 'F4'], ['ELA', 'F4'], ['F4'], ['ELA', 'F4'], ['ELA', 'F4']], ['F4']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 4.0, 6, [['F4'], ['ERB', 'F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'mm', 5.0, 6, [[], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], []], ['S_14/night_1\\EEG_raw', 3000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 3000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 1.0, 1, [['ELB', 'ERA', 'ERC', 'ERI', 'F3', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'ERI', 'F4', 'O2'], ['ELA', 'ELB', 'ERC', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ERA', 'ERE', 'EOGr', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'F3', 'O1', 'F4', 'O2']], ['ELB', 'ERA', 'ERC', 'ERI', 'F3', 'C3', 'O1', 'F4', 'ELA', 'EMGl', 'EMGr', 'O2', 'EMGc', 'ERE', 'C4', 'EOGr', 'ELT', 'ELI', 'ERB']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 1.0, 1, [['ELB', 'ERC', 'F4'], ['ELA', 'ELB', 'EMGl', 'F4', 'O2'], ['ELA', 'ERA', 'F4', 'O2'], ['ELA', 'ELB', 'ERC', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'EOGr', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'F3', 'C3', 'F4', 'C4']], ['ELB', 'ERC', 'F4', 'ELA', 'EMGl', 'O2', 'ERA', 'F3', 'C3', 'C4', 'EOGr']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EMGl', 'EMGr', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'ELI', 'EOGl', 'EMGr', 'O2', 'EMGc', 'ELE', 'C4']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ERB', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERB', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'EMGc', 'ERA', 'ERC', 'EOGl', 'EMGr', 'ELE', 'ELI', 'C3', 'C4', 'O2']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 1.5, 1, [['ERC', 'F4'], ['ELA', 'F4'], ['ELA', 'F4', 'O2'], ['ELB', 'F3', 'F4'], ['ELA', 'F3', 'C3', 'F4', 'C4'], ['F4']], ['ERC', 'F4', 'ELA', 'O2', 'ELB', 'F3', 'C3', 'C4']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 1.5, 1, [['ERC', 'F4'], ['ELA', 'F4'], ['ERA', 'F4', 'O2'], ['ELA', 'ELB', 'F3', 'F4'], ['ELA', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'F4']], ['ERC', 'F4', 'ELA', 'ERA', 'O2', 'ELB', 'F3', 'C3', 'C4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 1.5, 1, [['ERI'], ['ELA', 'ERC'], ['ELA', 'ERA', 'O2'], ['ELA', 'ELB', 'ERC'], ['ELA'], ['ELA', 'ELB']], ['ERI', 'ELA', 'ERC', 'ERA', 'O2', 'ELB']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 1.5, 1, [['ERB', 'ERI', 'M1', 'O1'], ['ELB', 'ERC', 'EMGl', 'EMGr', 'EMGc'], ['ELB', 'ERA', 'ERC'], ['ERA', 'ERB', 'C4', 'EMGc'], ['ELB', 'ERE', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'EMGc']], ['ERB', 'ERI', 'M1', 'O1', 'ELB', 'ERC', 'EMGl', 'EMGr', 'EMGc', 'ERA', 'C4', 'ERE', 'ELA']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 2.0, 1, [['F4'], ['F4'], ['F4', 'O2'], ['ELB', 'F4'], ['F3', 'F4'], ['F4']], ['F4', 'O2', 'ELB', 'F3']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 2.0, 1, [['ERC', 'F4'], ['F4'], ['F4', 'O2'], ['ELB', 'F4'], ['ELA', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'F4']], ['ERC', 'F4', 'O2', 'ELB', 'ELA', 'F3', 'C3', 'C4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], ['ERB']], ['ERB']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 2.5, 1, [['F4'], ['F4'], ['F4', 'O2'], ['ELB', 'F4'], ['F4'], ['F4']], ['F4', 'O2', 'ELB']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 2.5, 1, [['ERC', 'F4'], ['F4'], ['F4', 'O2'], ['ELB', 'F4'], ['F3', 'F4'], ['ELA', 'F4']], ['ERC', 'F4', 'O2', 'ELB', 'F3', 'ELA']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 3.0, 1, [['F4'], [], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 3.0, 1, [['F4'], ['F4'], ['F4', 'O2'], ['F4'], ['F4'], ['F4']], ['F4', 'O2']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 4.0, 1, [['F4'], [], ['F4'], [], [], []], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 4.0, 1, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 5.0, 1, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 1.0, 2, [['ELB', 'ERA', 'ERC', 'ERI', 'F3', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'ERI', 'F4', 'O2'], ['ELA', 'ELB', 'ERC', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ERA', 'ERE', 'EOGr', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'F3', 'O1', 'F4', 'O2']], ['ELB', 'ERC', 'ERI', 'F3', 'C3', 'F4', 'ELA', 'O2', 'ERE', 'C4']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 1.0, 2, [['ELB', 'ERC', 'F4'], ['ELA', 'ELB', 'EMGl', 'F4', 'O2'], ['ELA', 'ERA', 'F4', 'O2'], ['ELA', 'ELB', 'ERC', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'EOGr', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'F3', 'C3', 'F4', 'C4']], ['ELB', 'F4', 'ELA', 'O2', 'F3', 'C3', 'C4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EMGl', 'EMGr', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'ELI', 'EMGr', 'O2', 'EMGc', 'C4']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ERB', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'EMGc', 'ERA', 'ERC', 'C3', 'C4', 'O2']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 1.5, 2, [['ERC', 'F4'], ['ELA', 'F4'], ['ELA', 'F4', 'O2'], ['ELB', 'F3', 'F4'], ['ELA', 'F3', 'C3', 'F4', 'C4'], ['F4']], ['F4', 'ELA', 'F3']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 1.5, 2, [['ERC', 'F4'], ['ELA', 'F4'], ['ERA', 'F4', 'O2'], ['ELA', 'ELB', 'F3', 'F4'], ['ELA', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'F4']], ['F4', 'ELA', 'F3']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 1.5, 2, [['ERI'], ['ELA', 'ERC'], ['ELA', 'ERA', 'O2'], ['ELA', 'ELB', 'ERC'], ['ELA'], ['ELA', 'ELB']], ['ELA']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 1.5, 2, [['ERB', 'ERI', 'M1', 'O1'], ['ELB', 'ERC', 'EMGl', 'EMGr', 'EMGc'], ['ELB', 'ERA', 'ERC'], ['ERA', 'ERB', 'C4', 'EMGc'], ['ELB', 'ERE', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'EMGc']], ['ELB', 'ERC', 'EMGc', 'ERA', 'C4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 2.0, 2, [['F4'], ['F4'], ['F4', 'O2'], ['ELB', 'F4'], ['F3', 'F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 2.0, 2, [['ERC', 'F4'], ['F4'], ['F4', 'O2'], ['ELB', 'F4'], ['ELA', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'F4']], ['F4', 'ELA']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], ['ERB']], []], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 2.5, 2, [['F4'], ['F4'], ['F4', 'O2'], ['ELB', 'F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 2.5, 2, [['ERC', 'F4'], ['F4'], ['F4', 'O2'], ['ELB', 'F4'], ['F3', 'F4'], ['ELA', 'F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 3.0, 2, [['F4'], [], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 3.0, 2, [['F4'], ['F4'], ['F4', 'O2'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 4.0, 2, [['F4'], [], ['F4'], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 4.0, 2, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 5.0, 2, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 1.0, 3, [['ELB', 'ERA', 'ERC', 'ERI', 'F3', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'ERI', 'F4', 'O2'], ['ELA', 'ELB', 'ERC', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ERA', 'ERE', 'EOGr', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'F3', 'O1', 'F4', 'O2']], ['ELB', 'ERI', 'F3', 'F4', 'ELA']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 1.0, 3, [['ELB', 'ERC', 'F4'], ['ELA', 'ELB', 'EMGl', 'F4', 'O2'], ['ELA', 'ERA', 'F4', 'O2'], ['ELA', 'ELB', 'ERC', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'EOGr', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'F3', 'C3', 'F4', 'C4']], ['F4', 'ELA', 'F3', 'C3', 'C4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EMGl', 'EMGr', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'O2', 'EMGc', 'C4']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ERB', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'EMGc', 'ERA', 'ERC', 'C4', 'O2']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 1.5, 3, [['ERC', 'F4'], ['ELA', 'F4'], ['ELA', 'F4', 'O2'], ['ELB', 'F3', 'F4'], ['ELA', 'F3', 'C3', 'F4', 'C4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 1.5, 3, [['ERC', 'F4'], ['ELA', 'F4'], ['ERA', 'F4', 'O2'], ['ELA', 'ELB', 'F3', 'F4'], ['ELA', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'F4']], ['F4', 'ELA']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 1.5, 3, [['ERI'], ['ELA', 'ERC'], ['ELA', 'ERA', 'O2'], ['ELA', 'ELB', 'ERC'], ['ELA'], ['ELA', 'ELB']], ['ELA']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 1.5, 3, [['ERB', 'ERI', 'M1', 'O1'], ['ELB', 'ERC', 'EMGl', 'EMGr', 'EMGc'], ['ELB', 'ERA', 'ERC'], ['ERA', 'ERB', 'C4', 'EMGc'], ['ELB', 'ERE', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'EMGc']], ['EMGc']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 2.0, 3, [['F4'], ['F4'], ['F4', 'O2'], ['ELB', 'F4'], ['F3', 'F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 2.0, 3, [['ERC', 'F4'], ['F4'], ['F4', 'O2'], ['ELB', 'F4'], ['ELA', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], ['ERB']], []], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 2.5, 3, [['F4'], ['F4'], ['F4', 'O2'], ['ELB', 'F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 2.5, 3, [['ERC', 'F4'], ['F4'], ['F4', 'O2'], ['ELB', 'F4'], ['F3', 'F4'], ['ELA', 'F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 3.0, 3, [['F4'], [], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 3.0, 3, [['F4'], ['F4'], ['F4', 'O2'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 4.0, 3, [['F4'], [], ['F4'], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 4.0, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 5.0, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 1.0, 4, [['ELB', 'ERA', 'ERC', 'ERI', 'F3', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'ERI', 'F4', 'O2'], ['ELA', 'ELB', 'ERC', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ERA', 'ERE', 'EOGr', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'F3', 'O1', 'F4', 'O2']], ['ELB', 'F4', 'ELA']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 1.0, 4, [['ELB', 'ERC', 'F4'], ['ELA', 'ELB', 'EMGl', 'F4', 'O2'], ['ELA', 'ERA', 'F4', 'O2'], ['ELA', 'ELB', 'ERC', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'EOGr', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'F3', 'C3', 'F4', 'C4']], ['F4', 'ELA']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EMGl', 'EMGr', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERC', 'ERE', 'C3', 'O2', 'EMGc']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ERB', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERE', 'EOGr', 'EMGl', 'F3', 'ERA', 'ERC']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 1.5, 4, [['ERC', 'F4'], ['ELA', 'F4'], ['ELA', 'F4', 'O2'], ['ELB', 'F3', 'F4'], ['ELA', 'F3', 'C3', 'F4', 'C4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 1.5, 4, [['ERC', 'F4'], ['ELA', 'F4'], ['ERA', 'F4', 'O2'], ['ELA', 'ELB', 'F3', 'F4'], ['ELA', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 1.5, 4, [['ERI'], ['ELA', 'ERC'], ['ELA', 'ERA', 'O2'], ['ELA', 'ELB', 'ERC'], ['ELA'], ['ELA', 'ELB']], ['ELA']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 1.5, 4, [['ERB', 'ERI', 'M1', 'O1'], ['ELB', 'ERC', 'EMGl', 'EMGr', 'EMGc'], ['ELB', 'ERA', 'ERC'], ['ERA', 'ERB', 'C4', 'EMGc'], ['ELB', 'ERE', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'EMGc']], []], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 2.0, 4, [['F4'], ['F4'], ['F4', 'O2'], ['ELB', 'F4'], ['F3', 'F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 2.0, 4, [['ERC', 'F4'], ['F4'], ['F4', 'O2'], ['ELB', 'F4'], ['ELA', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], ['ERB']], []], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 2.5, 4, [['F4'], ['F4'], ['F4', 'O2'], ['ELB', 'F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 2.5, 4, [['ERC', 'F4'], ['F4'], ['F4', 'O2'], ['ELB', 'F4'], ['F3', 'F4'], ['ELA', 'F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 3.0, 4, [['F4'], [], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 3.0, 4, [['F4'], ['F4'], ['F4', 'O2'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 4.0, 4, [['F4'], [], ['F4'], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 4.0, 4, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 5.0, 4, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 1.0, 5, [['ELB', 'ERA', 'ERC', 'ERI', 'F3', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'ERI', 'F4', 'O2'], ['ELA', 'ELB', 'ERC', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ERA', 'ERE', 'EOGr', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'F3', 'O1', 'F4', 'O2']], ['F4', 'ELA']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 1.0, 5, [['ELB', 'ERC', 'F4'], ['ELA', 'ELB', 'EMGl', 'F4', 'O2'], ['ELA', 'ERA', 'F4', 'O2'], ['ELA', 'ELB', 'ERC', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'EOGr', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'F3', 'C3', 'F4', 'C4']], ['F4', 'ELA']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EMGl', 'EMGr', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERC', 'C3', 'O2', 'EMGc']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ERB', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'EOGr', 'EMGl', 'F3', 'ERA']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 1.5, 5, [['ERC', 'F4'], ['ELA', 'F4'], ['ELA', 'F4', 'O2'], ['ELB', 'F3', 'F4'], ['ELA', 'F3', 'C3', 'F4', 'C4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 1.5, 5, [['ERC', 'F4'], ['ELA', 'F4'], ['ERA', 'F4', 'O2'], ['ELA', 'ELB', 'F3', 'F4'], ['ELA', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 1.5, 5, [['ERI'], ['ELA', 'ERC'], ['ELA', 'ERA', 'O2'], ['ELA', 'ELB', 'ERC'], ['ELA'], ['ELA', 'ELB']], ['ELA']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 1.5, 5, [['ERB', 'ERI', 'M1', 'O1'], ['ELB', 'ERC', 'EMGl', 'EMGr', 'EMGc'], ['ELB', 'ERA', 'ERC'], ['ERA', 'ERB', 'C4', 'EMGc'], ['ELB', 'ERE', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'EMGc']], []], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 2.0, 5, [['F4'], ['F4'], ['F4', 'O2'], ['ELB', 'F4'], ['F3', 'F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 2.0, 5, [['ERC', 'F4'], ['F4'], ['F4', 'O2'], ['ELB', 'F4'], ['ELA', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], ['ERB']], []], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 2.5, 5, [['F4'], ['F4'], ['F4', 'O2'], ['ELB', 'F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 2.5, 5, [['ERC', 'F4'], ['F4'], ['F4', 'O2'], ['ELB', 'F4'], ['F3', 'F4'], ['ELA', 'F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 3.0, 5, [['F4'], [], ['F4'], ['F4'], ['F4'], ['F4']], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 3.0, 5, [['F4'], ['F4'], ['F4', 'O2'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 4.0, 5, [['F4'], [], ['F4'], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 4.0, 5, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 5.0, 5, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 1.0, 6, [['ELB', 'ERA', 'ERC', 'ERI', 'F3', 'C3', 'O1', 'F4'], ['ELA', 'ELB', 'ERC', 'ERI', 'EMGl', 'EMGr', 'F3', 'C3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERE', 'ERI', 'F4', 'O2'], ['ELA', 'ELB', 'ERC', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ERA', 'ERE', 'EOGr', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELT', 'ELI', 'ERB', 'ERC', 'ERE', 'F3', 'O1', 'F4', 'O2']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 1.0, 6, [['ELB', 'ERC', 'F4'], ['ELA', 'ELB', 'EMGl', 'F4', 'O2'], ['ELA', 'ERA', 'F4', 'O2'], ['ELA', 'ELB', 'ERC', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'EOGr', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'F3', 'C3', 'F4', 'C4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERC', 'ERI', 'EOGl', 'EMGl', 'EMGr', 'F3', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EMGl', 'EMGr', 'C3', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'C3']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ERB', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'O1', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'EMGc'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERE', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'C3'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'EOGr', 'EMGl', 'F3']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 1.5, 6, [['ERC', 'F4'], ['ELA', 'F4'], ['ELA', 'F4', 'O2'], ['ELB', 'F3', 'F4'], ['ELA', 'F3', 'C3', 'F4', 'C4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 1.5, 6, [['ERC', 'F4'], ['ELA', 'F4'], ['ERA', 'F4', 'O2'], ['ELA', 'ELB', 'F3', 'F4'], ['ELA', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 1.5, 6, [['ERI'], ['ELA', 'ERC'], ['ELA', 'ERA', 'O2'], ['ELA', 'ELB', 'ERC'], ['ELA'], ['ELA', 'ELB']], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 1.5, 6, [['ERB', 'ERI', 'M1', 'O1'], ['ELB', 'ERC', 'EMGl', 'EMGr', 'EMGc'], ['ELB', 'ERA', 'ERC'], ['ERA', 'ERB', 'C4', 'EMGc'], ['ELB', 'ERE', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'EMGc']], []], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 2.0, 6, [['F4'], ['F4'], ['F4', 'O2'], ['ELB', 'F4'], ['F3', 'F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 2.0, 6, [['ERC', 'F4'], ['F4'], ['F4', 'O2'], ['ELB', 'F4'], ['ELA', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], ['ERB']], []], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 2.5, 6, [['F4'], ['F4'], ['F4', 'O2'], ['ELB', 'F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 2.5, 6, [['ERC', 'F4'], ['F4'], ['F4', 'O2'], ['ELB', 'F4'], ['F3', 'F4'], ['ELA', 'F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 3.0, 6, [['F4'], [], ['F4'], ['F4'], ['F4'], ['F4']], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 3.0, 6, [['F4'], ['F4'], ['F4', 'O2'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 4.0, 6, [['F4'], [], ['F4'], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 4.0, 6, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'mm', 5.0, 6, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 4000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 4000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 1.0, 1, [['ELB', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERE', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERB', 'ERE', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ERB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELB', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'ERE', 'ELA', 'ERB', 'EOGl']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 1.0, 1, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'ERB', 'ELT', 'EMGl']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERE', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'ERB', 'ERA', 'EOGl', 'EMGl', 'ELT', 'EMGc']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 1.5, 1, [['F4'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['ERB', 'ERE', 'F4'], ['F3', 'C3', 'F4', 'C4'], ['F4', 'O2'], ['EOGr', 'F3', 'C3', 'F4']], ['F4', 'F3', 'C3', 'C4', 'O2', 'ERB', 'ERE', 'EOGr']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 1.5, 1, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 1.5, 1, [['C4'], ['ERB', 'ERE'], ['ERB', 'ERE', 'C3'], ['ERE', 'F3'], ['O2'], ['ERB', 'F3', 'C4']], ['C4', 'ERB', 'ERE', 'C3', 'F3', 'O2']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 1.5, 1, [['ELB', 'ERE', 'F3', 'M2'], ['ELA', 'ELB', 'ERB', 'M2'], ['ELA', 'ERB', 'C3'], ['ELA', 'ELB', 'ERE', 'EOGr', 'O2'], ['ELA', 'ERB', 'ERE', 'M2'], ['ELB', 'ERB']], ['ELB', 'ERE', 'F3', 'M2', 'ELA', 'ERB', 'C3', 'EOGr', 'O2']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 2.0, 1, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['C3', 'F4']], ['F4', 'C3']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 2.0, 1, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 2.0, 1, [['M2'], ['ELB', 'ERB'], [], [], [], ['ERB']], ['M2', 'ELB', 'ERB']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 2.5, 1, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 2.5, 1, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 3.0, 1, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 3.0, 1, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 4.0, 1, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 4.0, 1, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 5.0, 1, [['F4'], [], ['F4'], [], ['F4'], []], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 5.0, 1, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 1.0, 2, [['ELB', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERE', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERB', 'ERE', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ERB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ERE', 'ELA', 'ERB']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 1.0, 2, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2']], ['ELA', 'ELB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'ERB']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERE', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'ERB', 'EMGl', 'ELT']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 1.5, 2, [['F4'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['ERB', 'ERE', 'F4'], ['F3', 'C3', 'F4', 'C4'], ['F4', 'O2'], ['EOGr', 'F3', 'C3', 'F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 1.5, 2, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 1.5, 2, [['C4'], ['ERB', 'ERE'], ['ERB', 'ERE', 'C3'], ['ERE', 'F3'], ['O2'], ['ERB', 'F3', 'C4']], ['ERB', 'ERE']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 1.5, 2, [['ELB', 'ERE', 'F3', 'M2'], ['ELA', 'ELB', 'ERB', 'M2'], ['ELA', 'ERB', 'C3'], ['ELA', 'ELB', 'ERE', 'EOGr', 'O2'], ['ELA', 'ERB', 'ERE', 'M2'], ['ELB', 'ERB']], ['ELB', 'ERE', 'M2', 'ELA', 'ERB']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 2.0, 2, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['C3', 'F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 2.0, 2, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 2.0, 2, [['M2'], ['ELB', 'ERB'], [], [], [], ['ERB']], []], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 2.5, 2, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 2.5, 2, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 3.0, 2, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 3.0, 2, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 4.0, 2, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 4.0, 2, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 5.0, 2, [['F4'], [], ['F4'], [], ['F4'], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 5.0, 2, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 1.0, 3, [['ELB', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERE', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERB', 'ERE', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ERB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ERE', 'ERB']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 1.0, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2']], ['ELA', 'ELB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'ERB']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERE', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'ERB']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 1.5, 3, [['F4'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['ERB', 'ERE', 'F4'], ['F3', 'C3', 'F4', 'C4'], ['F4', 'O2'], ['EOGr', 'F3', 'C3', 'F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 1.5, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 1.5, 3, [['C4'], ['ERB', 'ERE'], ['ERB', 'ERE', 'C3'], ['ERE', 'F3'], ['O2'], ['ERB', 'F3', 'C4']], ['ERE']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 1.5, 3, [['ELB', 'ERE', 'F3', 'M2'], ['ELA', 'ELB', 'ERB', 'M2'], ['ELA', 'ERB', 'C3'], ['ELA', 'ELB', 'ERE', 'EOGr', 'O2'], ['ELA', 'ERB', 'ERE', 'M2'], ['ELB', 'ERB']], ['ELA']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 2.0, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['C3', 'F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 2.0, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 2.0, 3, [['M2'], ['ELB', 'ERB'], [], [], [], ['ERB']], []], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 2.5, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 2.5, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 3.0, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 3.0, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 4.0, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 4.0, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 5.0, 3, [['F4'], [], ['F4'], [], ['F4'], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 5.0, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 1.0, 4, [['ELB', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERE', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERB', 'ERE', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ERB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ERE', 'ERB']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 1.0, 4, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2']], ['ELA', 'ELB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'ERB']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERE', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'ERB']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 1.5, 4, [['F4'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['ERB', 'ERE', 'F4'], ['F3', 'C3', 'F4', 'C4'], ['F4', 'O2'], ['EOGr', 'F3', 'C3', 'F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 1.5, 4, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 1.5, 4, [['C4'], ['ERB', 'ERE'], ['ERB', 'ERE', 'C3'], ['ERE', 'F3'], ['O2'], ['ERB', 'F3', 'C4']], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 1.5, 4, [['ELB', 'ERE', 'F3', 'M2'], ['ELA', 'ELB', 'ERB', 'M2'], ['ELA', 'ERB', 'C3'], ['ELA', 'ELB', 'ERE', 'EOGr', 'O2'], ['ELA', 'ERB', 'ERE', 'M2'], ['ELB', 'ERB']], ['ELA']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 2.0, 4, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['C3', 'F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 2.0, 4, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 2.0, 4, [['M2'], ['ELB', 'ERB'], [], [], [], ['ERB']], []], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 2.5, 4, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 2.5, 4, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 3.0, 4, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 3.0, 4, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 4.0, 4, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 4.0, 4, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 5.0, 4, [['F4'], [], ['F4'], [], ['F4'], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 5.0, 4, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 1.0, 5, [['ELB', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERE', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERB', 'ERE', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ERB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'O1', 'F4', 'C4', 'O2']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 1.0, 5, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2']], ['ELA', 'ELB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'ERB']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERE', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'ERB']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 1.5, 5, [['F4'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['ERB', 'ERE', 'F4'], ['F3', 'C3', 'F4', 'C4'], ['F4', 'O2'], ['EOGr', 'F3', 'C3', 'F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 1.5, 5, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 1.5, 5, [['C4'], ['ERB', 'ERE'], ['ERB', 'ERE', 'C3'], ['ERE', 'F3'], ['O2'], ['ERB', 'F3', 'C4']], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 1.5, 5, [['ELB', 'ERE', 'F3', 'M2'], ['ELA', 'ELB', 'ERB', 'M2'], ['ELA', 'ERB', 'C3'], ['ELA', 'ELB', 'ERE', 'EOGr', 'O2'], ['ELA', 'ERB', 'ERE', 'M2'], ['ELB', 'ERB']], []], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 2.0, 5, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['C3', 'F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 2.0, 5, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 2.0, 5, [['M2'], ['ELB', 'ERB'], [], [], [], ['ERB']], []], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 2.5, 5, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 2.5, 5, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 3.0, 5, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 3.0, 5, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 4.0, 5, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 4.0, 5, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 5.0, 5, [['F4'], [], ['F4'], [], ['F4'], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 5.0, 5, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 1.0, 6, [['ELB', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERE', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERB', 'ERE', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELB', 'ERB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'O1', 'F4', 'C4', 'O2']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 1.0, 6, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ERA', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'C4', 'O2']], ['ELA', 'ELB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERE', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 1.5, 6, [['F4'], ['F3', 'C3', 'F4', 'C4', 'O2'], ['ERB', 'ERE', 'F4'], ['F3', 'C3', 'F4', 'C4'], ['F4', 'O2'], ['EOGr', 'F3', 'C3', 'F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 1.5, 6, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 1.5, 6, [['C4'], ['ERB', 'ERE'], ['ERB', 'ERE', 'C3'], ['ERE', 'F3'], ['O2'], ['ERB', 'F3', 'C4']], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 1.5, 6, [['ELB', 'ERE', 'F3', 'M2'], ['ELA', 'ELB', 'ERB', 'M2'], ['ELA', 'ERB', 'C3'], ['ELA', 'ELB', 'ERE', 'EOGr', 'O2'], ['ELA', 'ERB', 'ERE', 'M2'], ['ELB', 'ERB']], []], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 2.0, 6, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['C3', 'F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 2.0, 6, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 2.0, 6, [['M2'], ['ELB', 'ERB'], [], [], [], ['ERB']], []], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 2.5, 6, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 2.5, 6, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 3.0, 6, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 3.0, 6, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 4.0, 6, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 4.0, 6, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'mm', 5.0, 6, [['F4'], [], ['F4'], [], ['F4'], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'mm', 5.0, 6, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']], ['S_14/night_1\\EEG_raw', 5500, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 5500, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERT ', 'M1', 'F4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERT ', 'ERI', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELT', 'ERB', 'EOGl', 'ELE', 'ERA', 'ELI', 'ERT ', 'M1', 'ERI', 'M2', 'EMGc']], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERE', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'F4'], ['ELA', 'F4', 'O2'], ['ELA', 'ELB', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERT ', 'C4']], ['ELA', 'ELB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EOGl', 'M1', 'ELE', 'ELI', 'ERT ']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERI', 'F3', 'C3', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'M1', 'C3', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'M1', 'F3', 'C3', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERT ', 'ERI', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ERC', 'EOGl', 'ELE', 'ELI', 'ERI', 'ERT ', 'M1', 'M2', 'EMGc']], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'M1', 'C3', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERI', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERI', 'EMGl', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGl', 'ELE', 'ELI', 'ERA', 'ERI', 'M1', 'ERT ', 'EMGc', 'ELT', 'M2']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 1.5, 1, [['ELA', 'F4'], ['ELA'], ['ELA', 'ELB', 'F4'], ['ELA', 'ELB', 'M1', 'F4', 'O2'], ['ELA', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERT ']], ['ELA', 'F4', 'ELB', 'M1', 'O2', 'C3', 'C4', 'ERT ']], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 1.5, 1, [['ELA', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'F4'], ['F4', 'O2'], ['ELB', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERT ']], ['ELA', 'F3', 'C3', 'F4', 'C4', 'ELB', 'O2', 'M1', 'ELE', 'ELI', 'ERT ']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 1.5, 1, [['ERE'], ['ERE'], ['ELB', 'ELE', 'ERA', 'ERC', 'O2'], ['ELA', 'ELB', 'M1', 'O2'], ['ELB', 'ELT', 'ELE', 'M1', 'C3', 'C4', 'O2'], ['ELI', 'ERT ']], ['ERE', 'ELB', 'ELE', 'ERA', 'ERC', 'O2', 'ELA', 'M1', 'ELT', 'C3', 'C4', 'ELI', 'ERT ']], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 1.5, 1, [['ELB', 'EOGr', 'O1', 'O2'], ['ERE', 'EOGr'], ['ERA', 'EMGl', 'O2'], ['ELE', 'ELI', 'ERT ', 'ERE', 'ERI', 'C3'], ['ELT', 'ELE', 'ELI', 'ERI', 'EMGl'], ['ELE', 'ERT ', 'ERI', 'EMGc']], ['ELB', 'EOGr', 'O1', 'O2', 'ERE', 'ERA', 'EMGl', 'ELE', 'ELI', 'ERT ', 'ERI', 'C3', 'ELT', 'EMGc']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 2.0, 1, [[], [], ['ELA', 'F4'], ['ELA', 'F4', 'O2'], ['F4', 'C4', 'O2'], []], ['ELA', 'F4', 'O2', 'C4']], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 2.0, 1, [['F3', 'F4'], ['ELA'], ['ELA', 'ELB', 'F4'], ['F4'], ['M1', 'F4', 'C4', 'O2'], ['ELA', 'ELI']], ['F3', 'F4', 'ELA', 'ELB', 'M1', 'C4', 'O2', 'ELI']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 2.0, 1, [[], [], [], ['M1'], [], []], ['M1']], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 2.0, 1, [[], ['ERE'], [], ['ERT '], ['ELI', 'ERI'], []], ['ERE', 'ERT ', 'ELI', 'ERI']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 2.5, 1, [[], [], ['F4'], ['F4', 'O2'], [], []], ['F4', 'O2']], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 2.5, 1, [[], [], ['ELA', 'F4'], ['F4'], ['F4', 'C4', 'O2'], ['ELA', 'ELI']], ['ELA', 'F4', 'C4', 'O2', 'ELI']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 3.0, 1, [[], [], ['F4'], ['F4'], [], []], ['F4']], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 3.0, 1, [[], [], ['ELA', 'F4'], ['F4'], ['F4'], ['ELA']], ['ELA', 'F4']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 4.0, 1, [[], [], ['ELA', 'F4'], ['F4'], ['F4'], ['ELA']], ['ELA', 'F4']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 5.0, 1, [[], [], ['F4'], ['F4'], ['F4'], ['ELA']], ['F4', 'ELA']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERT ', 'M1', 'F4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERT ', 'ERI', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELE', 'M1']], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERE', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'F4'], ['ELA', 'F4', 'O2'], ['ELA', 'ELB', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERT ', 'C4']], ['ELA', 'ELB', 'ERE', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERI', 'F3', 'C3', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'M1', 'C3', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'M1', 'F3', 'C3', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERT ', 'ERI', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ERC', 'ELE', 'ELI', 'ERI', 'M1']], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'M1', 'C3', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERI', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERI', 'EMGl', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGl', 'ELE', 'ELI', 'ERA', 'ERI', 'M1', 'ERT ', 'EMGc']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 1.5, 2, [['ELA', 'F4'], ['ELA'], ['ELA', 'ELB', 'F4'], ['ELA', 'ELB', 'M1', 'F4', 'O2'], ['ELA', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERT ']], ['ELA', 'F4', 'ELB', 'M1', 'O2']], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 1.5, 2, [['ELA', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'F4'], ['F4', 'O2'], ['ELB', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERT ']], ['ELA', 'F3', 'C3', 'F4', 'C4', 'ELB', 'O2']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 1.5, 2, [['ERE'], ['ERE'], ['ELB', 'ELE', 'ERA', 'ERC', 'O2'], ['ELA', 'ELB', 'M1', 'O2'], ['ELB', 'ELT', 'ELE', 'M1', 'C3', 'C4', 'O2'], ['ELI', 'ERT ']], ['ERE', 'ELB', 'O2', 'M1']], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 1.5, 2, [['ELB', 'EOGr', 'O1', 'O2'], ['ERE', 'EOGr'], ['ERA', 'EMGl', 'O2'], ['ELE', 'ELI', 'ERT ', 'ERE', 'ERI', 'C3'], ['ELT', 'ELE', 'ELI', 'ERI', 'EMGl'], ['ELE', 'ERT ', 'ERI', 'EMGc']], ['EOGr', 'ELE', 'ELI', 'ERI']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 2.0, 2, [[], [], ['ELA', 'F4'], ['ELA', 'F4', 'O2'], ['F4', 'C4', 'O2'], []], ['ELA', 'F4', 'O2']], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 2.0, 2, [['F3', 'F4'], ['ELA'], ['ELA', 'ELB', 'F4'], ['F4'], ['M1', 'F4', 'C4', 'O2'], ['ELA', 'ELI']], ['F4', 'ELA']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 2.0, 2, [[], [], [], ['M1'], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 2.0, 2, [[], ['ERE'], [], ['ERT '], ['ELI', 'ERI'], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 2.5, 2, [[], [], ['F4'], ['F4', 'O2'], [], []], ['F4']], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 2.5, 2, [[], [], ['ELA', 'F4'], ['F4'], ['F4', 'C4', 'O2'], ['ELA', 'ELI']], ['F4']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 3.0, 2, [[], [], ['F4'], ['F4'], [], []], ['F4']], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 3.0, 2, [[], [], ['ELA', 'F4'], ['F4'], ['F4'], ['ELA']], ['F4']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 4.0, 2, [[], [], ['ELA', 'F4'], ['F4'], ['F4'], ['ELA']], ['F4']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 5.0, 2, [[], [], ['F4'], ['F4'], ['F4'], ['ELA']], ['F4']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERT ', 'M1', 'F4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERT ', 'ERI', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'F3', 'F4', 'C4', 'O2']], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERE', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'F4'], ['ELA', 'F4', 'O2'], ['ELA', 'ELB', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERT ', 'C4']], ['ELA', 'F4']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERI', 'F3', 'C3', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'M1', 'C3', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'M1', 'F3', 'C3', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERT ', 'ERI', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'F3', 'C3', 'O2', 'ERC', 'ELE', 'ELI']], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'M1', 'C3', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERI', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERI', 'EMGl', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELB', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'O2', 'ELE', 'ELI', 'ERI', 'M1', 'ERT ', 'EMGc']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 1.5, 3, [['ELA', 'F4'], ['ELA'], ['ELA', 'ELB', 'F4'], ['ELA', 'ELB', 'M1', 'F4', 'O2'], ['ELA', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERT ']], ['ELA', 'F4']], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 1.5, 3, [['ELA', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'F4'], ['F4', 'O2'], ['ELB', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERT ']], ['ELA', 'F4']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 1.5, 3, [['ERE'], ['ERE'], ['ELB', 'ELE', 'ERA', 'ERC', 'O2'], ['ELA', 'ELB', 'M1', 'O2'], ['ELB', 'ELT', 'ELE', 'M1', 'C3', 'C4', 'O2'], ['ELI', 'ERT ']], ['ELB', 'O2']], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 1.5, 3, [['ELB', 'EOGr', 'O1', 'O2'], ['ERE', 'EOGr'], ['ERA', 'EMGl', 'O2'], ['ELE', 'ELI', 'ERT ', 'ERE', 'ERI', 'C3'], ['ELT', 'ELE', 'ELI', 'ERI', 'EMGl'], ['ELE', 'ERT ', 'ERI', 'EMGc']], ['ELE', 'ERI']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 2.0, 3, [[], [], ['ELA', 'F4'], ['ELA', 'F4', 'O2'], ['F4', 'C4', 'O2'], []], ['F4']], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 2.0, 3, [['F3', 'F4'], ['ELA'], ['ELA', 'ELB', 'F4'], ['F4'], ['M1', 'F4', 'C4', 'O2'], ['ELA', 'ELI']], ['F4']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 2.0, 3, [[], [], [], ['M1'], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 2.0, 3, [[], ['ERE'], [], ['ERT '], ['ELI', 'ERI'], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 2.5, 3, [[], [], ['F4'], ['F4', 'O2'], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 2.5, 3, [[], [], ['ELA', 'F4'], ['F4'], ['F4', 'C4', 'O2'], ['ELA', 'ELI']], ['F4']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 3.0, 3, [[], [], ['F4'], ['F4'], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 3.0, 3, [[], [], ['ELA', 'F4'], ['F4'], ['F4'], ['ELA']], ['F4']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 4.0, 3, [[], [], ['ELA', 'F4'], ['F4'], ['F4'], ['ELA']], ['F4']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 5.0, 3, [[], [], ['F4'], ['F4'], ['F4'], ['ELA']], ['F4']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERT ', 'M1', 'F4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERT ', 'ERI', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'F4', 'O2']], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERE', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'F4'], ['ELA', 'F4', 'O2'], ['ELA', 'ELB', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERT ', 'C4']], ['ELA', 'F4']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERI', 'F3', 'C3', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'M1', 'C3', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'M1', 'F3', 'C3', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERT ', 'ERI', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'C3', 'O2', 'ELE', 'ELI']], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'M1', 'C3', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERI', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERI', 'EMGl', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELB', 'ERC', 'C3', 'ELE', 'ELI', 'ERI']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 1.5, 4, [['ELA', 'F4'], ['ELA'], ['ELA', 'ELB', 'F4'], ['ELA', 'ELB', 'M1', 'F4', 'O2'], ['ELA', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERT ']], ['ELA']], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 1.5, 4, [['ELA', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'F4'], ['F4', 'O2'], ['ELB', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERT ']], ['F4']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 1.5, 4, [['ERE'], ['ERE'], ['ELB', 'ELE', 'ERA', 'ERC', 'O2'], ['ELA', 'ELB', 'M1', 'O2'], ['ELB', 'ELT', 'ELE', 'M1', 'C3', 'C4', 'O2'], ['ELI', 'ERT ']], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 1.5, 4, [['ELB', 'EOGr', 'O1', 'O2'], ['ERE', 'EOGr'], ['ERA', 'EMGl', 'O2'], ['ELE', 'ELI', 'ERT ', 'ERE', 'ERI', 'C3'], ['ELT', 'ELE', 'ELI', 'ERI', 'EMGl'], ['ELE', 'ERT ', 'ERI', 'EMGc']], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 2.0, 4, [[], [], ['ELA', 'F4'], ['ELA', 'F4', 'O2'], ['F4', 'C4', 'O2'], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 2.0, 4, [['F3', 'F4'], ['ELA'], ['ELA', 'ELB', 'F4'], ['F4'], ['M1', 'F4', 'C4', 'O2'], ['ELA', 'ELI']], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 2.0, 4, [[], [], [], ['M1'], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 2.0, 4, [[], ['ERE'], [], ['ERT '], ['ELI', 'ERI'], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 2.5, 4, [[], [], ['F4'], ['F4', 'O2'], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 2.5, 4, [[], [], ['ELA', 'F4'], ['F4'], ['F4', 'C4', 'O2'], ['ELA', 'ELI']], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 3.0, 4, [[], [], ['F4'], ['F4'], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 3.0, 4, [[], [], ['ELA', 'F4'], ['F4'], ['F4'], ['ELA']], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 4.0, 4, [[], [], ['ELA', 'F4'], ['F4'], ['F4'], ['ELA']], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 5.0, 4, [[], [], ['F4'], ['F4'], ['F4'], ['ELA']], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERT ', 'M1', 'F4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERT ', 'ERI', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'F4', 'O2']], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERE', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'F4'], ['ELA', 'F4', 'O2'], ['ELA', 'ELB', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERT ', 'C4']], ['ELA', 'F4']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERI', 'F3', 'C3', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'M1', 'C3', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'M1', 'F3', 'C3', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERT ', 'ERI', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'C3', 'O2']], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'M1', 'C3', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERI', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERI', 'EMGl', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELB', 'C3']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 1.5, 5, [['ELA', 'F4'], ['ELA'], ['ELA', 'ELB', 'F4'], ['ELA', 'ELB', 'M1', 'F4', 'O2'], ['ELA', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERT ']], ['ELA']], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 1.5, 5, [['ELA', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'F4'], ['F4', 'O2'], ['ELB', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERT ']], ['F4']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 1.5, 5, [['ERE'], ['ERE'], ['ELB', 'ELE', 'ERA', 'ERC', 'O2'], ['ELA', 'ELB', 'M1', 'O2'], ['ELB', 'ELT', 'ELE', 'M1', 'C3', 'C4', 'O2'], ['ELI', 'ERT ']], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 1.5, 5, [['ELB', 'EOGr', 'O1', 'O2'], ['ERE', 'EOGr'], ['ERA', 'EMGl', 'O2'], ['ELE', 'ELI', 'ERT ', 'ERE', 'ERI', 'C3'], ['ELT', 'ELE', 'ELI', 'ERI', 'EMGl'], ['ELE', 'ERT ', 'ERI', 'EMGc']], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 2.0, 5, [[], [], ['ELA', 'F4'], ['ELA', 'F4', 'O2'], ['F4', 'C4', 'O2'], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 2.0, 5, [['F3', 'F4'], ['ELA'], ['ELA', 'ELB', 'F4'], ['F4'], ['M1', 'F4', 'C4', 'O2'], ['ELA', 'ELI']], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 2.0, 5, [[], [], [], ['M1'], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 2.0, 5, [[], ['ERE'], [], ['ERT '], ['ELI', 'ERI'], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 2.5, 5, [[], [], ['F4'], ['F4', 'O2'], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 2.5, 5, [[], [], ['ELA', 'F4'], ['F4'], ['F4', 'C4', 'O2'], ['ELA', 'ELI']], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 3.0, 5, [[], [], ['F4'], ['F4'], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 3.0, 5, [[], [], ['ELA', 'F4'], ['F4'], ['F4'], ['ELA']], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 4.0, 5, [[], [], ['ELA', 'F4'], ['F4'], ['F4'], ['ELA']], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 5.0, 5, [[], [], ['F4'], ['F4'], ['F4'], ['ELA']], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELT', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ERA', 'ERC', 'F3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELI', 'ERT ', 'M1', 'F4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERT ', 'ERI', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'F4', 'O2']], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERE', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'F4'], ['ELA', 'F4', 'O2'], ['ELA', 'ELB', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERT ', 'C4']], ['ELA']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELT', 'ERA', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERI', 'F3', 'C3', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERI', 'M1', 'C3', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'M1', 'F3', 'C3', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERT ', 'ERI', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'C3', 'O2']], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ERB', 'ERC', 'ERE', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERI', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'O2'], ['ELB', 'ELE', 'ELI', 'ERA', 'ERC', 'ERT ', 'ERE', 'ERI', 'M1', 'C3', 'EMGc'], ['ELB', 'ELT', 'ELE', 'ELI', 'ERT ', 'ERI', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'EMGc'], ['ELB', 'ELE', 'ELI', 'ERT ', 'ERI', 'EMGl', 'C3', 'M2', 'C4', 'O2', 'EMGc']], ['ELB', 'C3']], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 1.5, 6, [['ELA', 'F4'], ['ELA'], ['ELA', 'ELB', 'F4'], ['ELA', 'ELB', 'M1', 'F4', 'O2'], ['ELA', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERT ']], ['ELA']], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 1.5, 6, [['ELA', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'F4'], ['F4', 'O2'], ['ELB', 'M1', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELE', 'ELI', 'ERT ']], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 1.5, 6, [['ERE'], ['ERE'], ['ELB', 'ELE', 'ERA', 'ERC', 'O2'], ['ELA', 'ELB', 'M1', 'O2'], ['ELB', 'ELT', 'ELE', 'M1', 'C3', 'C4', 'O2'], ['ELI', 'ERT ']], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 1.5, 6, [['ELB', 'EOGr', 'O1', 'O2'], ['ERE', 'EOGr'], ['ERA', 'EMGl', 'O2'], ['ELE', 'ELI', 'ERT ', 'ERE', 'ERI', 'C3'], ['ELT', 'ELE', 'ELI', 'ERI', 'EMGl'], ['ELE', 'ERT ', 'ERI', 'EMGc']], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 2.0, 6, [[], [], ['ELA', 'F4'], ['ELA', 'F4', 'O2'], ['F4', 'C4', 'O2'], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 2.0, 6, [['F3', 'F4'], ['ELA'], ['ELA', 'ELB', 'F4'], ['F4'], ['M1', 'F4', 'C4', 'O2'], ['ELA', 'ELI']], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 2.0, 6, [[], [], [], ['M1'], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 2.0, 6, [[], ['ERE'], [], ['ERT '], ['ELI', 'ERI'], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 2.5, 6, [[], [], ['F4'], ['F4', 'O2'], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 2.5, 6, [[], [], ['ELA', 'F4'], ['F4'], ['F4', 'C4', 'O2'], ['ELA', 'ELI']], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 3.0, 6, [[], [], ['F4'], ['F4'], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 3.0, 6, [[], [], ['ELA', 'F4'], ['F4'], ['F4'], ['ELA']], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 4.0, 6, [[], [], ['ELA', 'F4'], ['F4'], ['F4'], ['ELA']], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'mm', 5.0, 6, [[], [], ['F4'], ['F4'], ['F4'], ['ELA']], []], ['S_14/night_1\\EEG_raw', 7500, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_14/night_1\\EEG_raw', 7500, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 1.0, 1, [['ERA', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERA', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'ELA', 'EOGl', 'M1', 'ERB']], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 1.0, 1, [['ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERE', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'M1']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELE', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELE', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELE', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'M1', 'ERB']], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 1.0, 1, [['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'ELC', 'ELE', 'M1']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 1.5, 1, [['ERE', 'F3', 'C4'], ['ERE'], ['ERE', 'EOGr'], ['ERE'], ['ERE', 'C3', 'O2'], ['ERE', 'F3', 'C4']], ['ERE', 'F3', 'C4', 'EOGr', 'C3', 'O2']], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 1.5, 1, [['ERE', 'F3', 'F4', 'C4'], ['ERE', 'F3', 'C3', 'F4', 'C4'], ['ERE', 'F3', 'F4'], ['ERE', 'F3', 'F4', 'C4'], ['ERE', 'F3', 'C3', 'F4'], ['ERE', 'F3']], ['ERE', 'F3', 'F4', 'C4', 'C3']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 1.5, 1, [['M2'], [], ['EOGr', 'C3'], ['M2', 'F4', 'O2'], ['F3', 'C3', 'C4'], ['EOGr', 'M2']], ['M2', 'EOGr', 'C3', 'F4', 'O2', 'F3', 'C4']], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 1.5, 1, [['O1', 'M2', 'O2', 'EMGc'], ['EOGl', 'O1', 'M2', 'O2', 'EMGc'], ['ERA', 'EOGl', 'C3', 'M2', 'EMGc'], ['EOGr', 'C3', 'M2', 'F4', 'EMGc'], ['C3', 'EMGc'], ['M2', 'O2', 'EMGc']], ['O1', 'M2', 'O2', 'EMGc', 'EOGl', 'ERA', 'C3', 'EOGr', 'F4']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 2.0, 1, [['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 2.0, 1, [['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 2.0, 1, [['EMGc'], ['EMGc'], [], ['EMGc'], ['EMGc'], ['M2', 'EMGc']], ['EMGc', 'M2']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 2.5, 1, [['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 3.0, 1, [['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 4.0, 1, [['ERE'], [], ['ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 5.0, 1, [[], [], [], ['ERE'], [], []], ['ERE']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 1.0, 2, [['ERA', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERA', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'ELA', 'EOGl']], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 1.0, 2, [['ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERE', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELE', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELE', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELE', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 1.0, 2, [['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 1.5, 2, [['ERE', 'F3', 'C4'], ['ERE'], ['ERE', 'EOGr'], ['ERE'], ['ERE', 'C3', 'O2'], ['ERE', 'F3', 'C4']], ['ERE']], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 1.5, 2, [['ERE', 'F3', 'F4', 'C4'], ['ERE', 'F3', 'C3', 'F4', 'C4'], ['ERE', 'F3', 'F4'], ['ERE', 'F3', 'F4', 'C4'], ['ERE', 'F3', 'C3', 'F4'], ['ERE', 'F3']], ['ERE', 'F3', 'F4', 'C4']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 1.5, 2, [['M2'], [], ['EOGr', 'C3'], ['M2', 'F4', 'O2'], ['F3', 'C3', 'C4'], ['EOGr', 'M2']], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 1.5, 2, [['O1', 'M2', 'O2', 'EMGc'], ['EOGl', 'O1', 'M2', 'O2', 'EMGc'], ['ERA', 'EOGl', 'C3', 'M2', 'EMGc'], ['EOGr', 'C3', 'M2', 'F4', 'EMGc'], ['C3', 'EMGc'], ['M2', 'O2', 'EMGc']], ['O1', 'M2', 'O2', 'EMGc', 'EOGl', 'C3']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 2.0, 2, [['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 2.0, 2, [['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 2.0, 2, [['EMGc'], ['EMGc'], [], ['EMGc'], ['EMGc'], ['M2', 'EMGc']], ['EMGc']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 2.5, 2, [['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 3.0, 2, [['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 4.0, 2, [['ERE'], [], ['ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 5.0, 2, [[], [], [], ['ERE'], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 1.0, 3, [['ERA', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERA', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'ELA', 'EOGl']], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 1.0, 3, [['ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERE', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELE', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELE', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 1.0, 3, [['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 1.5, 3, [['ERE', 'F3', 'C4'], ['ERE'], ['ERE', 'EOGr'], ['ERE'], ['ERE', 'C3', 'O2'], ['ERE', 'F3', 'C4']], ['ERE']], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 1.5, 3, [['ERE', 'F3', 'F4', 'C4'], ['ERE', 'F3', 'C3', 'F4', 'C4'], ['ERE', 'F3', 'F4'], ['ERE', 'F3', 'F4', 'C4'], ['ERE', 'F3', 'C3', 'F4'], ['ERE', 'F3']], ['ERE', 'F3', 'F4']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 1.5, 3, [['M2'], [], ['EOGr', 'C3'], ['M2', 'F4', 'O2'], ['F3', 'C3', 'C4'], ['EOGr', 'M2']], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 1.5, 3, [['O1', 'M2', 'O2', 'EMGc'], ['EOGl', 'O1', 'M2', 'O2', 'EMGc'], ['ERA', 'EOGl', 'C3', 'M2', 'EMGc'], ['EOGr', 'C3', 'M2', 'F4', 'EMGc'], ['C3', 'EMGc'], ['M2', 'O2', 'EMGc']], ['M2', 'EMGc', 'C3']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 2.0, 3, [['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 2.0, 3, [['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 2.0, 3, [['EMGc'], ['EMGc'], [], ['EMGc'], ['EMGc'], ['M2', 'EMGc']], ['EMGc']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 2.5, 3, [['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 3.0, 3, [['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 4.0, 3, [['ERE'], [], ['ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 5.0, 3, [[], [], [], ['ERE'], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 1.0, 4, [['ERA', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERA', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EOGl']], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 1.0, 4, [['ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERE', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELE', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELE', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 1.0, 4, [['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 1.5, 4, [['ERE', 'F3', 'C4'], ['ERE'], ['ERE', 'EOGr'], ['ERE'], ['ERE', 'C3', 'O2'], ['ERE', 'F3', 'C4']], ['ERE']], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 1.5, 4, [['ERE', 'F3', 'F4', 'C4'], ['ERE', 'F3', 'C3', 'F4', 'C4'], ['ERE', 'F3', 'F4'], ['ERE', 'F3', 'F4', 'C4'], ['ERE', 'F3', 'C3', 'F4'], ['ERE', 'F3']], ['ERE', 'F3', 'F4']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 1.5, 4, [['M2'], [], ['EOGr', 'C3'], ['M2', 'F4', 'O2'], ['F3', 'C3', 'C4'], ['EOGr', 'M2']], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 1.5, 4, [['O1', 'M2', 'O2', 'EMGc'], ['EOGl', 'O1', 'M2', 'O2', 'EMGc'], ['ERA', 'EOGl', 'C3', 'M2', 'EMGc'], ['EOGr', 'C3', 'M2', 'F4', 'EMGc'], ['C3', 'EMGc'], ['M2', 'O2', 'EMGc']], ['M2', 'EMGc']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 2.0, 4, [['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 2.0, 4, [['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 2.0, 4, [['EMGc'], ['EMGc'], [], ['EMGc'], ['EMGc'], ['M2', 'EMGc']], []], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 2.5, 4, [['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 3.0, 4, [['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 4.0, 4, [['ERE'], [], ['ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 5.0, 4, [[], [], [], ['ERE'], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 1.0, 5, [['ERA', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERA', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 1.0, 5, [['ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERE', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELE', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELE', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 1.0, 5, [['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 1.5, 5, [['ERE', 'F3', 'C4'], ['ERE'], ['ERE', 'EOGr'], ['ERE'], ['ERE', 'C3', 'O2'], ['ERE', 'F3', 'C4']], ['ERE']], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 1.5, 5, [['ERE', 'F3', 'F4', 'C4'], ['ERE', 'F3', 'C3', 'F4', 'C4'], ['ERE', 'F3', 'F4'], ['ERE', 'F3', 'F4', 'C4'], ['ERE', 'F3', 'C3', 'F4'], ['ERE', 'F3']], ['ERE', 'F3', 'F4']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 1.5, 5, [['M2'], [], ['EOGr', 'C3'], ['M2', 'F4', 'O2'], ['F3', 'C3', 'C4'], ['EOGr', 'M2']], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 1.5, 5, [['O1', 'M2', 'O2', 'EMGc'], ['EOGl', 'O1', 'M2', 'O2', 'EMGc'], ['ERA', 'EOGl', 'C3', 'M2', 'EMGc'], ['EOGr', 'C3', 'M2', 'F4', 'EMGc'], ['C3', 'EMGc'], ['M2', 'O2', 'EMGc']], ['EMGc']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 2.0, 5, [['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 2.0, 5, [['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 2.0, 5, [['EMGc'], ['EMGc'], [], ['EMGc'], ['EMGc'], ['M2', 'EMGc']], []], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 2.5, 5, [['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 3.0, 5, [['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 4.0, 5, [['ERE'], [], ['ERE'], ['ERE'], ['ERE'], ['ERE']], []], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 5.0, 5, [[], [], [], ['ERE'], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 1.0, 6, [['ERA', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERA', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 1.0, 6, [['ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERE', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERE', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELE', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELE', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 1.0, 6, [['ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ELE', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 1.5, 6, [['ERE', 'F3', 'C4'], ['ERE'], ['ERE', 'EOGr'], ['ERE'], ['ERE', 'C3', 'O2'], ['ERE', 'F3', 'C4']], ['ERE']], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 1.5, 6, [['ERE', 'F3', 'F4', 'C4'], ['ERE', 'F3', 'C3', 'F4', 'C4'], ['ERE', 'F3', 'F4'], ['ERE', 'F3', 'F4', 'C4'], ['ERE', 'F3', 'C3', 'F4'], ['ERE', 'F3']], ['ERE', 'F3']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 1.5, 6, [['M2'], [], ['EOGr', 'C3'], ['M2', 'F4', 'O2'], ['F3', 'C3', 'C4'], ['EOGr', 'M2']], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 1.5, 6, [['O1', 'M2', 'O2', 'EMGc'], ['EOGl', 'O1', 'M2', 'O2', 'EMGc'], ['ERA', 'EOGl', 'C3', 'M2', 'EMGc'], ['EOGr', 'C3', 'M2', 'F4', 'EMGc'], ['C3', 'EMGc'], ['M2', 'O2', 'EMGc']], ['EMGc']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 2.0, 6, [['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 2.0, 6, [['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 2.0, 6, [['EMGc'], ['EMGc'], [], ['EMGc'], ['EMGc'], ['M2', 'EMGc']], []], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 2.5, 6, [['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 3.0, 6, [['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE'], ['ERE']], ['ERE']], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 4.0, 6, [['ERE'], [], ['ERE'], ['ERE'], ['ERE'], ['ERE']], []], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'mm', 5.0, 6, [[], [], [], ['ERE'], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_15/night_1\\EEG_raw', 5000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 1.0, 1, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc', 'ELB', 'ELA', 'ERA', 'ELC', 'ERE', 'ELT', 'ELE', 'ELI', 'ERC']], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 1.0, 1, [['EOGr', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'O2'], ['EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'M1', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'O2'], ['ELA', 'ELT', 'ELI', 'EOGr', 'EOGl', 'M1', 'O1', 'O2'], ['ELA', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'O2', 'M2', 'EMGc', 'ELA', 'ERE', 'ELT', 'ELI', 'ERC', 'EMGl', 'EMGr']], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 1.0, 1, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc', 'ELB', 'ELC', 'ELA', 'ERE', 'ELT', 'ELI', 'ERC', 'ERT ']], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 1.0, 1, [['ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERT ', 'ERE', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc']], ['ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc', 'ELA', 'ELB', 'ERE', 'ELT', 'ELI', 'ERT ']], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 1.5, 1, [['EOGl', 'O2'], ['O1', 'O2'], ['O1', 'O2'], ['ELA', 'EOGr', 'EOGl', 'O1', 'O2'], ['ELA', 'O1', 'O2'], ['ELA']], ['EOGl', 'O2', 'O1', 'ELA', 'EOGr']], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 1.5, 1, [['EOGr', 'EOGl', 'O1', 'O2'], ['O1', 'O2'], ['EOGr', 'O1', 'O2'], ['ELA', 'EOGr', 'EOGl', 'O1', 'O2'], ['ELA', 'O1', 'O2'], ['ELA', 'EOGl', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'O1', 'O2', 'ELA', 'EMGc']], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 1.5, 1, [['O1', 'O2'], ['EOGr', 'EOGl', 'M1', 'O1'], ['EOGr', 'O1', 'O2'], ['ELA', 'O1', 'O2'], ['ELA', 'O1', 'O2'], ['ELA', 'O2']], ['O1', 'O2', 'EOGr', 'EOGl', 'M1', 'ELA']], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 1.5, 1, [['EMGr', 'M1', 'M2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'M1', 'EMGc'], ['EOGl', 'M2', 'EMGc'], ['ELA', 'ERE', 'M1', 'O1', 'M2'], ['M1', 'O1', 'M2'], ['ELT', 'M1', 'O1', 'O2']], ['EMGr', 'M1', 'M2', 'EMGc', 'EOGr', 'EOGl', 'EMGl', 'ELA', 'ERE', 'O1', 'ELT', 'O2']], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 2.0, 1, [['EOGl', 'O1', 'O2'], ['O1', 'O2'], ['O1', 'O2'], ['EOGr', 'EOGl', 'O2'], ['ELA', 'O2'], ['ELA']], ['EOGl', 'O1', 'O2', 'EOGr', 'ELA']], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 2.5, 1, [['EOGl', 'O2'], ['O1', 'O2'], ['O1', 'O2'], ['EOGr', 'EOGl', 'O2'], ['ELA', 'O2'], ['ELA']], ['EOGl', 'O2', 'O1', 'EOGr', 'ELA']], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 3.0, 1, [['O2'], ['O1', 'O2'], ['O2'], [], [], ['ELA']], ['O2', 'O1', 'ELA']], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 1.0, 2, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'O2', 'EMGc', 'ELB', 'ELA', 'ELT']], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 1.0, 2, [['EOGr', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'O2'], ['EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'M1', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'O2'], ['ELA', 'ELT', 'ELI', 'EOGr', 'EOGl', 'M1', 'O1', 'O2'], ['ELA', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'M1', 'O1', 'F4', 'O2', 'M2', 'ELA']], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 1.0, 2, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc', 'ELB', 'ELC', 'ELA', 'ELT', 'ELI']], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 1.0, 2, [['ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERT ', 'ERE', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc']], ['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc', 'ELA', 'ELB', 'ELT', 'ELI', 'ERT ']], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 1.5, 2, [['EOGl', 'O2'], ['O1', 'O2'], ['O1', 'O2'], ['ELA', 'EOGr', 'EOGl', 'O1', 'O2'], ['ELA', 'O1', 'O2'], ['ELA']], ['O2', 'O1', 'ELA']], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 1.5, 2, [['EOGr', 'EOGl', 'O1', 'O2'], ['O1', 'O2'], ['EOGr', 'O1', 'O2'], ['ELA', 'EOGr', 'EOGl', 'O1', 'O2'], ['ELA', 'O1', 'O2'], ['ELA', 'EOGl', 'O2', 'EMGc']], ['EOGr', 'O1', 'O2', 'ELA']], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 1.5, 2, [['O1', 'O2'], ['EOGr', 'EOGl', 'M1', 'O1'], ['EOGr', 'O1', 'O2'], ['ELA', 'O1', 'O2'], ['ELA', 'O1', 'O2'], ['ELA', 'O2']], ['O1', 'O2', 'EOGr', 'ELA']], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 1.5, 2, [['EMGr', 'M1', 'M2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'M1', 'EMGc'], ['EOGl', 'M2', 'EMGc'], ['ELA', 'ERE', 'M1', 'O1', 'M2'], ['M1', 'O1', 'M2'], ['ELT', 'M1', 'O1', 'O2']], ['M1', 'M2', 'EMGc', 'EOGl', 'O1']], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 2.0, 2, [['EOGl', 'O1', 'O2'], ['O1', 'O2'], ['O1', 'O2'], ['EOGr', 'EOGl', 'O2'], ['ELA', 'O2'], ['ELA']], ['O1', 'O2', 'ELA']], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 2.5, 2, [['EOGl', 'O2'], ['O1', 'O2'], ['O1', 'O2'], ['EOGr', 'EOGl', 'O2'], ['ELA', 'O2'], ['ELA']], ['O2', 'O1', 'ELA']], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 3.0, 2, [['O2'], ['O1', 'O2'], ['O2'], [], [], ['ELA']], ['O2']], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 1.0, 3, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'O2', 'EMGc', 'ELA']], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 1.0, 3, [['EOGr', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'O2'], ['EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'M1', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'O2'], ['ELA', 'ELT', 'ELI', 'EOGr', 'EOGl', 'M1', 'O1', 'O2'], ['ELA', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'M1', 'O1', 'O2', 'ELA']], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 1.0, 3, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'O2', 'EMGc', 'ELC', 'ELA']], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 1.0, 3, [['ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERT ', 'ERE', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc']], ['ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc', 'ELA']], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 1.5, 3, [['EOGl', 'O2'], ['O1', 'O2'], ['O1', 'O2'], ['ELA', 'EOGr', 'EOGl', 'O1', 'O2'], ['ELA', 'O1', 'O2'], ['ELA']], ['O2', 'O1', 'ELA']], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 1.5, 3, [['EOGr', 'EOGl', 'O1', 'O2'], ['O1', 'O2'], ['EOGr', 'O1', 'O2'], ['ELA', 'EOGr', 'EOGl', 'O1', 'O2'], ['ELA', 'O1', 'O2'], ['ELA', 'EOGl', 'O2', 'EMGc']], ['O1', 'O2', 'ELA']], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 1.5, 3, [['O1', 'O2'], ['EOGr', 'EOGl', 'M1', 'O1'], ['EOGr', 'O1', 'O2'], ['ELA', 'O1', 'O2'], ['ELA', 'O1', 'O2'], ['ELA', 'O2']], ['O1', 'O2', 'ELA']], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 1.5, 3, [['EMGr', 'M1', 'M2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'M1', 'EMGc'], ['EOGl', 'M2', 'EMGc'], ['ELA', 'ERE', 'M1', 'O1', 'M2'], ['M1', 'O1', 'M2'], ['ELT', 'M1', 'O1', 'O2']], ['M1', 'M2', 'EMGc', 'O1']], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 2.0, 3, [['EOGl', 'O1', 'O2'], ['O1', 'O2'], ['O1', 'O2'], ['EOGr', 'EOGl', 'O2'], ['ELA', 'O2'], ['ELA']], ['O1', 'O2']], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 2.5, 3, [['EOGl', 'O2'], ['O1', 'O2'], ['O1', 'O2'], ['EOGr', 'EOGl', 'O2'], ['ELA', 'O2'], ['ELA']], ['O2']], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 3.0, 3, [['O2'], ['O1', 'O2'], ['O2'], [], [], ['ELA']], ['O2']], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 1.0, 4, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'M1', 'O1', 'M2', 'F4', 'O2', 'ELA']], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 1.0, 4, [['EOGr', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'O2'], ['EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'M1', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'O2'], ['ELA', 'ELT', 'ELI', 'EOGr', 'EOGl', 'M1', 'O1', 'O2'], ['ELA', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'M1', 'O1', 'O2']], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 1.0, 4, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc', 'ELA']], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 1.0, 4, [['ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERT ', 'ERE', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc', 'ELA']], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 1.5, 4, [['EOGl', 'O2'], ['O1', 'O2'], ['O1', 'O2'], ['ELA', 'EOGr', 'EOGl', 'O1', 'O2'], ['ELA', 'O1', 'O2'], ['ELA']], ['O2', 'O1']], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 1.5, 4, [['EOGr', 'EOGl', 'O1', 'O2'], ['O1', 'O2'], ['EOGr', 'O1', 'O2'], ['ELA', 'EOGr', 'EOGl', 'O1', 'O2'], ['ELA', 'O1', 'O2'], ['ELA', 'EOGl', 'O2', 'EMGc']], ['O1', 'O2']], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 1.5, 4, [['O1', 'O2'], ['EOGr', 'EOGl', 'M1', 'O1'], ['EOGr', 'O1', 'O2'], ['ELA', 'O1', 'O2'], ['ELA', 'O1', 'O2'], ['ELA', 'O2']], ['O1', 'O2']], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 1.5, 4, [['EMGr', 'M1', 'M2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'M1', 'EMGc'], ['EOGl', 'M2', 'EMGc'], ['ELA', 'ERE', 'M1', 'O1', 'M2'], ['M1', 'O1', 'M2'], ['ELT', 'M1', 'O1', 'O2']], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 2.0, 4, [['EOGl', 'O1', 'O2'], ['O1', 'O2'], ['O1', 'O2'], ['EOGr', 'EOGl', 'O2'], ['ELA', 'O2'], ['ELA']], ['O2']], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 2.5, 4, [['EOGl', 'O2'], ['O1', 'O2'], ['O1', 'O2'], ['EOGr', 'EOGl', 'O2'], ['ELA', 'O2'], ['ELA']], ['O2']], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 3.0, 4, [['O2'], ['O1', 'O2'], ['O2'], [], [], ['ELA']], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 1.0, 5, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2']], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 1.0, 5, [['EOGr', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'O2'], ['EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'M1', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'O2'], ['ELA', 'ELT', 'ELI', 'EOGr', 'EOGl', 'M1', 'O1', 'O2'], ['ELA', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'M1', 'O1', 'O2']], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 1.0, 5, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 1.0, 5, [['ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERT ', 'ERE', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc', 'ELA']], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 1.5, 5, [['EOGl', 'O2'], ['O1', 'O2'], ['O1', 'O2'], ['ELA', 'EOGr', 'EOGl', 'O1', 'O2'], ['ELA', 'O1', 'O2'], ['ELA']], ['O2']], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 1.5, 5, [['EOGr', 'EOGl', 'O1', 'O2'], ['O1', 'O2'], ['EOGr', 'O1', 'O2'], ['ELA', 'EOGr', 'EOGl', 'O1', 'O2'], ['ELA', 'O1', 'O2'], ['ELA', 'EOGl', 'O2', 'EMGc']], ['O1', 'O2']], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 1.5, 5, [['O1', 'O2'], ['EOGr', 'EOGl', 'M1', 'O1'], ['EOGr', 'O1', 'O2'], ['ELA', 'O1', 'O2'], ['ELA', 'O1', 'O2'], ['ELA', 'O2']], ['O1']], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 1.5, 5, [['EMGr', 'M1', 'M2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'M1', 'EMGc'], ['EOGl', 'M2', 'EMGc'], ['ELA', 'ERE', 'M1', 'O1', 'M2'], ['M1', 'O1', 'M2'], ['ELT', 'M1', 'O1', 'O2']], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 2.0, 5, [['EOGl', 'O1', 'O2'], ['O1', 'O2'], ['O1', 'O2'], ['EOGr', 'EOGl', 'O2'], ['ELA', 'O2'], ['ELA']], ['O2']], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 2.5, 5, [['EOGl', 'O2'], ['O1', 'O2'], ['O1', 'O2'], ['EOGr', 'EOGl', 'O2'], ['ELA', 'O2'], ['ELA']], ['O2']], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 3.0, 5, [['O2'], ['O1', 'O2'], ['O2'], [], [], ['ELA']], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 1.0, 6, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ELB', 'ELT', 'ELE', 'ELI', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ERC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'M1', 'O1', 'O2']], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 1.0, 6, [['EOGr', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'O2'], ['EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'M1', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'O1', 'F4', 'O2'], ['ELA', 'ELT', 'ELI', 'EOGr', 'EOGl', 'M1', 'O1', 'O2'], ['ELA', 'ERC', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc']], ['EOGl', 'O1', 'O2']], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 1.0, 6, [['EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELB', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O1', 'M2', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'M1', 'O1', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 1.0, 6, [['ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELT', 'ELI', 'ERT ', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELT', 'ELI', 'ERT ', 'ERE', 'EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc']], ['EOGl', 'M1', 'O1', 'M2', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 1.5, 6, [['EOGl', 'O2'], ['O1', 'O2'], ['O1', 'O2'], ['ELA', 'EOGr', 'EOGl', 'O1', 'O2'], ['ELA', 'O1', 'O2'], ['ELA']], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 1.5, 6, [['EOGr', 'EOGl', 'O1', 'O2'], ['O1', 'O2'], ['EOGr', 'O1', 'O2'], ['ELA', 'EOGr', 'EOGl', 'O1', 'O2'], ['ELA', 'O1', 'O2'], ['ELA', 'EOGl', 'O2', 'EMGc']], ['O2']], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 1.5, 6, [['O1', 'O2'], ['EOGr', 'EOGl', 'M1', 'O1'], ['EOGr', 'O1', 'O2'], ['ELA', 'O1', 'O2'], ['ELA', 'O1', 'O2'], ['ELA', 'O2']], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 1.5, 6, [['EMGr', 'M1', 'M2', 'EMGc'], ['EOGr', 'EOGl', 'EMGl', 'M1', 'EMGc'], ['EOGl', 'M2', 'EMGc'], ['ELA', 'ERE', 'M1', 'O1', 'M2'], ['M1', 'O1', 'M2'], ['ELT', 'M1', 'O1', 'O2']], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 2.0, 6, [['EOGl', 'O1', 'O2'], ['O1', 'O2'], ['O1', 'O2'], ['EOGr', 'EOGl', 'O2'], ['ELA', 'O2'], ['ELA']], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 2.5, 6, [['EOGl', 'O2'], ['O1', 'O2'], ['O1', 'O2'], ['EOGr', 'EOGl', 'O2'], ['ELA', 'O2'], ['ELA']], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 3.0, 6, [['O2'], ['O1', 'O2'], ['O2'], [], [], ['ELA']], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 100, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'ERI', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERE', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERE', 'EOGr', 'EOGl', 'F3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'ERI', 'O1', 'M2', 'C4', 'O2', 'ELE', 'EMGl', 'EMGr', 'M1', 'EMGc', 'EOGr', 'EOGl', 'F4', 'F3']], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELC', 'ERC', 'ERE', 'ERI', 'O1', 'C4'], ['ELA', 'ELC', 'ELI'], ['ELA', 'ELC', 'ELE', 'ELI', 'O1', 'C4', 'O2'], ['ELA', 'ELI', 'ERE', 'EOGr', 'EOGl', 'O1', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERE', 'EOGr', 'EOGl'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc']], ['ELA', 'ELC', 'ERC', 'ERE', 'ERI', 'O1', 'C4', 'ELI', 'ELE', 'O2', 'EOGr', 'EOGl', 'EMGc', 'EMGl', 'EMGr', 'F4']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'ERI', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERE', 'EOGl', 'M1', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc']], ['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'ERI', 'M1', 'O1', 'M2', 'C4', 'O2', 'ELB', 'ELE', 'EOGl', 'EMGc', 'EOGr', 'EMGr', 'EMGl']], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 1.0, 1, [['ELC', 'ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'M1', 'M2', 'C4', 'O2'], ['ELB', 'ELE', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2'], ['ELC', 'ERC', 'ERE', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELC', 'ELI', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc']], ['ELC', 'ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'M1', 'M2', 'C4', 'O2', 'ELB', 'ELE', 'ERE', 'O1', 'EMGl', 'EMGr', 'EMGc', 'ELA']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 1.5, 1, [['ELA', 'ERE', 'O1', 'C4'], ['ELA', 'ELC', 'ELI', 'ERE'], ['ELA', 'ELI', 'O1'], ['ELA', 'ELI'], ['ELA', 'ELI', 'ERE', 'EOGr'], []], ['ELA', 'ERE', 'O1', 'C4', 'ELC', 'ELI', 'EOGr']], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 1.5, 1, [['ELA', 'ERE', 'ERI', 'O1', 'C4'], ['ELA', 'ELC', 'ELI'], ['ELA', 'ELI', 'O1', 'O2'], ['ELA', 'O1'], ['ELA', 'ERE', 'EOGr', 'EOGl'], ['ELA', 'EMGl', 'EMGr', 'EMGc']], ['ELA', 'ERE', 'ERI', 'O1', 'C4', 'ELC', 'ELI', 'O2', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 1.5, 1, [['ELA', 'ERI'], ['ELA', 'ERE'], ['ELA', 'O1', 'C4'], ['ELA', 'ELI', 'EMGc'], ['ELA', 'EMGc'], []], ['ELA', 'ERI', 'ERE', 'O1', 'C4', 'ELI', 'EMGc']], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 1.5, 1, [['ERC', 'M2', 'O2'], ['EOGl', 'M1', 'O1', 'M2', 'O2'], ['ELC', 'ERC', 'ERE', 'M2', 'C4'], ['M1', 'EMGc'], ['ELI', 'M1', 'O2', 'EMGc'], ['ELA', 'ELC', 'EMGl']], ['ERC', 'M2', 'O2', 'EOGl', 'M1', 'O1', 'ELC', 'ERE', 'C4', 'EMGc', 'ELI', 'ELA', 'EMGl']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 2.0, 1, [['ELA'], ['ELA', 'ELC'], [], [], [], []], ['ELA', 'ELC']], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 2.0, 1, [['ELA', 'ERE', 'O1', 'C4'], ['ELA', 'ELC'], ['ELA', 'ELI', 'O1'], ['ELA'], ['ELA', 'ERE', 'EOGr', 'EOGl'], ['EMGl', 'EMGr', 'EMGc']], ['ELA', 'ERE', 'O1', 'C4', 'ELC', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'EMGc']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 2.5, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ERE'], ['EMGc']], ['ELA', 'ERE', 'EMGc']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 3.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ERE'], []], ['ELA', 'ERE']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 4.0, 1, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], ['ELA']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 5.0, 1, [['ELA'], ['ELA'], [], ['ELA'], [], []], ['ELA']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'ERI', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERE', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERE', 'EOGr', 'EOGl', 'F3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ELI', 'ERE', 'O1', 'M2', 'O2', 'EMGr', 'M1', 'EMGc', 'EOGr', 'EOGl', 'F4', 'F3']], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELC', 'ERC', 'ERE', 'ERI', 'O1', 'C4'], ['ELA', 'ELC', 'ELI'], ['ELA', 'ELC', 'ELE', 'ELI', 'O1', 'C4', 'O2'], ['ELA', 'ELI', 'ERE', 'EOGr', 'EOGl', 'O1', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERE', 'EOGr', 'EOGl'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc']], ['ELA', 'ELC', 'ERE', 'O1', 'ELI', 'O2', 'EOGr', 'EOGl']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'ERI', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERE', 'EOGl', 'M1', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc']], ['ELA', 'ELC', 'ELI', 'ERE', 'M1', 'O1', 'M2', 'O2', 'EOGl', 'EMGc', 'EOGr', 'EMGl']], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 1.0, 2, [['ELC', 'ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'M1', 'M2', 'C4', 'O2'], ['ELB', 'ELE', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2'], ['ELC', 'ERC', 'ERE', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELC', 'ELI', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc']], ['ELC', 'ELI', 'EOGr', 'EOGl', 'M1', 'M2', 'O2', 'ELB', 'ERE', 'O1', 'EMGl', 'EMGr', 'EMGc']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 1.5, 2, [['ELA', 'ERE', 'O1', 'C4'], ['ELA', 'ELC', 'ELI', 'ERE'], ['ELA', 'ELI', 'O1'], ['ELA', 'ELI'], ['ELA', 'ELI', 'ERE', 'EOGr'], []], ['ELA', 'ERE', 'ELI']], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 1.5, 2, [['ELA', 'ERE', 'ERI', 'O1', 'C4'], ['ELA', 'ELC', 'ELI'], ['ELA', 'ELI', 'O1', 'O2'], ['ELA', 'O1'], ['ELA', 'ERE', 'EOGr', 'EOGl'], ['ELA', 'EMGl', 'EMGr', 'EMGc']], ['ELA', 'O1', 'ELI']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 1.5, 2, [['ELA', 'ERI'], ['ELA', 'ERE'], ['ELA', 'O1', 'C4'], ['ELA', 'ELI', 'EMGc'], ['ELA', 'EMGc'], []], ['ELA', 'EMGc']], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 1.5, 2, [['ERC', 'M2', 'O2'], ['EOGl', 'M1', 'O1', 'M2', 'O2'], ['ELC', 'ERC', 'ERE', 'M2', 'C4'], ['M1', 'EMGc'], ['ELI', 'M1', 'O2', 'EMGc'], ['ELA', 'ELC', 'EMGl']], ['M2', 'O2', 'M1', 'EMGc']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 2.0, 2, [['ELA'], ['ELA', 'ELC'], [], [], [], []], ['ELA']], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 2.0, 2, [['ELA', 'ERE', 'O1', 'C4'], ['ELA', 'ELC'], ['ELA', 'ELI', 'O1'], ['ELA'], ['ELA', 'ERE', 'EOGr', 'EOGl'], ['EMGl', 'EMGr', 'EMGc']], ['ELA']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 2.5, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ERE'], ['EMGc']], ['ELA']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 3.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ERE'], []], ['ELA']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 4.0, 2, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], ['ELA']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 5.0, 2, [['ELA'], ['ELA'], [], ['ELA'], [], []], ['ELA']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'ERI', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERE', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERE', 'EOGr', 'EOGl', 'F3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ELI', 'ERE', 'O1', 'M2', 'O2', 'EMGc', 'EOGr', 'EOGl', 'F4']], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELC', 'ERC', 'ERE', 'ERI', 'O1', 'C4'], ['ELA', 'ELC', 'ELI'], ['ELA', 'ELC', 'ELE', 'ELI', 'O1', 'C4', 'O2'], ['ELA', 'ELI', 'ERE', 'EOGr', 'EOGl', 'O1', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERE', 'EOGr', 'EOGl'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc']], ['ELA', 'ELC', 'ELI', 'EOGr', 'EOGl']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'ERI', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERE', 'EOGl', 'M1', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc']], ['ELA', 'ELC', 'ELI', 'ERE', 'M1', 'O1', 'M2', 'O2', 'EOGl', 'EMGc', 'EOGr']], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 1.0, 3, [['ELC', 'ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'M1', 'M2', 'C4', 'O2'], ['ELB', 'ELE', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2'], ['ELC', 'ERC', 'ERE', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELC', 'ELI', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc']], ['ELC', 'ELI', 'EOGl', 'M1', 'M2', 'O2', 'ERE', 'O1', 'EMGl', 'EMGr', 'EMGc']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 1.5, 3, [['ELA', 'ERE', 'O1', 'C4'], ['ELA', 'ELC', 'ELI', 'ERE'], ['ELA', 'ELI', 'O1'], ['ELA', 'ELI'], ['ELA', 'ELI', 'ERE', 'EOGr'], []], ['ELA', 'ELI']], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 1.5, 3, [['ELA', 'ERE', 'ERI', 'O1', 'C4'], ['ELA', 'ELC', 'ELI'], ['ELA', 'ELI', 'O1', 'O2'], ['ELA', 'O1'], ['ELA', 'ERE', 'EOGr', 'EOGl'], ['ELA', 'EMGl', 'EMGr', 'EMGc']], ['ELA']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 1.5, 3, [['ELA', 'ERI'], ['ELA', 'ERE'], ['ELA', 'O1', 'C4'], ['ELA', 'ELI', 'EMGc'], ['ELA', 'EMGc'], []], ['ELA']], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 1.5, 3, [['ERC', 'M2', 'O2'], ['EOGl', 'M1', 'O1', 'M2', 'O2'], ['ELC', 'ERC', 'ERE', 'M2', 'C4'], ['M1', 'EMGc'], ['ELI', 'M1', 'O2', 'EMGc'], ['ELA', 'ELC', 'EMGl']], ['M2']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 2.0, 3, [['ELA'], ['ELA', 'ELC'], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 2.0, 3, [['ELA', 'ERE', 'O1', 'C4'], ['ELA', 'ELC'], ['ELA', 'ELI', 'O1'], ['ELA'], ['ELA', 'ERE', 'EOGr', 'EOGl'], ['EMGl', 'EMGr', 'EMGc']], ['ELA']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 2.5, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ERE'], ['EMGc']], ['ELA']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 3.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ERE'], []], ['ELA']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 4.0, 3, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], ['ELA']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 5.0, 3, [['ELA'], ['ELA'], [], ['ELA'], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'ERI', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERE', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERE', 'EOGr', 'EOGl', 'F3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ELI', 'O1', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELC', 'ERC', 'ERE', 'ERI', 'O1', 'C4'], ['ELA', 'ELC', 'ELI'], ['ELA', 'ELC', 'ELE', 'ELI', 'O1', 'C4', 'O2'], ['ELA', 'ELI', 'ERE', 'EOGr', 'EOGl', 'O1', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERE', 'EOGr', 'EOGl'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc']], ['ELA', 'ELI']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'ERI', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERE', 'EOGl', 'M1', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc']], ['ELA', 'ELC', 'ELI', 'ERE', 'M1', 'O1', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 1.0, 4, [['ELC', 'ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'M1', 'M2', 'C4', 'O2'], ['ELB', 'ELE', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2'], ['ELC', 'ERC', 'ERE', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELC', 'ELI', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc']], ['ELC', 'M1', 'M2', 'O2']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 1.5, 4, [['ELA', 'ERE', 'O1', 'C4'], ['ELA', 'ELC', 'ELI', 'ERE'], ['ELA', 'ELI', 'O1'], ['ELA', 'ELI'], ['ELA', 'ELI', 'ERE', 'EOGr'], []], ['ELA', 'ELI']], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 1.5, 4, [['ELA', 'ERE', 'ERI', 'O1', 'C4'], ['ELA', 'ELC', 'ELI'], ['ELA', 'ELI', 'O1', 'O2'], ['ELA', 'O1'], ['ELA', 'ERE', 'EOGr', 'EOGl'], ['ELA', 'EMGl', 'EMGr', 'EMGc']], ['ELA']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 1.5, 4, [['ELA', 'ERI'], ['ELA', 'ERE'], ['ELA', 'O1', 'C4'], ['ELA', 'ELI', 'EMGc'], ['ELA', 'EMGc'], []], ['ELA']], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 1.5, 4, [['ERC', 'M2', 'O2'], ['EOGl', 'M1', 'O1', 'M2', 'O2'], ['ELC', 'ERC', 'ERE', 'M2', 'C4'], ['M1', 'EMGc'], ['ELI', 'M1', 'O2', 'EMGc'], ['ELA', 'ELC', 'EMGl']], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 2.0, 4, [['ELA'], ['ELA', 'ELC'], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 2.0, 4, [['ELA', 'ERE', 'O1', 'C4'], ['ELA', 'ELC'], ['ELA', 'ELI', 'O1'], ['ELA'], ['ELA', 'ERE', 'EOGr', 'EOGl'], ['EMGl', 'EMGr', 'EMGc']], ['ELA']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 2.5, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ERE'], ['EMGc']], ['ELA']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 3.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ERE'], []], ['ELA']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 4.0, 4, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], ['ELA']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 5.0, 4, [['ELA'], ['ELA'], [], ['ELA'], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'ERI', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERE', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERE', 'EOGr', 'EOGl', 'F3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc']], ['ELA', 'ELI', 'O2']], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELC', 'ERC', 'ERE', 'ERI', 'O1', 'C4'], ['ELA', 'ELC', 'ELI'], ['ELA', 'ELC', 'ELE', 'ELI', 'O1', 'C4', 'O2'], ['ELA', 'ELI', 'ERE', 'EOGr', 'EOGl', 'O1', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERE', 'EOGr', 'EOGl'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc']], ['ELA']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'ERI', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERE', 'EOGl', 'M1', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc']], ['ELA', 'ELC', 'ELI', 'ERE', 'M1', 'O2']], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 1.0, 5, [['ELC', 'ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'M1', 'M2', 'C4', 'O2'], ['ELB', 'ELE', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2'], ['ELC', 'ERC', 'ERE', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELC', 'ELI', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc']], ['M1', 'O2']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 1.5, 5, [['ELA', 'ERE', 'O1', 'C4'], ['ELA', 'ELC', 'ELI', 'ERE'], ['ELA', 'ELI', 'O1'], ['ELA', 'ELI'], ['ELA', 'ELI', 'ERE', 'EOGr'], []], ['ELA']], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 1.5, 5, [['ELA', 'ERE', 'ERI', 'O1', 'C4'], ['ELA', 'ELC', 'ELI'], ['ELA', 'ELI', 'O1', 'O2'], ['ELA', 'O1'], ['ELA', 'ERE', 'EOGr', 'EOGl'], ['ELA', 'EMGl', 'EMGr', 'EMGc']], ['ELA']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 1.5, 5, [['ELA', 'ERI'], ['ELA', 'ERE'], ['ELA', 'O1', 'C4'], ['ELA', 'ELI', 'EMGc'], ['ELA', 'EMGc'], []], ['ELA']], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 1.5, 5, [['ERC', 'M2', 'O2'], ['EOGl', 'M1', 'O1', 'M2', 'O2'], ['ELC', 'ERC', 'ERE', 'M2', 'C4'], ['M1', 'EMGc'], ['ELI', 'M1', 'O2', 'EMGc'], ['ELA', 'ELC', 'EMGl']], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 2.0, 5, [['ELA'], ['ELA', 'ELC'], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 2.0, 5, [['ELA', 'ERE', 'O1', 'C4'], ['ELA', 'ELC'], ['ELA', 'ELI', 'O1'], ['ELA'], ['ELA', 'ERE', 'EOGr', 'EOGl'], ['EMGl', 'EMGr', 'EMGc']], ['ELA']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 2.5, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ERE'], ['EMGc']], ['ELA']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 3.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ERE'], []], ['ELA']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 4.0, 5, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], ['ELA']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 5.0, 5, [['ELA'], ['ELA'], [], ['ELA'], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'ERI', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELC', 'ELE', 'ELI', 'ERE', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERE', 'EOGr', 'EOGl', 'F3', 'F4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F3', 'O1', 'F4', 'O2', 'EMGc']], ['ELA', 'O2']], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELC', 'ERC', 'ERE', 'ERI', 'O1', 'C4'], ['ELA', 'ELC', 'ELI'], ['ELA', 'ELC', 'ELE', 'ELI', 'O1', 'C4', 'O2'], ['ELA', 'ELI', 'ERE', 'EOGr', 'EOGl', 'O1', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERE', 'EOGr', 'EOGl'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'F4', 'EMGc']], ['ELA']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'ERI', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ELE', 'ELI', 'ERE', 'EOGl', 'M1', 'O1', 'M2', 'O2'], ['ELA', 'ELC', 'ELI', 'ERC', 'ERE', 'M1', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERE', 'EOGr', 'EOGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'M1', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERE', 'O2']], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 1.0, 6, [['ELC', 'ELI', 'ERC', 'ERI', 'EOGr', 'EOGl', 'M1', 'M2', 'C4', 'O2'], ['ELB', 'ELE', 'ERE', 'EOGr', 'EOGl', 'M1', 'O1', 'M2', 'O2'], ['ELC', 'ERC', 'ERE', 'M1', 'O1', 'M2', 'C4', 'O2'], ['ELC', 'ELI', 'ERE', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'M2', 'O2', 'EMGc'], ['ELB', 'ELC', 'ELI', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ELI', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'O1', 'O2', 'EMGc']], ['M1', 'O2']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 1.5, 6, [['ELA', 'ERE', 'O1', 'C4'], ['ELA', 'ELC', 'ELI', 'ERE'], ['ELA', 'ELI', 'O1'], ['ELA', 'ELI'], ['ELA', 'ELI', 'ERE', 'EOGr'], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 1.5, 6, [['ELA', 'ERE', 'ERI', 'O1', 'C4'], ['ELA', 'ELC', 'ELI'], ['ELA', 'ELI', 'O1', 'O2'], ['ELA', 'O1'], ['ELA', 'ERE', 'EOGr', 'EOGl'], ['ELA', 'EMGl', 'EMGr', 'EMGc']], ['ELA']], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 1.5, 6, [['ELA', 'ERI'], ['ELA', 'ERE'], ['ELA', 'O1', 'C4'], ['ELA', 'ELI', 'EMGc'], ['ELA', 'EMGc'], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 1.5, 6, [['ERC', 'M2', 'O2'], ['EOGl', 'M1', 'O1', 'M2', 'O2'], ['ELC', 'ERC', 'ERE', 'M2', 'C4'], ['M1', 'EMGc'], ['ELI', 'M1', 'O2', 'EMGc'], ['ELA', 'ELC', 'EMGl']], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 2.0, 6, [['ELA'], ['ELA', 'ELC'], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 2.0, 6, [['ELA', 'ERE', 'O1', 'C4'], ['ELA', 'ELC'], ['ELA', 'ELI', 'O1'], ['ELA'], ['ELA', 'ERE', 'EOGr', 'EOGl'], ['EMGl', 'EMGr', 'EMGc']], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 2.5, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ERE'], ['EMGc']], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 3.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA', 'ERE'], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 4.0, 6, [['ELA'], ['ELA'], ['ELA'], ['ELA'], ['ELA'], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'mm', 5.0, 6, [['ELA'], ['ELA'], [], ['ELA'], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 1000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc', 'O2', 'EOGl']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'EMGl']], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'O1', 'O2', 'EMGc', 'ERB', 'EMGr', 'M1', 'C3', 'C4', 'F4', 'ERE']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 1.5, 1, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 1.5, 1, [['ELA', 'EMGc'], ['F3', 'F4', 'EMGc'], ['F4', 'EMGc'], ['ELA', 'F3', 'F4', 'EMGc'], ['F3', 'F4', 'EMGc'], ['EMGc']], ['ELA', 'EMGc', 'F3', 'F4']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 1.5, 1, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 1.5, 1, [['ELA', 'ELC', 'ERC'], ['ELA', 'ERA'], ['ERA', 'O1'], ['ELA', 'EOGl'], ['ELC'], ['ERA']], ['ELA', 'ELC', 'ERC', 'ERA', 'O1', 'EOGl']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 2.0, 1, [['EMGc'], ['EMGc'], ['F4', 'EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc', 'F4']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 2.0, 1, [['ERC'], [], [], [], [], []], ['ERC']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 2.5, 1, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 3.0, 1, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc', 'O2', 'EOGl']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'EMGl']], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'O1', 'O2', 'EMGc', 'EMGr', 'C3']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 1.5, 2, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 1.5, 2, [['ELA', 'EMGc'], ['F3', 'F4', 'EMGc'], ['F4', 'EMGc'], ['ELA', 'F3', 'F4', 'EMGc'], ['F3', 'F4', 'EMGc'], ['EMGc']], ['EMGc', 'F3', 'F4']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 1.5, 2, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 1.5, 2, [['ELA', 'ELC', 'ERC'], ['ELA', 'ERA'], ['ERA', 'O1'], ['ELA', 'EOGl'], ['ELC'], ['ERA']], ['ELA', 'ERA']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 2.0, 2, [['EMGc'], ['EMGc'], ['F4', 'EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 2.0, 2, [['ERC'], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 2.5, 2, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 3.0, 2, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'EMGl']], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'O1', 'O2', 'EMGc', 'EMGr']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 1.5, 3, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 1.5, 3, [['ELA', 'EMGc'], ['F3', 'F4', 'EMGc'], ['F4', 'EMGc'], ['ELA', 'F3', 'F4', 'EMGc'], ['F3', 'F4', 'EMGc'], ['EMGc']], ['EMGc', 'F4']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 1.5, 3, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 1.5, 3, [['ELA', 'ELC', 'ERC'], ['ELA', 'ERA'], ['ERA', 'O1'], ['ELA', 'EOGl'], ['ELC'], ['ERA']], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 2.0, 3, [['EMGc'], ['EMGc'], ['F4', 'EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 2.0, 3, [['ERC'], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 2.5, 3, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 3.0, 3, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'O1', 'O2', 'EMGc', 'EMGr']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 1.5, 4, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 1.5, 4, [['ELA', 'EMGc'], ['F3', 'F4', 'EMGc'], ['F4', 'EMGc'], ['ELA', 'F3', 'F4', 'EMGc'], ['F3', 'F4', 'EMGc'], ['EMGc']], ['EMGc', 'F4']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 1.5, 4, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 1.5, 4, [['ELA', 'ELC', 'ERC'], ['ELA', 'ERA'], ['ERA', 'O1'], ['ELA', 'EOGl'], ['ELC'], ['ERA']], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 2.0, 4, [['EMGc'], ['EMGc'], ['F4', 'EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 2.0, 4, [['ERC'], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 2.5, 4, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 3.0, 4, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'O1', 'O2', 'EMGc', 'EMGr']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 1.5, 5, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 1.5, 5, [['ELA', 'EMGc'], ['F3', 'F4', 'EMGc'], ['F4', 'EMGc'], ['ELA', 'F3', 'F4', 'EMGc'], ['F3', 'F4', 'EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 1.5, 5, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 1.5, 5, [['ELA', 'ELC', 'ERC'], ['ELA', 'ERA'], ['ERA', 'O1'], ['ELA', 'EOGl'], ['ELC'], ['ERA']], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 2.0, 5, [['EMGc'], ['EMGc'], ['F4', 'EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 2.0, 5, [['ERC'], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 2.5, 5, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 3.0, 5, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELC', 'ERA', 'ERC', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ERA', 'ERC', 'EOGr', 'EOGl', 'EMGl', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'O1', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'C3', 'O1', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'O1', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 1.5, 6, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 1.5, 6, [['ELA', 'EMGc'], ['F3', 'F4', 'EMGc'], ['F4', 'EMGc'], ['ELA', 'F3', 'F4', 'EMGc'], ['F3', 'F4', 'EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 1.5, 6, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 1.5, 6, [['ELA', 'ELC', 'ERC'], ['ELA', 'ERA'], ['ERA', 'O1'], ['ELA', 'EOGl'], ['ELC'], ['ERA']], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 2.0, 6, [['EMGc'], ['EMGc'], ['F4', 'EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 2.0, 6, [['ERC'], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 2.5, 6, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 3.0, 6, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 2000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'EOGl']], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 1.0, 1, [['ELB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ELA', 'ERA']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'EOGl', 'EMGl']], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4']], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'EOGl', 'O1', 'M2', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 1.5, 1, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 1.5, 1, [['EMGc'], ['F3', 'F4', 'C4', 'EMGc'], ['F4', 'EMGc'], ['F3', 'C3', 'F4', 'EMGc'], ['F3', 'F4', 'C4', 'EMGc'], ['F3', 'F4', 'C4', 'EMGc']], ['EMGc', 'F3', 'F4', 'C4', 'C3']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 1.5, 1, [[], ['F3'], [], ['EMGc'], ['EMGc'], []], ['F3', 'EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 1.5, 1, [['ELC', 'M1'], ['ELB', 'M1'], ['ELA', 'ELB', 'ELC', 'M1'], ['ELA', 'ELB', 'M1', 'C4'], ['ELC', 'M1'], ['ERA', 'M1']], ['ELC', 'M1', 'ELB', 'ELA', 'C4', 'ERA']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 2.0, 1, [['EMGc'], ['F3', 'EMGc'], ['EMGc'], ['F4', 'EMGc'], ['F4', 'EMGc'], ['EMGc']], ['EMGc', 'F3', 'F4']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 2.5, 1, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 3.0, 1, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 4.0, 1, [[], [], ['EMGc'], [], [], []], ['EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'EOGl']], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 1.0, 2, [['ELB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ELA']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'EOGl']], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4']], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4', 'EOGl', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 1.5, 2, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 1.5, 2, [['EMGc'], ['F3', 'F4', 'C4', 'EMGc'], ['F4', 'EMGc'], ['F3', 'C3', 'F4', 'EMGc'], ['F3', 'F4', 'C4', 'EMGc'], ['F3', 'F4', 'C4', 'EMGc']], ['EMGc', 'F3', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 1.5, 2, [[], ['F3'], [], ['EMGc'], ['EMGc'], []], ['EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 1.5, 2, [['ELC', 'M1'], ['ELB', 'M1'], ['ELA', 'ELB', 'ELC', 'M1'], ['ELA', 'ELB', 'M1', 'C4'], ['ELC', 'M1'], ['ERA', 'M1']], ['M1', 'ELB', 'ELA']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 2.0, 2, [['EMGc'], ['F3', 'EMGc'], ['EMGc'], ['F4', 'EMGc'], ['F4', 'EMGc'], ['EMGc']], ['EMGc', 'F4']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 2.5, 2, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 3.0, 2, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 4.0, 2, [[], [], ['EMGc'], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'EOGl']], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 1.0, 3, [['ELB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'EOGl']], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4']], ['ELA', 'ELB', 'ERA', 'EOGr', 'EMGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'EOGl']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 1.5, 3, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 1.5, 3, [['EMGc'], ['F3', 'F4', 'C4', 'EMGc'], ['F4', 'EMGc'], ['F3', 'C3', 'F4', 'EMGc'], ['F3', 'F4', 'C4', 'EMGc'], ['F3', 'F4', 'C4', 'EMGc']], ['EMGc', 'F3', 'F4']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 1.5, 3, [[], ['F3'], [], ['EMGc'], ['EMGc'], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 1.5, 3, [['ELC', 'M1'], ['ELB', 'M1'], ['ELA', 'ELB', 'ELC', 'M1'], ['ELA', 'ELB', 'M1', 'C4'], ['ELC', 'M1'], ['ERA', 'M1']], ['M1', 'ELB']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 2.0, 3, [['EMGc'], ['F3', 'EMGc'], ['EMGc'], ['F4', 'EMGc'], ['F4', 'EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 2.5, 3, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 3.0, 3, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 4.0, 3, [[], [], ['EMGc'], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 1.0, 4, [['ELB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['F3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4']], ['ELA', 'ELB', 'ERA', 'M1', 'F3', 'C3', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 1.5, 4, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 1.5, 4, [['EMGc'], ['F3', 'F4', 'C4', 'EMGc'], ['F4', 'EMGc'], ['F3', 'C3', 'F4', 'EMGc'], ['F3', 'F4', 'C4', 'EMGc'], ['F3', 'F4', 'C4', 'EMGc']], ['EMGc', 'F4']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 1.5, 4, [[], ['F3'], [], ['EMGc'], ['EMGc'], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 1.5, 4, [['ELC', 'M1'], ['ELB', 'M1'], ['ELA', 'ELB', 'ELC', 'M1'], ['ELA', 'ELB', 'M1', 'C4'], ['ELC', 'M1'], ['ERA', 'M1']], ['M1']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 2.0, 4, [['EMGc'], ['F3', 'EMGc'], ['EMGc'], ['F4', 'EMGc'], ['F4', 'EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 2.5, 4, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 3.0, 4, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 4.0, 4, [[], [], ['EMGc'], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 1.0, 5, [['ELB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['F3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4']], ['ELA', 'ELB', 'ERA', 'M1', 'F3', 'C3', 'C4']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 1.5, 5, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 1.5, 5, [['EMGc'], ['F3', 'F4', 'C4', 'EMGc'], ['F4', 'EMGc'], ['F3', 'C3', 'F4', 'EMGc'], ['F3', 'F4', 'C4', 'EMGc'], ['F3', 'F4', 'C4', 'EMGc']], ['EMGc', 'F4']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 1.5, 5, [[], ['F3'], [], ['EMGc'], ['EMGc'], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 1.5, 5, [['ELC', 'M1'], ['ELB', 'M1'], ['ELA', 'ELB', 'ELC', 'M1'], ['ELA', 'ELB', 'M1', 'C4'], ['ELC', 'M1'], ['ERA', 'M1']], ['M1']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 2.0, 5, [['EMGc'], ['F3', 'EMGc'], ['EMGc'], ['F4', 'EMGc'], ['F4', 'EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 2.5, 5, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 3.0, 5, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 4.0, 5, [[], [], ['EMGc'], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 1.0, 6, [['ELB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['F3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'EMGr', 'M1', 'F3', 'C3', 'F4', 'C4']], ['ELA', 'ELB', 'ERA', 'M1', 'F3', 'C3', 'C4']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 1.5, 6, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 1.5, 6, [['EMGc'], ['F3', 'F4', 'C4', 'EMGc'], ['F4', 'EMGc'], ['F3', 'C3', 'F4', 'EMGc'], ['F3', 'F4', 'C4', 'EMGc'], ['F3', 'F4', 'C4', 'EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 1.5, 6, [[], ['F3'], [], ['EMGc'], ['EMGc'], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 1.5, 6, [['ELC', 'M1'], ['ELB', 'M1'], ['ELA', 'ELB', 'ELC', 'M1'], ['ELA', 'ELB', 'M1', 'C4'], ['ELC', 'M1'], ['ERA', 'M1']], ['M1']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 2.0, 6, [['EMGc'], ['F3', 'EMGc'], ['EMGc'], ['F4', 'EMGc'], ['F4', 'EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 2.5, 6, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 3.0, 6, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 4.0, 6, [[], [], ['EMGc'], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 3000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 1.0, 1, [['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERT ', 'ERE', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'ELA', 'ELI', 'ERC', 'ERT ', 'ERE', 'ERA', 'M1', 'ELB']], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 1.0, 1, [['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERE', 'F3', 'F4', 'C4'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ELA', 'ELI', 'ERE', 'M1']], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc', 'ELC', 'ELI', 'ERB', 'ERC', 'ERT ', 'O2', 'ERE', 'EMGr']], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4']], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc', 'ELC', 'ELI', 'ERC', 'EMGl', 'O1', 'O2', 'ERE', 'ERT ']], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 1.5, 1, [[], ['ELA', 'ERE', 'C4'], [], ['F4'], [], []], ['ELA', 'ERE', 'C4', 'F4']], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 1.5, 1, [['F3', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ERE', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4']], ['F3', 'C3', 'F4', 'C4', 'EMGc', 'ELA', 'ERE']], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 1.5, 1, [[], ['ELA'], [], [], [], []], ['ELA']], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 1.5, 1, [['M1', 'M2'], ['M2'], ['EOGr', 'M1'], [], ['ELA', 'M1'], ['M1']], ['M1', 'M2', 'EOGr', 'ELA']], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 2.0, 1, [['F4'], ['ERE', 'C4'], ['F4'], ['F3', 'F4'], ['F4'], ['F3', 'F4']], ['F4', 'ERE', 'C4', 'F3']], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 2.0, 1, [['M1'], ['M2'], [], [], [], []], ['M1', 'M2']], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 2.5, 1, [[], ['ERE', 'C4'], [], ['F3', 'F4'], [], []], ['ERE', 'C4', 'F3', 'F4']], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 3.0, 1, [[], ['ERE', 'C4'], [], ['F4'], [], []], ['ERE', 'C4', 'F4']], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 1.0, 2, [['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERT ', 'ERE', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'ELA', 'ERE', 'ERA', 'M1', 'ELB']], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 1.0, 2, [['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERE', 'F3', 'F4', 'C4'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELA', 'ERE']], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4']], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'ELI']], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 1.5, 2, [[], ['ELA', 'ERE', 'C4'], [], ['F4'], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 1.5, 2, [['F3', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ERE', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4']], ['F3', 'C3', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 1.5, 2, [[], ['ELA'], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 1.5, 2, [['M1', 'M2'], ['M2'], ['EOGr', 'M1'], [], ['ELA', 'M1'], ['M1']], ['M1', 'M2']], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 2.0, 2, [['F4'], ['ERE', 'C4'], ['F4'], ['F3', 'F4'], ['F4'], ['F3', 'F4']], ['F4']], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 2.0, 2, [['M1'], ['M2'], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 2.5, 2, [[], ['ERE', 'C4'], [], ['F3', 'F4'], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 3.0, 2, [[], ['ERE', 'C4'], [], ['F4'], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 1.0, 3, [['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERT ', 'ERE', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELA', 'ERA', 'M1']], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 1.0, 3, [['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERE', 'F3', 'F4', 'C4'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4']], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 1.5, 3, [[], ['ELA', 'ERE', 'C4'], [], ['F4'], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 1.5, 3, [['F3', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ERE', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4']], ['F3', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 1.5, 3, [[], ['ELA'], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 1.5, 3, [['M1', 'M2'], ['M2'], ['EOGr', 'M1'], [], ['ELA', 'M1'], ['M1']], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 2.0, 3, [['F4'], ['ERE', 'C4'], ['F4'], ['F3', 'F4'], ['F4'], ['F3', 'F4']], ['F4']], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 2.0, 3, [['M1'], ['M2'], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 2.5, 3, [[], ['ERE', 'C4'], [], ['F3', 'F4'], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 3.0, 3, [[], ['ERE', 'C4'], [], ['F4'], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 1.0, 4, [['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERT ', 'ERE', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELA', 'ERA', 'M1']], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 1.0, 4, [['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERE', 'F3', 'F4', 'C4'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4']], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 1.5, 4, [[], ['ELA', 'ERE', 'C4'], [], ['F4'], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 1.5, 4, [['F3', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ERE', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4']], ['F3', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 1.5, 4, [[], ['ELA'], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 1.5, 4, [['M1', 'M2'], ['M2'], ['EOGr', 'M1'], [], ['ELA', 'M1'], ['M1']], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 2.0, 4, [['F4'], ['ERE', 'C4'], ['F4'], ['F3', 'F4'], ['F4'], ['F3', 'F4']], ['F4']], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 2.0, 4, [['M1'], ['M2'], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 2.5, 4, [[], ['ERE', 'C4'], [], ['F3', 'F4'], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 3.0, 4, [[], ['ERE', 'C4'], [], ['F4'], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 1.0, 5, [['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERT ', 'ERE', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['F3', 'F4', 'C4', 'O2', 'ELA']], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 1.0, 5, [['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERE', 'F3', 'F4', 'C4'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4']], ['ELA', 'ERA', 'EOGr', 'F3', 'C3']], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 1.5, 5, [[], ['ELA', 'ERE', 'C4'], [], ['F4'], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 1.5, 5, [['F3', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ERE', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4']], ['C4']], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 1.5, 5, [[], ['ELA'], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 1.5, 5, [['M1', 'M2'], ['M2'], ['EOGr', 'M1'], [], ['ELA', 'M1'], ['M1']], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 2.0, 5, [['F4'], ['ERE', 'C4'], ['F4'], ['F3', 'F4'], ['F4'], ['F3', 'F4']], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 2.0, 5, [['M1'], ['M2'], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 2.5, 5, [[], ['ERE', 'C4'], [], ['F3', 'F4'], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 3.0, 5, [[], ['ERE', 'C4'], [], ['F4'], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 1.0, 6, [['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERC', 'ERT ', 'ERE', 'F3', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['F3', 'F4', 'C4', 'O2']], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 1.0, 6, [['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELI', 'ERE', 'F3', 'F4', 'C4'], ['ELA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELC', 'ELI', 'ERA', 'ERC', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'O2'], ['ELA', 'ERA', 'ERE', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'C4'], ['ELA', 'ELB', 'ELI', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4']], ['ELA', 'ERA', 'EOGr', 'F3', 'C3']], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 1.5, 6, [[], ['ELA', 'ERE', 'C4'], [], ['F4'], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 1.5, 6, [['F3', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ERE', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4']], ['C4']], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 1.5, 6, [[], ['ELA'], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 1.5, 6, [['M1', 'M2'], ['M2'], ['EOGr', 'M1'], [], ['ELA', 'M1'], ['M1']], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 2.0, 6, [['F4'], ['ERE', 'C4'], ['F4'], ['F3', 'F4'], ['F4'], ['F3', 'F4']], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 2.0, 6, [['M1'], ['M2'], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 2.5, 6, [[], ['ERE', 'C4'], [], ['F3', 'F4'], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 3.0, 6, [[], ['ERE', 'C4'], [], ['F4'], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 4000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'M1', 'ERA']], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 1.0, 1, [['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc']], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ELA', 'ELB']], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 1.0, 1, [['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'ERA', 'ELB']], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc', 'O1']], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 1.5, 1, [['C4'], ['C3'], ['C3', 'F4'], [], ['C4'], []], ['C4', 'C3', 'F4']], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 1.5, 1, [['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['EOGr', 'F3', 'C3', 'F4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4']], ['F3', 'C3', 'F4', 'C4', 'EOGr']], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 1.5, 1, [[], ['C3'], ['F3', 'C3'], [], [], []], ['C3', 'F3']], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 1.5, 1, [['M1', 'M2', 'EMGc'], ['M1', 'EMGc'], ['M1', 'F3', 'M2', 'EMGc'], ['M1', 'M2', 'EMGc'], ['ERA', 'M1', 'M2', 'EMGc'], ['M1', 'M2', 'EMGc']], ['M1', 'M2', 'EMGc', 'F3', 'ERA']], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 2.0, 1, [['F3', 'C4'], ['F3', 'C3', 'F4'], ['C3', 'F4'], ['F3', 'F4'], ['C3', 'C4'], ['C3', 'C4']], ['F3', 'C4', 'C3', 'F4']], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 2.0, 1, [[], ['M1', 'EMGc'], [], [], ['ERA'], ['EMGc']], ['M1', 'EMGc', 'ERA']], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 2.5, 1, [[], ['F4'], [], [], ['C4'], []], ['F4', 'C4']], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 1.0, 2, [['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ELA']], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 1.0, 2, [['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'ERA']], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc']], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 1.5, 2, [['C4'], ['C3'], ['C3', 'F4'], [], ['C4'], []], ['C3']], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 1.5, 2, [['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['EOGr', 'F3', 'C3', 'F4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4']], ['F3', 'C3', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 1.5, 2, [[], ['C3'], ['F3', 'C3'], [], [], []], ['C3']], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 1.5, 2, [['M1', 'M2', 'EMGc'], ['M1', 'EMGc'], ['M1', 'F3', 'M2', 'EMGc'], ['M1', 'M2', 'EMGc'], ['ERA', 'M1', 'M2', 'EMGc'], ['M1', 'M2', 'EMGc']], ['M1', 'M2', 'EMGc']], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 2.0, 2, [['F3', 'C4'], ['F3', 'C3', 'F4'], ['C3', 'F4'], ['F3', 'F4'], ['C3', 'C4'], ['C3', 'C4']], ['F3', 'C4', 'C3', 'F4']], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 2.0, 2, [[], ['M1', 'EMGc'], [], [], ['ERA'], ['EMGc']], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 2.5, 2, [[], ['F4'], [], [], ['C4'], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 1.0, 3, [['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 1.0, 3, [['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'ERA']], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc']], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 1.5, 3, [['C4'], ['C3'], ['C3', 'F4'], [], ['C4'], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 1.5, 3, [['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['EOGr', 'F3', 'C3', 'F4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4']], ['F3', 'C3', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 1.5, 3, [[], ['C3'], ['F3', 'C3'], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 1.5, 3, [['M1', 'M2', 'EMGc'], ['M1', 'EMGc'], ['M1', 'F3', 'M2', 'EMGc'], ['M1', 'M2', 'EMGc'], ['ERA', 'M1', 'M2', 'EMGc'], ['M1', 'M2', 'EMGc']], ['M1', 'M2', 'EMGc']], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 2.0, 3, [['F3', 'C4'], ['F3', 'C3', 'F4'], ['C3', 'F4'], ['F3', 'F4'], ['C3', 'C4'], ['C3', 'C4']], ['F4']], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 2.0, 3, [[], ['M1', 'EMGc'], [], [], ['ERA'], ['EMGc']], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 2.5, 3, [[], ['F4'], [], [], ['C4'], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 1.0, 4, [['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 1.0, 4, [['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'ERA']], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc']], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 1.5, 4, [['C4'], ['C3'], ['C3', 'F4'], [], ['C4'], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 1.5, 4, [['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['EOGr', 'F3', 'C3', 'F4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4']], ['F3', 'C3', 'F4']], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 1.5, 4, [[], ['C3'], ['F3', 'C3'], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 1.5, 4, [['M1', 'M2', 'EMGc'], ['M1', 'EMGc'], ['M1', 'F3', 'M2', 'EMGc'], ['M1', 'M2', 'EMGc'], ['ERA', 'M1', 'M2', 'EMGc'], ['M1', 'M2', 'EMGc']], ['M1', 'M2', 'EMGc']], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 2.0, 4, [['F3', 'C4'], ['F3', 'C3', 'F4'], ['C3', 'F4'], ['F3', 'F4'], ['C3', 'C4'], ['C3', 'C4']], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 2.0, 4, [[], ['M1', 'EMGc'], [], [], ['ERA'], ['EMGc']], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 2.5, 4, [[], ['F4'], [], [], ['C4'], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 1.0, 5, [['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 1.0, 5, [['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'ERA']], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc']], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 1.5, 5, [['C4'], ['C3'], ['C3', 'F4'], [], ['C4'], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 1.5, 5, [['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['EOGr', 'F3', 'C3', 'F4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4']], ['F3', 'C3', 'F4']], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 1.5, 5, [[], ['C3'], ['F3', 'C3'], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 1.5, 5, [['M1', 'M2', 'EMGc'], ['M1', 'EMGc'], ['M1', 'F3', 'M2', 'EMGc'], ['M1', 'M2', 'EMGc'], ['ERA', 'M1', 'M2', 'EMGc'], ['M1', 'M2', 'EMGc']], ['M1', 'EMGc']], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 2.0, 5, [['F3', 'C4'], ['F3', 'C3', 'F4'], ['C3', 'F4'], ['F3', 'F4'], ['C3', 'C4'], ['C3', 'C4']], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 2.0, 5, [[], ['M1', 'EMGc'], [], [], ['ERA'], ['EMGc']], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 2.5, 5, [[], ['F4'], [], [], ['C4'], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 1.0, 6, [['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc']], ['EOGr', 'F3', 'C3', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 1.0, 6, [['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc']], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 1.5, 6, [['C4'], ['C3'], ['C3', 'F4'], [], ['C4'], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 1.5, 6, [['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['EOGr', 'F3', 'C3', 'F4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4']], ['F3', 'C3', 'F4']], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 1.5, 6, [[], ['C3'], ['F3', 'C3'], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 1.5, 6, [['M1', 'M2', 'EMGc'], ['M1', 'EMGc'], ['M1', 'F3', 'M2', 'EMGc'], ['M1', 'M2', 'EMGc'], ['ERA', 'M1', 'M2', 'EMGc'], ['M1', 'M2', 'EMGc']], ['M1', 'EMGc']], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 2.0, 6, [['F3', 'C4'], ['F3', 'C3', 'F4'], ['C3', 'F4'], ['F3', 'F4'], ['C3', 'C4'], ['C3', 'C4']], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 2.0, 6, [[], ['M1', 'EMGc'], [], [], ['ERA'], ['EMGc']], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 2.5, 6, [[], ['F4'], [], [], ['C4'], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 5000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'M1', 'ERA']], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 1.0, 1, [['ELA', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc', 'O1', 'O2', 'EOGl']], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'ELB', 'M1']], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc', 'ERA', 'O2', 'O1']], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 1.5, 1, [['F4'], ['C4'], [], ['C3'], ['C4'], ['C3']], ['F4', 'C4', 'C3']], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 1.5, 1, [['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['EOGr', 'F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'O1', 'F4', 'C4'], ['EOGr', 'F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4']], ['F3', 'C3', 'F4', 'C4', 'EOGr', 'O1']], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 1.5, 1, [[], [], [], [], ['F4'], []], ['F4']], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 1.5, 1, [['M2', 'EMGc'], ['M1', 'EMGc'], ['EOGr', 'M1', 'M2', 'EMGc'], ['M1', 'M2', 'EMGc'], ['M1', 'F4', 'EMGc'], ['M1', 'M2', 'EMGc']], ['M2', 'EMGc', 'M1', 'EOGr', 'F4']], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 2.0, 1, [['F3', 'F4'], ['F4', 'C4'], ['F4'], ['F4'], [], ['F4', 'C4']], ['F3', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 2.0, 1, [[], ['EMGc'], [], [], [], ['M1']], ['EMGc', 'M1']], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 2.5, 1, [['F4'], ['C4'], [], [], [], []], ['F4', 'C4']], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 3.0, 1, [[], ['C4'], [], [], [], []], ['C4']], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'M1']], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 1.0, 2, [['ELA', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc', 'O1', 'O2']], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'M1']], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc', 'ERA']], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 1.5, 2, [['F4'], ['C4'], [], ['C3'], ['C4'], ['C3']], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 1.5, 2, [['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['EOGr', 'F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'O1', 'F4', 'C4'], ['EOGr', 'F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4']], ['F3', 'C3', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 1.5, 2, [[], [], [], [], ['F4'], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 1.5, 2, [['M2', 'EMGc'], ['M1', 'EMGc'], ['EOGr', 'M1', 'M2', 'EMGc'], ['M1', 'M2', 'EMGc'], ['M1', 'F4', 'EMGc'], ['M1', 'M2', 'EMGc']], ['M2', 'EMGc', 'M1']], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 2.0, 2, [['F3', 'F4'], ['F4', 'C4'], ['F4'], ['F4'], [], ['F4', 'C4']], ['F4']], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 2.0, 2, [[], ['EMGc'], [], [], [], ['M1']], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 2.5, 2, [['F4'], ['C4'], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 3.0, 2, [[], ['C4'], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 1.0, 3, [['ELA', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc', 'O1', 'O2']], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'M1']], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc', 'ERA']], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 1.5, 3, [['F4'], ['C4'], [], ['C3'], ['C4'], ['C3']], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 1.5, 3, [['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['EOGr', 'F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'O1', 'F4', 'C4'], ['EOGr', 'F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4']], ['F3', 'C3', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 1.5, 3, [[], [], [], [], ['F4'], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 1.5, 3, [['M2', 'EMGc'], ['M1', 'EMGc'], ['EOGr', 'M1', 'M2', 'EMGc'], ['M1', 'M2', 'EMGc'], ['M1', 'F4', 'EMGc'], ['M1', 'M2', 'EMGc']], ['EMGc', 'M1']], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 2.0, 3, [['F3', 'F4'], ['F4', 'C4'], ['F4'], ['F4'], [], ['F4', 'C4']], ['F4']], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 2.0, 3, [[], ['EMGc'], [], [], [], ['M1']], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 2.5, 3, [['F4'], ['C4'], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 3.0, 3, [[], ['C4'], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 1.0, 4, [['ELA', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc', 'O2']], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'M1']], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc']], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 1.5, 4, [['F4'], ['C4'], [], ['C3'], ['C4'], ['C3']], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 1.5, 4, [['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['EOGr', 'F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'O1', 'F4', 'C4'], ['EOGr', 'F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4']], ['F3', 'C3', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 1.5, 4, [[], [], [], [], ['F4'], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 1.5, 4, [['M2', 'EMGc'], ['M1', 'EMGc'], ['EOGr', 'M1', 'M2', 'EMGc'], ['M1', 'M2', 'EMGc'], ['M1', 'F4', 'EMGc'], ['M1', 'M2', 'EMGc']], ['EMGc', 'M1']], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 2.0, 4, [['F3', 'F4'], ['F4', 'C4'], ['F4'], ['F4'], [], ['F4', 'C4']], ['F4']], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 2.0, 4, [[], ['EMGc'], [], [], [], ['M1']], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 2.5, 4, [['F4'], ['C4'], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 3.0, 4, [[], ['C4'], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 1.0, 5, [['ELA', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc', 'O2']], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'M1']], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc']], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 1.5, 5, [['F4'], ['C4'], [], ['C3'], ['C4'], ['C3']], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 1.5, 5, [['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['EOGr', 'F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'O1', 'F4', 'C4'], ['EOGr', 'F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4']], ['F3', 'C3', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 1.5, 5, [[], [], [], [], ['F4'], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 1.5, 5, [['M2', 'EMGc'], ['M1', 'EMGc'], ['EOGr', 'M1', 'M2', 'EMGc'], ['M1', 'M2', 'EMGc'], ['M1', 'F4', 'EMGc'], ['M1', 'M2', 'EMGc']], ['EMGc', 'M1']], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 2.0, 5, [['F3', 'F4'], ['F4', 'C4'], ['F4'], ['F4'], [], ['F4', 'C4']], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 2.0, 5, [[], ['EMGc'], [], [], [], ['M1']], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 2.5, 5, [['F4'], ['C4'], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 3.0, 5, [[], ['C4'], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 1.0, 6, [['ELA', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc']], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc']], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 1.5, 6, [['F4'], ['C4'], [], ['C3'], ['C4'], ['C3']], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 1.5, 6, [['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['EOGr', 'F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'O1', 'F4', 'C4'], ['EOGr', 'F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4']], ['F3', 'C3', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 1.5, 6, [[], [], [], [], ['F4'], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 1.5, 6, [['M2', 'EMGc'], ['M1', 'EMGc'], ['EOGr', 'M1', 'M2', 'EMGc'], ['M1', 'M2', 'EMGc'], ['M1', 'F4', 'EMGc'], ['M1', 'M2', 'EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 2.0, 6, [['F3', 'F4'], ['F4', 'C4'], ['F4'], ['F4'], [], ['F4', 'C4']], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 2.0, 6, [[], ['EMGc'], [], [], [], ['M1']], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 2.5, 6, [['F4'], ['C4'], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 3.0, 6, [[], ['C4'], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 6000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELC', 'ERA', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'ELB', 'C3', 'EOGl']], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELC', 'ERA', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'EOGr', 'ELB', 'EOGl']], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'ELB', 'ERE']], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ERA', 'EMGl', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EMGl', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERA', 'EMGl', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc', 'ERE', 'EOGr', 'EOGl', 'O1', 'O2']], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 1.5, 1, [['ELA', 'F3', 'EMGc'], ['F3', 'F4', 'C4', 'EMGc'], ['F4', 'EMGc'], ['F3', 'F4'], ['F3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4']], ['ELA', 'F3', 'EMGc', 'F4', 'C4', 'C3']], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 1.5, 1, [[], ['EMGc'], ['EMGc'], [], [], []], ['EMGc']], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 1.5, 1, [['M2', 'EMGc'], ['ERA', 'EMGl', 'EMGc'], ['ELC', 'EMGc'], ['ELA', 'EMGc'], ['ELA', 'ELC', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EMGc']], ['M2', 'EMGc', 'ERA', 'EMGl', 'ELC', 'ELA', 'ELB']], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 2.0, 1, [[], [], [], ['F4'], [], []], ['F4']], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 2.0, 1, [[], ['EMGc'], ['EMGc'], ['EMGc'], [], []], ['EMGc']], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELC', 'ERA', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ELB', 'C3', 'EOGl']], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELC', 'ERA', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'EOGr']], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'ELB']], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ERA', 'EMGl', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EMGl', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERA', 'EMGl', 'F3', 'C3', 'M2', 'C4', 'EMGc', 'EOGr', 'EOGl', 'O1', 'O2']], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 1.5, 2, [['ELA', 'F3', 'EMGc'], ['F3', 'F4', 'C4', 'EMGc'], ['F4', 'EMGc'], ['F3', 'F4'], ['F3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4']], ['F3', 'EMGc', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 1.5, 2, [[], ['EMGc'], ['EMGc'], [], [], []], ['EMGc']], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 1.5, 2, [['M2', 'EMGc'], ['ERA', 'EMGl', 'EMGc'], ['ELC', 'EMGc'], ['ELA', 'EMGc'], ['ELA', 'ELC', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EMGc']], ['EMGc', 'ELC', 'ELA']], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 2.0, 2, [[], [], [], ['F4'], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 2.0, 2, [[], ['EMGc'], ['EMGc'], ['EMGc'], [], []], ['EMGc']], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELC', 'ERA', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ELB', 'C3']], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELC', 'ERA', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'EOGr']], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'ELB']], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ERA', 'EMGl', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EMGl', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERA', 'EMGl', 'F3', 'C3', 'M2', 'C4', 'EMGc', 'EOGr', 'EOGl', 'O1']], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 1.5, 3, [['ELA', 'F3', 'EMGc'], ['F3', 'F4', 'C4', 'EMGc'], ['F4', 'EMGc'], ['F3', 'F4'], ['F3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4']], ['F3', 'EMGc', 'F4']], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 1.5, 3, [[], ['EMGc'], ['EMGc'], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 1.5, 3, [['M2', 'EMGc'], ['ERA', 'EMGl', 'EMGc'], ['ELC', 'EMGc'], ['ELA', 'EMGc'], ['ELA', 'ELC', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EMGc']], ['EMGc', 'ELA']], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 2.0, 3, [[], [], [], ['F4'], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 2.0, 3, [[], ['EMGc'], ['EMGc'], ['EMGc'], [], []], ['EMGc']], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELC', 'ERA', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'EOGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'C3']], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELC', 'ERA', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'EOGr']], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ERA', 'EMGl', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EMGl', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'ERA', 'EMGl', 'C3', 'M2', 'C4', 'EMGc', 'EOGr', 'EOGl', 'O1']], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 1.5, 4, [['ELA', 'F3', 'EMGc'], ['F3', 'F4', 'C4', 'EMGc'], ['F4', 'EMGc'], ['F3', 'F4'], ['F3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4']], ['F4']], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 1.5, 4, [[], ['EMGc'], ['EMGc'], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 1.5, 4, [['M2', 'EMGc'], ['ERA', 'EMGl', 'EMGc'], ['ELC', 'EMGc'], ['ELA', 'EMGc'], ['ELA', 'ELC', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 2.0, 4, [[], [], [], ['F4'], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 2.0, 4, [[], ['EMGc'], ['EMGc'], ['EMGc'], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELC', 'ERA', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'EOGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'C3']], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELC', 'ERA', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'EOGr']], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ERA', 'EMGl', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EMGl', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'EMGl', 'C3', 'M2', 'C4', 'EMGc']], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 1.5, 5, [['ELA', 'F3', 'EMGc'], ['F3', 'F4', 'C4', 'EMGc'], ['F4', 'EMGc'], ['F3', 'F4'], ['F3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4']], ['F4']], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 1.5, 5, [[], ['EMGc'], ['EMGc'], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 1.5, 5, [['M2', 'EMGc'], ['ERA', 'EMGl', 'EMGc'], ['ELC', 'EMGc'], ['ELA', 'EMGc'], ['ELA', 'ELC', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 2.0, 5, [[], [], [], ['F4'], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 2.0, 5, [[], ['EMGc'], ['EMGc'], ['EMGc'], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELC', 'ERA', 'ERE', 'EOGr', 'EMGl', 'F3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'EOGr', 'F3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELC', 'ERA', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELC', 'ERA', 'EOGr', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELC', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ERA', 'EMGl', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EMGl', 'F3', 'C3', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'EMGl', 'C3', 'O1', 'M2', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ELC', 'EMGl', 'C3', 'M2', 'C4', 'EMGc']], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 1.5, 6, [['ELA', 'F3', 'EMGc'], ['F3', 'F4', 'C4', 'EMGc'], ['F4', 'EMGc'], ['F3', 'F4'], ['F3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4']], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 1.5, 6, [[], ['EMGc'], ['EMGc'], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 1.5, 6, [['M2', 'EMGc'], ['ERA', 'EMGl', 'EMGc'], ['ELC', 'EMGc'], ['ELA', 'EMGc'], ['ELA', 'ELC', 'EMGc'], ['ELA', 'ELB', 'ELC', 'EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 2.0, 6, [[], [], [], ['F4'], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 2.0, 6, [[], ['EMGc'], ['EMGc'], ['EMGc'], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 7000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELC', 'ERB']], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 1.0, 1, [['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELB', 'ELC', 'EOGl']], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGl', 'ERC', 'ERE']], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'C3', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4']], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'ELC', 'EMGl', 'O1']], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 1.5, 1, [['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'C4', 'O2'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4', 'C4'], ['F4', 'C4', 'O2']], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ERA']], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 1.5, 1, [['ELA', 'ELB', 'ERA', 'ERT '], ['ELC'], ['ELA', 'ELB', 'ELC', 'ERA'], ['ELB', 'ERA'], ['ELB', 'ERA'], ['ELA', 'ELC', 'ERA', 'EOGl']], ['ELA', 'ELB', 'ERA', 'ERT ', 'ELC', 'EOGl']], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 2.0, 1, [['C3'], ['F4'], [], ['F3'], [], []], ['C3', 'F4', 'F3']], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELC', 'ERB']], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 1.0, 2, [['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELB']], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ERE']], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'C3', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4']], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'ELC']], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 1.5, 2, [['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'C4', 'O2'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4', 'C4'], ['F4', 'C4', 'O2']], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 1.5, 2, [['ELA', 'ELB', 'ERA', 'ERT '], ['ELC'], ['ELA', 'ELB', 'ELC', 'ERA'], ['ELB', 'ERA'], ['ELB', 'ERA'], ['ELA', 'ELC', 'ERA', 'EOGl']], ['ELB', 'ERA', 'ELC']], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 2.0, 2, [['C3'], ['F4'], [], ['F3'], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELC']], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 1.0, 3, [['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'C3', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4']], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'C3', 'C4', 'ELC']], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 1.5, 3, [['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'C4', 'O2'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4', 'C4'], ['F4', 'C4', 'O2']], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 1.5, 3, [['ELA', 'ELB', 'ERA', 'ERT '], ['ELC'], ['ELA', 'ELB', 'ELC', 'ERA'], ['ELB', 'ERA'], ['ELB', 'ERA'], ['ELA', 'ELC', 'ERA', 'EOGl']], ['ELB', 'ERA']], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 2.0, 3, [['C3'], ['F4'], [], ['F3'], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELC']], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 1.0, 4, [['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'F3', 'C3', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'C3', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4']], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'EOGr', 'F3', 'C3', 'C4', 'ELC']], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 1.5, 4, [['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'C4', 'O2'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4', 'C4'], ['F4', 'C4', 'O2']], ['F3', 'C3', 'O1', 'C4', 'O2']], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 1.5, 4, [['ELA', 'ELB', 'ERA', 'ERT '], ['ELC'], ['ELA', 'ELB', 'ELC', 'ERA'], ['ELB', 'ERA'], ['ELB', 'ERA'], ['ELA', 'ELC', 'ERA', 'EOGl']], ['ERA']], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 2.0, 4, [['C3'], ['F4'], [], ['F3'], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'ELC']], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 1.0, 5, [['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'F3', 'C3', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'C3', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4']], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'F3', 'C3', 'C4', 'ELC']], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 1.5, 5, [['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'C4', 'O2'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4', 'C4'], ['F4', 'C4', 'O2']], ['F3', 'C3', 'C4']], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 1.5, 5, [['ELA', 'ELB', 'ERA', 'ERT '], ['ELC'], ['ELA', 'ELB', 'ELC', 'ERA'], ['ELB', 'ERA'], ['ELB', 'ERA'], ['ELA', 'ELC', 'ERA', 'EOGl']], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 2.0, 5, [['C3'], ['F4'], [], ['F3'], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 1.0, 6, [['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ELC', 'ERA', 'EOGr', 'F3', 'C3', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERT ', 'EOGr', 'EMGl', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'F4', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'C3', 'C4'], ['ELA', 'ELB', 'ELC', 'ERA', 'ERB', 'ERC', 'ERE', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4']], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'F3', 'C3', 'C4']], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 1.5, 6, [['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ERA', 'F3', 'C3', 'O1', 'C4', 'O2'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['F3', 'C3', 'F4', 'C4'], ['F4', 'C4', 'O2']], ['C4']], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 1.5, 6, [['ELA', 'ELB', 'ERA', 'ERT '], ['ELC'], ['ELA', 'ELB', 'ELC', 'ERA'], ['ELB', 'ERA'], ['ELB', 'ERA'], ['ELA', 'ELC', 'ERA', 'EOGl']], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 2.0, 6, [['C3'], ['F4'], [], ['F3'], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 8000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ERA', 'ERB', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERB', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'EOGr', 'EOGl', 'ERE']], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 1.0, 1, [['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'EOGl', 'ERT ', 'ERC', 'ERE']], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'F3', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc', 'EOGl', 'F4', 'ERC', 'ERE']], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 1.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 1.5, 1, [['O1', 'F4', 'C4', 'O2', 'EMGc'], ['F3', 'C3', 'F4', 'C4', 'EMGc'], ['ERA', 'F3', 'C3', 'F4', 'EMGc'], ['F3', 'C3', 'F4', 'EMGc'], ['F3', 'C3', 'O1', 'F4', 'EMGc'], ['F3', 'C3', 'F4', 'C4']], ['O1', 'F4', 'C4', 'O2', 'EMGc', 'F3', 'C3', 'ERA']], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 1.5, 1, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], [], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 1.5, 1, [['ELA', 'ERA', 'EMGc'], ['ELA', 'ERA', 'EMGc'], ['ELA', 'EMGc'], ['ELA', 'EMGc'], ['ELA', 'ERA', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EMGc']], ['ELA', 'ERA', 'EMGc', 'ELB']], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 2.0, 1, [[], ['C3', 'C4'], [], [], [], ['F4', 'C4']], ['C3', 'C4', 'F4']], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ERA', 'ERB', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'EOGr', 'EOGl']], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 1.0, 2, [['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc']], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ERC']], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'F3', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc', 'ERC']], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 1.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 1.5, 2, [['O1', 'F4', 'C4', 'O2', 'EMGc'], ['F3', 'C3', 'F4', 'C4', 'EMGc'], ['ERA', 'F3', 'C3', 'F4', 'EMGc'], ['F3', 'C3', 'F4', 'EMGc'], ['F3', 'C3', 'O1', 'F4', 'EMGc'], ['F3', 'C3', 'F4', 'C4']], ['F4', 'C4', 'EMGc', 'F3', 'C3']], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 1.5, 2, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], [], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 1.5, 2, [['ELA', 'ERA', 'EMGc'], ['ELA', 'ERA', 'EMGc'], ['ELA', 'EMGc'], ['ELA', 'EMGc'], ['ELA', 'ERA', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EMGc']], ['ELA', 'ERA', 'EMGc']], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 2.0, 2, [[], ['C3', 'C4'], [], [], [], ['F4', 'C4']], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ERA', 'ERB', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'EOGr']], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 1.0, 3, [['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc']], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'F3', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'C4', 'EMGc']], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 1.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 1.5, 3, [['O1', 'F4', 'C4', 'O2', 'EMGc'], ['F3', 'C3', 'F4', 'C4', 'EMGc'], ['ERA', 'F3', 'C3', 'F4', 'EMGc'], ['F3', 'C3', 'F4', 'EMGc'], ['F3', 'C3', 'O1', 'F4', 'EMGc'], ['F3', 'C3', 'F4', 'C4']], ['F4', 'EMGc', 'F3', 'C3']], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 1.5, 3, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], [], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 1.5, 3, [['ELA', 'ERA', 'EMGc'], ['ELA', 'ERA', 'EMGc'], ['ELA', 'EMGc'], ['ELA', 'EMGc'], ['ELA', 'ERA', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EMGc']], ['ELA', 'EMGc']], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 2.0, 3, [[], ['C3', 'C4'], [], [], [], ['F4', 'C4']], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ERA', 'ERB', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'EOGr']], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 1.0, 4, [['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'F3', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'C4', 'EMGc']], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 1.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 1.5, 4, [['O1', 'F4', 'C4', 'O2', 'EMGc'], ['F3', 'C3', 'F4', 'C4', 'EMGc'], ['ERA', 'F3', 'C3', 'F4', 'EMGc'], ['F3', 'C3', 'F4', 'EMGc'], ['F3', 'C3', 'O1', 'F4', 'EMGc'], ['F3', 'C3', 'F4', 'C4']], ['F4', 'EMGc', 'F3', 'C3']], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 1.5, 4, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], [], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 1.5, 4, [['ELA', 'ERA', 'EMGc'], ['ELA', 'ERA', 'EMGc'], ['ELA', 'EMGc'], ['ELA', 'EMGc'], ['ELA', 'ERA', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EMGc']], ['ELA', 'EMGc']], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 2.0, 4, [[], ['C3', 'C4'], [], [], [], ['F4', 'C4']], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ERA', 'ERB', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'EOGr']], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 1.0, 5, [['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc']], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'F3', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'C4', 'EMGc']], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 1.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 1.5, 5, [['O1', 'F4', 'C4', 'O2', 'EMGc'], ['F3', 'C3', 'F4', 'C4', 'EMGc'], ['ERA', 'F3', 'C3', 'F4', 'EMGc'], ['F3', 'C3', 'F4', 'EMGc'], ['F3', 'C3', 'O1', 'F4', 'EMGc'], ['F3', 'C3', 'F4', 'C4']], ['F4', 'EMGc', 'F3', 'C3']], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 1.5, 5, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], [], ['EMGc']], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 1.5, 5, [['ELA', 'ERA', 'EMGc'], ['ELA', 'ERA', 'EMGc'], ['ELA', 'EMGc'], ['ELA', 'EMGc'], ['ELA', 'ERA', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EMGc']], ['ELA', 'EMGc']], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 2.0, 5, [[], ['C3', 'C4'], [], [], [], ['F4', 'C4']], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ERA', 'ERB', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 1.0, 6, [['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc']], ['F3', 'C3', 'F4', 'C4', 'EMGc']], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERC', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'O1', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERT ', 'EOGr', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERC', 'ERT ', 'ERE', 'EOGr', 'F3', 'C3', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'ERA', 'ERB', 'ERT ', 'EOGr', 'F3', 'C3', 'C4', 'EMGc']], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 1.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 1.5, 6, [['O1', 'F4', 'C4', 'O2', 'EMGc'], ['F3', 'C3', 'F4', 'C4', 'EMGc'], ['ERA', 'F3', 'C3', 'F4', 'EMGc'], ['F3', 'C3', 'F4', 'EMGc'], ['F3', 'C3', 'O1', 'F4', 'EMGc'], ['F3', 'C3', 'F4', 'C4']], ['F4']], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 1.5, 6, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], [], ['EMGc']], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 1.5, 6, [['ELA', 'ERA', 'EMGc'], ['ELA', 'ERA', 'EMGc'], ['ELA', 'EMGc'], ['ELA', 'EMGc'], ['ELA', 'ERA', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EMGc']], ['ELA', 'EMGc']], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 2.0, 6, [[], ['C3', 'C4'], [], [], [], ['F4', 'C4']], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 9000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'ELB', 'ERA']], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 1.0, 1, [['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EOGl', 'M1', 'M2', 'EMGc']], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'ERA']], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc', 'EMGl', 'O2', 'ERT ', 'M2', 'ERB']], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 1.5, 1, [[], ['C3'], [], [], [], ['C3']], ['C3']], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 1.5, 1, [['F3', 'F4', 'C4'], ['EOGr', 'F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['EOGr', 'F3', 'F4', 'C4'], ['F3', 'O1', 'F4', 'C4'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'F4', 'C4', 'EOGr', 'C3', 'O1', 'O2']], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 1.5, 1, [['ELA', 'EMGc'], ['EMGc'], ['EOGr', 'EMGc'], ['EOGl', 'C4', 'EMGc'], ['M1', 'EMGc'], ['EMGc']], ['ELA', 'EMGc', 'EOGr', 'EOGl', 'C4', 'M1']], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 2.0, 1, [['F4'], ['C3'], ['C4'], ['F3', 'F4'], [], []], ['F4', 'C3', 'C4', 'F3']], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 2.0, 1, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], [], []], ['EMGc']], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 2.5, 1, [[], [], [], ['F4'], [], []], ['F4']], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'ERA']], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 1.0, 2, [['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EOGl', 'M1', 'M2']], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'ERA']], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'EMGc', 'EMGl', 'M2']], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 1.5, 2, [[], ['C3'], [], [], [], ['C3']], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 1.5, 2, [['F3', 'F4', 'C4'], ['EOGr', 'F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['EOGr', 'F3', 'F4', 'C4'], ['F3', 'O1', 'F4', 'C4'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'F4', 'C4', 'C3', 'O1']], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 1.5, 2, [['ELA', 'EMGc'], ['EMGc'], ['EOGr', 'EMGc'], ['EOGl', 'C4', 'EMGc'], ['M1', 'EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 2.0, 2, [['F4'], ['C3'], ['C4'], ['F3', 'F4'], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 2.0, 2, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], [], []], ['EMGc']], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 2.5, 2, [[], [], [], ['F4'], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ERA']], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 1.0, 3, [['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'M1']], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ERA']], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'EMGc', 'M2']], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 1.5, 3, [[], ['C3'], [], [], [], ['C3']], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 1.5, 3, [['F3', 'F4', 'C4'], ['EOGr', 'F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['EOGr', 'F3', 'F4', 'C4'], ['F3', 'O1', 'F4', 'C4'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 1.5, 3, [['ELA', 'EMGc'], ['EMGc'], ['EOGr', 'EMGc'], ['EOGl', 'C4', 'EMGc'], ['M1', 'EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 2.0, 3, [['F4'], ['C3'], ['C4'], ['F3', 'F4'], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 2.0, 3, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], [], []], ['EMGc']], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 2.5, 3, [[], [], [], ['F4'], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 1.0, 4, [['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'M1']], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc', 'ERA']], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'EMGc']], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 1.5, 4, [[], ['C3'], [], [], [], ['C3']], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 1.5, 4, [['F3', 'F4', 'C4'], ['EOGr', 'F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['EOGr', 'F3', 'F4', 'C4'], ['F3', 'O1', 'F4', 'C4'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 1.5, 4, [['ELA', 'EMGc'], ['EMGc'], ['EOGr', 'EMGc'], ['EOGl', 'C4', 'EMGc'], ['M1', 'EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 2.0, 4, [['F4'], ['C3'], ['C4'], ['F3', 'F4'], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 2.0, 4, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], [], []], ['EMGc']], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 2.5, 4, [[], [], [], ['F4'], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 1.0, 5, [['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc', 'ERA']], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'EMGc']], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 1.5, 5, [[], ['C3'], [], [], [], ['C3']], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 1.5, 5, [['F3', 'F4', 'C4'], ['EOGr', 'F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['EOGr', 'F3', 'F4', 'C4'], ['F3', 'O1', 'F4', 'C4'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 1.5, 5, [['ELA', 'EMGc'], ['EMGc'], ['EOGr', 'EMGc'], ['EOGl', 'C4', 'EMGc'], ['M1', 'EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 2.0, 5, [['F4'], ['C3'], ['C4'], ['F3', 'F4'], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 2.0, 5, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 2.5, 5, [[], [], [], ['F4'], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 1.0, 6, [['ELA', 'EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['EOGr', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2'], ['EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ELB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc']], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERT ', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'F4', 'C4', 'EMGc']], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 1.5, 6, [[], ['C3'], [], [], [], ['C3']], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 1.5, 6, [['F3', 'F4', 'C4'], ['EOGr', 'F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['EOGr', 'F3', 'F4', 'C4'], ['F3', 'O1', 'F4', 'C4'], ['F3', 'C3', 'O1', 'F4', 'C4', 'O2']], ['F3', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 1.5, 6, [['ELA', 'EMGc'], ['EMGc'], ['EOGr', 'EMGc'], ['EOGl', 'C4', 'EMGc'], ['M1', 'EMGc'], ['EMGc']], ['EMGc']], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 2.0, 6, [['F4'], ['C3'], ['C4'], ['F3', 'F4'], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 2.0, 6, [['EMGc'], ['EMGc'], ['EMGc'], ['EMGc'], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 2.5, 6, [[], [], [], ['F4'], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 10000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 1.0, 1, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc']], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 1.0, 1, [['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'ELA', 'EOGl', 'M1', 'M2', 'O2']], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 1.0, 1, [['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4']], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EOGl', 'EMGc', 'ELB']], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 1.0, 1, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc', 'ERB', 'EMGl']], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 1.5, 1, [['C3', 'F4'], ['F3', 'C4'], [], ['F3'], [], ['C3']], ['C3', 'F4', 'F3', 'C4']], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 1.5, 1, [['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['EOGr', 'F3', 'O1', 'F4', 'O2'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4']], ['F3', 'C3', 'F4', 'C4', 'EOGr', 'O1', 'O2']], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 1.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 1.5, 1, [['ELA', 'M1', 'M2'], ['ELA', 'ERA', 'M1', 'M2', 'EMGc'], ['ERA', 'EOGr'], ['M1', 'M2'], ['ELA', 'ERA', 'M1', 'M2'], ['ELA', 'M1', 'M2']], ['ELA', 'M1', 'M2', 'ERA', 'EMGc', 'EOGr']], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 2.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 2.0, 1, [['C3', 'C4'], ['C4'], ['F3', 'C3'], ['F3', 'F4'], ['F4'], ['F3', 'C3', 'C4']], ['C3', 'C4', 'F3', 'F4']], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 2.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 2.5, 1, [['C3'], [], [], [], [], ['C3']], ['C3']], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 2.5, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 3.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 4.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 5.0, 1, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 1.0, 2, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 1.0, 2, [['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'ELA', 'M1', 'M2', 'O2']], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 1.0, 2, [['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4']], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 1.0, 2, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc', 'ERB']], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 1.5, 2, [['C3', 'F4'], ['F3', 'C4'], [], ['F3'], [], ['C3']], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 1.5, 2, [['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['EOGr', 'F3', 'O1', 'F4', 'O2'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4']], ['F3', 'C3', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 1.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 1.5, 2, [['ELA', 'M1', 'M2'], ['ELA', 'ERA', 'M1', 'M2', 'EMGc'], ['ERA', 'EOGr'], ['M1', 'M2'], ['ELA', 'ERA', 'M1', 'M2'], ['ELA', 'M1', 'M2']], ['ELA', 'M1', 'M2', 'ERA']], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 2.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 2.0, 2, [['C3', 'C4'], ['C4'], ['F3', 'C3'], ['F3', 'F4'], ['F4'], ['F3', 'C3', 'C4']], ['C4', 'F3', 'F4']], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 2.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 2.5, 2, [['C3'], [], [], [], [], ['C3']], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 2.5, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 3.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 4.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 5.0, 2, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 1.0, 3, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 1.0, 3, [['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4', 'M1', 'M2', 'O2']], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 1.0, 3, [['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4']], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 1.0, 3, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ELB', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc']], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 1.5, 3, [['C3', 'F4'], ['F3', 'C4'], [], ['F3'], [], ['C3']], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 1.5, 3, [['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['EOGr', 'F3', 'O1', 'F4', 'O2'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4']], ['F3', 'C3', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 1.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 1.5, 3, [['ELA', 'M1', 'M2'], ['ELA', 'ERA', 'M1', 'M2', 'EMGc'], ['ERA', 'EOGr'], ['M1', 'M2'], ['ELA', 'ERA', 'M1', 'M2'], ['ELA', 'M1', 'M2']], ['M1', 'M2']], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 2.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 2.0, 3, [['C3', 'C4'], ['C4'], ['F3', 'C3'], ['F3', 'F4'], ['F4'], ['F3', 'C3', 'C4']], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 2.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 2.5, 3, [['C3'], [], [], [], [], ['C3']], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 2.5, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 3.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 4.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 5.0, 3, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 1.0, 4, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 1.0, 4, [['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 1.0, 4, [['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4']], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 1.0, 4, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc']], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 1.5, 4, [['C3', 'F4'], ['F3', 'C4'], [], ['F3'], [], ['C3']], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 1.5, 4, [['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['EOGr', 'F3', 'O1', 'F4', 'O2'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4']], ['F3', 'F4']], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 1.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 1.5, 4, [['ELA', 'M1', 'M2'], ['ELA', 'ERA', 'M1', 'M2', 'EMGc'], ['ERA', 'EOGr'], ['M1', 'M2'], ['ELA', 'ERA', 'M1', 'M2'], ['ELA', 'M1', 'M2']], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 2.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 2.0, 4, [['C3', 'C4'], ['C4'], ['F3', 'C3'], ['F3', 'F4'], ['F4'], ['F3', 'C3', 'C4']], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 2.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 2.5, 4, [['C3'], [], [], [], [], ['C3']], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 2.5, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 3.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 4.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 5.0, 4, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 1.0, 5, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 1.0, 5, [['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 1.0, 5, [['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4']], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 1.0, 5, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 1.5, 5, [['C3', 'F4'], ['F3', 'C4'], [], ['F3'], [], ['C3']], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 1.5, 5, [['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['EOGr', 'F3', 'O1', 'F4', 'O2'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4']], ['F3', 'F4']], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 1.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 1.5, 5, [['ELA', 'M1', 'M2'], ['ELA', 'ERA', 'M1', 'M2', 'EMGc'], ['ERA', 'EOGr'], ['M1', 'M2'], ['ELA', 'ERA', 'M1', 'M2'], ['ELA', 'M1', 'M2']], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 2.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 2.0, 5, [['C3', 'C4'], ['C4'], ['F3', 'C3'], ['F3', 'F4'], ['F4'], ['F3', 'C3', 'C4']], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 2.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 2.5, 5, [['C3'], [], [], [], [], ['C3']], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 2.5, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 3.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 4.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 5.0, 5, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 1.0, 6, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2']], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 1.0, 6, [['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'EOGr', 'EOGl', 'F3', 'C3', 'O1', 'F4', 'C4'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4']], ['EOGr', 'F3', 'C3', 'O1', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 1.0, 6, [['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4']], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 1.0, 6, [['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ERA', 'ERB', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'O2', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4', 'EMGc'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'O2'], ['ELA', 'ELB', 'ERA', 'EOGr', 'EOGl', 'EMGl', 'M1', 'F3', 'C3', 'O1', 'M2', 'F4', 'C4', 'EMGc']], ['ELA', 'ERA', 'EOGr', 'M1', 'F3', 'C3', 'M2', 'F4', 'C4']], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 1.5, 6, [['C3', 'F4'], ['F3', 'C4'], [], ['F3'], [], ['C3']], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 1.5, 6, [['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4'], ['EOGr', 'F3', 'O1', 'F4', 'O2'], ['F3', 'C3', 'F4', 'C4'], ['F3', 'C3', 'F4', 'C4']], ['F3', 'F4']], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 1.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 1.5, 6, [['ELA', 'M1', 'M2'], ['ELA', 'ERA', 'M1', 'M2', 'EMGc'], ['ERA', 'EOGr'], ['M1', 'M2'], ['ELA', 'ERA', 'M1', 'M2'], ['ELA', 'M1', 'M2']], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 2.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 2.0, 6, [['C3', 'C4'], ['C4'], ['F3', 'C3'], ['F3', 'F4'], ['F4'], ['F3', 'C3', 'C4']], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 2.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 2.5, 6, [['C3'], [], [], [], [], ['C3']], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 2.5, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 3.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 4.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'mm', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_idx', 'um', 5.0, 6, [[], [], [], [], [], []], []], ['S_17/night_1\\EEG_raw', 11000, 'M_max', 'um', 5.0, 6, [[], [], [], [], [], []], []]]
print( )

## Accuracies

$$
\text{ } \\
Accuracy = \frac{\text{Number of correct predictions}}{\text{Total number of predictions}} = \frac{TP + TN}{TP + TN + FP + FN}\\
$$

In [14]:
def binary_broken_electrode_accuracy(y_preds, y_labels):
    TP = 0 # True positive
    TN = 0 # True negative
    F = 0 # Incorrect - false
    accuracies = []
    
    for data_idx in range(len(y_preds)):
        if len(y_preds[data_idx]) == len(y_labels[data_idx]) == 0:
            # if both lists are empty. 0 labels and 0 predictions.
            TN += 1
            accuracies.append(1)
            
        elif len(y_preds[data_idx]) != len(y_labels[data_idx]):
            # if the lists are not same lenght. Not same number of predictions as number of labels
            F += 1
            accuracies.append(0)
        else:
            # check if predictions are correct.
            total_correct = 0
            for chn in range(len(y_preds[data_idx])):
                if y_preds[data_idx][chn] in y_labels[data_idx]:
                    total_correct += 1
            if total_correct == len(y_labels[data_idx]):
                TP += 1
                accuracies.append(1)
            else:
                F += 1
                accuracies.append(0)
    return (TP + TN) / (TP + TN + F), accuracies, {'TP':TP, 'TN':TN, 'F':F}

In [15]:
def broken_electrode_accuracy(y_preds, y_labels):
    TP = 0 #true_positive
    FP = 0 #false_positive
    TN = 0 #true_negative no broken channels predicted and none labelled.
    FN = 0 #false_negative
    accuracies = []
    
    for data_idx in range(len(y_preds)):
        # Running through each data point
        
        interval_accuracy = []
        for pred_chn in range(len(y_preds[data_idx])):
            # Go through each channel prediction
            
            if y_preds[data_idx][pred_chn] in y_labels[data_idx]:
                # Correctly predicts a broken channel.
                TP += 1
                interval_accuracy.append('TP')
            else:
                # Predict a broken channel, that was not broken.
                FP += 1
                interval_accuracy.append('FP')
        
        for label_chn in range(len(y_labels[data_idx])):
            if y_labels[data_idx][label_chn] not in y_preds[data_idx]:
                # Failed to predict a broken channel.
                FN += 1
                interval_accuracy.append('FN')
        
        if len(y_labels[data_idx]) == 0 and len(y_preds[data_idx]) == 0:
            # Predict no broken and no broken is correct.
            TN += 1
            interval_accuracy.append('TN')
        accuracies.append(interval_accuracy)
    return (TP + TN) / (TP + FP + FN + TN), accuracies, {'TP':TP, 'TN':TN, 'FP':FP, 'FN':FN}

In [16]:
def both_broken_electrode_accuracy(y_preds, y_labels, binary):
    if binary:
        acc_score, acc, types = binary_broken_electrode_accuracy(y_preds, y_labels)
    else:
        acc_score, acc, types = broken_electrode_accuracy(y_preds, y_labels)
    return acc_score, acc, types

## Table 1

In [17]:
def accuracy_per_combination(outputs, data, binary, parameters_boolean):
    #outputs = predictions.
    acc_scores = []
    
    if parameters_boolean:
        # Making empty list for the data.
        # Measures
        acc_scores_single = {}
        y_pre_M_idx = []
        y_lab_M_idx = []
        y_pre_M_max = []
        y_lab_M_max = []
        
        # Matrices
        y_pre_um = []
        y_lab_um = []
        y_pre_mm = []
        y_lab_mm = []
        
        # Relative procents [1.00, 1.5, 2.00, 2.5, 3.00, 4.00, 5.00]
        y_pre_Rel_1 = []
        y_lab_Rel_1 = []
        y_pre_Rel_1_5 = []
        y_lab_Rel_1_5 = []
        y_pre_Rel_2 = []
        y_lab_Rel_2 = []
        y_pre_Rel_2_5 = []
        y_lab_Rel_2_5 = []
        y_pre_Rel_3 = []
        y_lab_Rel_3 = []
        y_pre_Rel_4 = []
        y_lab_Rel_4 = []
        y_pre_Rel_5 = []
        y_lab_Rel_5 = []
        
        # intervals [1, 2, 3, 4, 5, 6]
        y_pre_1 = []
        y_lab_1 = []
        y_pre_2 = []
        y_lab_2 = []
        y_pre_3 = []
        y_lab_3 = []
        y_pre_4 = []
        y_lab_4 = []
        y_pre_5 = []
        y_lab_5 = []
        y_pre_6 = []
        y_lab_6 = []
        
        
    for start_combination in range(0, 168):
        if parameters_boolean:
            # Calculating an accuracy for all combinations of each of the single parameters.
            
            # Measures
            if outputs[start_combination][2] == "M_idx":
                [y_pre_M_idx.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_M_idx.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][2] == "M_max":
                [y_pre_M_max.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_M_max.append(data[j][3]) for j in range(0, len(data))]
            
            # Matrices
            if outputs[start_combination][3] == "um":
                [y_pre_um.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_um.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][3] == "mm":
                [y_pre_mm.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_mm.append(data[j][3]) for j in range(0, len(data))]
                
            # Relative procents
            if outputs[start_combination][4] == 1.00:
                [y_pre_Rel_1.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_Rel_1.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][4] == 1.5:
                [y_pre_Rel_1_5.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_Rel_1_5.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][4] == 2.00:
                [y_pre_Rel_2.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_Rel_2.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][4] == 2.5:
                [y_pre_Rel_2_5.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_Rel_2_5.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][4] == 3.00:
                [y_pre_Rel_3.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_Rel_3.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][4] == 4.00:
                [y_pre_Rel_4.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_Rel_4.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][4] == 5.00:
                [y_pre_Rel_5.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_Rel_5.append(data[j][3]) for j in range(0, len(data))]
            
            # intervals
            if outputs[start_combination][5] == 1:
                [y_pre_1.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_1.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][5] == 2:
                [y_pre_2.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_2.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][5] == 3:
                [y_pre_3.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_3.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][5] == 4:
                [y_pre_4.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_4.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][5] == 5:
                [y_pre_5.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_5.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][5] == 6:
                [y_pre_6.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_6.append(data[j][3]) for j in range(0, len(data))]
        else:
            y_pre_each_combination = [outputs[i][7] for i in range(start_combination, len(outputs), 168)]
            y_lab_each_combination = [data[j][3] for j in range(0, len(data))]
            #print("Combination: " + str(outputs[start_combination][2:6]))
        
            acc_score, acc, types = both_broken_electrode_accuracy(y_pre_each_combination, y_lab_each_combination, binary)
            acc_scores.append(acc_score)
            #print(str(acc_score)[:5])
    
    if parameters_boolean:
        acc_scores_single['M_idx'] = both_broken_electrode_accuracy(y_pre_M_idx, y_lab_M_idx, binary)
        acc_scores_single['M_max'] = both_broken_electrode_accuracy(y_pre_M_max, y_lab_M_max, binary)
        acc_scores_single['um'] = both_broken_electrode_accuracy(y_pre_um, y_lab_um, binary)
        acc_scores_single['mm'] = both_broken_electrode_accuracy(y_pre_mm, y_lab_mm, binary)
        acc_scores_single['Rel_1'] = both_broken_electrode_accuracy(y_pre_Rel_1, y_lab_Rel_1, binary)
        acc_scores_single['Rel_1.5'] = both_broken_electrode_accuracy(y_pre_Rel_1_5, y_lab_Rel_1_5, binary)
        acc_scores_single['Rel_2'] = both_broken_electrode_accuracy(y_pre_Rel_2, y_lab_Rel_2, binary)
        acc_scores_single['Rel_2.5'] = both_broken_electrode_accuracy(y_pre_Rel_2_5, y_lab_Rel_2_5, binary)
        acc_scores_single['Rel_3'] = both_broken_electrode_accuracy(y_pre_Rel_3, y_lab_Rel_3, binary)
        acc_scores_single['Rel_4'] = both_broken_electrode_accuracy(y_pre_Rel_4, y_lab_Rel_4, binary)
        acc_scores_single['Rel_5'] = both_broken_electrode_accuracy(y_pre_Rel_5, y_lab_Rel_5, binary)
        acc_scores_single['Int_1'] = both_broken_electrode_accuracy(y_pre_1, y_lab_1, binary)
        acc_scores_single['Int_2'] = both_broken_electrode_accuracy(y_pre_2, y_lab_2, binary)
        acc_scores_single['Int_3'] = both_broken_electrode_accuracy(y_pre_3, y_lab_3, binary)
        acc_scores_single['Int_4'] = both_broken_electrode_accuracy(y_pre_4, y_lab_4, binary)
        acc_scores_single['Int_5'] = both_broken_electrode_accuracy(y_pre_5, y_lab_5, binary)
        acc_scores_single['Int_6'] = both_broken_electrode_accuracy(y_pre_6, y_lab_6, binary)
       
        return acc_scores_single
    else:
        return outputs[np.argmax(acc_scores)][2:6], max(acc_scores)

In [18]:
dictionary = accuracy_per_combination(training_predictions, training_data, binary=True, parameters_boolean=True)
for key in dictionary.keys():
    print(str(key) + "=" + str(dictionary.get(key)[0])[:5])

M_idx=0.333
M_max=0.362
um=0.265
mm=0.430
Rel_1=0.042
Rel_1.5=0.310
Rel_2=0.442
Rel_2.5=0.437
Rel_3=0.415
Rel_4=0.404
Rel_5=0.382
Int_1=0.283
Int_2=0.340
Int_3=0.367
Int_4=0.369
Int_5=0.360
Int_6=0.365


In [19]:
dictionary = accuracy_per_combination(training_predictions, training_data, binary=False, parameters_boolean=True)
for key in dictionary.keys():
    print(str(key) + "=" + str(dictionary.get(key)[0])[:5])

M_idx=0.188
M_max=0.215
um=0.099
mm=0.330
Rel_1=0.066
Rel_1.5=0.218
Rel_2=0.400
Rel_2.5=0.409
Rel_3=0.385
Rel_4=0.357
Rel_5=0.331
Int_1=0.143
Int_2=0.189
Int_3=0.215
Int_4=0.234
Int_5=0.237
Int_6=0.245


Accuracy of combinations

## Table 2

In [20]:
def accuracy_per_combination(outputs, data, binary, parameters_boolean):
    #outputs = predictions.
    acc_scores = []
    types_list = []
    
    if parameters_boolean:
        # Making empty list for the data.
        # Measures
        acc_scores_single = {}
        y_pre_M_idx = []
        y_lab_M_idx = []
        y_pre_M_max = []
        y_lab_M_max = []
        
        # Matrices
        y_pre_mm = []
        y_lab_mm = []
        
        # Relative procents [1.00, 1.5, 2.00, 2.5, 3.00, 4.00, 5.00]
        y_pre_Rel_1_5 = []
        y_lab_Rel_1_5 = []
        y_pre_Rel_2 = []
        y_lab_Rel_2 = []
        y_pre_Rel_2_5 = []
        y_lab_Rel_2_5 = []
        y_pre_Rel_3 = []
        y_lab_Rel_3 = []
        y_pre_Rel_4 = []
        y_lab_Rel_4 = []
        y_pre_Rel_5 = []
        y_lab_Rel_5 = []
        
        # intervals [1, 2, 3, 4, 5, 6]
        y_pre_2 = []
        y_lab_2 = []
        y_pre_3 = []
        y_lab_3 = []
        y_pre_4 = []
        y_lab_4 = []
        y_pre_5 = []
        y_lab_5 = []
        y_pre_6 = []
        y_lab_6 = []
        
        
    for start_combination in range(0, 168):
        if parameters_boolean:
            # Calculating an accuracy for all combinations of each of the single parameters.
            if outputs[start_combination][3] == "um":
                continue
            elif outputs[start_combination][4] == 1.00:
                continue
            elif outputs[start_combination][5] == 1:
                continue
            #elif outputs[start_combination][5] == 2:
            #    continue
            
            
            # Measures
            if outputs[start_combination][2] == "M_idx":
                [y_pre_M_idx.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_M_idx.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][2] == "M_max":
                [y_pre_M_max.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_M_max.append(data[j][3]) for j in range(0, len(data))]
            
            # Matrices
            if outputs[start_combination][3] == "mm":
                [y_pre_mm.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_mm.append(data[j][3]) for j in range(0, len(data))]
                
            # Relative procents
            if outputs[start_combination][4] == 1.5:
                [y_pre_Rel_1_5.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_Rel_1_5.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][4] == 2.00:
                [y_pre_Rel_2.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_Rel_2.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][4] == 2.5:
                [y_pre_Rel_2_5.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_Rel_2_5.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][4] == 3.00:
                [y_pre_Rel_3.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_Rel_3.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][4] == 4.00:
                [y_pre_Rel_4.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_Rel_4.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][4] == 5.00:
                [y_pre_Rel_5.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_Rel_5.append(data[j][3]) for j in range(0, len(data))]
            
            # intervals
            if outputs[start_combination][5] == 2:
                [y_pre_2.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_2.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][5] == 3:
                [y_pre_3.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_3.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][5] == 4:
                [y_pre_4.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_4.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][5] == 5:
                [y_pre_5.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_5.append(data[j][3]) for j in range(0, len(data))]
            elif outputs[start_combination][5] == 6:
                [y_pre_6.append(outputs[i][7]) for i in range(start_combination, len(outputs), 168)]
                [y_lab_6.append(data[j][3]) for j in range(0, len(data))]
        else:
            y_pre_each_combination = [outputs[i][7] for i in range(start_combination, len(outputs), 168)]
            y_lab_each_combination = [data[j][3] for j in range(0, len(data))]
            #print("Combination: " + str(outputs[start_combination][2:6]))
        
            acc_score, acc, types = both_broken_electrode_accuracy(y_pre_each_combination, y_lab_each_combination, binary)
            acc_scores.append(acc_score)
            types_list.append(types)
            #print(str(acc_score)[:5])
    
    if parameters_boolean:
        acc_scores_single['M_idx'] = both_broken_electrode_accuracy(y_pre_M_idx, y_lab_M_idx, binary)
        acc_scores_single['M_max'] = both_broken_electrode_accuracy(y_pre_M_max, y_lab_M_max, binary)
        acc_scores_single['mm'] = both_broken_electrode_accuracy(y_pre_mm, y_lab_mm, binary)
        acc_scores_single['Rel_1.5'] = both_broken_electrode_accuracy(y_pre_Rel_1_5, y_lab_Rel_1_5, binary)
        acc_scores_single['Rel_2'] = both_broken_electrode_accuracy(y_pre_Rel_2, y_lab_Rel_2, binary)
        acc_scores_single['Rel_2.5'] = both_broken_electrode_accuracy(y_pre_Rel_2_5, y_lab_Rel_2_5, binary)
        acc_scores_single['Rel_3'] = both_broken_electrode_accuracy(y_pre_Rel_3, y_lab_Rel_3, binary)
        acc_scores_single['Rel_4'] = both_broken_electrode_accuracy(y_pre_Rel_4, y_lab_Rel_4, binary)
        acc_scores_single['Rel_5'] = both_broken_electrode_accuracy(y_pre_Rel_5, y_lab_Rel_5, binary)
        acc_scores_single['Int_2'] = both_broken_electrode_accuracy(y_pre_2, y_lab_2, binary)
        acc_scores_single['Int_3'] = both_broken_electrode_accuracy(y_pre_3, y_lab_3, binary)
        acc_scores_single['Int_4'] = both_broken_electrode_accuracy(y_pre_4, y_lab_4, binary)
        acc_scores_single['Int_5'] = both_broken_electrode_accuracy(y_pre_5, y_lab_5, binary)
        acc_scores_single['Int_6'] = both_broken_electrode_accuracy(y_pre_6, y_lab_6, binary)
       
        return acc_scores_single
    else:
        # Extract the 10 combination with highest accuracies.
        arg_sort = np.argsort(acc_scores)
        for idx in arg_sort[-10:]:
            print(str(outputs[idx][2:6]) + " " + str(acc_scores[idx]) + str(types_list[idx]))
        return outputs[np.argmax(acc_scores)][2:6], max(acc_scores)

In [21]:
dictionary = accuracy_per_combination(training_predictions, training_data, binary = True, parameters_boolean = True)
for key in dictionary.keys():
    print(str(key) + "=" + str(dictionary.get(key)[0])[:5])

M_idx=0.463
M_max=0.556
mm=0.509
Rel_1.5=0.507
Rel_2=0.603
Rel_2.5=0.56
Rel_3=0.501
Rel_4=0.469
Rel_5=0.416
Int_2=0.496
Int_3=0.539
Int_4=0.528
Int_5=0.492
Int_6=0.492


In [22]:
dictionary = accuracy_per_combination(training_predictions, training_data, binary = False, parameters_boolean = True)
for key in dictionary.keys():
    print(str(key) + "=" + str(dictionary.get(key)[0])[:5])

M_idx=0.457
M_max=0.583
mm=0.522
Rel_1.5=0.509
Rel_2=0.653
Rel_2.5=0.591
Rel_3=0.525
Rel_4=0.456
Rel_5=0.398
Int_2=0.509
Int_3=0.549
Int_4=0.544
Int_5=0.507
Int_6=0.504


## Best combinations on measured training set

In [23]:
accuracy_per_combination(training_predictions, training_data, binary=True, parameters_boolean=False)

['M_max', 'mm', 2.5, 4] 0.6615384615384615{'TP': 25, 'TN': 18, 'F': 22}
['M_idx', 'mm', 1.5, 3] 0.6615384615384615{'TP': 25, 'TN': 18, 'F': 22}
['M_max', 'mm', 2.0, 6] 0.6615384615384615{'TP': 23, 'TN': 20, 'F': 22}
['M_max', 'mm', 2.0, 4] 0.676923076923077{'TP': 28, 'TN': 16, 'F': 21}
['M_idx', 'mm', 1.5, 6] 0.676923076923077{'TP': 24, 'TN': 20, 'F': 21}
['M_max', 'mm', 2.0, 5] 0.676923076923077{'TP': 26, 'TN': 18, 'F': 21}
['M_max', 'mm', 2.0, 3] 0.6923076923076923{'TP': 33, 'TN': 12, 'F': 20}
['M_idx', 'mm', 1.5, 4] 0.7076923076923077{'TP': 28, 'TN': 18, 'F': 19}
['M_max', 'mm', 2.5, 2] 0.7230769230769231{'TP': 32, 'TN': 15, 'F': 18}
['M_max', 'mm', 2.5, 3] 0.7384615384615385{'TP': 30, 'TN': 18, 'F': 17}


(['M_max', 'mm', 2.5, 3], 0.7384615384615385)

In [24]:
accuracy_per_combination(training_predictions, training_data, binary=False, parameters_boolean=False)

['M_idx', 'mm', 1.5, 4] 0.7115384615384616{'TP': 56, 'TN': 18, 'FP': 19, 'FN': 11}
['M_max', 'mm', 2.0, 6] 0.7191011235955056{'TP': 44, 'TN': 20, 'FP': 2, 'FN': 23}
['M_max', 'mm', 2.5, 4] 0.7222222222222222{'TP': 47, 'TN': 18, 'FP': 5, 'FN': 20}
['M_max', 'mm', 3.0, 2] 0.7252747252747253{'TP': 51, 'TN': 15, 'FP': 9, 'FN': 16}
['M_idx', 'mm', 1.5, 6] 0.7263157894736842{'TP': 49, 'TN': 20, 'FP': 8, 'FN': 18}
['M_max', 'mm', 2.0, 5] 0.7303370786516854{'TP': 47, 'TN': 18, 'FP': 4, 'FN': 20}
['M_max', 'mm', 2.0, 4] 0.7692307692307693{'TP': 54, 'TN': 16, 'FP': 8, 'FN': 13}
['M_max', 'mm', 2.0, 3] 0.7717391304347826{'TP': 59, 'TN': 12, 'FP': 13, 'FN': 8}
['M_max', 'mm', 2.5, 2] 0.7731958762886598{'TP': 60, 'TN': 15, 'FP': 15, 'FN': 7}
['M_max', 'mm', 2.5, 3] 0.7777777777777778{'TP': 52, 'TN': 18, 'FP': 5, 'FN': 15}


(['M_max', 'mm', 2.5, 3], 0.7777777777777778)

## Result of the best combination on measured test set.

In [25]:
check_file_exists(test_data, False)
type(['mm', 'um']) == list

True

In [26]:
#test_prediction = optimise_set(test_data, False, ['M_max', 'mm', 2.5, 3]) # calculation prediction of the best combination

In [27]:
test_prediction = [['S_01/night_1\\EEG_raw', 170, 'M_max', 'mm', 2.5, 3, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], ['EMGr', 'C3']], ['C3']], ['S_01/night_1\\EEG_raw', 960, 'M_max', 'mm', 2.5, 3, [['ERB', 'C3'], ['ERB'], ['ERB'], ['ERB', 'C3'], ['C3'], ['C3']], ['ERB', 'C3']], ['S_01/night_1\\EEG_raw', 1830, 'M_max', 'mm', 2.5, 3, [['C3'], ['C3'], ['C3'], ['C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_1\\EEG_raw', 6290, 'M_max', 'mm', 2.5, 3, [['ELI', 'C3'], ['C3'], ['C3'], ['ELI', 'C3'], ['ELI', 'C3'], ['ELI', 'C3']], ['ELI', 'C3']], ['S_01/night_1\\EEG_raw', 7050, 'M_max', 'mm', 2.5, 3, [['C3'], ['C3'], ['C3'], ['ERB', 'C3'], ['C3'], ['C3']], ['C3']], ['S_01/night_3\\EEG_raw', 240, 'M_max', 'mm', 2.5, 3, [['O2'], [], ['EOGr'], [], ['O1'], []], []], ['S_01/night_4\\EEG_raw', 15320, 'M_max', 'mm', 2.5, 3, [['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl'], ['ERA', 'EOGl']], ['ERA', 'EOGl']], ['S_01/night_4\\EEG_raw', 1670, 'M_max', 'mm', 2.5, 3, [['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl'], ['EOGl']], ['EOGl']], ['S_12/night_1\\EEG_raw', 10, 'M_max', 'mm', 2.5, 3, [['F4'], ['EOGr', 'F4'], ['EOGr', 'F4'], ['EOGr', 'F4'], ['F4'], ['EOGr', 'F4']], ['F4', 'EOGr']], ['S_12/night_1\\EEG_raw', 510, 'M_max', 'mm', 2.5, 3, [['EOGr', 'F4'], ['EOGr', 'F4'], ['EOGr', 'F4'], ['EOGr', 'F4'], ['EOGr', 'F4'], ['EOGr', 'F4']], ['EOGr', 'F4']], ['S_12/night_1\\EEG_raw', 2000, 'M_max', 'mm', 2.5, 3, [['EOGr', 'F4'], ['EOGr', 'F4'], ['F4'], ['EOGr', 'EMGl', 'F4'], ['F4'], ['EOGr', 'F4']], ['F4']], ['S_12/night_1\\EEG_raw', 3000, 'M_max', 'mm', 2.5, 3, [['EOGr', 'F4'], ['F4'], ['EOGr', 'F4'], ['EOGr', 'F4'], ['EOGr', 'F4'], ['EOGr', 'F4']], ['EOGr', 'F4']], ['S_13/night_1\\EEG_raw', 8000, 'M_max', 'mm', 2.5, 3, [['F3', 'F4'], [], [], [], [], []], []], ['S_13/night_1\\EEG_raw', 10000, 'M_max', 'mm', 2.5, 3, [['F4'], ['ELC', 'EMGl', 'EMGc'], ['F3', 'F4'], [], ['F3', 'F4'], ['F3']], []], ['S_14/night_1\\EEG_raw', 2500, 'M_max', 'mm', 2.5, 3, [['ELC', 'EMGc'], ['ELC', 'EMGc'], ['ELC', 'EMGc'], ['ELC', 'EMGc'], ['ELC', 'EMGc'], ['ELC', 'EMGc']], ['ELC', 'EMGc']], ['S_14/night_1\\EEG_raw', 4500, 'M_max', 'mm', 2.5, 3, [['F4'], ['F4'], ['F4'], ['F4'], ['F4'], ['F4']], ['F4']]]

In [28]:
def test_accuracies(outputs, data, binary):        
    y_pre_each_combination = [outputs[i][7] for i in range(0, len(outputs))]
    y_lab_each_combination = [data[j][3] for j in range(0, len(data))]
    acc_score, acc, types = both_broken_electrode_accuracy(y_pre_each_combination, y_lab_each_combination, binary)
    return acc_score, types
print(test_accuracies(test_prediction, test_data, binary=True))
print(test_accuracies(test_prediction, test_data, binary=False))

(0.75, {'TP': 10, 'TN': 2, 'F': 4})
(0.8076923076923077, {'TP': 19, 'TN': 2, 'FP': 1, 'FN': 4})
